# Part II: DC Performance Prediction

In [ ]:
from pprint import pprint
import sys

import matplotlib
from matplotlib import pyplot
import numpy
import pandas
import pvlib

import pvfit
from pvfit.common import E_hemispherical_tilted_W_per_m2_stc, T_degC_stc
from pvfit.measurement.iv.types import FTData, IVPerformanceMatrix
from pvfit.measurement.spectral_correction.spectra import E_60904_3_hemispherical_tilted
from pvfit.measurement.spectral_correction.computation import (
    SpectralIrradiance,
    SpectralResponsivity,
    extrapolate_spectral_irradiance,
    inner_product,
)
from pvfit.modeling.dc.common import (
    Material,
    iam_factory,
)
import pvfit.modeling.dc.single_diode.equation.simple.simulation as sde_simple_sim
import pvfit.modeling.dc.single_diode.model.simple.auxiliary_equations as sdm_simple_ae
import pvfit.modeling.dc.single_diode.model.simple.inference_matrix as sdm_simple_inf_matrix

print("Python version: %s.%s.%s" % sys.version_info[:3])
print("matplotlib version: " + matplotlib.__version__)
print("numpy version: " + numpy.__version__)
print("pandas version: " + pandas.__version__)
print("pvlib version: " + pvlib.__version__)
print("pvfit version: " + pvfit.__version__)

## Configuration

### Location

In [ ]:
# Albuquerque, NM, USA
location = pvlib.location.Location(
    latitude=35.0546,
    longitude=-106.5401,
    tz="Etc/GMT+7",
    altitude=1600,
    name="Albuquerque, NM, USA",
)

### Array (here, just one module)

In [ ]:
array = {
    "albedo": 0.19,  # Average of monthly values.
    "surface_tilt_deg": 35.0,
    "surface_azimuth_deg": 180.0,
    "commission_date": pandas.Timestamp(tz='Etc/GMT+7', year=2021, month=7, day=1),
}

### Module-Specific Parameters

In [ ]:
# Special thanks to PVPMC: https://pvpmc.sandia.gov/pv-research/pv-lifetime-project/pv-lifetime-modules/
# Special thanks to PV Performance Labs: https://datahub.duramat.org/en/dataset/gni-spectra-abq
# LG320N1K-A5 320W N-PERT Si, CFV Report #s 19074-PR-E-005 and 20125-PR-E-005
module_spec = {
    "make": "Canadian Solar",
    "model": "CS6K-275M",
    "length_mm": 1650,
    "width_mm": 992,
    "thickness_mm": 40,
    "material": Material.monoSi,
    "N_s": 60,
    "T_degC_0": T_degC_stc,
    "E_W_per_m2_0": E_hemispherical_tilted_W_per_m2_stc,
    # I-V Performance from IEC 61853-1.
    "I_sc_A_0": 9.299,
    "I_mp_A_0": 8.810,
    "P_mp_W_0": 277.37,
    "V_mp_V_0": 31.48,
    "V_oc_V_0": 38.29,
    "dI_sc_dT_A_per_degC_0": 0.00330,  # 0.0355 %/degC
    "dP_mp_dT_W_per_degC_0": -1.1522,  # -0.4155 %/degC
    "dV_oc_dT_V_per_degC_0": -0.1178,  # -0.3078 %/degC
    "I_sc_A": numpy.array([
        1.02432156592117, 2.04220329161805, 4.08876231489616, 6.16742104333028, 8.23620585618444, 10.3278357876755,
        1.02757453945704, 2.04102689488776, 4.0971414004165, 6.18719186830119, 8.26393082981699, 10.3551722873482, 11.395279503447,
        1.03913626907321, 2.06392255863692, 4.13658036822046, 6.23104469564041, 8.32545975924577, 10.4337969237837, 11.4824044494735,
        1.04491230559491, 2.0826858185294, 4.17166165651346, 6.27316031746301, 8.38355926709284, 10.4998465959987, 11.5723836702704,
    ]),
    "I_mp_A": numpy.array([
        0.97751989541629, 1.95726960371108, 3.92418023622383, 5.88128100651897, 7.83318220601609, 9.79347790957467,
        0.98916629640619, 1.94985466389299, 3.91453936347665, 5.86450863231455, 7.82315841412953, 9.78856875676654, 10.7347437361592,
        0.979284585841039, 1.9562110868987, 3.89343224765022, 5.85540132593549, 7.80002485702661, 9.7568852648073, 10.7216370906218,
        0.974720372318489, 1.952125140497, 3.88896895682736, 5.83030275932059, 7.76392690540913, 9.68589365355965, 10.6291077113922,
    ]),
    "V_mp_V": numpy.array([
        32.483559723954, 33.258802022921, 33.8340393593184, 34.0645574854619, 34.0867190147778, 33.9728268087745,
        30.9778660821221, 31.9811420769189, 32.594898159062, 32.8194215533058, 32.8550150165226, 32.7239339231629, 32.7049519389132,
        27.8308170148312, 28.7322982525907, 29.5538016200052, 29.6987353242727, 29.6990451824235, 29.5689893366802, 29.4925560748204,
        24.4491348564815, 25.448012412096, 26.219831977235, 26.427468974117, 26.4770877509191, 26.4207371130951, 26.4033443018523,
    ]),
    "V_oc_V": numpy.array([
        37.5801692934928, 38.6911107010062, 39.8079483178206, 40.4518064164751, 40.9107513118948, 41.2545791691551,
        36.3299943128803, 37.469502764561, 38.62911737847, 39.2968966959977, 39.7727741849426, 40.1323305920718, 40.2987316268181,
        33.1224743848133, 34.3345551259948, 35.5886134033995, 36.3181133114656, 36.8413940279752, 37.2287178272536, 37.4032160396215,
        29.7485796556728, 31.0865085511878, 32.4495485404176, 33.2484353792992, 33.8118471371682, 34.2675187516404, 34.4526444363561,
    ]),
    "E_W_per_m2": numpy.array([
        100, 200, 400, 600, 800, 1000,
        100, 200, 400, 600, 800, 1000, 1100,
        100, 200, 400, 600, 800, 1000, 1100,
        100, 200, 400, 600, 800, 1000, 1100,
    ]),
    "T_degC": numpy.array([
        15, 15, 15, 15, 15, 15,
        25, 25, 25, 25, 25, 25, 25,
        50, 50, 50, 50, 50, 50, 50,
        75, 75, 75, 75, 75, 75, 75,
    ]),
    "iam_angle_deg": numpy.array([0, 10, 20, 30, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]),
    # IAM from IEC 61853-2.
    "iam_frac": numpy.array([
        1.0000, 0.9989, 1.0014, 1.0002, 0.9984, 0.9941, 0.9911, 0.9815, 0.9631, 0.9352,
        0.8922, 0.8134, 0.6778, 0.4541, 0.0000,
    ]),
    # DeltaT  measured in IEC 61853 standard.
    "sapm": {
        "DeltaT_degC": 3.0,  # Stock value.
    },
    # Temperature parameters from IEC 61853-2.
    "faiman": {
        "u0": 24.229,  # IEC 61853-2, actually installation dependent.
        "u1": 7.182,  # IEC 61853-2, actually installation dependent.
    },
    # Spectral responsivity.
    "sr": {
        "lambda_nm": numpy.array([
            300,  305,  310,  315,  320,  325,  330,  335,  340,  345,  350,
            355,  360,  365,  370,  375,  380,  385,  390,  395,  400,  405,
            410,  415,  420,  425,  430,  435,  440,  445,  450,  455,  460,
            465,  470,  475,  480,  485,  490,  495,  500,  505,  510,  515,
            520,  525,  530,  535,  540,  545,  550,  555,  560,  565,  570,
            575,  580,  585,  590,  595,  600,  605,  610,  615,  620,  625,
            630,  635,  640,  645,  650,  655,  660,  665,  670,  675,  680,
            685,  690,  695,  700,  705,  710,  715,  720,  725,  730,  735,
            740,  745,  750,  755,  760,  765,  770,  775,  780,  785,  790,
            795,  800,  805,  810,  815,  820,  825,  830,  835,  840,  845,
            850,  855,  860,  865,  870,  875,  880,  885,  890,  895,  900,
            905,  910,  915,  920,  925,  930,  935,  940,  945,  950,  955,
            960,  965,  970,  975,  980,  985,  990,  995, 1000, 1005, 1010,
        1015, 1020, 1025, 1030, 1035, 1040, 1045, 1050, 1055, 1060, 1065,
        1070, 1075, 1080, 1085, 1090, 1095, 1100, 1105, 1110, 1115, 1120,
        1125, 1130, 1135, 1140, 1145, 1150, 1155, 1160, 1165, 1170, 1175,
        1180, 1185, 1190, 1195, 1200
        ]),
        # Spectral responsivity is actually normalized here.
        "S_A_per_W": numpy.array([
            0.        , 0.03730679, 0.07626343, 0.11847496, 0.16214371,
            0.20241547, 0.24121857, 0.27735138, 0.30130768, 0.31400299,
            0.3235321 , 0.33115387, 0.33811951, 0.34568787, 0.35356903,
            0.36102295, 0.36860657, 0.37688446, 0.38674927, 0.39775848,
            0.40844727, 0.41732788, 0.42365265, 0.42974091, 0.43779755,
            0.44664764, 0.45428467, 0.45993805, 0.46470642, 0.46937561,
            0.47473907, 0.4812851 , 0.48860168, 0.4961853 , 0.50349426,
            0.51046753, 0.51734924, 0.52415466, 0.53088379, 0.53749084,
            0.54402161, 0.55049133, 0.5569458 , 0.56338501, 0.56977844,
            0.57606506, 0.58227539, 0.58839417, 0.5944519 , 0.60043335,
            0.60636902, 0.61228943, 0.61819458, 0.62406921, 0.62991333,
            0.63572693, 0.64151001, 0.64724731, 0.65293884, 0.65858459,
            0.66418457, 0.66973877, 0.67523193, 0.68057251, 0.6857605 ,
            0.69091797, 0.69612122, 0.70152283, 0.70704651, 0.71263123,
            0.71836853, 0.72431946, 0.7305603 , 0.73770142, 0.74452209,
            0.74964905, 0.75404358, 0.75839233, 0.76341248, 0.76992798,
            0.77748108, 0.78486633, 0.7908783 , 0.79501343, 0.79846191,
            0.80264282, 0.80912781, 0.81687927, 0.82342529, 0.82795715,
            0.83171082, 0.83528137, 0.83921814, 0.84378052, 0.84854126,
            0.85316467, 0.85771179, 0.86224365, 0.86677551, 0.87138367,
            0.87609863, 0.88096619, 0.8860321 , 0.8914032 , 0.89706421,
            0.90284729, 0.90861511, 0.91445923, 0.9203949 , 0.92622375,
            0.9316864 , 0.93670654, 0.94146729, 0.9460907 , 0.95069885,
            0.95556641, 0.96055603, 0.96516418, 0.96893311, 0.97180176,
            0.97418213, 0.9763031 , 0.97834778, 0.98054504, 0.98309326,
            0.9866333 , 0.99104309, 0.99537659, 0.99868774, 1.        ,
            0.99981689, 0.99931335, 0.99848938, 0.99739075, 0.99601746,
            0.99200439, 0.98422241, 0.97460938, 0.96508789, 0.95632935,
            0.94744873, 0.93798828, 0.92750549, 0.91557312, 0.9017334 ,
            0.8842926 , 0.86245728, 0.83709717, 0.80908203, 0.77926636,
            0.74855042, 0.71430969, 0.67553711, 0.63542175, 0.59719849,
            0.5609436 , 0.52516174, 0.49068451, 0.45831299, 0.42822266,
            0.39994812, 0.3733902 , 0.34738159, 0.32099152, 0.29441071,
            0.26786041, 0.24153137, 0.21562958, 0.19035721, 0.16592407,
            0.14252472, 0.12036705, 0.09965515, 0.08058929, 0.06337738,
            0.04821968, 0.03532124, 0.02321959, 0.01180935, 0.00332475,
            0.
        ]),
    }
}

#### Incident-Angle Modifier (IAM) Function

In [ ]:
# Use Piecewise Cubic Hermite Interpolating Polynomial (PCHIP) interpolation with zero tail.
iam = iam_factory(iam_angle_deg=module_spec["iam_angle_deg"], iam_frac=module_spec["iam_frac"])

angle_deg = numpy.linspace(0, 95, 200)
fig, ax = pyplot.subplots()
ax.plot(module_spec["iam_angle_deg"], module_spec["iam_frac"], '.', angle_deg, iam(angle_deg=angle_deg))
ax.set_ylabel("Modifier")
ax.set_xlabel("Incident Angle (deg)")
ax.set_title("Incident-Angle Modifier (IAM)")
pass

#### Module Spectral Responsivity

In [ ]:
# PVfit provides SpectralResponsivity class to wrap SR data.
spectral_responsivity = SpectralResponsivity(
    lambda_nm=module_spec["sr"]["lambda_nm"],
    S_A_per_W=module_spec["sr"]["S_A_per_W"],
)

angle_deg = numpy.linspace(0, 95, 200)
fig, ax = pyplot.subplots()
ax.plot(spectral_responsivity.lambda_nm, spectral_responsivity.S_A_per_W, '.')
ax.set_ylabel('S (1)')
ax.set_xlabel('Wavelength (nm)')
ax.set_title('Normalized Spectral Responsivity')
pass

### Fit to simple SDM

In [ ]:
# Fit simple SDM to IEC-81853-1 data.
iv_performance_matrix = IVPerformanceMatrix(
    material=Material.monoSi,
    N_s=60,
    I_sc_A=module_spec["I_sc_A"],
    I_mp_A=module_spec["I_mp_A"],
    V_mp_V=module_spec["V_mp_V"],
    V_oc_V=module_spec["V_oc_V"],
    E_W_per_m2=module_spec["E_W_per_m2"],
    T_degC=module_spec["T_degC"],
    E_W_per_m2_0=E_hemispherical_tilted_W_per_m2_stc,
    T_degC_0=T_degC_stc,
)
sdm_simple_fit_matrix = sdm_simple_inf_matrix.fit(iv_performance_matrix=iv_performance_matrix)

# Examine fit parameters and fit quality.
pprint(sdm_simple_fit_matrix["model_parameters"])
pprint(sdm_simple_inf_matrix.compute_fit_quality(
    iv_performance_matrix=iv_performance_matrix,
    model_parameters=sdm_simple_fit_matrix["model_parameters"],
)[0])
sdm_simple_fit_matrix_iv_curve_parameters = sde_simple_sim.iv_curve_parameters(
    model_parameters=sdm_simple_ae.compute_sde_model_parameters(
        ft_data=FTData(F=iv_performance_matrix.F, T_degC=iv_performance_matrix.T_degC),
        model_parameters=sdm_simple_fit_matrix["model_parameters"],
    )
)
# Plot the data fits.
fig, ax = pyplot.subplots(figsize=(8, 6))
for idx, (F, T_degC) in enumerate(
    zip(iv_performance_matrix.F, iv_performance_matrix.T_degC)
):
    # Plot Isc, Pmp, and Voc with same colors as fit lines.
    color = next(ax._get_lines.prop_cycler)["color"]
    ax.plot(
        iv_performance_matrix.ivft_data.V_V[3 * idx : 3 * idx + 3],
        iv_performance_matrix.ivft_data.I_A[3 * idx : 3 * idx + 3],
        "o",
        color=color,
        mfc='none',
    )
    V_V_plot = numpy.linspace(0, iv_performance_matrix.ivft_data.V_V[3 * idx + 2], 101)
    ax.plot(
        V_V_plot,
        sde_simple_sim.I_at_V(
            V_V=V_V_plot,
            model_parameters=sdm_simple_ae.compute_sde_model_parameters(
                ft_data=FTData(F=F, T_degC=T_degC),
                model_parameters=sdm_simple_fit_matrix["model_parameters"],
            ),
        )["I_A"],
        label=f"F={F:.2f}, T={T_degC:.0f} °C",
        color=color,
    )
    ax.plot(0, sdm_simple_fit_matrix_iv_curve_parameters["I_sc_A"][idx], 'x', color=color)
    ax.plot(
        sdm_simple_fit_matrix_iv_curve_parameters["V_mp_V"][idx],
        sdm_simple_fit_matrix_iv_curve_parameters["I_mp_A"][idx],
        'x', color=color
    )
    ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_oc_V"][idx], 0, 'x', color=color)

ax.set_title("6-Parameter Simple SDM Fit to Performance Matrix", fontdict={"fontsize": 14})
ax.set_xlabel("V (V)")
ax.set_ylabel("I (A)")

pyplot.show()

## Indroductory Direct-Current (DC) Maximum-Power Simulation

Before getting into the weeds of deriving F and T_degC from MET-station data, let's assume that an oracle gave us the "PV module's hourly average weather", i.e., average F and T_degC for each hour, for one day.

In [ ]:
# Here is some synthetic hourly-averaged F-T data,
power_timestamps = pandas.date_range(start="2024-06-22 03:30:00", end="2024-06-22 20:30:00", freq="H", tz="Etc/GMT+7")
dawn_hour = 4
daylight_length_hours = 24 - 2*dawn_hour
daylight_hours = numpy.linspace(dawn_hour - 0.5, 24 - dawn_hour + 0.5, num=daylight_length_hours+2)
F_hourly = 1.1 * numpy.sin(numpy.pi/daylight_length_hours*(daylight_hours - dawn_hour))**2
F_hourly[0] = F_hourly[-1] = 0.0  # Nighttime. 
T_degC_hourly = 10 + 30 * numpy.sin(numpy.pi/daylight_length_hours*(daylight_hours - (dawn_hour+2)))**2
T_degC_hourly[0:2] = 10.0  # 2-hour lag in temperature.

# Compute the maximum DC power for the F-T timeseries. Note that power is selected from larger result dictionary.
P_mp_W = sde_simple_sim.P_mp(
    model_parameters=sdm_simple_ae.compute_sde_model_parameters(
        ft_data=FTData(F=F_hourly, T_degC=T_degC_hourly),
        model_parameters=sdm_simple_fit_matrix["model_parameters"],
    ),
)["P_mp_W"]

# These are the "fenceposts" of the piecewise-constant power, used to plot accumulated total energy.
energy_timestamps = pandas.date_range(start="2024-06-22 04:00:00", end="2024-06-22 21:00:00", freq="H", tz="Etc/GMT+7")
energy_W_h = numpy.hstack((0.0, numpy.cumsum(P_mp_W)))

power_sim_hourly = pandas.DataFrame(
    data={
        "F": F_hourly,
        "T_degC": T_degC_hourly,
        "P_mp_W": P_mp_W,
        "energy_W_h": energy_W_h[:-1],
    },
    index=power_timestamps,
)

power_sim_hourly

#### Plot the "weather" with the DC power.

In [ ]:
fig, ax_left = pyplot.subplots(nrows=2, ncols=1)
ax_left[0].step(power_timestamps, power_sim_hourly["F"], '.-', where="mid", label="F")
ax_left[0].set_ylabel("$F$ [1]")
ax_left[0].legend(loc="upper left")

ax_right = ax_left[0].twinx()
color = next(ax_left[0]._get_lines.prop_cycler)["color"]
ax_right.step(power_timestamps, power_sim_hourly["T_degC"], 'x-', where="mid", label="T_degC", color=color)
ax_right.set_ylabel("$T$ [$^\\circ$C]")
ax_right.legend(loc="upper right")

color = next(ax_left[0]._get_lines.prop_cycler)["color"]
ax_left[1].step(power_timestamps, power_sim_hourly["P_mp_W"], '.-', where="mid", label="P_pm_W", color=color)
ax_left[1].set_ylabel("$P_\\mathrm{mp}$ [W]")
ax_left[1].legend(loc="upper left")

ax_right = ax_left[1].twinx()
color = next(ax_left[0]._get_lines.prop_cycler)["color"]
color = next(ax_left[0]._get_lines.prop_cycler)["color"]  # Avoid red next to green.
ax_right.plot(energy_timestamps, power_sim_hourly["energy_W_h"], 'x-', label="energy_W_h", color=color)
ax_right.set_ylabel("Energy [W$\\cdot$h]")
ax_right.legend(loc=(0.7, 0.6))

ax_left[0].set_title('Average hourly $(F,T)$ "weather" and $P_\\mathrm{mp}$ & Energy')
fig.autofmt_xdate()
fig.tight_layout()

## Weather

### GHI, DNI, DHI, etc. Measurements

In [ ]:
weather = pandas.DataFrame(
    data={
        "ghi": numpy.array(
            [0.,1.2299805,3.9371948,6.796631,9.779785,36.526367,48.010742,62.092773,76.359375,91.0293,105.99609,121.41797,136.78906,153.79688,169.48828,185.,202.,217.50781,233.52734,250.09375,264.45312,280.27344,295.6953,310.89844,326.23438,339.70312,354.32812,368.09375,381.9297,395.60938,408.22656,420.875,435.2578,448.4453,460.3672,471.84375,481.97656,494.1797,504.83594,515.1406,524.0625,532.65625,541.9219,551.03125,559.0781,563.9844,571.1875,577.65625,583.40625,591.3906,595.4531,601.46875,605.78125,610.71875,611.09375,615.1406,615.8281,619.6094,621.5625,619.28125,624.0469,625.59375,626.1094,624.625,624.6406,622.71875,620.8125,617.90625,613.875,612.2656,607.375,601.8281,598.0469,593.5469,587.84375,580.2969,573.0156,568.1094,559.59375,550.78125,543.6719,533.40625,523.53125,514.3594,503.3672,494.65625,488.5547,475.47656,465.1797,453.09375,440.45312,429.16406,414.78906,404.57812,390.6328,375.57812,360.64844,346.64844,329.64062,315.07812,298.08594,282.34375,267.32812,250.42188,235.09375,217.67969,203.21875,189.15625,173.28906,156.50781,140.71875,124.21875,109.0918,94.11328,79.4707,64.68555,50.936523,38.123047,27.212402,17.893555,7.343872,1.2299805],
            dtype=numpy.float32,
        ),
        "dhi": numpy.array(
            [0.,0.56999207,3.2161255,6.716675,9.5998535,12.368652,14.345459,16.264648,17.998535,18.524414,19.871094,19.872559,20.30957,22.029785,23.03955,23.869629,24.889648,25.589844,26.03955,26.717285,26.916992,27.098145,27.849121,28.277832,28.46582,29.100586,29.259766,29.701172,30.121094,30.924316,31.152344,31.519531,32.70996,33.492188,34.42383,34.467773,35.973633,36.532227,36.620117,37.427734,36.29297,37.70703,37.155273,37.38965,38.48828,38.461914,38.9209,39.569336,39.15918,39.38965,40.679688,41.039062,39.719727,42.239258,42.121094,40.90918,41.103516,40.2959,40.848633,41.21875,40.57715,40.91992,40.51953,40.279297,40.15918,39.729492,39.649414,39.757812,39.911133,38.66504,39.80664,37.899414,38.739258,37.759766,37.839844,37.089844,37.629883,35.859375,35.663086,34.541016,34.55078,33.137695,32.448242,31.28955,31.8667,31.624023,31.739746,31.509766,31.54541,31.541016,31.435547,30.909668,30.369629,30.649902,30.30957,28.639648,28.299805,26.009766,25.666992,26.415527,24.82373,23.742676,24.86914,23.5625,23.15625,22.0083,21.22998,21.149902,20.59961,18.59961,17.419922,15.079834,13.889893,12.617676,11.189941,9.45166,7.416992,5.1499023,3.5637817,2.2299805,0.,0.],
            dtype=numpy.float32,
        ),
        "dni": numpy.array(
            [2.99997330e-02,4.99992371e-02,0.00000000e+00,5.48143387e-02,2.79998779e-01,4.35757812e+02,4.91296875e+02,5.47015625e+02,5.89328125e+02,6.29171875e+02,6.63078125e+02,6.96765625e+02,7.22578125e+02,7.51375000e+02,7.70015625e+02,7.88546875e+02,8.11312500e+02,8.27984375e+02,8.44500000e+02,8.61453125e+02,8.72296875e+02,8.86609375e+02,8.97328125e+02,9.10984375e+02,9.20656250e+02,9.29593750e+02,9.37718750e+02,9.46140625e+02,9.53093750e+02,9.59703125e+02,9.66187500e+02,9.70828125e+02,9.77046875e+02,9.84109375e+02,9.87703125e+02,9.93703125e+02,9.96312500e+02,1.00235938e+03,1.00545312e+03,1.00796875e+03,1.01246875e+03,1.01418750e+03,1.01796875e+03,1.02128125e+03,1.02326562e+03,1.02431250e+03,1.02478125e+03,1.02556250e+03,1.02737500e+03,1.02868750e+03,1.03093750e+03,1.03390625e+03,1.03525000e+03,1.03531250e+03,1.03300000e+03,1.03218750e+03,1.03171875e+03,1.03640625e+03,1.03509375e+03,1.03131250e+03,1.03565625e+03,1.03543750e+03,1.03890625e+03,1.03700000e+03,1.03875000e+03,1.03975000e+03,1.03618750e+03,1.03493750e+03,1.03593750e+03,1.03515625e+03,1.03215625e+03,1.03309375e+03,1.03340625e+03,1.03365625e+03,1.03115625e+03,1.02515625e+03,1.02471875e+03,1.02618750e+03,1.02281250e+03,1.01973438e+03,1.01606250e+03,1.01557812e+03,1.00976562e+03,1.00660938e+03,9.99890625e+02,9.98484375e+02,9.95312500e+02,9.91234375e+02,9.83828125e+02,9.78546875e+02,9.73265625e+02,9.66250000e+02,9.59140625e+02,9.53031250e+02,9.44656250e+02,9.38656250e+02,9.28031250e+02,9.20500000e+02,9.05671875e+02,8.92140625e+02,8.81265625e+02,8.68578125e+02,8.54437500e+02,8.40140625e+02,8.27468750e+02,8.07578125e+02,7.89281250e+02,7.66796875e+02,7.44796875e+02,7.20015625e+02,6.93937500e+02,6.64296875e+02,6.33343750e+02,6.01484375e+02,5.66484375e+02,5.19078125e+02,4.72453125e+02,4.09796875e+02,3.65617188e+02,2.99085938e+02,8.70332031e+01,1.35086060e+00],
            dtype=numpy.float32,
        ),
        "temperature": numpy.array(
            [2.09999084e-01,2.09999084e-01,-2.59256363e-03,2.79998779e-01,2.16663361e-01,1.59999847e-01,2.56332397e-01,8.79989624e-01,1.13156128e+00,5.29998779e-01,1.78997803e+00,2.31182861e+00,2.81994629e+00,2.94995117e+00,3.26995850e+00,3.52996826e+00,3.76995850e+00,4.01989746e+00,4.08996582e+00,4.21447754e+00,4.29992676e+00,4.50256348e+00,4.46997070e+00,4.73999023e+00,4.80737305e+00,4.84729004e+00,4.92993164e+00,5.33374023e+00,5.30993652e+00,5.58776855e+00,5.78735352e+00,5.94995117e+00,6.02990723e+00,6.20996094e+00,6.37988281e+00,6.22656250e+00,6.58142090e+00,6.76831055e+00,6.79309082e+00,6.46154785e+00,6.91589355e+00,6.94738770e+00,7.12377930e+00,7.02990723e+00,7.22253418e+00,7.78991699e+00,7.65405273e+00,7.65991211e+00,7.81994629e+00,8.03979492e+00,8.30981445e+00,8.14990234e+00,8.39990234e+00,8.59985352e+00,8.84375000e+00,9.41992188e+00,8.89868164e+00,9.67553711e+00,9.69921875e+00,9.78173828e+00,9.64331055e+00,9.58984375e+00,9.39990234e+00,1.02099609e+01,9.92993164e+00,9.79980469e+00,1.01398926e+01,1.01191406e+01,1.04851074e+01,1.03125000e+01,1.04882812e+01,1.01599121e+01,1.05798340e+01,1.10297852e+01,1.06398926e+01,1.11298828e+01,1.16198730e+01,1.13999023e+01,1.14199219e+01,1.17961426e+01,1.17409668e+01,1.14455566e+01,1.17998047e+01,1.20310059e+01,1.23037109e+01,1.26738281e+01,1.26425781e+01,1.21699219e+01,1.23525391e+01,1.20498047e+01,1.20256348e+01,1.20998535e+01,1.23698730e+01,1.22299805e+01,1.20197754e+01,1.20998535e+01,1.23298340e+01,1.20278320e+01,1.21757812e+01,1.20397949e+01,1.22072754e+01,1.25126953e+01,1.24899902e+01,1.24038086e+01,1.24638672e+01,1.23417969e+01,1.22597656e+01,1.23898926e+01,1.20998535e+01,1.20598145e+01,1.20498047e+01,1.20798340e+01,1.17451172e+01,1.16999512e+01,1.17199707e+01,1.16499023e+01,1.15625000e+01,1.13999023e+01,1.12897949e+01,1.11599121e+01,1.09499512e+01,1.07299805e+01],
            dtype=numpy.float32,
        ),
        "humidity": numpy.array(
            [41.359375,41.34961,41.729492,41.692383,41.966797,42.333008,40.283203,39.323242,38.299805,38.072266,36.493164,35.032227,34.129883,34.259766,33.479492,33.259766,33.069336,32.649414,32.149414,32.08789,31.799805,31.416992,31.775879,31.07959,30.609863,30.739746,30.604004,30.18457,29.845215,29.489746,28.868652,29.03955,28.72998,28.732422,28.506348,28.53711,27.99414,27.929688,27.395996,27.815918,27.312012,27.529785,27.047363,26.845703,26.736328,26.269531,26.219727,26.30957,26.09961,25.699707,25.509766,25.779785,25.299805,24.989746,25.4292,23.35254,23.952148,22.88623,22.953125,23.106445,22.621094,22.369629,22.78955,21.779785,21.689941,21.219727,21.269531,21.254395,21.784668,20.55957,19.88086,20.189941,19.84961,19.78955,20.,20.07959,19.53955,19.169922,19.489746,18.436523,18.467285,18.362793,17.526855,17.182617,16.32959,15.924072,16.152832,16.32959,16.34961,16.896973,17.103516,17.049805,17.239746,17.419922,17.529785,17.489746,17.429688,17.202637,17.356934,17.069824,16.929688,16.646973,17.004395,17.04248,17.299805,17.095703,17.139648,17.259766,17.47998,16.95996,17.509766,17.609863,17.908203,18.141602,18.219727,18.386719,18.399902,18.589844,18.759766,19.174805,19.341797,19.626953],
            dtype=numpy.float32,
        ),
        "windspeed": numpy.array(
            [2.039978,2.209961,2.0704956,1.4692383,1.1277771,1.5216675,1.6022034,2.4000854,2.433838,2.1861572,1.631897,2.0941162,1.8299866,2.7399902,1.9599915,2.3599854,3.3899536,2.6399536,3.4099731,2.7443848,2.4386597,3.3336792,2.8257446,2.4089355,2.4366455,2.2629395,2.8149414,2.6311646,2.326294,3.3988647,2.842163,1.8299866,0.7599945,4.3199463,3.993164,3.0299683,3.7590942,4.265991,2.4240723,4.1950684,2.4107666,3.9855347,3.1346436,3.3370972,3.2949219,2.7114258,2.4916992,2.5599976,1.539978,3.6399536,2.1499634,2.2199707,2.0099487,4.379883,2.7736206,3.3773193,2.4730225,3.2019043,4.033325,4.140991,2.4057007,4.7999268,3.669983,4.0499268,3.6799927,2.9799805,3.1599731,3.157959,3.5255127,2.944519,1.6662292,1.2399902,1.3499756,2.169983,1.039978,2.2699585,1.75,0.7599945,1.4899902,2.8623657,1.5499878,0.47306824,0.8608856,1.4169617,0.83296204,2.02594,3.2792358,3.163391,2.7785034,2.552002,1.7001648,1.7299805,2.0199585,2.1499634,2.5299683,2.0599976,1.4599915,0.774353,1.0888672,2.0612183,1.5644836,1.885376,1.9756775,2.116272,2.1588745,1.7556152,1.5499878,1.8799744,2.5299683,1.6899719,1.0099792,1.4799805,1.9843445,2.0825806,1.9329529,2.4557495,1.5014038,1.7289429,1.9159241,2.135315,1.0955505,1.1899719],
            dtype=numpy.float32,
        ),
        "pressure": numpy.array(
            [83682.,83682.,83680.,83682.,83682.,83680.,83680.,83682.,83680.,83680.,83680.,83680.,83680.,83680.,83680.,83734.,83734.,83734.,83734.,83734.,83734.,83734.,83732.,83732.,83732.,83732.,83732.,83732.,83732.,83742.,83754.,83786.,83786.,83786.,83786.,83786.,83786.,83786.,83784.,83786.,83784.,83784.,83784.,83784.,83784.,83784.,83784.,83780.,83730.,83730.,83730.,83728.,83728.,83728.,83728.,83728.,83728.,83674.,83674.,83674.,83674.,83674.,83674.,83674.,83620.,83620.,83620.,83620.,83620.,83620.,83574.,83566.,83566.,83512.,83512.,83512.,83512.,83512.,83514.,83514.,83460.,83460.,83460.,83460.,83460.,83460.,83460.,83460.,83460.,83460.,83460.,83460.,83462.,83460.,83462.,83462.,83460.,83442.,83444.,83444.,83444.,83462.,83460.,83462.,83462.,83460.,83462.,83462.,83460.,83462.,83462.,83460.,83454.,83432.,83406.,83408.,83408.,83406.,83408.,83408.,83408.,83406.],
            dtype=numpy.float32,
        ),
        "azimuth": numpy.array(
            [114.66211,115.37305,116.08789,116.8125,117.54297,118.2832,119.03125,119.78711,120.552734,121.32617,122.11133,122.90625,123.708984,124.52539,125.35156,126.1875,127.03711,127.89844,128.76953,129.65625,130.55469,131.46484,132.39062,133.33203,134.28516,135.25,136.23438,137.23047,138.24219,139.26953,140.3125,141.3711,142.44531,143.53906,144.64453,145.76562,146.90625,148.0586,149.23047,150.41797,151.6211,152.83984,154.07031,155.32031,156.58203,157.85938,159.15234,160.45703,161.77344,163.10156,164.44531,165.79688,167.15625,168.52734,169.91016,171.29688,172.6875,174.08594,175.49219,176.89844,178.30469,179.71484,181.125,182.53516,183.94531,185.34766,186.75,188.14453,189.53516,190.91797,192.29297,193.66016,195.01562,196.36328,197.69922,199.01953,200.33203,201.6289,202.91406,204.1836,205.4375,206.67969,207.90625,209.11328,210.3086,211.48828,212.64844,213.79688,214.92578,216.03906,217.13672,218.21875,219.28516,220.33594,221.3711,222.39062,223.39453,224.38672,225.35938,226.32031,227.26562,228.19922,229.11719,230.02344,230.91406,231.79297,232.66016,233.51562,234.35938,235.1914,236.01172,236.82031,237.6211,238.41016,239.1914,239.96094,240.72266,241.47266,242.21875,242.95312,243.6836,244.40234],
            dtype=numpy.float32,
        ),
        "zenith": numpy.array(
            [91.484375,90.55664,89.634766,88.71875,87.80859,86.9043,86.00586,85.11328,84.228516,83.35156,82.48047,81.61719,80.76172,79.91406,79.07617,78.24414,77.42383,76.61133,75.80859,75.015625,74.23242,73.458984,72.697266,71.947266,71.208984,70.48242,69.76758,69.06641,68.37695,67.70117,67.041016,66.39453,65.76367,65.146484,64.54492,63.961914,63.39453,62.84375,62.310547,61.7959,61.299805,60.822266,60.365234,59.926758,59.509766,59.112305,58.737305,58.38379,58.051758,57.742188,57.456055,57.192383,56.953125,56.73633,56.54492,56.376953,56.2334,56.115234,56.021484,55.953125,55.910156,55.8916,55.898438,55.930664,55.987305,56.069336,56.176758,56.308594,56.46582,56.646484,56.851562,57.081055,57.333984,57.609375,57.90918,58.23047,58.57422,58.94043,59.32715,59.73535,60.164062,60.61328,61.081055,61.569336,62.075195,62.600586,63.14258,63.703125,64.2793,64.87305,65.48242,66.10742,66.74609,67.40039,68.07031,68.75195,69.447266,70.15625,70.87891,71.61133,72.35742,73.11328,73.88086,74.6582,75.447266,76.24609,77.05469,77.87109,78.69922,79.5332,80.37695,81.228516,82.08789,82.95703,83.83008,84.71289,85.60156,86.49609,87.39844,88.30469,89.21875,90.13867],
            dtype=numpy.float32,
        ),
        "apparent_zenith": numpy.array(
            [91.484375,89.98828,89.203125,88.38281,87.53906,86.68164,85.81641,84.95117,84.08594,83.22266,82.365234,81.51172,80.666016,79.82617,78.99219,78.16797,77.35156,76.54297,75.74414,74.953125,74.17383,73.4043,72.64453,71.896484,71.16016,70.43555,69.72266,69.021484,68.33594,67.66211,67.00195,66.35547,65.72656,65.11133,64.51172,63.927734,63.36133,62.811523,62.279297,61.76465,61.26953,60.79297,60.33496,59.898438,59.481445,59.08496,58.70996,58.356445,58.024414,57.71582,57.429688,57.166992,56.926758,56.710938,56.51953,56.351562,56.208984,56.089844,55.99707,55.92871,55.884766,55.86621,55.874023,55.905273,55.96289,56.04492,56.152344,56.28418,56.44043,56.621094,56.82617,57.054688,57.307617,57.583008,57.881836,58.203125,58.546875,58.91211,59.29883,59.70703,60.134766,60.583008,61.05078,61.538086,62.043945,62.56836,63.11035,63.668945,64.24414,64.83789,65.44531,66.06836,66.70703,67.36133,68.0293,68.708984,69.4043,70.11133,70.83008,71.5625,72.30469,73.05859,73.82422,74.59961,75.384766,76.17969,76.984375,77.796875,78.61914,79.447266,80.28516,81.12891,81.978516,82.833984,83.69531,84.55859,85.42383,86.291016,87.15234,88.00391,88.83594,89.63672],
            dtype=numpy.float32,
        ),
    },
    index=pandas.date_range(start="2014-01-16 07:12:30", end="2014-01-16 17:17:30", freq="5min", tz="Etc/GMT+7"),
)
weather

### Plane-of-Array (POA) Irradiance Calculation

#### Decomposition into POA components

In [ ]:
# Compute angle of incidence of sun on POA.
aoi_deg = pvlib.irradiance.aoi(
    array["surface_tilt_deg"],
    array["surface_azimuth_deg"],
    weather["apparent_zenith"],
    weather["azimuth"],
)
# Compute direct component on POA.
poa_components = pandas.DataFrame(
    data={"direct": numpy.maximum(weather["dni"] * numpy.cos(numpy.radians(aoi_deg)), 0)},
    index=weather.index
)
# Compute sky-diffuse components on POA using Perez allsitescomposite1990.
poa_components_perez = pvlib.irradiance.perez(
    array["surface_tilt_deg"],
    array["surface_azimuth_deg"],
    weather["dhi"],
    weather["dni"],
    pvlib.irradiance.get_extra_radiation(weather.index),
    weather["apparent_zenith"],
    weather["azimuth"],
    airmass=pvlib.atmosphere.get_relative_airmass(weather["apparent_zenith"]),
    model="allsitescomposite1990",  # Cf. albuquerque1988
    return_components=True,
)
poa_components["circumsolar"] = poa_components_perez["circumsolar"]
poa_components["isotropic"] = poa_components_perez["isotropic"]
poa_components["horizon"] = poa_components_perez["horizon"]

# Compute ground-diffuse component on POA.
poa_components["ground_diffuse"] = pvlib.irradiance.get_ground_diffuse(
    array["surface_tilt_deg"],
    weather["ghi"],
    albedo=array["albedo"],
)

# Compute total POA irradiance.
poa_components["total"] = numpy.sum(poa_components.to_numpy(), axis=1)

poa_components

#### Irradiances Plot

In [ ]:
fig, ax = pyplot.subplots()
ax.plot_date(weather.index, weather["dni"], '.', label="DNI")
ax.plot_date(weather.index, weather["ghi"], '.', label="GHI")
ax.plot_date(weather.index, weather["dhi"], '.', label="DHI")
ax.plot_date(weather.index, poa_components["total"], '.', label="POA")
ax.legend(loc=(0.425, 0.2))
ax.set_title("Clear-Sky Irradiances")
ax.set_ylabel("Irradiance [W/m$^2$]")
fig.autofmt_xdate()

## Performance Simulation

### Module Temperature - Faiman Model

In [ ]:
module_temperature = pvlib.temperature.faiman(
    poa_global=poa_components["total"].to_numpy(),
    temp_air=weather["temperature"].to_numpy(),
    wind_speed=weather["windspeed"].to_numpy(),
    u0=module_spec["faiman"]["u0"],
    u1=module_spec["faiman"]["u1"],
)

### Cell Temperature - SAPM

In [ ]:
T_degC = pvlib.temperature.sapm_cell_from_module(
    module_temperature=module_temperature,
    poa_global=poa_components["total"].to_numpy(),
    deltaT=module_spec["sapm"]["DeltaT_degC"],
    irrad_ref=E_hemispherical_tilted_W_per_m2_stc,
)

#### Temperatures plot

In [ ]:
fig, ax = pyplot.subplots()
ax.plot_date(weather.index, T_degC, '.', label="cell")
ax.plot_date(weather.index, module_temperature, '.', label="module")
ax.plot_date(weather.index, weather["temperature"], '.', label="ambient")
ax.legend(loc="upper left")
ax.set_title("Temperatures")
ax.set_ylabel("Temperature [$^\\circ$C]")
fig.autofmt_xdate()

### Effective Irradiance Ratio

#### Incident-angle-modifier (IAM) corrections

In [ ]:
# Compute IAMs for diffuse components.
# FIXME Explain alternatives here.
# diffuse_iam = pvlib.iam.marion_diffuse('physical', surface_tilt=array["surface_tilt_deg"])
diffuse_iam = {}
for region in ["sky", "horizon", "ground"]:
    diffuse_iam[region] = pvlib.iam.marion_integrate(
        lambda angle_deg: iam(angle_deg=angle_deg), array["surface_tilt_deg"], region
    )
pprint(diffuse_iam)

In [ ]:
# Compute IAM-corrected effective POA irradiance.
poa_components["total_eff"] = iam(angle_deg=aoi_deg) * (
    poa_components["direct"] + poa_components["circumsolar"])  + \
        diffuse_iam["sky"] * poa_components["isotropic"] + \
        diffuse_iam["horizon"] * poa_components["horizon"] + \
        diffuse_iam["ground"] * poa_components["ground_diffuse"]

#### Global Normal Spectra

In [ ]:
spectra = pandas.DataFrame(
    data=numpy.array([
        [2.39795446e-03,2.48938799e-03,2.62749195e-03,3.16226482e-03,  3.34006548e-03,3.33112478e-03,3.35568190e-03,3.35085392e-03,  3.70275974e-03,4.34648991e-03,5.83791733e-03,6.67595863e-03,  7.11441040e-03,7.50887394e-03,7.62665272e-03,7.52449036e-03,  7.52723217e-03,8.20922852e-03,8.98814201e-03,9.74082947e-03,  1.03061199e-02,1.05547905e-02,1.06725693e-02,1.07100010e-02,  1.07305050e-02,1.07223988e-02,1.05764866e-02,1.02384090e-02,  1.03392601e-02,1.03485584e-02,9.77301598e-03,9.40680504e-03,  9.34672356e-03,8.93521309e-03,8.66651535e-03,8.35156441e-03,  7.96008110e-03,7.73859024e-03,7.48157501e-03,7.24637508e-03,  7.03227520e-03,6.67083263e-03,6.16276264e-03,5.68485260e-03,  5.25140762e-03,5.06842136e-03,5.27679920e-03,5.47671318e-03,  5.28192520e-03,5.03683090e-03,4.86958027e-03,4.87065315e-03,  5.05983829e-03,5.28609753e-03,5.41281700e-03,5.34331799e-03,  5.35070896e-03,5.80763817e-03,6.20651245e-03,6.38389587e-03,  6.41715527e-03,6.45470619e-03,6.81042671e-03,7.13312626e-03,  7.28964806e-03,7.36248493e-03,7.29632378e-03,6.36589527e-03,  5.95033169e-03,6.65402412e-03,6.84463978e-03,6.97803497e-03,  6.98029995e-03,6.37030602e-03,5.82957268e-03,5.75435162e-03,  5.98657131e-03,6.37352467e-03,6.47723675e-03,6.50894642e-03,  6.49607182e-03,5.74493408e-03,3.41016054e-03,3.48031521e-03,  5.34617901e-03,5.95772266e-03,5.78391552e-03,5.68461418e-03,  5.56361675e-03,5.38611412e-03,5.33998013e-03,5.23757935e-03,  4.78434563e-03,4.25267220e-03,4.21392918e-03,4.19723988e-03,  4.27210331e-03,4.42302227e-03,4.49287891e-03,4.66287136e-03,  4.20069695e-03,4.29415703e-03,4.28843498e-03,4.38594818e-03,  4.40144539e-03,4.57704067e-03,4.35888767e-03,4.12452221e-03,  4.02557850e-03,3.74227762e-03,3.50105762e-03,3.32033634e-03,  3.29536200e-03,3.30454111e-03,3.11273336e-03,2.83861160e-03,  1.63462758e-03,9.83595848e-04,9.58442688e-04,9.76443291e-04,  1.00469589e-03,1.09708309e-03,1.23572350e-03,1.44633651e-03,  1.63695216e-03,1.69327855e-03,1.78545713e-03,1.92320347e-03,  2.00563669e-03,2.02339888e-03,1.99568272e-03,1.93750858e-03,  1.90365314e-03,1.88142061e-03,1.84664130e-03,1.79234147e-03,  1.74093246e-03,1.70099735e-03,1.65274739e-03,1.58640742e-03,  1.49598718e-03,1.40514970e-03,1.32921338e-03,1.26221776e-03,  1.25291944e-03,1.28078461e-03,1.27634406e-03,1.24689937e-03,  1.18583441e-03,1.09922886e-03,9.94980335e-04,8.90403986e-04,  7.46652484e-04,5.38021326e-04,3.72067094e-04,3.10614705e-04,  3.17476690e-04,3.62627208e-04,4.06913459e-04,3.83317471e-04,  3.95730138e-04,4.89652157e-04,6.10753894e-04,6.95407391e-04,  7.47710466e-04,7.75590539e-04,7.46771693e-04,7.14346766e-04,  7.24554062e-04,7.31453300e-04,7.02440739e-04,6.75916672e-04,  6.87286258e-04,7.12186098e-04,7.13169575e-04,7.15404749e-04,  7.29918480e-04,7.34716654e-04,7.20247626e-04,6.71327114e-04,  6.17802143e-04,5.51789999e-04,4.50082123e-04,3.55735421e-04,  3.80590558e-04,4.87424433e-04,5.51477075e-04,5.86554408e-04,  6.09457493e-04,6.02707267e-04,5.63740730e-04,5.15282154e-04,  4.75779176e-04,4.38839197e-04,3.91468406e-04,3.59289348e-04,  3.03260982e-04,2.85051763e-04,2.49169767e-04,1.85951591e-04,  1.29479915e-04,9.07350332e-05,8.54060054e-05,9.12044197e-05,  9.71835107e-05,8.07717443e-05,7.88439065e-05,9.88133252e-05,  1.00614503e-04,8.84719193e-05,9.29217786e-05,1.00931153e-04,  8.78833234e-05,9.92268324e-05,1.20289624e-04,1.25799328e-04,  1.09469518e-04,1.07541680e-04,1.18477270e-04,1.12488866e-04,  1.28258020e-04,1.60291791e-04,1.66978687e-04,1.40983611e-04,  1.43289566e-04,1.58783048e-04,1.59449875e-04,1.75539404e-04,  2.17307359e-04,2.34626234e-04,2.49348581e-04,2.69211829e-04,  2.76371837e-04,2.68064439e-04,2.77966261e-04,2.96071172e-04,  2.90907919e-04,2.96153128e-04,3.17417085e-04,3.28592956e-04,  3.28667462e-04,3.42175364e-04,3.32258642e-04,2.95981765e-04,  2.75664032e-04,2.72512436e-04,2.77101994e-04,3.11106443e-04,  3.34426761e-04,3.03179026e-04,2.70858407e-04,2.88046896e-04,  2.91116536e-04,3.07410955e-04,3.51883471e-04,3.40595841e-04,  3.29852104e-04,3.77185643e-04,4.03247774e-04,4.16882336e-04,  4.86917794e-04,5.71340322e-04,5.79833984e-04,7.19949603e-04,  8.27416778e-04,8.13215971e-04,9.46164131e-04,1.01038814e-03,  9.15840268e-04,9.62242484e-04,1.19829178e-03],
        [3.90338898e-03,4.20761108e-03,4.40347195e-03,4.97198105e-03,  5.42354584e-03,5.49221039e-03,5.54299355e-03,5.59914112e-03,  6.11495972e-03,7.15744495e-03,9.53030586e-03,1.09398365e-02,  1.17127895e-02,1.23181343e-02,1.25668049e-02,1.22950077e-02,  1.22303963e-02,1.33740902e-02,1.45540237e-02,1.57670975e-02,  1.66950226e-02,1.70550346e-02,1.72805786e-02,1.72648430e-02,  1.72820091e-02,1.72505379e-02,1.70536041e-02,1.65190697e-02,  1.66149139e-02,1.66430473e-02,1.58023834e-02,1.52673721e-02,  1.51622295e-02,1.44407749e-02,1.40459538e-02,1.36427879e-02,  1.30562782e-02,1.27527714e-02,1.23677254e-02,1.19833946e-02,  1.16553307e-02,1.10731125e-02,1.02865696e-02,9.55224037e-03,  8.85438919e-03,8.56614113e-03,8.84079933e-03,9.05013084e-03,  8.76784325e-03,8.39257240e-03,8.08119774e-03,8.08572769e-03,  8.24379921e-03,8.51297379e-03,8.65578651e-03,8.47458839e-03,  8.54039192e-03,9.06682014e-03,9.56153870e-03,9.72390175e-03,  9.70721245e-03,9.74893570e-03,1.01783276e-02,1.04804039e-02,  1.06642246e-02,1.06790066e-02,1.05304718e-02,9.30881500e-03,  8.63242149e-03,9.57179070e-03,9.91511345e-03,9.94181633e-03,  9.88030434e-03,8.99672508e-03,8.25262070e-03,8.23974609e-03,  8.36873055e-03,8.89134407e-03,9.02128220e-03,8.88919830e-03,  8.80336761e-03,7.66158104e-03,4.40502167e-03,4.73284721e-03,  7.09557533e-03,7.85279274e-03,7.89284706e-03,7.69698620e-03,  7.53867626e-03,7.35116005e-03,7.09903240e-03,7.13300705e-03,  6.51693344e-03,5.69689274e-03,5.78284264e-03,5.77640533e-03,  5.86414337e-03,6.11650944e-03,6.00254536e-03,6.00087643e-03,  5.79655170e-03,5.46956062e-03,5.90002537e-03,5.56242466e-03,  5.54180145e-03,5.91015816e-03,5.32686710e-03,5.48458099e-03,  5.58316708e-03,4.19151783e-03,4.42373753e-03,4.35435772e-03,  3.94737720e-03,4.13000584e-03,4.18722630e-03,3.66443396e-03,  2.22778320e-03,1.37403607e-03,1.36533380e-03,1.40064955e-03,  1.44219398e-03,1.54709816e-03,1.73848867e-03,2.04026699e-03,  2.30097771e-03,2.36320496e-03,2.45630741e-03,2.60639191e-03,  2.69132853e-03,2.69627571e-03,2.63947248e-03,2.57122517e-03,  2.52801180e-03,2.49511003e-03,2.44343281e-03,2.38758326e-03,  2.32338905e-03,2.25055218e-03,2.18909979e-03,2.12031603e-03,  2.04479694e-03,1.93464756e-03,1.83290243e-03,1.77127123e-03,  1.75178051e-03,1.76122785e-03,1.74692273e-03,1.70132518e-03,  1.63993239e-03,1.53115392e-03,1.40091777e-03,1.29267573e-03,  1.08805299e-03,7.68586993e-04,5.30079007e-04,4.53904271e-04,  4.82514501e-04,5.40062785e-04,6.00382686e-04,5.65871596e-04,  5.86658716e-04,7.18846917e-04,8.92326236e-04,1.02740526e-03,  1.10018253e-03,1.11645460e-03,1.08304620e-03,1.05318427e-03,  1.07574463e-03,1.07491016e-03,1.01983547e-03,9.92417336e-04,  1.01685524e-03,1.03911757e-03,1.02502108e-03,1.02970004e-03,  1.05324388e-03,1.06018782e-03,1.02922320e-03,9.87678766e-04,  9.27761197e-04,8.32989812e-04,6.82294369e-04,5.59374690e-04,  5.97104430e-04,7.33718276e-04,8.32989812e-04,8.79853964e-04,  8.95321369e-04,8.80122185e-04,8.39054585e-04,7.75456429e-04,  7.04154372e-04,6.53266907e-04,6.18517399e-04,5.52028418e-04,  4.44233418e-04,4.19504941e-04,3.91952693e-04,2.75105238e-04,  1.52014196e-04,1.00648031e-04,9.89325345e-05,9.54121351e-05,  8.20700079e-05,8.46572220e-05,9.44901258e-05,9.64514911e-05,  9.20668244e-05,9.42200422e-05,1.01028010e-04,1.01551414e-04,  9.97763127e-05,1.14779919e-04,1.33723021e-04,1.48501247e-04,  1.27673149e-04,1.22118741e-04,1.43181533e-04,1.62705779e-04,  1.77767128e-04,2.25301832e-04,2.38880515e-04,2.10169703e-04,  1.82915479e-04,2.10925937e-04,2.39685178e-04,2.65672803e-04,  3.11359763e-04,3.60339880e-04,3.90939415e-04,3.99142504e-04,  4.08478081e-04,4.14587557e-04,4.28408384e-04,4.21173871e-04,  4.35113907e-04,4.54701483e-04,4.60833311e-04,4.58776951e-04,  4.72046435e-04,4.83252108e-04,4.65616584e-04,4.25308943e-04,  3.86670232e-04,3.77371907e-04,3.89128923e-04,4.25346196e-04,  4.58851457e-04,4.40128148e-04,3.68639827e-04,3.65890563e-04,  4.07308340e-04,4.21673059e-04,4.38444316e-04,4.69565392e-04,  4.80510294e-04,4.56020236e-04,4.85271215e-04,5.50791621e-04,  5.78865409e-04,6.05061650e-04,7.34090805e-04,8.20219517e-04,  7.90968537e-04,9.10133123e-04,1.10068917e-03,1.06519461e-03,  9.62510705e-04,1.10030174e-03,1.26743317e-03],
        [6.04403019e-03,6.18994236e-03,6.38353825e-03,7.63428211e-03,  8.10265541e-03,8.33964348e-03,8.39829445e-03,8.39948654e-03,  9.28306580e-03,1.07426643e-02,1.44855976e-02,1.65882111e-02,  1.76649094e-02,1.86572075e-02,1.89213753e-02,1.85718536e-02,  1.85332298e-02,2.01783180e-02,2.20599174e-02,2.38389969e-02,  2.52056122e-02,2.58450508e-02,2.61750221e-02,2.62131691e-02,  2.62265205e-02,2.62322426e-02,2.61464119e-02,2.53286362e-02,  2.55427361e-02,2.56915092e-02,2.44793892e-02,2.38599777e-02,  2.37078667e-02,2.26473808e-02,2.21481323e-02,2.16336250e-02,  2.09498405e-02,2.05345154e-02,1.99675560e-02,1.94778442e-02,  1.89847946e-02,1.81856155e-02,1.70221329e-02,1.58519745e-02,  1.47912502e-02,1.42557621e-02,1.46894455e-02,1.51081085e-02,  1.45840645e-02,1.41091347e-02,1.36747360e-02,1.36442184e-02,  1.38943195e-02,1.41689777e-02,1.43189430e-02,1.38838291e-02,  1.37615204e-02,1.46837234e-02,1.54256821e-02,1.55742168e-02,  1.55885220e-02,1.54976845e-02,1.61166191e-02,1.66826248e-02,  1.67636871e-02,1.68204308e-02,1.65047646e-02,1.41942501e-02,  1.31261349e-02,1.46737099e-02,1.51202679e-02,1.53198242e-02,  1.51591301e-02,1.37350559e-02,1.24292374e-02,1.22320652e-02,  1.26688480e-02,1.34973526e-02,1.37047768e-02,1.38201714e-02,  1.34203434e-02,1.16808414e-02,6.76894188e-03,6.93333149e-03,  1.08182430e-02,1.20155811e-02,1.19063854e-02,1.17506981e-02,  1.12988949e-02,1.09777451e-02,1.09434128e-02,1.06592178e-02,  9.85503197e-03,8.49890709e-03,7.92598724e-03,8.26001167e-03,  8.57234001e-03,8.76736641e-03,9.08780098e-03,8.96358490e-03,  8.32962990e-03,8.53967667e-03,8.46815109e-03,8.02445412e-03,  8.43763351e-03,8.32486153e-03,8.21566582e-03,8.46076012e-03,  7.55119324e-03,6.83891773e-03,6.34503365e-03,5.81300259e-03,  5.94651699e-03,5.72621822e-03,5.67018986e-03,5.33187389e-03,  3.22067738e-03,1.87894702e-03,1.85117126e-03,1.87233090e-03,  1.94528699e-03,2.12824345e-03,2.45136023e-03,2.95424461e-03,  3.33207846e-03,3.42816114e-03,3.63063812e-03,3.92425060e-03,  4.08828259e-03,4.07135487e-03,3.97050381e-03,3.88801098e-03,  3.85081768e-03,3.79663706e-03,3.70967388e-03,3.60709429e-03,  3.50993872e-03,3.42965126e-03,3.33046913e-03,3.19057703e-03,  3.05372477e-03,2.90369987e-03,2.75254250e-03,2.64739990e-03,  2.62290239e-03,2.63375044e-03,2.62784958e-03,2.58308649e-03,  2.47842073e-03,2.28172541e-03,2.07042694e-03,1.89065933e-03,  1.59379840e-03,1.09556317e-03,6.92039728e-04,5.64917922e-04,  6.23434782e-04,7.33956695e-04,8.12262297e-04,7.48604536e-04,  8.07940960e-04,1.03831291e-03,1.31207705e-03,1.50245428e-03,  1.61203742e-03,1.65551901e-03,1.61814690e-03,1.57094002e-03,  1.61004066e-03,1.61087513e-03,1.53622031e-03,1.51988864e-03,  1.53830647e-03,1.54587626e-03,1.54951215e-03,1.58205628e-03,  1.61156058e-03,1.59215927e-03,1.53958797e-03,1.48198009e-03,  1.39403343e-03,1.22785568e-03,9.81211662e-04,7.94261694e-04,  8.58858228e-04,1.09115243e-03,1.24210119e-03,1.32095814e-03,  1.35442615e-03,1.33281946e-03,1.26516819e-03,1.15999579e-03,  1.04182959e-03,9.47892666e-04,9.02473927e-04,8.03560019e-04,  6.27055764e-04,5.89460135e-04,5.39749861e-04,3.58544290e-04,  1.76344067e-04,1.02005899e-04,9.44416970e-05,1.01462007e-04,  8.58809799e-05,7.22352415e-05,7.82087445e-05,8.77492130e-05,  9.61776823e-05,8.22134316e-05,8.21929425e-05,9.29664820e-05,  1.09096989e-04,1.16003677e-04,1.31849200e-04,1.56026334e-04,  1.52003020e-04,1.35317445e-04,1.48195773e-04,1.77573413e-04,  2.12207437e-04,2.73622572e-04,2.91548669e-04,2.54899263e-04,  2.35952437e-04,2.62930989e-04,2.94901431e-04,3.46973538e-04,  4.23826277e-04,4.95418906e-04,5.43877482e-04,5.69969416e-04,  6.00561500e-04,5.92157245e-04,5.99458814e-04,6.19471073e-04,  6.31928444e-04,6.43506646e-04,6.62267208e-04,6.78330660e-04,  6.82637095e-04,6.68361783e-04,6.57558441e-04,6.20007515e-04,  5.31092286e-04,4.90844250e-04,5.38393855e-04,6.00725412e-04,  6.12109900e-04,5.75140119e-04,5.25191426e-04,5.20989299e-04,  5.18038869e-04,5.52475452e-04,5.96225262e-04,6.18368387e-04,  6.05329871e-04,6.00323081e-04,6.03690743e-04,6.49169087e-04,  6.75633550e-04,7.01859593e-04,7.98195601e-04,8.86842608e-04,  8.64267349e-04,8.77574086e-04,1.03977323e-03,1.12456083e-03,  1.01965666e-03,9.90450382e-04,1.07911229e-03],
        [8.52870941e-03,8.91852379e-03,9.28997993e-03,1.08542442e-02,  1.16448402e-02,1.18465424e-02,1.19299889e-02,1.20289326e-02,  1.31990910e-02,1.53095722e-02,2.05826759e-02,2.35238075e-02,  2.51135826e-02,2.64434814e-02,2.68449783e-02,2.63142586e-02,  2.61850357e-02,2.85468102e-02,3.11417580e-02,3.37123871e-02,  3.57198715e-02,3.65686417e-02,3.70397568e-02,3.70645523e-02,  3.70931625e-02,3.71541977e-02,3.70874405e-02,3.60450745e-02,  3.63378525e-02,3.65877151e-02,3.50751877e-02,3.42779160e-02,  3.41224670e-02,3.26499939e-02,3.20205688e-02,3.15160751e-02,  3.06820869e-02,3.01589966e-02,2.94270515e-02,2.87780762e-02,  2.81786919e-02,2.71406174e-02,2.55236626e-02,2.38943100e-02,  2.23140717e-02,2.15158463e-02,2.21810341e-02,2.27355957e-02,  2.19931602e-02,2.14099884e-02,2.08845139e-02,2.09584236e-02,  2.11968422e-02,2.14567184e-02,2.15821266e-02,2.07881927e-02,  2.05636024e-02,2.18672752e-02,2.29620934e-02,2.31304169e-02,  2.29182243e-02,2.27942467e-02,2.37665176e-02,2.45518684e-02,  2.46858597e-02,2.46338844e-02,2.40664482e-02,2.06365585e-02,  1.88722610e-02,2.10947990e-02,2.18491554e-02,2.21109390e-02,  2.19779015e-02,1.97396278e-02,1.76472664e-02,1.74355507e-02,  1.80964470e-02,1.94129944e-02,2.00042725e-02,2.01239586e-02,  1.97758675e-02,1.71647072e-02,9.62376595e-03,1.00276470e-02,  1.59611702e-02,1.76815987e-02,1.75571442e-02,1.71780586e-02,  1.65109634e-02,1.61137581e-02,1.57594681e-02,1.55551434e-02,  1.43482685e-02,1.21967793e-02,1.17442608e-02,1.21009350e-02,  1.23918056e-02,1.29704475e-02,1.33540630e-02,1.33116245e-02,  1.28774643e-02,1.26707554e-02,1.29470825e-02,1.25575066e-02,  1.26748085e-02,1.24568939e-02,1.20117664e-02,1.16930008e-02,  1.11477375e-02,9.64546204e-03,8.70132446e-03,8.30769539e-03,  7.79950619e-03,7.60579109e-03,7.85183907e-03,7.06565380e-03,  4.51695919e-03,2.66069174e-03,2.61604786e-03,2.69871950e-03,  2.80594826e-03,3.10140848e-03,3.60000134e-03,4.45270538e-03,  5.18834591e-03,5.41722775e-03,5.80048561e-03,6.34896755e-03,  6.68418407e-03,6.75714016e-03,6.64961338e-03,6.49642944e-03,  6.43491745e-03,6.36208057e-03,6.23500347e-03,6.11352921e-03,  5.97405434e-03,5.80978394e-03,5.64312935e-03,5.43606281e-03,  5.19835949e-03,4.91964817e-03,4.62460518e-03,4.43339348e-03,  4.43303585e-03,4.51815128e-03,4.49740887e-03,4.38451767e-03,  4.20451164e-03,3.87483835e-03,3.46374512e-03,3.09175253e-03,  2.55286694e-03,1.71542168e-03,1.06835365e-03,8.72269273e-04,  9.33945179e-04,1.10262632e-03,1.22323632e-03,1.14855170e-03,  1.24311447e-03,1.61525607e-03,2.10416317e-03,2.51787901e-03,  2.75927782e-03,2.83896923e-03,2.74658203e-03,2.69424915e-03,  2.79074907e-03,2.82114744e-03,2.67159939e-03,2.63875723e-03,  2.70396471e-03,2.76499987e-03,2.77936459e-03,2.81864405e-03,  2.89267302e-03,2.90054083e-03,2.82591581e-03,2.69311666e-03,  2.48831511e-03,2.15649605e-03,1.69792771e-03,1.34813786e-03,  1.45104527e-03,1.88252330e-03,2.22045183e-03,2.42811441e-03,  2.47478485e-03,2.40302086e-03,2.28399038e-03,2.11375952e-03,  1.89206004e-03,1.69402361e-03,1.57693028e-03,1.39603019e-03,  1.07270479e-03,9.81241465e-04,9.02459025e-04,6.04689121e-04,  3.43814492e-04,2.49624252e-04,2.37096101e-04,2.31653452e-04,  2.21274793e-04,2.09234655e-04,2.10855156e-04,2.10147351e-04,  2.25909054e-04,2.24146992e-04,2.20276415e-04,2.20026821e-04,  2.46293843e-04,2.63430178e-04,2.81795859e-04,3.06881964e-04,  2.97978520e-04,2.69085169e-04,2.88806856e-04,3.31178308e-04,  3.85411084e-04,4.84921038e-04,5.14492393e-04,4.60065901e-04,  4.28296626e-04,4.82209027e-04,5.33923507e-04,6.31660223e-04,  7.81729817e-04,9.58830118e-04,1.06307864e-03,1.13525987e-03,  1.22064352e-03,1.23092532e-03,1.26969814e-03,1.31589174e-03,  1.35713816e-03,1.37022138e-03,1.40300393e-03,1.45801902e-03,  1.47774816e-03,1.47449970e-03,1.45035982e-03,1.34775043e-03,  1.14431977e-03,1.08560920e-03,1.19364262e-03,1.30566955e-03,  1.37037039e-03,1.32909417e-03,1.18032098e-03,1.12420321e-03,  1.18461251e-03,1.29559636e-03,1.35535002e-03,1.38908625e-03,  1.38661265e-03,1.40017271e-03,1.43951178e-03,1.50248408e-03,  1.59108639e-03,1.72412395e-03,1.91470981e-03,2.08425522e-03,  2.24816799e-03,2.41947174e-03,2.51221657e-03,2.53057480e-03,  2.58255005e-03,2.64513493e-03,2.69222260e-03],
        [1.15754604e-02,1.18896961e-02,1.25286579e-02,1.47700310e-02,  1.56180859e-02,1.60536766e-02,1.60770416e-02,1.61342621e-02,  1.77731514e-02,2.06050873e-02,2.77776718e-02,3.17573547e-02,  3.38602066e-02,3.56712341e-02,3.61099243e-02,3.54299545e-02,  3.51524353e-02,3.82299423e-02,4.17537689e-02,4.50735092e-02,  4.76961136e-02,4.88166809e-02,4.94174957e-02,4.94604111e-02,  4.93640900e-02,4.94079590e-02,4.93478775e-02,4.79116440e-02,  4.83303070e-02,4.86240387e-02,4.67233658e-02,4.57868576e-02,  4.54912186e-02,4.35352325e-02,4.27160263e-02,4.21466827e-02,  4.12549973e-02,4.06227112e-02,3.96566391e-02,3.88097763e-02,  3.80153656e-02,3.67155075e-02,3.46355438e-02,3.25975418e-02,  3.06015015e-02,2.94675827e-02,3.03125381e-02,3.09867859e-02,  2.98981667e-02,2.92220116e-02,2.85720825e-02,2.85334587e-02,  2.87423134e-02,2.88939476e-02,2.90069580e-02,2.79178619e-02,  2.74748802e-02,2.90861130e-02,3.02968025e-02,3.03864479e-02,  3.01012993e-02,2.97417641e-02,3.09581757e-02,3.18336487e-02,  3.18956375e-02,3.18670273e-02,3.09767723e-02,2.65326500e-02,  2.43549347e-02,2.70786285e-02,2.80189514e-02,2.83908844e-02,  2.80957222e-02,2.53458023e-02,2.26573944e-02,2.22449303e-02,  2.31561661e-02,2.47778893e-02,2.55327225e-02,2.57205963e-02,  2.51488686e-02,2.19469070e-02,1.23167038e-02,1.26523972e-02,  2.03657150e-02,2.25610733e-02,2.24261284e-02,2.21252441e-02,  2.11715698e-02,2.07071304e-02,2.03795433e-02,1.96933746e-02,  1.82619095e-02,1.55947208e-02,1.48422718e-02,1.54690742e-02,  1.57880783e-02,1.65181160e-02,1.72491074e-02,1.68461800e-02,  1.62110329e-02,1.60775185e-02,1.59778595e-02,1.58061981e-02,  1.58314705e-02,1.54337883e-02,1.55346394e-02,1.51741505e-02,  1.40533447e-02,1.26719475e-02,1.05783939e-02,1.03592873e-02,  1.02348328e-02,9.22274590e-03,1.02596283e-02,1.01914406e-02,  6.06644154e-03,3.38119268e-03,3.34459543e-03,3.44586372e-03,  3.61651182e-03,4.02784348e-03,4.70519066e-03,5.84614277e-03,  6.84309006e-03,7.15363026e-03,7.66086578e-03,8.38851929e-03,  8.80169868e-03,8.89778137e-03,8.76379013e-03,8.57162476e-03,  8.48317146e-03,8.40187073e-03,8.25095177e-03,8.08477402e-03,  7.90429115e-03,7.70127773e-03,7.49528408e-03,7.23993778e-03,  6.93333149e-03,6.57474995e-03,6.20877743e-03,5.97107410e-03,  5.96106052e-03,6.06203079e-03,6.02495670e-03,5.88095188e-03,  5.64491749e-03,5.19680977e-03,4.66930866e-03,4.18162346e-03,  3.43048573e-03,2.26360559e-03,1.36131048e-03,1.09431148e-03,  1.17799640e-03,1.41391158e-03,1.60560012e-03,1.50048733e-03,  1.64493918e-03,2.18015909e-03,2.87210941e-03,3.43436003e-03,  3.75533104e-03,3.85570526e-03,3.75378132e-03,3.67826223e-03,  3.81511450e-03,3.84604931e-03,3.64923477e-03,3.59070301e-03,  3.68916988e-03,3.78161669e-03,3.81046534e-03,3.86279821e-03,  3.95655632e-03,3.96859646e-03,3.87495756e-03,3.70103121e-03,  3.43185663e-03,2.99000740e-03,2.35921144e-03,1.85900927e-03,  2.00432539e-03,2.59280205e-03,3.04603577e-03,3.32665443e-03,  3.39800119e-03,3.30859423e-03,3.13574076e-03,2.88218260e-03,  2.58278847e-03,2.31927633e-03,2.14761496e-03,1.89247727e-03,  1.43975019e-03,1.32295489e-03,1.20490789e-03,7.69093633e-04,  3.87869775e-04,2.61850655e-04,2.45265663e-04,2.44341791e-04,  2.19926238e-04,1.96911395e-04,2.04782933e-04,2.15552747e-04,  2.25979835e-04,2.21028924e-04,2.25525349e-04,2.24642456e-04,  2.42635608e-04,2.66753137e-04,2.98216939e-04,3.32780182e-04,  3.18706036e-04,2.94841826e-04,3.25739384e-04,3.80061567e-04,  4.53673303e-04,5.89758158e-04,6.42791390e-04,5.61013818e-04,  5.21257520e-04,5.90458512e-04,6.70284033e-04,8.07702541e-04,  1.05139613e-03,1.30501390e-03,1.45232677e-03,1.55410171e-03,  1.67378783e-03,1.69906020e-03,1.74465775e-03,1.80599093e-03,  1.85465813e-03,1.87984109e-03,1.90645456e-03,1.96439028e-03,  1.99890137e-03,1.99472904e-03,1.95491314e-03,1.80888176e-03,  1.54626369e-03,1.45572424e-03,1.58745050e-03,1.75881386e-03,  1.82488561e-03,1.75231695e-03,1.54533982e-03,1.47730112e-03,  1.55124068e-03,1.69828534e-03,1.79302692e-03,1.81242824e-03,  1.81084871e-03,1.82187557e-03,1.83597207e-03,1.89718604e-03,  1.99776888e-03,2.12275982e-03,2.25293636e-03,2.38531828e-03,  2.56490707e-03,2.71850824e-03,2.78073549e-03,2.85482407e-03,  2.86453962e-03,2.82651186e-03,2.82394886e-03],
        [1.76115036e-02,1.78112984e-02,1.89542770e-02,2.25896835e-02,  2.38280296e-02,2.52523422e-02,2.58450508e-02,2.67734528e-02,  3.04093361e-02,3.62663269e-02,5.09471893e-02,6.05812073e-02,  6.79702759e-02,7.59048462e-02,8.15811157e-02,8.53443146e-02,  9.06085968e-02,1.05205536e-01,1.22650146e-01,1.41174316e-01,  1.59824371e-01,1.75136566e-01,1.89113617e-01,2.01644897e-01,  2.13165283e-01,2.25097656e-01,2.41172791e-01,2.50770569e-01,  2.68676758e-01,2.86529541e-01,2.91519165e-01,3.00842285e-01,  3.15261841e-01,3.18817139e-01,3.29856873e-01,3.41499329e-01,  3.51280212e-01,3.63662720e-01,3.71185303e-01,3.79898071e-01,  3.89869690e-01,3.93638611e-01,3.87420654e-01,3.77227783e-01,  3.62686157e-01,3.59626770e-01,3.88252258e-01,4.12765503e-01,  4.07867432e-01,4.15702820e-01,4.29450989e-01,4.47929382e-01,  4.63943481e-01,4.78630066e-01,4.89753723e-01,4.75921631e-01,  4.78408813e-01,5.27755737e-01,5.69854736e-01,5.82794189e-01,  5.88775635e-01,6.01150513e-01,6.46759033e-01,6.84188843e-01,  7.02606201e-01,7.17147827e-01,7.12341309e-01,6.05926514e-01,  5.52505493e-01,6.38900757e-01,6.81121826e-01,7.11547852e-01,  7.24227905e-01,6.41586304e-01,5.61401367e-01,5.61187744e-01,  6.10214233e-01,6.96334839e-01,7.48840332e-01,7.78335571e-01,  7.79251099e-01,6.83258057e-01,3.59878540e-01,3.76518250e-01,  6.65328979e-01,7.66510010e-01,7.76565552e-01,7.65350342e-01,  7.43362427e-01,7.36907959e-01,7.32009888e-01,7.27828979e-01,  6.65390015e-01,5.54199219e-01,5.39779663e-01,5.75149536e-01,  6.07086182e-01,6.58004761e-01,7.02270508e-01,7.14172363e-01,  6.93557739e-01,6.91696167e-01,7.11029053e-01,7.02728271e-01,  7.13363647e-01,7.16308594e-01,7.12738037e-01,7.05429077e-01,  6.64962769e-01,5.42694092e-01,4.64271545e-01,4.59533691e-01,  4.15473938e-01,4.05014038e-01,4.52178955e-01,4.29382324e-01,  2.55836487e-01,1.19548798e-01,1.19270325e-01,1.28772736e-01,  1.42795563e-01,1.72241211e-01,2.21412659e-01,3.07373047e-01,  3.85513306e-01,4.15275574e-01,4.62348938e-01,5.29144287e-01,  5.72555542e-01,5.89248657e-01,5.86471558e-01,5.79193115e-01,  5.80093384e-01,5.81008911e-01,5.76583862e-01,5.71075439e-01,  5.64697266e-01,5.56015015e-01,5.45532227e-01,5.30899048e-01,  5.11886597e-01,4.87152100e-01,4.61425781e-01,4.45724487e-01,  4.49348450e-01,4.61647034e-01,4.62089539e-01,4.52072144e-01,  4.32487488e-01,3.92715454e-01,3.42407227e-01,2.96447754e-01,  2.30384827e-01,1.31053925e-01,5.75256348e-02,3.64999771e-02,  4.38442230e-02,6.16645813e-02,7.49664307e-02,6.75487518e-02,  7.98873901e-02,1.27796173e-01,1.95564270e-01,2.51808167e-01,  2.86796570e-01,3.00735474e-01,2.94982910e-01,2.90748596e-01,  3.09097290e-01,3.17222595e-01,3.02680969e-01,3.02055359e-01,  3.14857483e-01,3.28247070e-01,3.36334229e-01,3.46603394e-01,  3.61259460e-01,3.66294861e-01,3.58741760e-01,3.42292786e-01,  3.14895630e-01,2.68005371e-01,1.98715210e-01,1.44634247e-01,  1.60503387e-01,2.27001190e-01,2.79457092e-01,3.12904358e-01,  3.22052002e-01,3.13110352e-01,2.94799805e-01,2.68341064e-01,  2.36034393e-01,2.08248138e-01,1.89682007e-01,1.61560059e-01,  1.10685349e-01,9.82837677e-02,8.66947174e-02,4.38528061e-02,  1.21300220e-02,5.14376163e-03,4.64713573e-03,4.47249413e-03,  3.68702412e-03,2.99948454e-03,3.26895714e-03,3.65871191e-03,  4.07958031e-03,4.15444374e-03,4.04727459e-03,4.35388088e-03,  4.84454632e-03,5.39231300e-03,6.53958321e-03,8.65030289e-03,  8.87584686e-03,7.13372231e-03,8.01014900e-03,1.25513077e-02,  2.03442574e-02,3.34453583e-02,3.84807587e-02,3.00970078e-02,  2.63328552e-02,3.46174240e-02,4.33349609e-02,5.97324371e-02,  9.28363800e-02,1.29146576e-01,1.52793884e-01,1.70570374e-01,  1.91005707e-01,2.02033997e-01,2.14965820e-01,2.23613739e-01,  2.27157593e-01,2.30251312e-01,2.35408783e-01,2.44155884e-01,  2.48935699e-01,2.48596191e-01,2.44541168e-01,2.24479675e-01,  1.88114166e-01,1.76498413e-01,1.94389343e-01,2.17006683e-01,  2.26280212e-01,2.14488983e-01,1.85070038e-01,1.75373077e-01,  1.84818268e-01,2.04730988e-01,2.17327118e-01,2.20390320e-01,  2.16995239e-01,2.10975647e-01,2.04738617e-01,2.01717377e-01,  1.99119568e-01,1.98898315e-01,1.92192078e-01,1.78592682e-01,  1.74823761e-01,1.68502808e-01,1.62368774e-01,1.55452728e-01,  1.47747040e-01,1.42063141e-01,1.38347626e-01],
        [2.25839615e-02,2.40998268e-02,2.61664391e-02,3.08074951e-02,  3.34796906e-02,3.51133347e-02,3.66859436e-02,3.83348465e-02,  4.38098907e-02,5.35116196e-02,7.53021240e-02,9.05532837e-02,  1.02285385e-01,1.14078522e-01,1.22934341e-01,1.28318787e-01,  1.36089325e-01,1.57611847e-01,1.82559967e-01,2.09300995e-01,  2.35198975e-01,2.55523682e-01,2.74322510e-01,2.89878845e-01,  3.04473877e-01,3.19831848e-01,3.39179993e-01,3.48899841e-01,  3.69949341e-01,3.91525269e-01,3.96141052e-01,4.06646729e-01,  4.22630310e-01,4.23316956e-01,4.35165405e-01,4.49150085e-01,  4.59884644e-01,4.73175049e-01,4.80545044e-01,4.89555359e-01,  4.99801636e-01,5.02090454e-01,4.93476868e-01,4.81918335e-01,  4.65560913e-01,4.62303162e-01,4.92584229e-01,5.18341064e-01,  5.12588501e-01,5.19317627e-01,5.31661987e-01,5.49987793e-01,  5.65658569e-01,5.80917358e-01,5.92773438e-01,5.77392578e-01,  5.79788208e-01,6.30249023e-01,6.73004150e-01,6.86447144e-01,  6.91085815e-01,7.00439453e-01,7.46612549e-01,7.84454346e-01,  8.02398682e-01,8.15383911e-01,8.08547974e-01,6.95434570e-01,  6.40716553e-01,7.32986450e-01,7.76046753e-01,8.05160522e-01,  8.16589355e-01,7.31964111e-01,6.50573730e-01,6.51336670e-01,  6.99050903e-01,7.82546997e-01,8.30261230e-01,8.55560303e-01,  8.55789185e-01,7.51510620e-01,4.05250549e-01,4.28253174e-01,  7.32192993e-01,8.35449219e-01,8.44421387e-01,8.33602905e-01,  8.12850952e-01,8.05984497e-01,8.01177979e-01,7.96997070e-01,  7.35168457e-01,6.23519897e-01,6.08261108e-01,6.42074585e-01,  6.72897339e-01,7.20748901e-01,7.60101318e-01,7.69683838e-01,  7.45132446e-01,7.40188599e-01,7.62084961e-01,7.50732422e-01,  7.58880615e-01,7.64022827e-01,7.57064819e-01,7.53570557e-01,  7.17453003e-01,5.96771240e-01,5.23269653e-01,5.18951416e-01,  4.70710754e-01,4.68482971e-01,5.11047363e-01,4.81781006e-01,  3.01048279e-01,1.55796051e-01,1.57726288e-01,1.68796539e-01,  1.83883667e-01,2.14908600e-01,2.65953064e-01,3.53698730e-01,  4.32212830e-01,4.61257935e-01,5.05386353e-01,5.67245483e-01,  6.06292725e-01,6.20223999e-01,6.16195679e-01,6.08398438e-01,  6.08718872e-01,6.08932495e-01,6.03942871e-01,5.98098755e-01,  5.91308594e-01,5.82183838e-01,5.72174072e-01,5.58502197e-01,  5.40924072e-01,5.17868042e-01,4.94010925e-01,4.78889465e-01,  4.80911255e-01,4.90623474e-01,4.89761353e-01,4.79362488e-01,  4.60380554e-01,4.22981262e-01,3.76037598e-01,3.31817627e-01,  2.63854980e-01,1.58313751e-01,7.79609680e-02,5.39865494e-02,  6.26678467e-02,8.42075348e-02,1.00738525e-01,9.15336609e-02,  1.05463028e-01,1.58012390e-01,2.28168488e-01,2.84446716e-01,  3.17695618e-01,3.29605103e-01,3.23799133e-01,3.19786072e-01,  3.37051392e-01,3.43940735e-01,3.28865051e-01,3.27430725e-01,  3.39874268e-01,3.52859497e-01,3.59794617e-01,3.68827820e-01,  3.81950378e-01,3.86024475e-01,3.78692627e-01,3.63662720e-01,  3.38615417e-01,2.94746399e-01,2.27485657e-01,1.72531128e-01,  1.87660217e-01,2.53257751e-01,3.02047729e-01,3.32603455e-01,  3.41201782e-01,3.32519531e-01,3.14453125e-01,2.89207458e-01,  2.58575439e-01,2.31445312e-01,2.13188171e-01,1.85005188e-01,  1.32888794e-01,1.20534897e-01,1.07364655e-01,5.69896698e-02,  1.71017647e-02,7.29489326e-03,6.35373592e-03,6.07383251e-03,  5.32245636e-03,4.61232662e-03,4.97770309e-03,5.46228886e-03,  5.76627254e-03,5.91897964e-03,5.91969490e-03,6.14726543e-03,  7.13706017e-03,8.26907158e-03,1.00448132e-02,1.36103630e-02,  1.43587589e-02,1.18215084e-02,1.32219791e-02,1.96704865e-02,  3.01680565e-02,4.72145081e-02,5.30996323e-02,4.18148041e-02,  3.66353989e-02,4.67472076e-02,5.74264526e-02,7.55538940e-02,  1.10506058e-01,1.46961212e-01,1.69181824e-01,1.85714722e-01,  2.05257416e-01,2.15545654e-01,2.27081299e-01,2.35290527e-01,  2.38662720e-01,2.41069794e-01,2.45609283e-01,2.53189087e-01,  2.56851196e-01,2.56210327e-01,2.52304077e-01,2.33715057e-01,  1.99867249e-01,1.88358307e-01,2.04799652e-01,2.25936890e-01,  2.34180450e-01,2.23659515e-01,1.96464539e-01,1.87191010e-01,  1.96235657e-01,2.14908600e-01,2.26490021e-01,2.29515076e-01,  2.26936340e-01,2.21920013e-01,2.16835022e-01,2.15244293e-01,  2.13802338e-01,2.13668823e-01,2.08301544e-01,1.95663452e-01,  1.90979004e-01,1.87164307e-01,1.80728912e-01,1.73389435e-01,  1.67949677e-01,1.61289215e-01,1.56158447e-01],
        [3.01642418e-02,3.18460464e-02,3.43809128e-02,4.17585373e-02,  4.57429886e-02,4.88042831e-02,5.13229370e-02,5.42478561e-02,  6.25801086e-02,7.62176514e-02,1.07864380e-01,1.29951477e-01,  1.46503448e-01,1.63463593e-01,1.75487518e-01,1.82235718e-01,  1.91932678e-01,2.20870972e-01,2.54318237e-01,2.89520264e-01,  3.22654724e-01,3.47938538e-01,3.70666504e-01,3.88656616e-01,  4.05693054e-01,4.24278259e-01,4.46220398e-01,4.54391479e-01,  4.77851868e-01,5.02410889e-01,5.06072998e-01,5.17410278e-01,  5.34027100e-01,5.31250000e-01,5.43258667e-01,5.59478760e-01,  5.71517944e-01,5.85067749e-01,5.91796875e-01,6.01104736e-01,  6.10916138e-01,6.11862183e-01,6.00952148e-01,5.87951660e-01,  5.70892334e-01,5.67214966e-01,5.98220825e-01,6.24923706e-01,  6.18286133e-01,6.24481201e-01,6.34368896e-01,6.51184082e-01,  6.66122437e-01,6.80694580e-01,6.93069458e-01,6.77047729e-01,  6.78329468e-01,7.29187012e-01,7.71392822e-01,7.84896851e-01,  7.88681030e-01,7.94097900e-01,8.39874268e-01,8.78051758e-01,  8.94332886e-01,9.05715942e-01,8.97186279e-01,7.79067993e-01,  7.23571777e-01,8.20602417e-01,8.63876343e-01,8.91113281e-01,  9.00314331e-01,8.17398071e-01,7.37014771e-01,7.38098145e-01,  7.83813477e-01,8.62335205e-01,9.03289795e-01,9.25277710e-01,  9.23233032e-01,8.13766479e-01,4.49165344e-01,4.73388672e-01,  7.90206909e-01,8.95431519e-01,9.03793335e-01,8.94927979e-01,  8.75762939e-01,8.68286133e-01,8.64166260e-01,8.59008789e-01,  7.99926758e-01,6.90673828e-01,6.71340942e-01,7.02758789e-01,  7.31674194e-01,7.75497437e-01,8.11340332e-01,8.16711426e-01,  7.87826538e-01,7.82699585e-01,8.03115845e-01,7.88192749e-01,  7.97271729e-01,8.03100586e-01,7.97531128e-01,7.94204712e-01,  7.59918213e-01,6.45690918e-01,5.74691772e-01,5.69671631e-01,  5.26199341e-01,5.20065308e-01,5.62225342e-01,5.36834717e-01,  3.48716736e-01,1.95999146e-01,2.00637817e-01,2.12623596e-01,  2.28145599e-01,2.60406494e-01,3.12873840e-01,4.00657654e-01,  4.78141785e-01,5.06103516e-01,5.46768188e-01,6.03256226e-01,  6.37832642e-01,6.49108887e-01,6.43814087e-01,6.35528564e-01,  6.35238647e-01,6.34811401e-01,6.29196167e-01,6.22909546e-01,  6.15554810e-01,6.06384277e-01,5.96527100e-01,5.83221436e-01,  5.66848755e-01,5.45623779e-01,5.23010254e-01,5.07843018e-01,  5.08743286e-01,5.17105103e-01,5.15258789e-01,5.04760742e-01,  4.86579895e-01,4.52224731e-01,4.09149170e-01,3.66828918e-01,  2.97531128e-01,1.87355042e-01,1.01324081e-01,7.46593475e-02,  8.44440460e-02,1.09689713e-01,1.29646301e-01,1.18539810e-01,  1.33499146e-01,1.89750671e-01,2.61177063e-01,3.16596985e-01,  3.47915649e-01,3.57963562e-01,3.52020264e-01,3.48197937e-01,  3.64356995e-01,3.70063782e-01,3.54484558e-01,3.52081299e-01,  3.64044189e-01,3.76548767e-01,3.82255554e-01,3.89770508e-01,  4.01176453e-01,4.04174805e-01,3.96842957e-01,3.82812500e-01,  3.59718323e-01,3.18344116e-01,2.53044128e-01,1.97788239e-01,  2.12162018e-01,2.76527405e-01,3.22059631e-01,3.50479126e-01,  3.59054565e-01,3.50990295e-01,3.33343506e-01,3.09272766e-01,  2.80326843e-01,2.54165649e-01,2.36392975e-01,2.08503723e-01,  1.55799866e-01,1.43524170e-01,1.28639221e-01,7.11193085e-02,  2.23131180e-02,8.55779648e-03,7.03787804e-03,6.76202774e-03,  6.02638721e-03,5.22339344e-03,5.50758839e-03,6.13510609e-03,  6.52873516e-03,6.57820702e-03,6.66546822e-03,7.09354877e-03,  8.81147385e-03,1.07550621e-02,1.36816502e-02,1.91435814e-02,  2.03328133e-02,1.71356201e-02,1.93600655e-02,2.82101631e-02,  4.15763855e-02,6.25572205e-02,6.92081451e-02,5.50212860e-02,  4.79812622e-02,5.98220825e-02,7.27024078e-02,9.23500061e-02,  1.28276825e-01,1.64512634e-01,1.85375214e-01,2.00267792e-01,  2.18872070e-01,2.28424072e-01,2.38620758e-01,2.46002197e-01,  2.49092102e-01,2.50991821e-01,2.54707336e-01,2.61306763e-01,  2.64312744e-01,2.63259888e-01,2.59193420e-01,2.41882324e-01,  2.09880829e-01,1.98493958e-01,2.13977814e-01,2.33673096e-01,  2.41325378e-01,2.31510162e-01,2.05955505e-01,1.97254181e-01,  2.05703735e-01,2.23033905e-01,2.34146118e-01,2.37194061e-01,  2.34867096e-01,2.30884552e-01,2.26627350e-01,2.24925995e-01,  2.24212646e-01,2.24655151e-01,2.18578339e-01,2.06340790e-01,  2.03109741e-01,1.97750092e-01,1.90414429e-01,1.84616089e-01,  1.78504944e-01,1.70944214e-01,1.67312622e-01],
        [3.64789963e-02,3.92856598e-02,4.36115265e-02,5.40399551e-02,  5.95083237e-02,6.41670227e-02,6.85024261e-02,7.21225739e-02,  8.41236115e-02,1.02903366e-01,1.45050049e-01,1.74182892e-01,  1.95121765e-01,2.16468811e-01,2.31010437e-01,2.38101959e-01,  2.49343872e-01,2.85179138e-01,3.26431274e-01,3.69262695e-01,  4.08660889e-01,4.38148499e-01,4.63920593e-01,4.83436584e-01,  5.02426147e-01,5.23696899e-01,5.47439575e-01,5.52764893e-01,  5.78155518e-01,6.05194092e-01,6.06948853e-01,6.19247437e-01,  6.36062622e-01,6.29364014e-01,6.41647339e-01,6.59912109e-01,  6.72927856e-01,6.86676025e-01,6.92169189e-01,7.01492310e-01,  7.10800171e-01,7.09915161e-01,6.96975708e-01,6.82662964e-01,  6.64596558e-01,6.60293579e-01,6.91558838e-01,7.17742920e-01,  7.09915161e-01,7.15637207e-01,7.23648071e-01,7.38800049e-01,  7.52395630e-01,7.65670776e-01,7.77954102e-01,7.60452271e-01,  7.61077881e-01,8.11660767e-01,8.52951050e-01,8.65386963e-01,  8.67706299e-01,8.69293213e-01,9.14459229e-01,9.52468872e-01,  9.66934204e-01,9.76074219e-01,9.65454102e-01,8.42636108e-01,  7.87582397e-01,8.87100220e-01,9.28512573e-01,9.53903198e-01,  9.61425781e-01,8.78524780e-01,7.99285889e-01,8.00125122e-01,  8.43887329e-01,9.17358398e-01,9.53399658e-01,9.71893311e-01,  9.68048096e-01,8.52752686e-01,4.77966309e-01,5.09170532e-01,  8.30398560e-01,9.34387207e-01,9.39971924e-01,9.31442261e-01,  9.12994385e-01,9.03701782e-01,9.00863647e-01,8.94866943e-01,  8.37066650e-01,7.32269287e-01,7.10968018e-01,7.40005493e-01,  7.68173218e-01,8.07495117e-01,8.40301514e-01,8.44375610e-01,  8.11630249e-01,8.06762695e-01,8.25149536e-01,8.07937622e-01,  8.18130493e-01,8.21853638e-01,8.15093994e-01,8.14437866e-01,  7.78884888e-01,6.68823242e-01,6.07894897e-01,6.01638794e-01,  5.60501099e-01,5.56396484e-01,5.91674805e-01,5.67626953e-01,  3.81240845e-01,2.26432800e-01,2.31971741e-01,2.44609833e-01,  2.60780334e-01,2.93403625e-01,3.45436096e-01,4.31640625e-01,  5.06942749e-01,5.33706665e-01,5.71716309e-01,6.23565674e-01,  6.54235840e-01,6.63726807e-01,6.57836914e-01,6.49185181e-01,  6.48376465e-01,6.47552490e-01,6.41952515e-01,6.35574341e-01,  6.27777100e-01,6.18270874e-01,6.08718872e-01,5.96298218e-01,  5.80917358e-01,5.60668945e-01,5.39413452e-01,5.25070190e-01,  5.24917603e-01,5.31707764e-01,5.28778076e-01,5.18295288e-01,  5.01144409e-01,4.68986511e-01,4.28939819e-01,3.88847351e-01,  3.20053101e-01,2.08168030e-01,1.19564056e-01,9.16290283e-02,  1.02180481e-01,1.29291534e-01,1.51195526e-01,1.39125824e-01,  1.54712677e-01,2.12493896e-01,2.83218384e-01,3.37211609e-01,  3.66775513e-01,3.75129700e-01,3.68911743e-01,3.65409851e-01,  3.80615234e-01,3.85337830e-01,3.69552612e-01,3.66729736e-01,  3.78280640e-01,3.90129089e-01,3.94836426e-01,4.01367188e-01,  4.11392212e-01,4.13619995e-01,4.06440735e-01,3.93463135e-01,  3.72245789e-01,3.33602905e-01,2.71003723e-01,2.16724396e-01,  2.30396271e-01,2.92366028e-01,3.34068298e-01,3.60267639e-01,  3.68492126e-01,3.60809326e-01,3.43620300e-01,3.20678711e-01,  2.93319702e-01,2.68310547e-01,2.51068115e-01,2.23678589e-01,  1.71463013e-01,1.59385681e-01,1.43676758e-01,8.17985535e-02,  2.64129639e-02,9.30881500e-03,7.24732876e-03,6.87026978e-03,  6.05583191e-03,5.36274910e-03,5.79881668e-03,6.18290901e-03,  6.52360916e-03,6.83510303e-03,7.08544254e-03,7.41648674e-03,  9.90843773e-03,1.29840374e-02,1.70779228e-02,2.36988068e-02,  2.52041817e-02,2.20193863e-02,2.48999596e-02,3.50713730e-02,  5.05037308e-02,7.43274689e-02,8.12950134e-02,6.50234222e-02,  5.70020676e-02,6.98127747e-02,8.37059021e-02,1.04009628e-01,  1.40205383e-01,1.75529480e-01,1.94587708e-01,2.08312988e-01,  2.26257324e-01,2.35042572e-01,2.44094849e-01,2.51045227e-01,  2.53845215e-01,2.55249023e-01,2.58377075e-01,2.64251709e-01,  2.66525269e-01,2.65121460e-01,2.61291504e-01,2.45105743e-01,  2.14797974e-01,2.03861237e-01,2.18383789e-01,2.36461639e-01,  2.43339539e-01,2.34535217e-01,2.10533142e-01,2.02095032e-01,  2.10189819e-01,2.26566315e-01,2.36618042e-01,2.39669800e-01,  2.37892151e-01,2.34069824e-01,2.30094910e-01,2.28897095e-01,  2.28096008e-01,2.27394104e-01,2.21431732e-01,2.09293365e-01,  2.04956055e-01,1.99241638e-01,1.92905426e-01,1.85894012e-01,  1.79328918e-01,1.72729492e-01,1.68876648e-01],
        [4.77409363e-02,5.24988174e-02,5.71804047e-02,7.04326630e-02,  7.94391632e-02,8.47492218e-02,9.01451111e-02,9.52453613e-02,  1.09790802e-01,1.34529114e-01,1.88587189e-01,2.24861145e-01,  2.50686646e-01,2.75924683e-01,2.93098450e-01,3.00056458e-01,  3.12202454e-01,3.55064392e-01,4.03343201e-01,4.53918457e-01,  4.99114990e-01,5.31311035e-01,5.59799194e-01,5.79620361e-01,  5.99960327e-01,6.23718262e-01,6.47918701e-01,6.50299072e-01,  6.76742554e-01,7.05245972e-01,7.05566406e-01,7.18063354e-01,  7.34313965e-01,7.23266602e-01,7.35778809e-01,7.55889893e-01,  7.69607544e-01,7.83081055e-01,7.87216187e-01,7.96142578e-01,  8.05023193e-01,8.02429199e-01,7.87475586e-01,7.73010254e-01,  7.54699707e-01,7.50991821e-01,7.82012939e-01,8.05953979e-01,  7.96875000e-01,8.02093506e-01,8.08609009e-01,8.23181152e-01,  8.34930420e-01,8.47869873e-01,8.59725952e-01,8.40667725e-01,  8.40621948e-01,8.90045166e-01,9.29412842e-01,9.40795898e-01,  9.40826416e-01,9.39926147e-01,9.85443115e-01,1.02233887e+00,  1.03549194e+00,1.04290771e+00,1.03018188e+00,9.04327393e-01,  8.50189209e-01,9.50378418e-01,9.90600586e-01,1.01416016e+00,  1.01983643e+00,9.34341431e-01,8.54736328e-01,8.56658936e-01,  8.99795532e-01,9.71282959e-01,1.00469971e+00,1.02136230e+00,  1.01687622e+00,8.94805908e-01,5.08361816e-01,5.49758911e-01,  8.75030518e-01,9.76028442e-01,9.81887817e-01,9.71664429e-01,  9.53933716e-01,9.46014404e-01,9.41680908e-01,9.34906006e-01,  8.76907349e-01,7.73422241e-01,7.54257202e-01,7.81127930e-01,  8.07342529e-01,8.46496582e-01,8.73687744e-01,8.77532959e-01,  8.43048096e-01,8.34213257e-01,8.57131958e-01,8.38546753e-01,  8.47702026e-01,8.52767944e-01,8.43307495e-01,8.45153809e-01,  8.14346313e-01,7.01690674e-01,6.35375977e-01,6.34872437e-01,  5.93429565e-01,5.95809937e-01,6.24801636e-01,5.94604492e-01,  4.08035278e-01,2.51434326e-01,2.58224487e-01,2.70935059e-01,  2.86979675e-01,3.19511414e-01,3.72360229e-01,4.58770752e-01,  5.33187866e-01,5.59036255e-01,5.95718384e-01,6.45736694e-01,  6.74652100e-01,6.82891846e-01,6.76300049e-01,6.67388916e-01,  6.66641235e-01,6.65405273e-01,6.59271240e-01,6.52526855e-01,  6.44622803e-01,6.35284424e-01,6.25900269e-01,6.13571167e-01,  5.98754883e-01,5.79849243e-01,5.59692383e-01,5.45196533e-01,  5.43853760e-01,5.49270630e-01,5.45532227e-01,5.34576416e-01,  5.17349243e-01,4.86053467e-01,4.47731018e-01,4.08584595e-01,  3.39241028e-01,2.25242615e-01,1.34510040e-01,1.05955124e-01,  1.16962433e-01,1.45111084e-01,1.68426514e-01,1.55982971e-01,  1.72088623e-01,2.31006622e-01,3.01780701e-01,3.55400085e-01,  3.84170532e-01,3.91624451e-01,3.85078430e-01,3.81492615e-01,  3.96331787e-01,4.00520325e-01,3.84552002e-01,3.81553650e-01,  3.92814636e-01,4.04571533e-01,4.08973694e-01,4.14749146e-01,  4.24110413e-01,4.26139832e-01,4.19082642e-01,4.06745911e-01,  3.86940002e-01,3.50479126e-01,2.90100098e-01,2.36328125e-01,  2.49122620e-01,3.09364319e-01,3.48152161e-01,3.72375488e-01,  3.80004883e-01,3.72299194e-01,3.55201721e-01,3.32595825e-01,  3.05763245e-01,2.81051636e-01,2.64045715e-01,2.36808777e-01,  1.84528351e-01,1.72351837e-01,1.56017303e-01,9.10739899e-02,  3.09014320e-02,1.11427307e-02,8.54849815e-03,8.13412666e-03,  7.44938850e-03,6.81936741e-03,7.14659691e-03,7.67683983e-03,  8.02779198e-03,8.34012032e-03,8.58068466e-03,9.13524628e-03,  1.22780800e-02,1.61404610e-02,2.09259987e-02,2.85987854e-02,  3.04083824e-02,2.71267891e-02,3.05290222e-02,4.17356491e-02,  5.84802628e-02,8.42189789e-02,9.12094116e-02,7.35797882e-02,  6.49757385e-02,7.85408020e-02,9.30347443e-02,1.13859177e-01,  1.50394440e-01,1.85405731e-01,2.03643799e-01,2.16907501e-01,  2.34573364e-01,2.42900848e-01,2.51312256e-01,2.58049011e-01,  2.61054993e-01,2.62306213e-01,2.65121460e-01,2.70515442e-01,  2.72399902e-01,2.70675659e-01,2.66868591e-01,2.51693726e-01,  2.22789764e-01,2.11971283e-01,2.25795746e-01,2.42862701e-01,  2.49256134e-01,2.41039276e-01,2.18418121e-01,2.10544586e-01,  2.18189240e-01,2.33711243e-01,2.43221283e-01,2.46452332e-01,  2.45117188e-01,2.41859436e-01,2.38704681e-01,2.38281250e-01,  2.38128662e-01,2.37770081e-01,2.32242584e-01,2.22064972e-01,  2.18711853e-01,2.12593079e-01,2.05986023e-01,2.00031281e-01,  1.93618774e-01,1.86813354e-01,1.84242249e-01],
        [6.28452301e-02,6.55708313e-02,7.21740723e-02,8.96911621e-02,  9.85260010e-02,1.07282639e-01,1.12974167e-01,1.19192123e-01,  1.37615204e-01,1.66984558e-01,2.34230042e-01,2.76466370e-01,  3.06388855e-01,3.35815430e-01,3.53591919e-01,3.60328674e-01,  3.73176575e-01,4.22241211e-01,4.77798462e-01,5.33752441e-01,  5.84014893e-01,6.18911743e-01,6.48742676e-01,6.69464111e-01,  6.89926147e-01,7.15347290e-01,7.40447998e-01,7.38647461e-01,  7.66281128e-01,7.95364380e-01,7.93716431e-01,8.06777954e-01,  8.21411133e-01,8.06671143e-01,8.19381714e-01,8.40362549e-01,  8.55590820e-01,8.68286133e-01,8.71017456e-01,8.79806519e-01,  8.87863159e-01,8.84414673e-01,8.67828369e-01,8.52264404e-01,  8.34442139e-01,8.30642700e-01,8.61968994e-01,8.84552002e-01,  8.72833252e-01,8.78311157e-01,8.84094238e-01,8.97384644e-01,  9.07699585e-01,9.18258667e-01,9.30221558e-01,9.10247803e-01,  9.09164429e-01,9.58557129e-01,9.95300293e-01,1.00405884e+00,  1.00341797e+00,9.99221802e-01,1.04473877e+00,1.08129883e+00,  1.09231567e+00,1.09860229e+00,1.08288574e+00,9.53414917e-01,  9.03823853e-01,1.00250244e+00,1.04043579e+00,1.06298828e+00,  1.06494141e+00,9.79324341e-01,9.01351929e-01,9.02282715e-01,  9.46212769e-01,1.01419067e+00,1.04418945e+00,1.06060791e+00,  1.05294800e+00,9.25170898e-01,5.31463623e-01,5.84243774e-01,  9.11819458e-01,1.00823975e+00,1.01141357e+00,1.00347900e+00,  9.84619141e-01,9.77233887e-01,9.75387573e-01,9.65209961e-01,  9.06982422e-01,8.04138184e-01,7.81738281e-01,8.12133789e-01,  8.36929321e-01,8.72573853e-01,9.02633667e-01,9.01000977e-01,  8.66668701e-01,8.60366821e-01,8.75778198e-01,8.59313965e-01,  8.69827271e-01,8.70620728e-01,8.69338989e-01,8.67340088e-01,  8.32748413e-01,7.30590820e-01,6.64978027e-01,6.60995483e-01,  6.26861572e-01,6.15997314e-01,6.50466919e-01,6.29531860e-01,  4.33319092e-01,2.71331787e-01,2.78465271e-01,2.91587830e-01,  3.07624817e-01,3.40286255e-01,3.92814636e-01,4.78401184e-01,  5.51574707e-01,5.76675415e-01,6.11816406e-01,6.59362793e-01,  6.86630249e-01,6.94000244e-01,6.86965942e-01,6.77719116e-01,  6.76666260e-01,6.75155640e-01,6.69082642e-01,6.62094116e-01,  6.53808594e-01,6.44500732e-01,6.35391235e-01,6.23657227e-01,  6.09405518e-01,5.91262817e-01,5.71899414e-01,5.57830811e-01,  5.55755615e-01,5.59738159e-01,5.55236816e-01,5.44296265e-01,  5.27297974e-01,4.96864319e-01,4.60014343e-01,4.22042847e-01,  3.53027344e-01,2.38506317e-01,1.46423340e-01,1.17290497e-01,  1.28810883e-01,1.57985687e-01,1.82262421e-01,1.69185638e-01,  1.85638428e-01,2.45399475e-01,3.15765381e-01,3.68247986e-01,  3.96011353e-01,4.02610779e-01,3.95881653e-01,3.92234802e-01,  4.06425476e-01,4.10270691e-01,3.94485474e-01,3.91471863e-01,  4.02168274e-01,4.13536072e-01,4.17739868e-01,4.22912598e-01,  4.31404114e-01,4.33135986e-01,4.26437378e-01,4.14871216e-01,  3.96270752e-01,3.61564636e-01,3.03642273e-01,2.51068115e-01,  2.63145447e-01,3.20983887e-01,3.56994629e-01,3.79653931e-01,  3.86734009e-01,3.78829956e-01,3.61877441e-01,3.39965820e-01,  3.13896179e-01,2.89733887e-01,2.73017883e-01,2.46070862e-01,  1.94160461e-01,1.81907654e-01,1.65088654e-01,9.79881287e-02,  3.40032578e-02,1.16758347e-02,8.55374336e-03,8.18443298e-03,  7.44807720e-03,6.82663918e-03,7.06827641e-03,7.59458542e-03,  8.22639465e-03,8.50629807e-03,8.91876221e-03,9.59944725e-03,  1.33721828e-02,1.79519653e-02,2.35905647e-02,3.20882797e-02,  3.41119766e-02,3.07660103e-02,3.47337723e-02,4.68254089e-02,  6.45523071e-02,9.15508270e-02,9.86862183e-02,8.02040100e-02,  7.09972382e-02,8.47911835e-02,9.98764038e-02,1.21091843e-01,  1.57482147e-01,1.91814423e-01,2.09148407e-01,2.21782684e-01,  2.39048004e-01,2.46898651e-01,2.54730225e-01,2.61360168e-01,  2.64221191e-01,2.65205383e-01,2.67707825e-01,2.72743225e-01,  2.74185181e-01,2.72178650e-01,2.68539429e-01,2.54211426e-01,  2.26428986e-01,2.15694427e-01,2.28923798e-01,2.45025635e-01,  2.50785828e-01,2.43247986e-01,2.21946716e-01,2.14210510e-01,  2.21347809e-01,2.36106873e-01,2.45201111e-01,2.48146057e-01,  2.46982574e-01,2.44251251e-01,2.41325378e-01,2.40455627e-01,  2.40253448e-01,2.40062714e-01,2.33982086e-01,2.22633362e-01,  2.19005585e-01,2.13645935e-01,2.06146240e-01,1.99611664e-01,  1.94149017e-01,1.87183380e-01,1.82754517e-01],
        [7.51171112e-02,8.01277161e-02,8.90731812e-02,1.09008789e-01,  1.22323990e-01,1.32236481e-01,1.38103485e-01,1.46320343e-01,  1.66786194e-01,2.01843262e-01,2.81219482e-01,3.30459595e-01,  3.64837646e-01,3.97277832e-01,4.17098999e-01,4.22637939e-01,  4.35066223e-01,4.90051270e-01,5.51528931e-01,6.14135742e-01,  6.68685913e-01,7.04971313e-01,7.36557007e-01,7.56622314e-01,  7.77328491e-01,8.04885864e-01,8.29269409e-01,8.24127197e-01,  8.51989746e-01,8.81439209e-01,8.78387451e-01,8.91281128e-01,  9.05181885e-01,8.86657715e-01,8.98773193e-01,9.21569824e-01,  9.37210083e-01,9.49264526e-01,9.51354980e-01,9.59823608e-01,  9.67514038e-01,9.62554932e-01,9.45114136e-01,9.29962158e-01,  9.11529541e-01,9.08355713e-01,9.39376831e-01,9.59671021e-01,  9.47860718e-01,9.52835083e-01,9.56710815e-01,9.69345093e-01,  9.77508545e-01,9.87030029e-01,9.98031616e-01,9.76364136e-01,  9.75753784e-01,1.02359009e+00,1.05752563e+00,1.06573486e+00,  1.06335449e+00,1.05664062e+00,1.10168457e+00,1.13653564e+00,  1.14672852e+00,1.15151978e+00,1.13381958e+00,1.00326538e+00,  9.54177856e-01,1.05352783e+00,1.09014893e+00,1.10989380e+00,  1.11123657e+00,1.02584839e+00,9.49172974e-01,9.51980591e-01,  9.93118286e-01,1.05737305e+00,1.08465576e+00,1.09756470e+00,  1.08978271e+00,9.57305908e-01,5.57281494e-01,6.18011475e-01,  9.46823120e-01,1.04098511e+00,1.04589844e+00,1.03552246e+00,  1.01721191e+00,1.01095581e+00,1.00637817e+00,9.97924805e-01,  9.40292358e-01,8.39675903e-01,8.19488525e-01,8.44070435e-01,  8.67309570e-01,9.04632568e-01,9.29260254e-01,9.25872803e-01,  8.92456055e-01,8.81988525e-01,8.99597168e-01,8.80355835e-01,  8.87252808e-01,8.94256592e-01,8.91845703e-01,8.87710571e-01,  8.58123779e-01,7.54196167e-01,6.92276001e-01,6.91192627e-01,  6.49017334e-01,6.47750854e-01,6.80328369e-01,6.48178101e-01,  4.56619263e-01,2.97393799e-01,3.05442810e-01,3.18534851e-01,  3.34579468e-01,3.66844177e-01,4.18998718e-01,5.03021240e-01,  5.74234009e-01,5.98144531e-01,6.31347656e-01,6.76040649e-01,  7.01049805e-01,7.07260132e-01,6.99768066e-01,6.90185547e-01,  6.88812256e-01,6.87164307e-01,6.80923462e-01,6.73828125e-01,  6.65222168e-01,6.55731201e-01,6.46728516e-01,6.35177612e-01,  6.21551514e-01,6.04110718e-01,5.85418701e-01,5.71395874e-01,  5.68710327e-01,5.72067261e-01,5.67214966e-01,5.56045532e-01,  5.39520264e-01,5.10711670e-01,4.75906372e-01,4.39346313e-01,  3.70750427e-01,2.55783081e-01,1.62815094e-01,1.33377075e-01,  1.45149231e-01,1.75041199e-01,2.00523376e-01,1.87046051e-01,  2.03590393e-01,2.63572693e-01,3.33030701e-01,3.84323120e-01,  4.10881042e-01,4.16503906e-01,4.09584045e-01,4.06005859e-01,  4.19578552e-01,4.22805786e-01,4.07005310e-01,4.03610229e-01,  4.13955688e-01,4.24911499e-01,4.28474426e-01,4.32846069e-01,  4.40559387e-01,4.41802979e-01,4.34997559e-01,4.23927307e-01,  4.06280518e-01,3.73237610e-01,3.17314148e-01,2.65632629e-01,  2.77076721e-01,3.33061218e-01,3.66798401e-01,3.88175964e-01,  3.95095825e-01,3.87474060e-01,3.70948792e-01,3.49632263e-01,  3.24378967e-01,3.00994873e-01,2.84736633e-01,2.58277893e-01,  2.06928253e-01,1.94755554e-01,1.77310944e-01,1.07650757e-01,  3.88069153e-02,1.32644176e-02,9.41371918e-03,9.05656815e-03,  8.27074051e-03,7.70449638e-03,7.96771049e-03,8.63075256e-03,  9.23824310e-03,9.68050957e-03,1.04894638e-02,1.13170147e-02,  1.57170296e-02,2.13642120e-02,2.81677246e-02,3.78856659e-02,  3.97806168e-02,3.64360809e-02,4.13055420e-02,5.45129776e-02,  7.33489990e-02,1.02045059e-01,1.09378815e-01,8.96968842e-02,  7.95555115e-02,9.37786102e-02,1.09569550e-01,1.31084442e-01,  1.67148590e-01,2.00576782e-01,2.17300415e-01,2.29190826e-01,  2.45590210e-01,2.52853394e-01,2.60375977e-01,2.66601562e-01,  2.69180298e-01,2.70149231e-01,2.72499084e-01,2.76748657e-01,  2.77671814e-01,2.75825500e-01,2.72315979e-01,2.58331299e-01,  2.31662750e-01,2.21420288e-01,2.33875275e-01,2.48905182e-01,  2.54432678e-01,2.47478485e-01,2.27012634e-01,2.19612122e-01,  2.26730347e-01,2.40821838e-01,2.49095917e-01,2.52357483e-01,  2.51724243e-01,2.49050140e-01,2.46486664e-01,2.46658325e-01,  2.46513367e-01,2.45635986e-01,2.40802765e-01,2.30690002e-01,  2.25505829e-01,2.20436096e-01,2.14817047e-01,2.07817078e-01,  2.01560974e-01,1.96949005e-01,1.93935394e-01],
        [8.81462097e-02,9.56249237e-02,1.05073929e-01,1.29837036e-01,  1.45408630e-01,1.54701233e-01,1.62483215e-01,1.71104431e-01,  1.94118500e-01,2.34298706e-01,3.24691772e-01,3.80790710e-01,  4.17877197e-01,4.52896118e-01,4.73533630e-01,4.77638245e-01,  4.89662170e-01,5.49072266e-01,6.15188599e-01,6.82388306e-01,  7.39715576e-01,7.77816772e-01,8.10531616e-01,8.29421997e-01,  8.50189209e-01,8.78112793e-01,9.02511597e-01,8.94180298e-01,  9.20883179e-01,9.51080322e-01,9.46762085e-01,9.59243774e-01,  9.72137451e-01,9.50088501e-01,9.61898804e-01,9.85534668e-01,  1.00170898e+00,1.01394653e+00,1.01473999e+00,1.02285767e+00,  1.03060913e+00,1.02429199e+00,1.00588989e+00,9.90356445e-01,  9.72900391e-01,9.70001221e-01,9.99176025e-01,1.01873779e+00,  1.00595093e+00,1.01010132e+00,1.01287842e+00,1.02401733e+00,  1.03036499e+00,1.03851318e+00,1.04824829e+00,1.02627563e+00,  1.02420044e+00,1.06948853e+00,1.10223389e+00,1.10958862e+00,  1.10574341e+00,1.09643555e+00,1.13958740e+00,1.17428589e+00,  1.18313599e+00,1.18603516e+00,1.16812134e+00,1.03808594e+00,  9.88739014e-01,1.08633423e+00,1.12109375e+00,1.14016724e+00,  1.14025879e+00,1.05593872e+00,9.81506348e-01,9.83871460e-01,  1.02188110e+00,1.08316040e+00,1.10671997e+00,1.11859131e+00,  1.11102295e+00,9.78103638e-01,5.78735352e-01,6.38366699e-01,  9.64614868e-01,1.05859375e+00,1.06253052e+00,1.05416870e+00,  1.03701782e+00,1.02725220e+00,1.02203369e+00,1.01483154e+00,  9.58297729e-01,8.61358643e-01,8.39218140e-01,8.62823486e-01,  8.86795044e-01,9.19326782e-01,9.41604614e-01,9.39514160e-01,  9.01077271e-01,8.92318726e-01,9.11941528e-01,8.89068604e-01,  8.98483276e-01,9.03717041e-01,8.94149780e-01,8.98223877e-01,  8.70773315e-01,7.68249512e-01,7.09564209e-01,7.02774048e-01,  6.66015625e-01,6.68167114e-01,6.88323975e-01,6.62399292e-01,  4.75601196e-01,3.16520691e-01,3.24958801e-01,3.37730408e-01,  3.53599548e-01,3.85749817e-01,4.37004089e-01,5.19180298e-01,  5.88363647e-01,6.11328125e-01,6.42883301e-01,6.84967041e-01,  7.08099365e-01,7.13302612e-01,7.05673218e-01,6.96060181e-01,  6.94412231e-01,6.92520142e-01,6.86233521e-01,6.79107666e-01,  6.70257568e-01,6.60690308e-01,6.51992798e-01,6.40853882e-01,  6.27395630e-01,6.10458374e-01,5.92498779e-01,5.79055786e-01,  5.75805664e-01,5.78033447e-01,5.72708130e-01,5.61950684e-01,  5.46005249e-01,5.18218994e-01,4.85191345e-01,4.50065613e-01,  3.82369995e-01,2.67814636e-01,1.74694061e-01,1.45351410e-01,  1.57455444e-01,1.87549591e-01,2.13733673e-01,2.00061798e-01,  2.16575623e-01,2.76458740e-01,3.44444275e-01,3.94363403e-01,  4.19998169e-01,4.24880981e-01,4.17594910e-01,4.13955688e-01,  4.27268982e-01,4.30038452e-01,4.14169312e-01,4.10522461e-01,  4.20471191e-01,4.31236267e-01,4.34280396e-01,4.37873840e-01,  4.44847107e-01,4.45938110e-01,4.39140320e-01,4.28512573e-01,  4.11926270e-01,3.80653381e-01,3.26828003e-01,2.76397705e-01,  2.87132263e-01,3.41056824e-01,3.72421265e-01,3.92379761e-01,  3.99101257e-01,3.91891479e-01,3.75671387e-01,3.54866028e-01,  3.30451965e-01,3.07723999e-01,2.92015076e-01,2.66098022e-01,  2.15587616e-01,2.03426361e-01,1.85806274e-01,1.14761353e-01,  4.25825119e-02,1.43663883e-02,9.87863541e-03,9.42826271e-03,  8.70537758e-03,8.36348534e-03,8.65435600e-03,9.21988487e-03,  9.96947289e-03,1.04758739e-02,1.15063190e-02,1.25188828e-02,  1.76582336e-02,2.41527557e-02,3.19156647e-02,4.23011780e-02,  4.41455841e-02,4.09421921e-02,4.66661453e-02,6.05249405e-02,  7.98778534e-02,1.09458923e-01,1.16916656e-01,9.65690613e-02,  8.59756470e-02,1.00299835e-01,1.16397858e-01,1.38019562e-01,  1.73522949e-01,2.06096649e-01,2.21828461e-01,2.32986450e-01,  2.48989105e-01,2.56111145e-01,2.62916565e-01,2.68608093e-01,  2.71080017e-01,2.72033691e-01,2.74070740e-01,2.77976990e-01,  2.78793335e-01,2.76794434e-01,2.73139954e-01,2.59796143e-01,  2.34420776e-01,2.24277496e-01,2.36049652e-01,2.50320435e-01,  2.55500793e-01,2.48901367e-01,2.29534149e-01,2.22396851e-01,  2.28939056e-01,2.42370605e-01,2.50358582e-01,2.53631592e-01,  2.53189087e-01,2.50923157e-01,2.48767853e-01,2.48931885e-01,  2.49233246e-01,2.48313904e-01,2.43080139e-01,2.33913422e-01,  2.29026794e-01,2.23022461e-01,2.17456818e-01,2.11551666e-01,  2.05310822e-01,1.98829651e-01,1.96311951e-01],
        [1.03876114e-01,1.12985611e-01,1.25541687e-01,1.51878357e-01,  1.68853760e-01,1.80519104e-01,1.88171387e-01,1.96861267e-01,  2.23350525e-01,2.67814636e-01,3.69598389e-01,4.32281494e-01,  4.73396301e-01,5.11138916e-01,5.31768799e-01,5.34729004e-01,  5.45593262e-01,6.09268188e-01,6.80236816e-01,7.52426147e-01,  8.13262939e-01,8.51898193e-01,8.84689331e-01,9.03274536e-01,  9.23553467e-01,9.52941895e-01,9.76837158e-01,9.64904785e-01,  9.90936279e-01,1.02139282e+00,1.01580811e+00,1.02746582e+00,  1.03955078e+00,1.01443481e+00,1.02548218e+00,1.05084229e+00,  1.06784058e+00,1.07952881e+00,1.07931519e+00,1.08712769e+00,  1.09408569e+00,1.08721924e+00,1.06799316e+00,1.05270386e+00,  1.03530884e+00,1.03204346e+00,1.06030273e+00,1.07836914e+00,  1.06448364e+00,1.06838989e+00,1.06924438e+00,1.07946777e+00,  1.08413696e+00,1.09027100e+00,1.09991455e+00,1.07754517e+00,  1.07427979e+00,1.11746216e+00,1.14862061e+00,1.15545654e+00,  1.14996338e+00,1.13772583e+00,1.18008423e+00,1.21432495e+00,  1.22222900e+00,1.22402954e+00,1.20550537e+00,1.07617188e+00,  1.02563477e+00,1.12265015e+00,1.15612793e+00,1.17272949e+00,  1.17269897e+00,1.09088135e+00,1.01626587e+00,1.01803589e+00,  1.05459595e+00,1.11264038e+00,1.13421631e+00,1.14425659e+00,  1.13565063e+00,1.00350952e+00,6.02050781e-01,6.59225464e-01,  9.86755371e-01,1.08071899e+00,1.08358765e+00,1.07461548e+00,  1.05813599e+00,1.04898071e+00,1.04339600e+00,1.03396606e+00,  9.80941772e-01,8.85513306e-01,8.62884521e-01,8.85040283e-01,  9.06845093e-01,9.38339233e-01,9.58694458e-01,9.55017090e-01,  9.15466309e-01,9.04495239e-01,9.22180176e-01,9.01199341e-01,  9.09988403e-01,9.15695190e-01,9.07562256e-01,9.09683228e-01,  8.83956909e-01,7.89108276e-01,7.28469849e-01,7.23434448e-01,  6.87789917e-01,6.85073853e-01,7.09701538e-01,6.80908203e-01,  4.94712830e-01,3.36441040e-01,3.45169067e-01,3.58177185e-01,  3.74076843e-01,4.05723572e-01,4.56413269e-01,5.37460327e-01,  6.04873657e-01,6.26968384e-01,6.56997681e-01,6.96990967e-01,  7.18704224e-01,7.23205566e-01,7.15179443e-01,7.05245972e-01,  7.03628540e-01,7.01644897e-01,6.95083618e-01,6.87515259e-01,  6.78741455e-01,6.69342041e-01,6.60659790e-01,6.49459839e-01,  6.36444092e-01,6.20193481e-01,6.02722168e-01,5.88989258e-01,  5.85281372e-01,5.87310791e-01,5.81695557e-01,5.70541382e-01,  5.54534912e-01,5.28152466e-01,4.96841431e-01,4.62677002e-01,  3.95118713e-01,2.80784607e-01,1.87473297e-01,1.57848358e-01,  1.69925690e-01,2.00481415e-01,2.27703094e-01,2.13699341e-01,  2.29999542e-01,2.89970398e-01,3.57284546e-01,4.06227112e-01,  4.30664062e-01,4.34776306e-01,4.27520752e-01,4.23927307e-01,  4.36553955e-01,4.38926697e-01,4.23263550e-01,4.19494629e-01,  4.29008484e-01,4.39468384e-01,4.42169189e-01,4.45243835e-01,  4.51461792e-01,4.52095032e-01,4.45487976e-01,4.35287476e-01,  4.19433594e-01,3.89328003e-01,3.37081909e-01,2.87544250e-01,  2.97843933e-01,3.49990845e-01,3.79585266e-01,3.98536682e-01,  4.05136108e-01,3.98094177e-01,3.82209778e-01,3.61793518e-01,  3.37928772e-01,3.15818787e-01,3.00369263e-01,2.74856567e-01,  2.24933624e-01,2.12757111e-01,1.94618225e-01,1.21936798e-01,  4.62503433e-02,1.52513981e-02,9.90056992e-03,9.33814049e-03,  8.73017311e-03,8.56685638e-03,8.64863396e-03,9.34457779e-03,  1.03080273e-02,1.09713078e-02,1.23178959e-02,1.34451389e-02,  1.91998482e-02,2.67305374e-02,3.53775024e-02,4.65259552e-02,  4.84695435e-02,4.54740524e-02,5.17768860e-02,6.62307739e-02,  8.63647461e-02,1.17235184e-01,1.24658585e-01,1.03351593e-01,  9.22584534e-02,1.07040405e-01,1.23434067e-01,1.44947052e-01,  1.80339813e-01,2.12295532e-01,2.26982117e-01,2.37606049e-01,  2.53372192e-01,2.60124207e-01,2.66372681e-01,2.72033691e-01,  2.74414062e-01,2.75001526e-01,2.76771545e-01,2.80380249e-01,  2.80792236e-01,2.78511047e-01,2.75108337e-01,2.62329102e-01,  2.37518311e-01,2.27535248e-01,2.38910675e-01,2.52395630e-01,  2.57171631e-01,2.51144409e-01,2.32437134e-01,2.25433350e-01,  2.31597900e-01,2.44541168e-01,2.52227783e-01,2.55363464e-01,  2.55004883e-01,2.53036499e-01,2.50915527e-01,2.50785828e-01,  2.50846863e-01,2.49588013e-01,2.43724823e-01,2.33661652e-01,  2.28633881e-01,2.23041534e-01,2.16747284e-01,2.10308075e-01,  2.04353333e-01,1.97891235e-01,1.94240570e-01],
        [1.21448517e-01,1.29249573e-01,1.41326904e-01,1.74110413e-01,  1.91692352e-01,2.03727722e-01,2.12806702e-01,2.20081329e-01,  2.49187469e-01,2.96508789e-01,4.09126282e-01,4.78218079e-01,  5.21179199e-01,5.61340332e-01,5.82412720e-01,5.83648682e-01,  5.92636108e-01,6.59454346e-01,7.34893799e-01,8.09936523e-01,  8.73077393e-01,9.12445068e-01,9.44931030e-01,9.63394165e-01,  9.83001709e-01,1.01214600e+00,1.03613281e+00,1.02056885e+00,  1.04553223e+00,1.07684326e+00,1.07012939e+00,1.08148193e+00,  1.09286499e+00,1.06527710e+00,1.07485962e+00,1.10101318e+00,  1.11904907e+00,1.13101196e+00,1.12966919e+00,1.13729858e+00,  1.14340210e+00,1.13641357e+00,1.11639404e+00,1.10073853e+00,  1.08404541e+00,1.08062744e+00,1.10748291e+00,1.12432861e+00,  1.10858154e+00,1.11260986e+00,1.11288452e+00,1.12127686e+00,  1.12524414e+00,1.12890625e+00,1.13787842e+00,1.11618042e+00,  1.11047363e+00,1.15188599e+00,1.18081665e+00,1.18612671e+00,  1.18041992e+00,1.16567993e+00,1.20660400e+00,1.24060059e+00,  1.24624634e+00,1.24774170e+00,1.22915649e+00,1.10137939e+00,  1.04904175e+00,1.14328003e+00,1.17605591e+00,1.19241333e+00,  1.19125366e+00,1.11251831e+00,1.03756714e+00,1.03796387e+00,  1.07363892e+00,1.12869263e+00,1.14892578e+00,1.15823364e+00,  1.14822388e+00,1.02087402e+00,6.21398926e-01,6.69937134e-01,  9.97207642e-01,1.09017944e+00,1.09176636e+00,1.08435059e+00,  1.06759644e+00,1.05813599e+00,1.05456543e+00,1.04324341e+00,  9.93255615e-01,8.98757935e-01,8.72482300e-01,8.95736694e-01,  9.14413452e-01,9.43878174e-01,9.65270996e-01,9.56420898e-01,  9.18746948e-01,9.07897949e-01,9.23889160e-01,9.07104492e-01,  9.12673950e-01,9.15222168e-01,9.13223267e-01,9.13101196e-01,  8.87176514e-01,7.97943115e-01,7.30895996e-01,7.25860596e-01,  6.96029663e-01,6.87545776e-01,7.18582153e-01,6.96685791e-01,  5.08499146e-01,3.50540161e-01,3.59573364e-01,3.72344971e-01,  3.88153076e-01,4.19441223e-01,4.69558716e-01,5.48919678e-01,  6.14776611e-01,6.35971069e-01,6.64779663e-01,7.03201294e-01,  7.23739624e-01,7.27630615e-01,7.19207764e-01,7.09228516e-01,  7.07794189e-01,7.05413818e-01,6.98562622e-01,6.91131592e-01,  6.82464600e-01,6.72897339e-01,6.64123535e-01,6.53259277e-01,  6.40869141e-01,6.25137329e-01,6.07894897e-01,5.94192505e-01,  5.90484619e-01,5.91918945e-01,5.85586548e-01,5.74478149e-01,  5.59204102e-01,5.33401489e-01,5.03005981e-01,4.69970703e-01,  4.03541565e-01,2.89855957e-01,1.96517944e-01,1.67083740e-01,  1.79492950e-01,2.09777832e-01,2.37152100e-01,2.23506927e-01,  2.39860535e-01,2.99301147e-01,3.65364075e-01,4.13513184e-01,  4.37217712e-01,4.40513611e-01,4.33135986e-01,4.29740906e-01,  4.41925049e-01,4.43840027e-01,4.28260803e-01,4.24674988e-01,  4.33731079e-01,4.43656921e-01,4.46327209e-01,4.49081421e-01,  4.54627991e-01,4.55162048e-01,4.48760986e-01,4.38896179e-01,  4.23637390e-01,3.94760132e-01,3.44520569e-01,2.96257019e-01,  3.05969238e-01,3.56300354e-01,3.83911133e-01,4.01695251e-01,  4.08134460e-01,4.01298523e-01,3.85589600e-01,3.65524292e-01,  3.42224121e-01,3.20602417e-01,3.05442810e-01,2.80479431e-01,  2.31422424e-01,2.19242096e-01,2.00870514e-01,1.27628326e-01,  4.98838425e-02,1.66730881e-02,1.06911659e-02,1.02343559e-02,  9.66095924e-03,9.55915451e-03,9.78088379e-03,1.04784966e-02,  1.14688873e-02,1.23088360e-02,1.39558315e-02,1.51109695e-02,  2.13127136e-02,2.97474861e-02,3.89986038e-02,5.04684448e-02,  5.24578094e-02,4.98428345e-02,5.64498901e-02,7.11612701e-02,  9.18083191e-02,1.23165131e-01,1.30397797e-01,1.08818054e-01,  9.76428986e-02,1.12226486e-01,1.28715515e-01,1.50409698e-01,  1.85192108e-01,2.16442108e-01,2.30628967e-01,2.40730286e-01,  2.56088257e-01,2.62687683e-01,2.68531799e-01,2.73826599e-01,  2.76268005e-01,2.76863098e-01,2.78236389e-01,2.81669617e-01,  2.82043457e-01,2.79563904e-01,2.76206970e-01,2.63999939e-01,  2.39974976e-01,2.30171204e-01,2.41222382e-01,2.53967285e-01,  2.58499146e-01,2.52876282e-01,2.34886169e-01,2.28065491e-01,  2.34222412e-01,2.46559143e-01,2.53921509e-01,2.57316589e-01,  2.56988525e-01,2.55279541e-01,2.53837585e-01,2.53776550e-01,  2.54173279e-01,2.53372192e-01,2.47650146e-01,2.38803864e-01,  2.34638214e-01,2.28462219e-01,2.22797394e-01,2.17407227e-01,  2.10517883e-01,2.05207825e-01,2.02823639e-01],
        [1.38267517e-01,1.46911621e-01,1.59931183e-01,1.95411682e-01,  2.16823578e-01,2.28519440e-01,2.37091064e-01,2.45273590e-01,  2.76382446e-01,3.28414917e-01,4.50668335e-01,5.24780273e-01,  5.70175171e-01,6.11511230e-01,6.32720947e-01,6.31973267e-01,  6.39297485e-01,7.09732056e-01,7.88436890e-01,8.67340088e-01,  9.32479858e-01,9.71878052e-01,1.00500488e+00,1.02194214e+00,  1.04095459e+00,1.07073975e+00,1.09423828e+00,1.07550049e+00,  1.09927368e+00,1.13058472e+00,1.12286377e+00,1.13357544e+00,  1.14419556e+00,1.11386108e+00,1.12289429e+00,1.15002441e+00,  1.16833496e+00,1.17993164e+00,1.17803955e+00,1.18502808e+00,  1.19094849e+00,1.18292236e+00,1.16220093e+00,1.14709473e+00,  1.13027954e+00,1.12710571e+00,1.15307617e+00,1.16870117e+00,  1.15307617e+00,1.15621948e+00,1.15533447e+00,1.16284180e+00,  1.16433716e+00,1.16741943e+00,1.17538452e+00,1.15264893e+00,  1.14688110e+00,1.18609619e+00,1.21356201e+00,1.21780396e+00,  1.21029663e+00,1.19381714e+00,1.23370361e+00,1.26736450e+00,  1.27239990e+00,1.27203369e+00,1.25302124e+00,1.12582397e+00,  1.07400513e+00,1.16784668e+00,1.19860840e+00,1.21237183e+00,  1.21115112e+00,1.13418579e+00,1.06112671e+00,1.06198120e+00,  1.09484863e+00,1.14788818e+00,1.16513062e+00,1.17315674e+00,  1.16336060e+00,1.03353882e+00,6.34689331e-01,6.86050415e-01,  1.01028442e+00,1.10290527e+00,1.10385132e+00,1.09585571e+00,  1.07955933e+00,1.06997681e+00,1.06579590e+00,1.05606079e+00,  1.00582886e+00,9.14215088e-01,8.88748169e-01,9.10186768e-01,  9.29870605e-01,9.55947876e-01,9.74380493e-01,9.66430664e-01,  9.25613403e-01,9.16137695e-01,9.34188843e-01,9.10964966e-01,  9.20120239e-01,9.27642822e-01,9.21203613e-01,9.20318604e-01,  8.94638062e-01,8.05419922e-01,7.47497559e-01,7.44369507e-01,  7.12158203e-01,7.06954956e-01,7.31353760e-01,7.06817627e-01,  5.23086548e-01,3.66653442e-01,3.75946045e-01,3.88565063e-01,  4.04106140e-01,4.34997559e-01,4.84191895e-01,5.61660767e-01,  6.25228882e-01,6.45248413e-01,6.72653198e-01,7.09091187e-01,  7.28210449e-01,7.31109619e-01,7.22229004e-01,7.12432861e-01,  7.10800171e-01,7.08282471e-01,7.01263428e-01,6.93969727e-01,  6.85165405e-01,6.75552368e-01,6.66915894e-01,6.56387329e-01,  6.44088745e-01,6.28555298e-01,6.12030029e-01,5.98892212e-01,  5.94467163e-01,5.95260620e-01,5.89111328e-01,5.78369141e-01,  5.63247681e-01,5.38436890e-01,5.09826660e-01,4.78096008e-01,  4.12513733e-01,2.99942017e-01,2.07401276e-01,1.78039551e-01,  1.90158844e-01,2.20428467e-01,2.48493195e-01,2.34657288e-01,  2.50823975e-01,3.09928894e-01,3.74694824e-01,4.21173096e-01,  4.43778992e-01,4.46670532e-01,4.39002991e-01,4.35371399e-01,  4.47235107e-01,4.48814392e-01,4.33280945e-01,4.29534912e-01,  4.38453674e-01,4.47998047e-01,4.50019836e-01,4.52217102e-01,  4.57412720e-01,4.57656860e-01,4.51126099e-01,4.41581726e-01,  4.27055359e-01,3.99276733e-01,3.50532532e-01,3.03359985e-01,  3.12705994e-01,3.61312866e-01,3.87298584e-01,4.04319763e-01,  4.10629272e-01,4.03938293e-01,3.88710022e-01,3.69316101e-01,  3.46557617e-01,3.25447083e-01,3.10935974e-01,2.86674500e-01,  2.38430023e-01,2.26303101e-01,2.07965851e-01,1.33899689e-01,  5.31759262e-02,1.73816681e-02,1.07364655e-02,9.93776321e-03,  9.41848755e-03,9.85908508e-03,9.84382629e-03,1.03356838e-02,  1.19509697e-02,1.29504204e-02,1.46629810e-02,1.61414146e-02,  2.32648849e-02,3.23629379e-02,4.24070358e-02,5.48553467e-02,  5.67550659e-02,5.39464951e-02,6.13346100e-02,7.66448975e-02,  9.73663330e-02,1.29493713e-01,1.36940002e-01,1.14665985e-01,  1.03183746e-01,1.18021011e-01,1.34407043e-01,1.55796051e-01,  1.90338135e-01,2.20642090e-01,2.33665466e-01,2.43434906e-01,  2.58476257e-01,2.64411926e-01,2.69897461e-01,2.75024414e-01,  2.77046204e-01,2.77503967e-01,2.78930664e-01,2.81845093e-01,  2.81982422e-01,2.79685974e-01,2.76046753e-01,2.64259338e-01,  2.41268158e-01,2.31380463e-01,2.41821289e-01,2.54257202e-01,  2.58430481e-01,2.52906799e-01,2.35816956e-01,2.29076385e-01,  2.34748840e-01,2.46921539e-01,2.53768921e-01,2.56950378e-01,  2.56942749e-01,2.55104065e-01,2.53517151e-01,2.53654480e-01,  2.53433228e-01,2.51976013e-01,2.46406555e-01,2.36812592e-01,  2.32334137e-01,2.26722717e-01,2.19806671e-01,2.13962555e-01,  2.08175659e-01,2.01320648e-01,1.99081421e-01],
        [1.56509399e-01,1.65454865e-01,1.82109833e-01,2.20233917e-01,  2.42450714e-01,2.56355286e-01,2.63702393e-01,2.73056030e-01,  3.06755066e-01,3.63082886e-01,4.97871399e-01,5.75942993e-01,  6.24038696e-01,6.66931152e-01,6.87408447e-01,6.84982300e-01,  6.91085815e-01,7.65686035e-01,8.48175049e-01,9.29962158e-01,  9.97863770e-01,1.03729248e+00,1.07086182e+00,1.08645630e+00,  1.10464478e+00,1.13583374e+00,1.15756226e+00,1.13494873e+00,  1.15954590e+00,1.19003296e+00,1.18029785e+00,1.19140625e+00,  1.19992065e+00,1.16650391e+00,1.17556763e+00,1.20358276e+00,  1.22229004e+00,1.23281860e+00,1.22982788e+00,1.23632812e+00,  1.24163818e+00,1.23260498e+00,1.21047974e+00,1.19595337e+00,  1.17971802e+00,1.17669678e+00,1.20281982e+00,1.21658325e+00,  1.20001221e+00,1.20324707e+00,1.20077515e+00,1.20788574e+00,  1.20739746e+00,1.20898438e+00,1.21691895e+00,1.19290161e+00,  1.18731689e+00,1.22528076e+00,1.25042725e+00,1.25433350e+00,  1.24533081e+00,1.22665405e+00,1.26696777e+00,1.29852295e+00,  1.30276489e+00,1.30209351e+00,1.28076172e+00,1.15313721e+00,  1.10415649e+00,1.19720459e+00,1.22708130e+00,1.23904419e+00,  1.23614502e+00,1.15985107e+00,1.08914185e+00,1.09100342e+00,  1.12289429e+00,1.17153931e+00,1.18682861e+00,1.19326782e+00,  1.18261719e+00,1.05010986e+00,6.48605347e-01,7.11212158e-01,  1.03240967e+00,1.12133789e+00,1.12280273e+00,1.11215210e+00,  1.09686279e+00,1.08898926e+00,1.08303833e+00,1.07348633e+00,  1.02398682e+00,9.33349609e-01,9.09866333e-01,9.28497314e-01,  9.46487427e-01,9.72824097e-01,9.86740112e-01,9.80941772e-01,  9.41055298e-01,9.26422119e-01,9.49172974e-01,9.25903320e-01,  9.31213379e-01,9.38980103e-01,9.32296753e-01,9.31243896e-01,  9.09851074e-01,8.21289062e-01,7.67852783e-01,7.62847900e-01,  7.28439331e-01,7.26943970e-01,7.48657227e-01,7.23632812e-01,  5.40832520e-01,3.85406494e-01,3.95370483e-01,4.07951355e-01,  4.22668457e-01,4.52972412e-01,5.01937866e-01,5.77651978e-01,  6.38778687e-01,6.58065796e-01,6.84478760e-01,7.18902588e-01,  7.36221313e-01,7.38815308e-01,7.30010986e-01,7.19696045e-01,  7.17788696e-01,7.15316772e-01,7.08450317e-01,7.00805664e-01,  6.91772461e-01,6.82281494e-01,6.73583984e-01,6.62979126e-01,  6.51138306e-01,6.36169434e-01,6.19812012e-01,6.06399536e-01,  6.01867676e-01,6.02523804e-01,5.95977783e-01,5.85235596e-01,  5.70541382e-01,5.46524048e-01,5.19317627e-01,4.88647461e-01,  4.23873901e-01,3.11973572e-01,2.19650269e-01,1.90582275e-01,  2.02789307e-01,2.32902527e-01,2.61390686e-01,2.47604370e-01,  2.63572693e-01,3.22242737e-01,3.85726929e-01,4.31053162e-01,  4.52957153e-01,4.55146790e-01,4.47395325e-01,4.43664551e-01,  4.55139160e-01,4.56565857e-01,4.40956116e-01,4.36935425e-01,  4.45495605e-01,4.54711914e-01,4.56390381e-01,4.58213806e-01,  4.62791443e-01,4.62593079e-01,4.56176758e-01,4.46975708e-01,  4.32922363e-01,4.05982971e-01,3.58535767e-01,3.12355042e-01,  3.21205139e-01,3.68408203e-01,3.93013000e-01,4.09408569e-01,  4.15634155e-01,4.09217834e-01,3.94279480e-01,3.75343323e-01,  3.53172302e-01,3.32534790e-01,3.18305969e-01,2.94532776e-01,  2.47135162e-01,2.35027313e-01,2.16426849e-01,1.41502380e-01,  5.81369400e-02,1.95331573e-02,1.18889809e-02,1.10132694e-02,  1.06623173e-02,1.14622116e-02,1.12423897e-02,1.20353699e-02,  1.40161514e-02,1.49652958e-02,1.70326233e-02,1.87063217e-02,  2.64029503e-02,3.65304947e-02,4.75654602e-02,6.04181290e-02,  6.21547699e-02,5.98001480e-02,6.77947998e-02,8.33644867e-02,  1.04703903e-01,1.37561798e-01,1.44737244e-01,1.22020721e-01,  1.10305786e-01,1.25072479e-01,1.41723633e-01,1.63223267e-01,  1.97021484e-01,2.26577759e-01,2.39143372e-01,2.48249054e-01,  2.62863159e-01,2.68707275e-01,2.73681641e-01,2.78533936e-01,  2.80570984e-01,2.80723572e-01,2.81929016e-01,2.84767151e-01,  2.84645081e-01,2.82264709e-01,2.78839111e-01,2.67288208e-01,  2.44823456e-01,2.35263824e-01,2.45414734e-01,2.57308960e-01,  2.61199951e-01,2.55958557e-01,2.39536285e-01,2.33108521e-01,  2.38746643e-01,2.50335693e-01,2.57011414e-01,2.60368347e-01,  2.60620117e-01,2.59170532e-01,2.58010864e-01,2.58468628e-01,  2.59132385e-01,2.58193970e-01,2.52853394e-01,2.44426727e-01,  2.40329742e-01,2.35427856e-01,2.29713440e-01,2.23495483e-01,  2.17956543e-01,2.12265015e-01,2.09205627e-01],
        [1.73320770e-01,1.83830261e-01,2.00065613e-01,2.42446899e-01,  2.65998840e-01,2.81143188e-01,2.88681030e-01,2.96882629e-01,  3.32283020e-01,3.93531799e-01,5.36880493e-01,6.18621826e-01,  6.67083740e-01,7.11044312e-01,7.30850220e-01,7.25769043e-01,  7.31872559e-01,8.08715820e-01,8.94180298e-01,9.78698730e-01,  1.04724121e+00,1.08770752e+00,1.12048340e+00,1.13507080e+00,  1.15341187e+00,1.18432617e+00,1.20547485e+00,1.17984009e+00,  1.20422363e+00,1.23471069e+00,1.22351074e+00,1.23455811e+00,  1.24169922e+00,1.20541382e+00,1.21517944e+00,1.24313354e+00,  1.26214600e+00,1.27200317e+00,1.26779175e+00,1.27465820e+00,  1.27960205e+00,1.26992798e+00,1.24795532e+00,1.23284912e+00,  1.21740723e+00,1.21463013e+00,1.23965454e+00,1.25262451e+00,  1.23532104e+00,1.23818970e+00,1.23513794e+00,1.24084473e+00,  1.23928833e+00,1.24029541e+00,1.24725342e+00,1.22323608e+00,  1.21752930e+00,1.25436401e+00,1.27789307e+00,1.28021240e+00,  1.27169800e+00,1.25128174e+00,1.29061890e+00,1.32153320e+00,  1.32412720e+00,1.32241821e+00,1.30044556e+00,1.17211914e+00,  1.12719727e+00,1.21881104e+00,1.24612427e+00,1.25817871e+00,  1.25335693e+00,1.17782593e+00,1.11019897e+00,1.11181641e+00,  1.14331055e+00,1.18850708e+00,1.20080566e+00,1.20718384e+00,  1.19497681e+00,1.05889893e+00,6.59057617e-01,7.29415894e-01,  1.04708862e+00,1.13189697e+00,1.13284302e+00,1.12475586e+00,  1.10833740e+00,1.09942627e+00,1.09591675e+00,1.08340454e+00,  1.03506470e+00,9.48730469e-01,9.23141479e-01,9.43222046e-01,  9.59777832e-01,9.83169556e-01,1.00125122e+00,9.90875244e-01,  9.47998047e-01,9.37408447e-01,9.53994751e-01,9.34829712e-01,  9.41238403e-01,9.43801880e-01,9.41070557e-01,9.42123413e-01,  9.18685913e-01,8.36517334e-01,7.81051636e-01,7.73040771e-01,  7.46841431e-01,7.42172241e-01,7.61901855e-01,7.37197876e-01,  5.54916382e-01,4.00573730e-01,4.10530090e-01,4.22798157e-01,  4.37591553e-01,4.67391968e-01,5.15151978e-01,5.88729858e-01,  6.47964478e-01,6.66244507e-01,6.91101074e-01,7.23754883e-01,  7.39913940e-01,7.41744995e-01,7.32635498e-01,7.22412109e-01,  7.20565796e-01,7.17758179e-01,7.10769653e-01,7.03262329e-01,  6.94168091e-01,6.84371948e-01,6.75933838e-01,6.65756226e-01,  6.53945923e-01,6.39144897e-01,6.23321533e-01,6.10107422e-01,  6.05163574e-01,6.05407715e-01,5.98999023e-01,5.88302612e-01,  5.73974609e-01,5.51101685e-01,5.25146484e-01,4.95605469e-01,  4.32006836e-01,3.21563721e-01,2.29797363e-01,2.01133728e-01,  2.13447571e-01,2.43091583e-01,2.71743774e-01,2.58316040e-01,  2.74078369e-01,3.31649780e-01,3.93577576e-01,4.37660217e-01,  4.58602905e-01,4.60319519e-01,4.52507019e-01,4.48837280e-01,  4.59777832e-01,4.60777283e-01,4.45404053e-01,4.41200256e-01,  4.49401855e-01,4.58290100e-01,4.59648132e-01,4.61021423e-01,  4.65072632e-01,4.64675903e-01,4.58236694e-01,4.49234009e-01,  4.35699463e-01,4.09759521e-01,3.63647461e-01,3.18420410e-01,  3.27011108e-01,3.72817993e-01,3.95980835e-01,4.11598206e-01,  4.17800903e-01,4.11582947e-01,3.97132874e-01,3.78822327e-01,  3.57055664e-01,3.36967468e-01,3.23356628e-01,3.00071716e-01,  2.53669739e-01,2.41786957e-01,2.22927094e-01,1.47579193e-01,  6.22444153e-02,2.10695267e-02,1.23445988e-02,1.15349293e-02,  1.13110542e-02,1.24330521e-02,1.21319294e-02,1.29306316e-02,  1.52006149e-02,1.63474083e-02,1.87988281e-02,2.07486153e-02,  2.91361809e-02,4.00314331e-02,5.17120361e-02,6.51302338e-02,  6.67667389e-02,6.45122528e-02,7.30991364e-02,8.90178680e-02,  1.10425949e-01,1.43825531e-01,1.50974274e-01,1.27967834e-01,  1.16073608e-01,1.30699158e-01,1.47342682e-01,1.68521881e-01,  2.01637268e-01,2.30594635e-01,2.42336273e-01,2.50839233e-01,  2.65266418e-01,2.70614624e-01,2.75115967e-01,2.79823303e-01,  2.81684875e-01,2.81929016e-01,2.82943726e-01,2.85369873e-01,  2.85171509e-01,2.82646179e-01,2.79174805e-01,2.68119812e-01,  2.46227264e-01,2.36778259e-01,2.46623993e-01,2.57736206e-01,  2.61566162e-01,2.56729126e-01,2.40707397e-01,2.34645844e-01,  2.39856720e-01,2.51190186e-01,2.57881165e-01,2.60948181e-01,  2.61413574e-01,2.60154724e-01,2.58995056e-01,2.59712219e-01,  2.60116577e-01,2.59223938e-01,2.54653931e-01,2.46273041e-01,  2.41897583e-01,2.35847473e-01,2.30659485e-01,2.25257874e-01,  2.18894958e-01,2.14515686e-01,2.11280823e-01],
        [1.89529419e-01,1.97498322e-01,2.16068268e-01,2.62825012e-01,  2.83599854e-01,3.01742554e-01,3.09181213e-01,3.18595886e-01,  3.56788635e-01,4.17831421e-01,5.69869995e-01,6.54281616e-01,  7.04238892e-01,7.50503540e-01,7.68447876e-01,7.62191772e-01,  7.66128540e-01,8.45596313e-01,9.35714722e-01,1.02264404e+00,  1.09356689e+00,1.13357544e+00,1.16723633e+00,1.18301392e+00,  1.20022583e+00,1.23333740e+00,1.25363159e+00,1.22424316e+00,  1.25057983e+00,1.28115845e+00,1.26940918e+00,1.28118896e+00,  1.28668213e+00,1.24859619e+00,1.25860596e+00,1.28756714e+00,  1.30825806e+00,1.31689453e+00,1.31176758e+00,1.31796265e+00,  1.32177734e+00,1.31143188e+00,1.28741455e+00,1.27252197e+00,  1.25689697e+00,1.25308228e+00,1.27767944e+00,1.28616333e+00,  1.26828003e+00,1.27316284e+00,1.26824951e+00,1.27420044e+00,  1.27148438e+00,1.26910400e+00,1.27615356e+00,1.25036621e+00,  1.24453735e+00,1.28134155e+00,1.30273438e+00,1.30554199e+00,  1.29531860e+00,1.27124023e+00,1.31234741e+00,1.34231567e+00,  1.34423828e+00,1.34088135e+00,1.31588745e+00,1.18832397e+00,  1.14404297e+00,1.23364258e+00,1.26144409e+00,1.26943970e+00,  1.26385498e+00,1.19149780e+00,1.12335205e+00,1.12612915e+00,  1.15600586e+00,1.19653320e+00,1.20996094e+00,1.21502686e+00,  1.20004272e+00,1.06454468e+00,6.66671753e-01,7.40768433e-01,  1.05517578e+00,1.13653564e+00,1.14157104e+00,1.13116455e+00,  1.11251831e+00,1.10675049e+00,1.09881592e+00,1.08953857e+00,  1.04641724e+00,9.55001831e-01,9.36157227e-01,9.54452515e-01,  9.61898804e-01,9.92736816e-01,1.00164795e+00,9.84329224e-01,  9.55566406e-01,9.37454224e-01,9.58328247e-01,9.39758301e-01,  9.31671143e-01,9.45693970e-01,9.41299438e-01,9.27917480e-01,  9.18014526e-01,8.37829590e-01,7.80670166e-01,7.89031982e-01,  7.36938477e-01,7.44400024e-01,7.65975952e-01,7.25875854e-01,  5.59478760e-01,4.12185669e-01,4.22233582e-01,4.34364319e-01,  4.49172974e-01,4.78538513e-01,5.25650024e-01,5.98007202e-01,  6.55990601e-01,6.73400879e-01,6.97311401e-01,7.28713989e-01,  7.43911743e-01,7.45223999e-01,7.36099243e-01,7.25891113e-01,  7.23876953e-01,7.21099854e-01,7.14004517e-01,7.06237793e-01,  6.97082520e-01,6.87454224e-01,6.78833008e-01,6.68670654e-01,  6.57180786e-01,6.42654419e-01,6.26953125e-01,6.14089966e-01,  6.08978271e-01,6.08886719e-01,6.02325439e-01,5.91659546e-01,  5.77468872e-01,5.55114746e-01,5.30105591e-01,5.01342773e-01,  4.38484192e-01,3.28666687e-01,2.37274170e-01,2.08862305e-01,  2.21099854e-01,2.50617981e-01,2.79747009e-01,2.66128540e-01,  2.81822205e-01,3.39172363e-01,3.99971008e-01,4.43199158e-01,  4.63722229e-01,4.64927673e-01,4.56817627e-01,4.53323364e-01,  4.63928223e-01,4.64599609e-01,4.49592590e-01,4.45297241e-01,  4.53071594e-01,4.61776733e-01,4.62829590e-01,4.63722229e-01,  4.67582703e-01,4.67018127e-01,4.60472107e-01,4.51881409e-01,  4.38774109e-01,4.13421631e-01,3.68576050e-01,3.24157715e-01,  3.32359314e-01,3.76937866e-01,3.98750305e-01,4.13864136e-01,  4.20066833e-01,4.13818359e-01,3.99711609e-01,3.81706238e-01,  3.60267639e-01,3.40606689e-01,3.27163696e-01,3.04290771e-01,  2.58628845e-01,2.46505737e-01,2.27443695e-01,1.52080536e-01,  6.48536682e-02,2.15067863e-02,1.21262074e-02,1.10411644e-02,  1.09622478e-02,1.24888420e-02,1.18606091e-02,1.29413605e-02,  1.56784058e-02,1.67088509e-02,1.94697380e-02,2.16789246e-02,  3.04012299e-02,4.20312881e-02,5.42545319e-02,6.80904388e-02,  6.96868896e-02,6.76460266e-02,7.66735077e-02,9.29222107e-02,  1.14511490e-01,1.48212433e-01,1.55250549e-01,1.32038116e-01,  1.19911194e-01,1.34647369e-01,1.51210785e-01,1.72267914e-01,  2.05101013e-01,2.33268738e-01,2.44472504e-01,2.52738953e-01,  2.66777039e-01,2.72010803e-01,2.76115417e-01,2.80555725e-01,  2.82402039e-01,2.82325745e-01,2.83256531e-01,2.85568237e-01,  2.85179138e-01,2.82585144e-01,2.79029846e-01,2.68333435e-01,  2.47005463e-01,2.37613678e-01,2.47093201e-01,2.57843018e-01,  2.61337280e-01,2.56599426e-01,2.41130829e-01,2.35176086e-01,  2.40135193e-01,2.51167297e-01,2.57347107e-01,2.60658264e-01,  2.60963440e-01,2.59628296e-01,2.58544922e-01,2.58651733e-01,  2.59147644e-01,2.57156372e-01,2.51754761e-01,2.43305206e-01,  2.37648010e-01,2.32769012e-01,2.26139069e-01,2.20424652e-01,  2.15278625e-01,2.08789825e-01,2.05513000e-01],
        [2.05684662e-01,2.17933655e-01,2.37319946e-01,2.86575317e-01,  3.11256409e-01,3.26728821e-01,3.35151672e-01,3.41934204e-01,  3.80630493e-01,4.47814941e-01,6.06735229e-01,6.95892334e-01,  7.45971680e-01,7.91458130e-01,8.10714722e-01,8.02337646e-01,  8.06655884e-01,8.87924194e-01,9.79888916e-01,1.06936646e+00,  1.14041138e+00,1.18151855e+00,1.21536255e+00,1.22860718e+00,  1.24630737e+00,1.27841187e+00,1.29925537e+00,1.26760864e+00,  1.29089355e+00,1.32196045e+00,1.30914307e+00,1.32159424e+00,  1.32659912e+00,1.28512573e+00,1.29519653e+00,1.32449341e+00,  1.34521484e+00,1.35452271e+00,1.34820557e+00,1.35473633e+00,  1.35855103e+00,1.34637451e+00,1.32330322e+00,1.30740356e+00,  1.29278564e+00,1.28967285e+00,1.31204224e+00,1.32067871e+00,  1.30264282e+00,1.30682373e+00,1.30187988e+00,1.30484009e+00,  1.30194092e+00,1.29953003e+00,1.30541992e+00,1.28024292e+00,  1.27261353e+00,1.30789185e+00,1.32864380e+00,1.32897949e+00,  1.31942749e+00,1.29592896e+00,1.33441162e+00,1.36398315e+00,  1.36502075e+00,1.36166382e+00,1.33804321e+00,1.20950317e+00,  1.16571045e+00,1.25292969e+00,1.27825928e+00,1.28887939e+00,  1.28182983e+00,1.20837402e+00,1.14208984e+00,1.14282227e+00,  1.17199707e+00,1.21203613e+00,1.22277832e+00,1.22836304e+00,  1.21368408e+00,1.07785034e+00,6.83700562e-01,7.55844116e-01,  1.06838989e+00,1.14828491e+00,1.14990234e+00,1.14279175e+00,  1.12542725e+00,1.11996460e+00,1.11611938e+00,1.10217285e+00,  1.05856323e+00,9.74380493e-01,9.49340820e-01,9.70138550e-01,  9.79278564e-01,1.00122070e+00,1.01617432e+00,1.00100708e+00,  9.64111328e-01,9.52850342e-01,9.64355469e-01,9.50347900e-01,  9.52377319e-01,9.57077026e-01,9.59014893e-01,9.52346802e-01,  9.29779053e-01,8.55010986e-01,7.99148560e-01,8.00231934e-01,  7.67822266e-01,7.58499146e-01,7.89428711e-01,7.67608643e-01,  5.76904297e-01,4.22805786e-01,4.32815552e-01,4.45106506e-01,  4.59770203e-01,4.88861084e-01,5.35781860e-01,6.07055664e-01,  6.63650513e-01,6.80633545e-01,7.03933716e-01,7.34466553e-01,  7.49176025e-01,7.50457764e-01,7.41058350e-01,7.30682373e-01,  7.28546143e-01,7.25524902e-01,7.18399048e-01,7.10784912e-01,  7.01477051e-01,6.91864014e-01,6.83502197e-01,6.73309326e-01,  6.61972046e-01,6.47705078e-01,6.32186890e-01,6.19201660e-01,  6.13845825e-01,6.13464355e-01,6.06643677e-01,5.95977783e-01,  5.81878662e-01,5.59890747e-01,5.35629272e-01,5.07415771e-01,  4.44885254e-01,3.35678101e-01,2.44522095e-01,2.15988159e-01,  2.28218079e-01,2.57690430e-01,2.86926270e-01,2.73468018e-01,  2.89154053e-01,3.46244812e-01,4.06387329e-01,4.49012756e-01,  4.69017029e-01,4.69833374e-01,4.61761475e-01,4.58129883e-01,  4.68574524e-01,4.69032288e-01,4.53895569e-01,4.49752808e-01,  4.57420349e-01,4.65812683e-01,4.66773987e-01,4.67559814e-01,  4.71015930e-01,4.70451355e-01,4.63981628e-01,4.55383301e-01,  4.42741394e-01,4.18075562e-01,3.74229431e-01,3.30772400e-01,  3.38607788e-01,3.81919861e-01,4.02839661e-01,4.17251587e-01,  4.23164368e-01,4.17106628e-01,4.02976990e-01,3.85162354e-01,  3.64112854e-01,3.44673157e-01,3.31367493e-01,3.08731079e-01,  2.63671875e-01,2.51846313e-01,2.32624054e-01,1.56600952e-01,  6.80580139e-02,2.32663155e-02,1.32541656e-02,1.19256973e-02,  1.21500492e-02,1.40593052e-02,1.32937431e-02,1.44205093e-02,  1.73726082e-02,1.82991028e-02,2.13537216e-02,2.36926079e-02,  3.28369141e-02,4.49066162e-02,5.75180054e-02,7.17620850e-02,  7.33776093e-02,7.13977814e-02,8.07743073e-02,9.70726013e-02,  1.18831635e-01,1.52698517e-01,1.59667969e-01,1.36310577e-01,  1.24013901e-01,1.38763428e-01,1.55433655e-01,1.76418304e-01,  2.09030151e-01,2.36763000e-01,2.47562408e-01,2.55706787e-01,  2.69470215e-01,2.74528503e-01,2.78518677e-01,2.82783508e-01,  2.84622192e-01,2.84576416e-01,2.85385132e-01,2.87719727e-01,  2.87139893e-01,2.84454346e-01,2.81044006e-01,2.70393372e-01,  2.49778748e-01,2.40398407e-01,2.49511719e-01,2.60101318e-01,  2.63496399e-01,2.59040833e-01,2.44037628e-01,2.38258362e-01,  2.43080139e-01,2.53684998e-01,2.59826660e-01,2.62985229e-01,  2.63671875e-01,2.62672424e-01,2.62062073e-01,2.62260437e-01,  2.63130188e-01,2.62008667e-01,2.56996155e-01,2.50694275e-01,  2.44972229e-01,2.40085602e-01,2.33825684e-01,2.28450775e-01,  2.23457336e-01,2.17811584e-01,2.16121674e-01],
        [2.20844269e-01,2.34714508e-01,2.54028320e-01,3.03459167e-01,  3.31809998e-01,3.46992493e-01,3.53942871e-01,3.61000061e-01,  4.01153564e-01,4.69749451e-01,6.34368896e-01,7.27966309e-01,  7.80471802e-01,8.26721191e-01,8.46572876e-01,8.36349487e-01,  8.37249756e-01,9.20654297e-01,1.01309204e+00,1.10571289e+00,  1.17834473e+00,1.21963501e+00,1.25308228e+00,1.26434326e+00,  1.28112793e+00,1.31338501e+00,1.33364868e+00,1.30099487e+00,  1.32263184e+00,1.35391235e+00,1.34054565e+00,1.35006714e+00,  1.35620117e+00,1.31411743e+00,1.32229614e+00,1.35372925e+00,  1.37374878e+00,1.38317871e+00,1.37661743e+00,1.38220215e+00,  1.38610840e+00,1.37384033e+00,1.35058594e+00,1.33578491e+00,  1.32086182e+00,1.31793213e+00,1.33847046e+00,1.34628296e+00,  1.32849121e+00,1.33056641e+00,1.32501221e+00,1.32803345e+00,  1.32327271e+00,1.32049561e+00,1.32498169e+00,1.29992676e+00,  1.29260254e+00,1.32556152e+00,1.34594727e+00,1.34460449e+00,  1.33331299e+00,1.30975342e+00,1.34652710e+00,1.37692261e+00,  1.37777710e+00,1.37231445e+00,1.34994507e+00,1.22265625e+00,  1.17584229e+00,1.26513672e+00,1.28784180e+00,1.29638672e+00,  1.29095459e+00,1.21697998e+00,1.15121460e+00,1.15121460e+00,  1.17764282e+00,1.21984863e+00,1.22726440e+00,1.23150635e+00,  1.21936035e+00,1.08639526e+00,6.96716309e-01,7.59887695e-01,  1.07055664e+00,1.15554810e+00,1.15203857e+00,1.14483643e+00,  1.13067627e+00,1.12017822e+00,1.11685181e+00,1.10632324e+00,  1.05920410e+00,9.81781006e-01,9.52056885e-01,9.69268799e-01,  9.85122681e-01,1.00271606e+00,1.01800537e+00,1.00964355e+00,  9.64843750e-01,9.54772949e-01,9.62860107e-01,9.37683105e-01,  9.54238892e-01,9.51553345e-01,9.52484131e-01,9.47753906e-01,  9.27001953e-01,8.57070923e-01,8.00262451e-01,7.87078857e-01,  7.68600464e-01,7.59338379e-01,7.78503418e-01,7.52578735e-01,  5.79376221e-01,4.31198120e-01,4.41497803e-01,4.53529358e-01,  4.67979431e-01,4.96902466e-01,5.43121338e-01,6.13388062e-01,  6.69387817e-01,6.85775757e-01,7.08312988e-01,7.38159180e-01,  7.52136230e-01,7.53021240e-01,7.43728638e-01,7.33276367e-01,  7.30972290e-01,7.28256226e-01,7.20916748e-01,7.13058472e-01,  7.03994751e-01,6.94244385e-01,6.85745239e-01,6.75796509e-01,  6.64535522e-01,6.50543213e-01,6.35437012e-01,6.22360229e-01,  6.16836548e-01,6.16210938e-01,6.09039307e-01,5.98449707e-01,  5.84671021e-01,5.62820435e-01,5.39215088e-01,5.11596680e-01,  4.49462891e-01,3.40950012e-01,2.50160217e-01,2.21797943e-01,  2.34085083e-01,2.63214111e-01,2.92526245e-01,2.79380798e-01,  2.94998169e-01,3.51669312e-01,4.11071777e-01,4.52995300e-01,  4.72656250e-01,4.73312378e-01,4.64927673e-01,4.61387634e-01,  4.71557617e-01,4.71870422e-01,4.56954956e-01,4.52713013e-01,  4.60182190e-01,4.68437195e-01,4.69223022e-01,4.69818115e-01,  4.73060608e-01,4.72251892e-01,4.65934753e-01,4.57527161e-01,  4.45190430e-01,4.21310425e-01,3.78364563e-01,3.35617065e-01,  3.43284607e-01,3.85292053e-01,4.05258179e-01,4.19250488e-01,  4.24957275e-01,4.19067383e-01,4.05059814e-01,3.87374878e-01,  3.66638184e-01,3.47358704e-01,3.34388733e-01,3.12072754e-01,  2.67318726e-01,2.55432129e-01,2.36175537e-01,1.59999847e-01,  7.04536438e-02,2.42657661e-02,1.36671066e-02,1.23748779e-02,  1.26228333e-02,1.48777962e-02,1.39358044e-02,1.50277615e-02,  1.83649063e-02,1.94001198e-02,2.25749016e-02,2.49977112e-02,  3.45792770e-02,4.70733643e-02,6.01015091e-02,7.45410919e-02,  7.60726929e-02,7.44209290e-02,8.39214325e-02,1.00337982e-01,  1.22121811e-01,1.56097412e-01,1.63154602e-01,1.39488220e-01,  1.27227783e-01,1.42005920e-01,1.58481598e-01,1.79393768e-01,  2.11524963e-01,2.38986969e-01,2.49469757e-01,2.57286072e-01,  2.70988464e-01,2.75764465e-01,2.79739380e-01,2.83866882e-01,  2.85629272e-01,2.85598755e-01,2.86277771e-01,2.88475037e-01,  2.87673950e-01,2.85034180e-01,2.81608582e-01,2.71293640e-01,  2.51075745e-01,2.41790771e-01,2.50793457e-01,2.60818481e-01,  2.64053345e-01,2.59605408e-01,2.45216370e-01,2.39509583e-01,  2.44293213e-01,2.54524231e-01,2.60414124e-01,2.63793945e-01,  2.64213562e-01,2.63542175e-01,2.62748718e-01,2.63450623e-01,  2.64190674e-01,2.63076782e-01,2.58316040e-01,2.51136780e-01,  2.46768951e-01,2.40871429e-01,2.35542297e-01,2.28939056e-01,  2.24960327e-01,2.19112396e-01,2.16259003e-01],
        [2.39543915e-01,2.51693726e-01,2.71644592e-01,3.26080322e-01,  3.55262756e-01,3.67828369e-01,3.75106812e-01,3.81835938e-01,  4.23171997e-01,4.93019104e-01,6.67419434e-01,7.66540527e-01,  8.19885254e-01,8.67172241e-01,8.85910034e-01,8.74649048e-01,  8.73062134e-01,9.58587646e-01,1.05389404e+00,1.14770508e+00,  1.22201538e+00,1.26266479e+00,1.29504395e+00,1.30621338e+00,  1.32214355e+00,1.35455322e+00,1.37429810e+00,1.33856201e+00,  1.35845947e+00,1.39001465e+00,1.37597656e+00,1.38571167e+00,  1.39096069e+00,1.34689331e+00,1.35372925e+00,1.38522339e+00,  1.40646362e+00,1.41604614e+00,1.40835571e+00,1.41384888e+00,  1.41690063e+00,1.40444946e+00,1.38012695e+00,1.36520386e+00,  1.35067749e+00,1.34777832e+00,1.36770630e+00,1.37420654e+00,  1.35482788e+00,1.35824585e+00,1.35275269e+00,1.35452271e+00,  1.34860229e+00,1.34375000e+00,1.34799194e+00,1.32284546e+00,  1.31372070e+00,1.34448242e+00,1.36404419e+00,1.36380005e+00,  1.35089111e+00,1.32492065e+00,1.36196899e+00,1.39291382e+00,  1.39190674e+00,1.38589478e+00,1.36297607e+00,1.23797607e+00,  1.18890381e+00,1.27536011e+00,1.29876709e+00,1.30755615e+00,  1.30255127e+00,1.23080444e+00,1.16464233e+00,1.16571045e+00,  1.19097900e+00,1.22921753e+00,1.23629761e+00,1.24050903e+00,  1.22769165e+00,1.10000610e+00,7.10556030e-01,7.65808105e-01,  1.07479858e+00,1.15960693e+00,1.15802002e+00,1.15197754e+00,  1.13711548e+00,1.12573242e+00,1.12005615e+00,1.10681152e+00,  1.06402588e+00,9.86968994e-01,9.57595825e-01,9.74792480e-01,  9.84024048e-01,1.00286865e+00,1.01913452e+00,1.00656128e+00,  9.66522217e-01,9.56771851e-01,9.66415405e-01,9.39285278e-01,  9.55841064e-01,9.59701538e-01,9.57534790e-01,9.65225220e-01,  9.40933228e-01,8.59756470e-01,8.04702759e-01,7.97988892e-01,  7.76718140e-01,7.73681641e-01,7.95883179e-01,7.66296387e-01,  5.90209961e-01,4.41505432e-01,4.51683044e-01,4.63493347e-01,  4.77989197e-01,5.06713867e-01,5.52261353e-01,6.21307373e-01,  6.75720215e-01,6.91375732e-01,7.13470459e-01,7.42126465e-01,  7.55401611e-01,7.55966187e-01,7.46200562e-01,7.35824585e-01,  7.33596802e-01,7.30453491e-01,7.23281860e-01,7.15484619e-01,  7.06085205e-01,6.96472168e-01,6.88140869e-01,6.78192139e-01,  6.67236328e-01,6.53350830e-01,6.38320923e-01,6.25350952e-01,  6.19735718e-01,6.18850708e-01,6.11724854e-01,6.01119995e-01,  5.87371826e-01,5.66284180e-01,5.43395996e-01,5.16296387e-01,  4.54940796e-01,3.47236633e-01,2.56973267e-01,2.29000092e-01,  2.41149902e-01,2.70011902e-01,2.99583435e-01,2.86369324e-01,  3.01849365e-01,3.58062744e-01,4.16511536e-01,4.57740784e-01,  4.76776123e-01,4.77005005e-01,4.68666077e-01,4.64942932e-01,  4.74906921e-01,4.75151062e-01,4.60311890e-01,4.55993652e-01,  4.63195801e-01,4.71160889e-01,4.71900940e-01,4.72145081e-01,  4.74975586e-01,4.74159241e-01,4.67720032e-01,4.59449768e-01,  4.47570801e-01,4.24148560e-01,3.82133484e-01,3.40232849e-01,  3.47557068e-01,3.88702393e-01,4.07592773e-01,4.21096802e-01,  4.26895142e-01,4.20906067e-01,4.07188416e-01,3.89846802e-01,  3.69285583e-01,3.50547791e-01,3.37707520e-01,3.15788269e-01,  2.71774292e-01,2.59719849e-01,2.40520477e-01,1.64123535e-01,  7.32936859e-02,2.53319740e-02,1.38072968e-02,1.25081539e-02,  1.28402710e-02,1.54371262e-02,1.44610405e-02,1.54812336e-02,  1.93743706e-02,2.05020905e-02,2.36635208e-02,2.65626907e-02,  3.64160538e-02,4.95223999e-02,6.30512238e-02,7.76290894e-02,  7.92732239e-02,7.76195526e-02,8.76255035e-02,1.04234695e-01,  1.25946045e-01,1.60282135e-01,1.67064667e-01,1.43386841e-01,  1.31080627e-01,1.45748138e-01,1.62258148e-01,1.82838440e-01,  2.14694977e-01,2.41580963e-01,2.51579285e-01,2.59086609e-01,  2.72544861e-01,2.77305603e-01,2.80776978e-01,2.84774780e-01,  2.86361694e-01,2.86277771e-01,2.86865234e-01,2.88841248e-01,  2.88101196e-01,2.85301208e-01,2.82012939e-01,2.71743774e-01,  2.51869202e-01,2.42801666e-01,2.51548767e-01,2.61367798e-01,  2.64137268e-01,2.60116577e-01,2.45971680e-01,2.40291595e-01,  2.44907379e-01,2.54997253e-01,2.60833740e-01,2.64091492e-01,  2.64633179e-01,2.63931274e-01,2.63237000e-01,2.63633728e-01,  2.64358521e-01,2.62863159e-01,2.58094788e-01,2.50289917e-01,  2.45601654e-01,2.40833282e-01,2.35649109e-01,2.29797363e-01,  2.24697113e-01,2.19074249e-01,2.16037750e-01],
        [2.57385254e-01,2.70011902e-01,2.86056519e-01,3.47007751e-01,  3.74000549e-01,3.87908936e-01,3.96652222e-01,3.99894714e-01,  4.44648743e-01,5.16555786e-01,6.98638916e-01,8.00827026e-01,  8.54492188e-01,9.03564453e-01,9.20684814e-01,9.07211304e-01,  9.05441284e-01,9.91439819e-01,1.08956909e+00,1.18368530e+00,  1.25857544e+00,1.29931641e+00,1.33065796e+00,1.34146118e+00,  1.35647583e+00,1.38906860e+00,1.40899658e+00,1.36987305e+00,  1.39013672e+00,1.42065430e+00,1.40512085e+00,1.41494751e+00,  1.41857910e+00,1.37310791e+00,1.37985229e+00,1.41137695e+00,  1.43365479e+00,1.44207764e+00,1.43453979e+00,1.44030762e+00,  1.44235229e+00,1.43002319e+00,1.40484619e+00,1.38973999e+00,  1.37625122e+00,1.37265015e+00,1.39273071e+00,1.39865112e+00,  1.37753296e+00,1.38085938e+00,1.37332153e+00,1.37466431e+00,  1.36889648e+00,1.36169434e+00,1.36584473e+00,1.34042358e+00,  1.33084106e+00,1.36267090e+00,1.37814331e+00,1.37686157e+00,  1.36593628e+00,1.33703613e+00,1.37387085e+00,1.40313721e+00,  1.40206909e+00,1.39813232e+00,1.37307739e+00,1.24981689e+00,  1.20199585e+00,1.28533936e+00,1.30947876e+00,1.31750488e+00,  1.31005859e+00,1.24172974e+00,1.17495728e+00,1.17349243e+00,  1.19958496e+00,1.23544312e+00,1.24322510e+00,1.24667358e+00,  1.23168945e+00,1.10449219e+00,7.15438843e-01,7.74368286e-01,  1.08377075e+00,1.16268921e+00,1.16421509e+00,1.15747070e+00,  1.13607788e+00,1.13305664e+00,1.12631226e+00,1.11026001e+00,  1.07159424e+00,9.90325928e-01,9.59991455e-01,9.82711792e-01,  9.89593506e-01,1.00802612e+00,1.02178955e+00,1.00469971e+00,  9.72900391e-01,9.58755493e-01,9.69650269e-01,9.55108643e-01,  9.56909180e-01,9.63088989e-01,9.59289551e-01,9.47631836e-01,  9.36279297e-01,8.68850708e-01,8.06915283e-01,8.11340332e-01,  7.81768799e-01,7.72521973e-01,7.87124634e-01,7.51419067e-01,  5.90042114e-01,4.48234558e-01,4.58740234e-01,4.70420837e-01,  4.84786987e-01,5.13061523e-01,5.57846069e-01,6.25747681e-01,  6.78985596e-01,6.94213867e-01,7.15713501e-01,7.43225098e-01,  7.55798340e-01,7.56103516e-01,7.46261597e-01,7.35870361e-01,  7.33825684e-01,7.30499268e-01,7.23373413e-01,7.15606689e-01,  7.06207275e-01,6.96624756e-01,6.88125610e-01,6.78314209e-01,  6.67556763e-01,6.53839111e-01,6.39114380e-01,6.26358032e-01,  6.20422363e-01,6.19415283e-01,6.12045288e-01,6.01516724e-01,  5.88027954e-01,5.67199707e-01,5.45043945e-01,5.18737793e-01,  4.58015442e-01,3.51379395e-01,2.61817932e-01,2.33917236e-01,  2.46112823e-01,2.74650574e-01,3.04283142e-01,2.91183472e-01,  3.06602478e-01,3.62174988e-01,4.19593811e-01,4.60235596e-01,  4.78797913e-01,4.78553772e-01,4.70336914e-01,4.66667175e-01,  4.76371765e-01,4.76409912e-01,4.61524963e-01,4.57466125e-01,  4.64324951e-01,4.72045898e-01,4.72755432e-01,4.72686768e-01,  4.75326538e-01,4.74411011e-01,4.68055725e-01,4.60037231e-01,  4.48257446e-01,4.25491333e-01,3.84475708e-01,3.43200684e-01,  3.50471497e-01,3.90365601e-01,4.08477783e-01,4.21531677e-01,  4.27070618e-01,4.21386719e-01,4.07844543e-01,3.90632629e-01,  3.70422363e-01,3.51829529e-01,3.39378357e-01,3.17741394e-01,  2.74436951e-01,2.62535095e-01,2.43251801e-01,1.66984558e-01,  7.53707886e-02,2.61054039e-02,1.38864517e-02,1.24363899e-02,  1.30400658e-02,1.59149170e-02,1.47333145e-02,1.59049034e-02,  2.01478004e-02,2.12421417e-02,2.46348381e-02,2.76637077e-02,  3.78875732e-02,5.13715744e-02,6.51454926e-02,8.00991058e-02,  8.16078186e-02,8.01544189e-02,9.03701782e-02,1.06847763e-01,  1.28582001e-01,1.62948608e-01,1.69784546e-01,1.45973206e-01,  1.33651733e-01,1.48319244e-01,1.64558411e-01,1.85043335e-01,  2.16350555e-01,2.42782593e-01,2.52433777e-01,2.59773254e-01,  2.72857666e-01,2.77458191e-01,2.80822754e-01,2.84591675e-01,  2.86186218e-01,2.85942078e-01,2.86506653e-01,2.88253784e-01,  2.87414551e-01,2.84629822e-01,2.81333923e-01,2.71415710e-01,  2.52052307e-01,2.42912292e-01,2.51296997e-01,2.60696411e-01,  2.63511658e-01,2.59681702e-01,2.45925903e-01,2.40356445e-01,  2.44728088e-01,2.54501343e-01,2.60108948e-01,2.63374329e-01,  2.63771057e-01,2.63084412e-01,2.62237549e-01,2.62901306e-01,  2.63328552e-01,2.61741638e-01,2.56607056e-01,2.49221802e-01,  2.43789673e-01,2.38574982e-01,2.32852936e-01,2.27416992e-01,  2.22270966e-01,2.17277527e-01,2.14710236e-01],
        [2.73986816e-01,2.85316467e-01,3.08532715e-01,3.67675781e-01,  3.98757935e-01,4.14390564e-01,4.18190002e-01,4.26170349e-01,  4.67681885e-01,5.44281006e-01,7.34741211e-01,8.36486816e-01,  8.93798828e-01,9.40917969e-01,9.57839966e-01,9.42977905e-01,  9.38735962e-01,1.02890015e+00,1.12805176e+00,1.22531128e+00,  1.30203247e+00,1.34017944e+00,1.37350464e+00,1.38165283e+00,  1.39566040e+00,1.42941284e+00,1.44674683e+00,1.40600586e+00,  1.42605591e+00,1.45642090e+00,1.44064331e+00,1.44921875e+00,  1.45260620e+00,1.40478516e+00,1.41104126e+00,1.44433594e+00,  1.46548462e+00,1.47329712e+00,1.46557617e+00,1.47000122e+00,  1.47235107e+00,1.45870972e+00,1.43399048e+00,1.41970825e+00,  1.40484619e+00,1.40280151e+00,1.42175293e+00,1.42553711e+00,  1.40563965e+00,1.40792847e+00,1.40042114e+00,1.40209961e+00,  1.39248657e+00,1.38763428e+00,1.39141846e+00,1.36422729e+00,  1.35562134e+00,1.38275146e+00,1.40002441e+00,1.39880371e+00,  1.38458252e+00,1.35739136e+00,1.39321899e+00,1.42105103e+00,  1.42181396e+00,1.41381836e+00,1.38787842e+00,1.26510620e+00,  1.21789551e+00,1.30401611e+00,1.32525635e+00,1.33062744e+00,  1.32427979e+00,1.25479126e+00,1.19281006e+00,1.19229126e+00,  1.21444702e+00,1.25161743e+00,1.25531006e+00,1.25640869e+00,  1.24630737e+00,1.10995483e+00,7.24288940e-01,7.94357300e-01,  1.09381104e+00,1.17398071e+00,1.17114258e+00,1.16180420e+00,  1.15283203e+00,1.14138794e+00,1.13442993e+00,1.12326050e+00,  1.07913208e+00,1.00488281e+00,9.76974487e-01,9.89257812e-01,  1.00857544e+00,1.02365112e+00,1.02859497e+00,1.02401733e+00,  9.72854614e-01,9.65606689e-01,9.86892700e-01,9.59564209e-01,  9.66842651e-01,9.70199585e-01,9.59320068e-01,9.67819214e-01,  9.42932129e-01,8.70758057e-01,8.28277588e-01,8.13415527e-01,  7.90328979e-01,7.89398193e-01,7.96417236e-01,7.73559570e-01,  6.07192993e-01,4.61906433e-01,4.72030640e-01,4.83879089e-01,  4.97856140e-01,5.25375366e-01,5.69869995e-01,6.35757446e-01,  6.87271118e-01,7.02072144e-01,7.22229004e-01,7.48641968e-01,  7.60345459e-01,7.60162354e-01,7.50488281e-01,7.39776611e-01,  7.37548828e-01,7.34466553e-01,7.27035522e-01,7.19268799e-01,  7.09732056e-01,6.99874878e-01,6.91909790e-01,6.81976318e-01,  6.71218872e-01,6.57821655e-01,6.43066406e-01,6.30340576e-01,  6.24298096e-01,6.23153687e-01,6.15936279e-01,6.05300903e-01,  5.92117310e-01,5.71914673e-01,5.50765991e-01,5.25314331e-01,  4.65393066e-01,3.60023499e-01,2.71095276e-01,2.43549347e-01,  2.55531311e-01,2.83668518e-01,3.13491821e-01,3.00537109e-01,  3.15757751e-01,3.70437622e-01,4.26589966e-01,4.66331482e-01,  4.84153748e-01,4.83741760e-01,4.75318909e-01,4.71557617e-01,  4.81040955e-01,4.80743408e-01,4.66102600e-01,4.61753845e-01,  4.68391418e-01,4.75936890e-01,4.76150513e-01,4.75898743e-01,  4.78271484e-01,4.77142334e-01,4.70863342e-01,4.62745667e-01,  4.51461792e-01,4.29130554e-01,3.88748169e-01,3.48213196e-01,  3.55064392e-01,3.94309998e-01,4.11582947e-01,4.24278259e-01,  4.29870605e-01,4.24209595e-01,4.11132812e-01,3.94226074e-01,  3.74427795e-01,3.56262207e-01,3.44009399e-01,3.23028564e-01,  2.80250549e-01,2.68455505e-01,2.49061584e-01,1.72740936e-01,  7.96794891e-02,2.81314850e-02,1.49176121e-02,1.32036209e-02,  1.40631199e-02,1.76076889e-02,1.61519051e-02,1.74956322e-02,  2.22525597e-02,2.33421326e-02,2.69293785e-02,3.02162170e-02,  4.10394669e-02,5.52310944e-02,6.95476532e-02,8.47969055e-02,  8.62350464e-02,8.49266052e-02,9.54875946e-02,1.12174988e-01,  1.34063721e-01,1.68575287e-01,1.75323486e-01,1.51416779e-01,  1.38957977e-01,1.53575897e-01,1.69822693e-01,1.89964294e-01,  2.20779419e-01,2.46585846e-01,2.55691528e-01,2.62611389e-01,  2.75428772e-01,2.79815674e-01,2.82905579e-01,2.86537170e-01,  2.88024902e-01,2.87757874e-01,2.88124084e-01,2.89794922e-01,  2.88742065e-01,2.85957336e-01,2.82592773e-01,2.72926331e-01,  2.53707886e-01,2.44838715e-01,2.53059387e-01,2.62268066e-01,  2.64892578e-01,2.61199951e-01,2.47760773e-01,2.42305756e-01,  2.46555328e-01,2.56286621e-01,2.61688232e-01,2.64900208e-01,  2.65419006e-01,2.64915466e-01,2.64526367e-01,2.65312195e-01,  2.66014099e-01,2.64343262e-01,2.59269714e-01,2.52456665e-01,  2.47524261e-01,2.42465973e-01,2.36953735e-01,2.31536865e-01,  2.26463318e-01,2.21843719e-01,2.18151093e-01],
        [2.90641785e-01,3.04100037e-01,3.23486328e-01,3.85818481e-01,  4.15298462e-01,4.31602478e-01,4.36546326e-01,4.42665100e-01,  4.88510132e-01,5.67382812e-01,7.63137817e-01,8.67675781e-01,  9.23461914e-01,9.72061157e-01,9.87136841e-01,9.68612671e-01,  9.64691162e-01,1.05618286e+00,1.15701294e+00,1.25622559e+00,  1.33267212e+00,1.37152100e+00,1.40298462e+00,1.41140747e+00,  1.42559814e+00,1.45889282e+00,1.47558594e+00,1.43176270e+00,  1.45227051e+00,1.48159790e+00,1.46435547e+00,1.47503662e+00,  1.47619629e+00,1.42642212e+00,1.43338013e+00,1.46539307e+00,  1.48788452e+00,1.49526978e+00,1.48614502e+00,1.49136353e+00,  1.49282837e+00,1.47869873e+00,1.45376587e+00,1.43981934e+00,  1.42587280e+00,1.42227173e+00,1.44134521e+00,1.44519043e+00,  1.42407227e+00,1.42742920e+00,1.41937256e+00,1.41888428e+00,  1.41009521e+00,1.40332031e+00,1.40759277e+00,1.38110352e+00,  1.37048340e+00,1.39913940e+00,1.41448975e+00,1.41210938e+00,  1.39871216e+00,1.36834717e+00,1.40377808e+00,1.43243408e+00,  1.43051147e+00,1.42352295e+00,1.39718628e+00,1.27359009e+00,  1.23138428e+00,1.31417847e+00,1.33407593e+00,1.34020996e+00,  1.33087158e+00,1.26211548e+00,1.20242310e+00,1.20269775e+00,  1.22671509e+00,1.25897217e+00,1.26193237e+00,1.26287842e+00,  1.24938965e+00,1.11489868e+00,7.30651855e-01,8.05908203e-01,  1.10369873e+00,1.17868042e+00,1.17837524e+00,1.17138672e+00,  1.15579224e+00,1.14633179e+00,1.13870239e+00,1.12704468e+00,  1.08889771e+00,1.01199341e+00,9.86633301e-01,1.00286865e+00,  1.01419067e+00,1.02880859e+00,1.03414917e+00,1.02313232e+00,  9.83459473e-01,9.68551636e-01,9.89303589e-01,9.64111328e-01,  9.68002319e-01,9.65988159e-01,9.67880249e-01,9.67285156e-01,  9.54055786e-01,8.86077881e-01,8.25042725e-01,8.29696655e-01,  8.02917480e-01,7.96661377e-01,8.18389893e-01,7.93823242e-01,  6.18072510e-01,4.70367432e-01,4.80796814e-01,4.92164612e-01,  5.05935669e-01,5.33172607e-01,5.76660156e-01,6.40960693e-01,  6.90841675e-01,7.04772949e-01,7.24380493e-01,7.49618530e-01,  7.60452271e-01,7.59918213e-01,7.50000000e-01,7.39471436e-01,  7.37106323e-01,7.33963013e-01,7.26684570e-01,7.18704224e-01,  7.09243774e-01,6.99569702e-01,6.91268921e-01,6.81610107e-01,  6.71005249e-01,6.57791138e-01,6.43234253e-01,6.30462646e-01,  6.24481201e-01,6.23092651e-01,6.15707397e-01,6.05392456e-01,  5.92346191e-01,5.72814941e-01,5.52413940e-01,5.27664185e-01,  4.68925476e-01,3.64746094e-01,2.77015686e-01,2.49877930e-01,  2.61604309e-01,2.89436340e-01,3.19137573e-01,3.06587219e-01,  3.21479797e-01,3.75106812e-01,4.30114746e-01,4.68704224e-01,  4.86122131e-01,4.85473633e-01,4.76928711e-01,4.73281860e-01,  4.82406616e-01,4.81933594e-01,4.67407227e-01,4.63058472e-01,  4.69512939e-01,4.76684570e-01,4.76737976e-01,4.76242065e-01,  4.78309631e-01,4.76997375e-01,4.70558167e-01,4.62760925e-01,  4.51683044e-01,4.29817200e-01,3.90327454e-01,3.50517273e-01,  3.57284546e-01,3.95484924e-01,4.11956787e-01,4.24293518e-01,  4.29771423e-01,4.24377441e-01,4.11529541e-01,3.95011902e-01,  3.75587463e-01,3.57772827e-01,3.45970154e-01,3.25370789e-01,  2.83561707e-01,2.71743774e-01,2.52235413e-01,1.76113129e-01,  8.23211670e-02,2.90665627e-02,1.48675442e-02,1.31349564e-02,  1.41134262e-02,1.82886124e-02,1.65047646e-02,1.79152489e-02,  2.33087540e-02,2.43477821e-02,2.82254219e-02,3.16581726e-02,  4.28600311e-02,5.74512482e-02,7.22103119e-02,8.77227783e-02,  8.90350342e-02,8.79821777e-02,9.88311768e-02,1.15688324e-01,  1.37382507e-01,1.72000885e-01,1.78535461e-01,1.54624939e-01,  1.42219543e-01,1.56635284e-01,1.72737122e-01,1.92565918e-01,  2.22774506e-01,2.47859955e-01,2.56553650e-01,2.63137817e-01,  2.75756836e-01,2.79922485e-01,2.82730103e-01,2.86056519e-01,  2.87498474e-01,2.87162781e-01,2.87452698e-01,2.88887024e-01,  2.87895203e-01,2.85018921e-01,2.81631470e-01,2.72125244e-01,  2.53463745e-01,2.44606018e-01,2.52532959e-01,2.61329651e-01,  2.63763428e-01,2.60185242e-01,2.47138977e-01,2.41725922e-01,  2.45830536e-01,2.55065918e-01,2.60482788e-01,2.63710022e-01,  2.64419556e-01,2.63656616e-01,2.63145447e-01,2.63389587e-01,  2.63801575e-01,2.61840820e-01,2.56904602e-01,2.49050140e-01,  2.43911743e-01,2.38231659e-01,2.32498169e-01,2.26844788e-01,  2.22679138e-01,2.16419220e-01,2.13539124e-01],
        [3.02352905e-01,3.16795349e-01,3.37669373e-01,4.05754089e-01,  4.34455872e-01,4.49211121e-01,4.55642700e-01,4.58374023e-01,  5.07019043e-01,5.88729858e-01,7.88742065e-01,8.95874023e-01,  9.50576782e-01,9.99542236e-01,1.01425171e+00,9.94491577e-01,  9.91317749e-01,1.08227539e+00,1.18511963e+00,1.28530884e+00,  1.36184692e+00,1.40191650e+00,1.43249512e+00,1.43954468e+00,  1.45382690e+00,1.48663330e+00,1.50363159e+00,1.45864868e+00,  1.47961426e+00,1.50918579e+00,1.48965454e+00,1.50021362e+00,  1.50015259e+00,1.44885254e+00,1.45651245e+00,1.48910522e+00,  1.51257324e+00,1.51876831e+00,1.50799561e+00,1.51373291e+00,  1.51428223e+00,1.50024414e+00,1.47549438e+00,1.46020508e+00,  1.44866943e+00,1.44528198e+00,1.46322632e+00,1.46636963e+00,  1.44439697e+00,1.44805908e+00,1.44039917e+00,1.43945312e+00,  1.43093872e+00,1.42166138e+00,1.42514038e+00,1.39938354e+00,  1.38839722e+00,1.41671753e+00,1.43075562e+00,1.42703247e+00,  1.41418457e+00,1.38235474e+00,1.41931152e+00,1.44680786e+00,  1.44326782e+00,1.43731689e+00,1.40899658e+00,1.28533936e+00,  1.24642944e+00,1.32601929e+00,1.34622192e+00,1.35260010e+00,  1.34185791e+00,1.27682495e+00,1.21539307e+00,1.21557617e+00,  1.23992920e+00,1.26815796e+00,1.27264404e+00,1.27667236e+00,  1.25970459e+00,1.12445068e+00,7.38327026e-01,8.15216064e-01,  1.11520386e+00,1.18518066e+00,1.18762207e+00,1.18438721e+00,  1.16058350e+00,1.15390015e+00,1.14926147e+00,1.13342285e+00,  1.09930420e+00,1.01983643e+00,9.91210938e-01,1.01229858e+00,  1.01550293e+00,1.03411865e+00,1.04650879e+00,1.02355957e+00,  9.88204956e-01,9.73587036e-01,9.85946655e-01,9.73663330e-01,  9.73571777e-01,9.78561401e-01,9.75082397e-01,9.64920044e-01,  9.56329346e-01,8.87176514e-01,8.31420898e-01,8.45092773e-01,  8.12515259e-01,7.91839600e-01,8.23669434e-01,7.86193848e-01,  6.19857788e-01,4.79171753e-01,4.89646912e-01,5.00823975e-01,  5.14541626e-01,5.41244507e-01,5.84106445e-01,6.47460938e-01,  6.96380615e-01,7.09991455e-01,7.28836060e-01,7.53051758e-01,  7.63656616e-01,7.62817383e-01,7.52822876e-01,7.42248535e-01,  7.39715576e-01,7.36495972e-01,7.29217529e-01,7.21359253e-01,  7.11807251e-01,7.02087402e-01,6.93984985e-01,6.84204102e-01,  6.73645020e-01,6.60522461e-01,6.46118164e-01,6.33392334e-01,  6.27288818e-01,6.25839233e-01,6.18469238e-01,6.08200073e-01,  5.95230103e-01,5.75973511e-01,5.56243896e-01,5.32089233e-01,  4.73655701e-01,3.70269775e-01,2.82951355e-01,2.55996704e-01,  2.67791748e-01,2.95272827e-01,3.24974060e-01,3.12583923e-01,  3.27400208e-01,3.80607605e-01,4.34806824e-01,4.72839355e-01,  4.89830017e-01,4.88967896e-01,4.80415344e-01,4.76661682e-01,  4.85641479e-01,4.84992981e-01,4.70565796e-01,4.66156006e-01,  4.72236633e-01,4.79270935e-01,4.79278564e-01,4.78630066e-01,  4.80415344e-01,4.79019165e-01,4.72618103e-01,4.64874268e-01,  4.54017639e-01,4.32586670e-01,3.93806458e-01,3.54469299e-01,  3.61114502e-01,3.98559570e-01,4.14329529e-01,4.26307678e-01,  4.31762695e-01,4.26483154e-01,4.13726807e-01,3.97529602e-01,  3.78295898e-01,3.60687256e-01,3.49029541e-01,3.28704834e-01,  2.87498474e-01,2.75680542e-01,2.56141663e-01,1.79950714e-01,  8.53652954e-02,3.07149887e-02,1.58257484e-02,1.39753819e-02,  1.51171684e-02,1.97634697e-02,1.77359581e-02,1.92852020e-02,  2.50430107e-02,2.61058807e-02,3.00397873e-02,3.37018967e-02,  4.51498032e-02,6.01797104e-02,7.53345490e-02,9.10129547e-02,  9.23538208e-02,9.13524628e-02,1.02443695e-01,1.19327545e-01,  1.41094208e-01,1.75708771e-01,1.82323456e-01,1.58252716e-01,  1.45812988e-01,1.60068512e-01,1.76120758e-01,1.95869446e-01,  2.25803375e-01,2.50495911e-01,2.58872986e-01,2.65258789e-01,  2.77725220e-01,2.81661987e-01,2.84339905e-01,2.87666321e-01,  2.89001465e-01,2.88520813e-01,2.88688660e-01,2.90229797e-01,  2.89085388e-01,2.86132812e-01,2.82844543e-01,2.73483276e-01,  2.55126953e-01,2.46341705e-01,2.54188538e-01,2.62733459e-01,  2.65243530e-01,2.61642456e-01,2.48992920e-01,2.43556976e-01,  2.47749329e-01,2.56683350e-01,2.62039185e-01,2.65304565e-01,  2.66067505e-01,2.65586853e-01,2.65296936e-01,2.65792847e-01,  2.66616821e-01,2.64701843e-01,2.60154724e-01,2.53158569e-01,  2.48680115e-01,2.43476868e-01,2.37915039e-01,2.32814789e-01,  2.27806091e-01,2.23232269e-01,2.19913483e-01],
        [3.17344666e-01,3.29391479e-01,3.51829529e-01,4.20059204e-01,  4.49363708e-01,4.65911865e-01,4.68284607e-01,4.73480225e-01,  5.20477295e-01,6.04003906e-01,8.10714722e-01,9.19296265e-01,  9.74838257e-01,1.02407837e+00,1.03805542e+00,1.01766968e+00,  1.01217651e+00,1.10504150e+00,1.20993042e+00,1.31057739e+00,  1.38772583e+00,1.42654419e+00,1.45739746e+00,1.46459961e+00,  1.47717285e+00,1.51104736e+00,1.52682495e+00,1.47903442e+00,  1.49978638e+00,1.52926636e+00,1.51016235e+00,1.52017212e+00,  1.51971436e+00,1.46762085e+00,1.47439575e+00,1.50744629e+00,  1.53009033e+00,1.53631592e+00,1.52648926e+00,1.53134155e+00,  1.53286743e+00,1.51864624e+00,1.49255371e+00,1.47848511e+00,  1.46627808e+00,1.46365356e+00,1.48168945e+00,1.48324585e+00,  1.46115112e+00,1.46441650e+00,1.45547485e+00,1.45541382e+00,  1.44448853e+00,1.43560791e+00,1.43984985e+00,1.41247559e+00,  1.40246582e+00,1.42977905e+00,1.44192505e+00,1.43920898e+00,  1.42465210e+00,1.39355469e+00,1.42941284e+00,1.45608521e+00,  1.45333862e+00,1.44613647e+00,1.41876221e+00,1.29632568e+00,  1.25482178e+00,1.33584595e+00,1.35504150e+00,1.35882568e+00,  1.34902954e+00,1.28228760e+00,1.22149658e+00,1.22372437e+00,  1.24584961e+00,1.27487183e+00,1.27685547e+00,1.27575684e+00,  1.26239014e+00,1.12915039e+00,7.45407104e-01,8.23043823e-01,  1.11703491e+00,1.18884277e+00,1.18844604e+00,1.18029785e+00,  1.16586304e+00,1.16012573e+00,1.15286255e+00,1.14181519e+00,  1.10131836e+00,1.02264404e+00,9.98672485e-01,1.00994873e+00,  1.02154541e+00,1.04183960e+00,1.04077148e+00,1.02453613e+00,  9.87197876e-01,9.75128174e-01,9.92385864e-01,9.66934204e-01,  9.71328735e-01,9.82650757e-01,9.76852417e-01,9.77401733e-01,  9.61486816e-01,8.82537842e-01,8.38088989e-01,8.32153320e-01,  7.99468994e-01,8.04855347e-01,8.20755005e-01,7.88543701e-01,  6.24572754e-01,4.84092712e-01,4.94590759e-01,5.05737305e-01,  5.19515991e-01,5.46005249e-01,5.88699341e-01,6.51535034e-01,  6.99874878e-01,7.13394165e-01,7.31765747e-01,7.55752563e-01,  7.65914917e-01,7.65029907e-01,7.55004883e-01,7.44201660e-01,  7.42034912e-01,7.38647461e-01,7.31246948e-01,7.23495483e-01,  7.13775635e-01,7.04116821e-01,6.95831299e-01,6.86203003e-01,  6.75781250e-01,6.62750244e-01,6.48574829e-01,6.35833740e-01,  6.29730225e-01,6.27899170e-01,6.20483398e-01,6.10229492e-01,  5.97183228e-01,5.78186035e-01,5.58517456e-01,5.34729004e-01,  4.76600647e-01,3.73466492e-01,2.86376953e-01,2.59452820e-01,  2.71347046e-01,2.98599243e-01,3.28491211e-01,3.16131592e-01,  3.30879211e-01,3.83842468e-01,4.37629700e-01,4.75524902e-01,  4.92195129e-01,4.91127014e-01,4.82429504e-01,4.78866577e-01,  4.87663269e-01,4.86953735e-01,4.72656250e-01,4.68223572e-01,  4.74296570e-01,4.81124878e-01,4.81033325e-01,4.80201721e-01,  4.81948853e-01,4.80491638e-01,4.74212646e-01,4.66629028e-01,  4.55825806e-01,4.34852600e-01,3.96614075e-01,3.57849121e-01,  3.64219666e-01,4.00924683e-01,4.16076660e-01,4.27749634e-01,  4.33151245e-01,4.27841187e-01,4.15252686e-01,3.99002075e-01,  3.80012512e-01,3.62541199e-01,3.50967407e-01,3.30680847e-01,  2.89649963e-01,2.77900696e-01,2.58361816e-01,1.82079315e-01,  8.70342255e-02,3.15704346e-02,1.62582397e-02,1.43861771e-02,  1.55866146e-02,2.04381943e-02,1.82847977e-02,1.99084282e-02,  2.58708000e-02,2.68497467e-02,3.08618546e-02,3.47023010e-02,  4.64849472e-02,6.18333817e-02,7.69119263e-02,9.27352905e-02,  9.40570831e-02,9.32197571e-02,1.04433060e-01,1.21320724e-01,  1.43009186e-01,1.77768707e-01,1.84242249e-01,1.60156250e-01,  1.47636414e-01,1.62002563e-01,1.78020477e-01,1.97628021e-01,  2.27310181e-01,2.51861572e-01,2.59979248e-01,2.66357422e-01,  2.78633118e-01,2.82569885e-01,2.85087585e-01,2.88391113e-01,  2.89718628e-01,2.89207458e-01,2.89390564e-01,2.90832520e-01,  2.89566040e-01,2.86621094e-01,2.83332825e-01,2.74215698e-01,  2.56118774e-01,2.47222900e-01,2.54875183e-01,2.63259888e-01,  2.65495300e-01,2.62191772e-01,2.49767303e-01,2.44655609e-01,  2.48611450e-01,2.57385254e-01,2.62451172e-01,2.65769958e-01,  2.66510010e-01,2.66197205e-01,2.66036987e-01,2.66517639e-01,  2.66914368e-01,2.65342712e-01,2.60581970e-01,2.53791809e-01,  2.48840332e-01,2.44045258e-01,2.38388062e-01,2.33257294e-01,  2.28961945e-01,2.23518372e-01,2.21473694e-01],
        [3.29055786e-01,3.42407227e-01,3.65226746e-01,4.32296753e-01,  4.66270447e-01,4.79927063e-01,4.82582092e-01,4.86488342e-01,  5.33630371e-01,6.17828369e-01,8.27102661e-01,9.39056396e-01,  9.96643066e-01,1.04583740e+00,1.05966187e+00,1.03692627e+00,  1.02990723e+00,1.12411499e+00,1.22793579e+00,1.32952881e+00,  1.40805054e+00,1.44616699e+00,1.47769165e+00,1.48370361e+00,  1.49633789e+00,1.52975464e+00,1.54434204e+00,1.49636841e+00,  1.51501465e+00,1.54446411e+00,1.52554321e+00,1.53402710e+00,  1.53421021e+00,1.48074341e+00,1.48672485e+00,1.52062988e+00,  1.54354858e+00,1.55075073e+00,1.54037476e+00,1.54467773e+00,  1.54644775e+00,1.53201294e+00,1.50588989e+00,1.49224854e+00,  1.47976685e+00,1.47760010e+00,1.49398804e+00,1.49499512e+00,  1.47467041e+00,1.47625732e+00,1.46694946e+00,1.46536255e+00,  1.45462036e+00,1.44726562e+00,1.44967651e+00,1.42279053e+00,  1.41232300e+00,1.43560791e+00,1.44900513e+00,1.44577026e+00,  1.43081665e+00,1.39877319e+00,1.43322754e+00,1.46096802e+00,  1.45758057e+00,1.45022583e+00,1.42453003e+00,1.30273438e+00,  1.25869751e+00,1.33920288e+00,1.35815430e+00,1.36267090e+00,  1.35211182e+00,1.28707886e+00,1.22808838e+00,1.22753906e+00,  1.24911499e+00,1.27920532e+00,1.27957153e+00,1.28063965e+00,  1.26544189e+00,1.13348389e+00,7.54806519e-01,8.25225830e-01,  1.11785889e+00,1.19161987e+00,1.18960571e+00,1.18392944e+00,  1.16711426e+00,1.15689087e+00,1.15115356e+00,1.13861084e+00,  1.10217285e+00,1.03192139e+00,9.98840332e-01,1.01513672e+00,  1.02661133e+00,1.03378296e+00,1.04391479e+00,1.03131104e+00,  9.87960815e-01,9.75097656e-01,9.91149902e-01,9.69772339e-01,  9.79034424e-01,9.79797363e-01,9.73144531e-01,9.77096558e-01,  9.56344604e-01,8.89450073e-01,8.45306396e-01,8.44680786e-01,  8.09677124e-01,8.04779053e-01,8.26797485e-01,7.99362183e-01,  6.31439209e-01,4.89791870e-01,5.00289917e-01,5.11428833e-01,  5.24780273e-01,5.51010132e-01,5.93307495e-01,6.55120850e-01,  7.02499390e-01,7.15255737e-01,7.33306885e-01,7.56698608e-01,  7.66326904e-01,7.65335083e-01,7.55157471e-01,7.44430542e-01,  7.42019653e-01,7.38616943e-01,7.31292725e-01,7.23327637e-01,  7.13851929e-01,7.03994751e-01,6.95846558e-01,6.86477661e-01,  6.76025391e-01,6.63192749e-01,6.49108887e-01,6.36688232e-01,  6.30187988e-01,6.28341675e-01,6.20880127e-01,6.10427856e-01,  5.97824097e-01,5.79132080e-01,5.60119629e-01,5.36743164e-01,  4.79171753e-01,3.76853943e-01,2.90237427e-01,2.63710022e-01,  2.75306702e-01,3.02452087e-01,3.32244873e-01,3.19961548e-01,  3.34625244e-01,3.87023926e-01,4.40124512e-01,4.77233887e-01,  4.93766785e-01,4.92492676e-01,4.83955383e-01,4.80308533e-01,  4.88868713e-01,4.88037109e-01,4.73663330e-01,4.69284058e-01,  4.75227356e-01,4.82070923e-01,4.81697083e-01,4.80766296e-01,  4.82269287e-01,4.80690002e-01,4.74456787e-01,4.66865540e-01,  4.56413269e-01,4.35737610e-01,3.98262024e-01,3.59916687e-01,  3.66310120e-01,4.02206421e-01,4.16702271e-01,4.28108215e-01,  4.33326721e-01,4.28153992e-01,4.15733337e-01,3.99818420e-01,  3.80874634e-01,3.63845825e-01,3.52333069e-01,3.32405090e-01,  2.91938782e-01,2.80166626e-01,2.60589600e-01,1.84532166e-01,  8.89282227e-02,3.25365067e-02,1.67164803e-02,1.46138668e-02,  1.60384178e-02,2.12645531e-02,1.90711021e-02,2.06708908e-02,  2.70748138e-02,2.78639793e-02,3.21645737e-02,3.60393524e-02,  4.80995178e-02,6.36062622e-02,7.90786743e-02,9.50088501e-02,  9.62829590e-02,9.54704285e-02,1.06843948e-01,1.23725891e-01,  1.45450592e-01,1.80084229e-01,1.86454773e-01,1.62479401e-01,  1.50032043e-01,1.64222717e-01,1.80099487e-01,1.99489594e-01,  2.28935242e-01,2.53036499e-01,2.60963440e-01,2.67082214e-01,  2.79251099e-01,2.82997131e-01,2.85339355e-01,2.88459778e-01,  2.89695740e-01,2.89222717e-01,2.89459229e-01,2.90718079e-01,  2.89489746e-01,2.86506653e-01,2.83187866e-01,2.74108887e-01,  2.56317139e-01,2.47585297e-01,2.55065918e-01,2.63175964e-01,  2.65396118e-01,2.62077332e-01,2.49969482e-01,2.44865417e-01,  2.48657227e-01,2.57255554e-01,2.62344360e-01,2.65586853e-01,  2.66448975e-01,2.66174316e-01,2.65922546e-01,2.66502380e-01,  2.66693115e-01,2.65090942e-01,2.60574341e-01,2.53875732e-01,  2.49118805e-01,2.43980408e-01,2.38948822e-01,2.32795715e-01,  2.29148865e-01,2.23545074e-01,2.20050812e-01],
        [3.43513489e-01,3.54103088e-01,3.79936218e-01,4.46372986e-01,  4.79270935e-01,4.95193481e-01,4.95140076e-01,4.98992920e-01,  5.46310425e-01,6.30676270e-01,8.46618652e-01,9.60281372e-01,  1.01910400e+00,1.06869507e+00,1.08154297e+00,1.05941772e+00,  1.04898071e+00,1.14385986e+00,1.24926758e+00,1.35144043e+00,  1.43124390e+00,1.46911621e+00,1.50006104e+00,1.50479126e+00,  1.51583862e+00,1.55035400e+00,1.56506348e+00,1.51593018e+00,  1.53302002e+00,1.56265259e+00,1.54504395e+00,1.55242920e+00,  1.55227661e+00,1.49829102e+00,1.50253296e+00,1.53799438e+00,  1.56066895e+00,1.56750488e+00,1.55795288e+00,1.56121826e+00,  1.56289673e+00,1.54794312e+00,1.52151489e+00,1.50964355e+00,  1.49658203e+00,1.49331665e+00,1.50872803e+00,1.50881958e+00,  1.48947144e+00,1.49026489e+00,1.48043823e+00,1.48046875e+00,  1.46746826e+00,1.45901489e+00,1.46136475e+00,1.43344116e+00,  1.42376709e+00,1.44677734e+00,1.45892334e+00,1.45639038e+00,  1.43917847e+00,1.40872192e+00,1.44171143e+00,1.46771240e+00,  1.46563721e+00,1.45632935e+00,1.43099976e+00,1.31158447e+00,  1.26464844e+00,1.34594727e+00,1.36318970e+00,1.36584473e+00,  1.36001587e+00,1.29302979e+00,1.23318481e+00,1.23507690e+00,  1.25170898e+00,1.28305054e+00,1.28308105e+00,1.28158569e+00,  1.26956177e+00,1.13867188e+00,7.62191772e-01,8.29483032e-01,  1.11688232e+00,1.19558716e+00,1.19409180e+00,1.18289185e+00,  1.17300415e+00,1.15893555e+00,1.15463257e+00,1.14428711e+00,  1.10235596e+00,1.03195190e+00,1.00518799e+00,1.01361084e+00,  1.03067017e+00,1.04049683e+00,1.04309082e+00,1.03512573e+00,  9.92675781e-01,9.78881836e-01,9.96215820e-01,9.62219238e-01,  9.79537964e-01,9.83047485e-01,9.65774536e-01,9.83245850e-01,  9.53536987e-01,8.86032104e-01,8.58551025e-01,8.36242676e-01,  8.18603516e-01,8.21990967e-01,8.11904907e-01,8.00369263e-01,  6.39266968e-01,4.97200012e-01,5.07522583e-01,5.18463135e-01,  5.31784058e-01,5.57876587e-01,5.99487305e-01,6.60385132e-01,  7.06954956e-01,7.19467163e-01,7.36999512e-01,7.59704590e-01,  7.68783569e-01,7.67517090e-01,7.57247925e-01,7.46505737e-01,  7.44186401e-01,7.40859985e-01,7.33398438e-01,7.25189209e-01,  7.15728760e-01,7.05993652e-01,6.97845459e-01,6.88461304e-01,  6.78161621e-01,6.65267944e-01,6.51107788e-01,6.38641357e-01,  6.32339478e-01,6.30477905e-01,6.22863770e-01,6.12594604e-01,  5.99884033e-01,5.81497192e-01,5.63064575e-01,5.40130615e-01,  4.83139038e-01,3.81408691e-01,2.95272827e-01,2.68836975e-01,  2.80471802e-01,3.07357788e-01,3.37043762e-01,3.25057983e-01,  3.39469910e-01,3.91510010e-01,4.43824768e-01,4.80567932e-01,  4.96734619e-01,4.95307922e-01,4.86480713e-01,4.82704163e-01,  4.91264343e-01,4.90264893e-01,4.76196289e-01,4.71717834e-01,  4.77478027e-01,4.83970642e-01,4.83718872e-01,4.82574463e-01,  4.83825684e-01,4.82238770e-01,4.75944519e-01,4.68574524e-01,  4.58183289e-01,4.37904358e-01,4.00856018e-01,3.63113403e-01,  3.69163513e-01,4.04548645e-01,4.18548584e-01,4.29679871e-01,  4.34928894e-01,4.29870605e-01,4.17564392e-01,4.01802063e-01,  3.83003235e-01,3.66058350e-01,3.54965210e-01,3.35220337e-01,  2.95196533e-01,2.83462524e-01,2.63832092e-01,1.87725067e-01,  9.15775299e-02,3.39040756e-02,1.74202919e-02,1.51555538e-02,  1.67441368e-02,2.24208832e-02,1.99818611e-02,2.16836929e-02,  2.85654068e-02,2.93178558e-02,3.36427689e-02,3.77445221e-02,  5.01375198e-02,6.58550262e-02,8.15620422e-02,9.77916718e-02,  9.91039276e-02,9.83924866e-02,1.10008240e-01,1.26842499e-01,  1.48445129e-01,1.83109283e-01,1.89495087e-01,1.65367126e-01,  1.52969360e-01,1.67095184e-01,1.82922363e-01,2.02190399e-01,  2.31315613e-01,2.55104065e-01,2.62634277e-01,2.68577576e-01,  2.80601501e-01,2.84294128e-01,2.86590576e-01,2.89627075e-01,  2.90878296e-01,2.90328979e-01,2.90336609e-01,2.91549683e-01,  2.90092468e-01,2.87208557e-01,2.83927917e-01,2.75047302e-01,  2.57484436e-01,2.48908997e-01,2.56179810e-01,2.64144897e-01,  2.66166687e-01,2.63038635e-01,2.51167297e-01,2.46109009e-01,  2.49950409e-01,2.58369446e-01,2.63427734e-01,2.66738892e-01,  2.67478943e-01,2.67272949e-01,2.67280579e-01,2.67768860e-01,  2.68478394e-01,2.66967773e-01,2.62527466e-01,2.56416321e-01,  2.51533508e-01,2.46749878e-01,2.41844177e-01,2.36049652e-01,  2.31407166e-01,2.26314545e-01,2.24006653e-01],
        [3.56704712e-01,3.69033813e-01,3.91670227e-01,4.61540222e-01,  4.93659973e-01,5.07751465e-01,5.11703491e-01,5.12329102e-01,  5.62057495e-01,6.46881104e-01,8.65768433e-01,9.83932495e-01,  1.04351807e+00,1.09405518e+00,1.10617065e+00,1.08270264e+00,  1.07125854e+00,1.16632080e+00,1.27368164e+00,1.37683105e+00,  1.45559692e+00,1.49435425e+00,1.52395630e+00,1.52899170e+00,  1.53973389e+00,1.57241821e+00,1.58786011e+00,1.53738403e+00,  1.55279541e+00,1.58297729e+00,1.56344604e+00,1.57107544e+00,  1.57083130e+00,1.51605225e+00,1.51989746e+00,1.55453491e+00,  1.57797241e+00,1.58493042e+00,1.57440186e+00,1.57879639e+00,  1.57977295e+00,1.56494141e+00,1.53845215e+00,1.52502441e+00,  1.51272583e+00,1.50973511e+00,1.52551270e+00,1.52551270e+00,  1.50369263e+00,1.50585938e+00,1.49478149e+00,1.49261475e+00,  1.47985840e+00,1.46899414e+00,1.47152710e+00,1.44464111e+00,  1.43237305e+00,1.45623779e+00,1.46820068e+00,1.46432495e+00,  1.44851685e+00,1.41467285e+00,1.44784546e+00,1.47613525e+00,  1.47265625e+00,1.46356201e+00,1.43682861e+00,1.31851196e+00,  1.27343750e+00,1.35076904e+00,1.36831665e+00,1.37231445e+00,  1.36404419e+00,1.30014038e+00,1.24090576e+00,1.24005127e+00,  1.25961304e+00,1.28823853e+00,1.28823853e+00,1.28677368e+00,  1.27172852e+00,1.14599609e+00,7.70080566e-01,8.31192017e-01,  1.12197876e+00,1.19732666e+00,1.19708252e+00,1.18875122e+00,  1.17340088e+00,1.16201782e+00,1.15652466e+00,1.14382935e+00,  1.10433960e+00,1.03512573e+00,1.00558472e+00,1.01956177e+00,  1.02987671e+00,1.04226685e+00,1.04840088e+00,1.03271484e+00,  9.89288330e-01,9.75418091e-01,9.93698120e-01,9.67803955e-01,  9.70321655e-01,9.79812622e-01,9.77600098e-01,9.76211548e-01,  9.58312988e-01,8.96011353e-01,8.48724365e-01,8.41888428e-01,  8.14285278e-01,8.24172974e-01,8.37203979e-01,8.20343018e-01,  6.48330688e-01,5.04272461e-01,5.14617920e-01,5.25497437e-01,  5.38528442e-01,5.64208984e-01,6.05285645e-01,6.64962769e-01,  7.10418701e-01,7.22305298e-01,7.39425659e-01,7.60986328e-01,  7.69668579e-01,7.68020630e-01,7.57858276e-01,7.47222900e-01,  7.44705200e-01,7.41256714e-01,7.33749390e-01,7.25784302e-01,  7.16156006e-01,7.06436157e-01,6.98226929e-01,6.88705444e-01,  6.78665161e-01,6.65985107e-01,6.52130127e-01,6.39419556e-01,  6.33010864e-01,6.30996704e-01,6.23626709e-01,6.13327026e-01,  6.00875854e-01,5.82824707e-01,5.64941406e-01,5.42572021e-01,  4.86267090e-01,3.85696411e-01,3.00102234e-01,2.74002075e-01,  2.85530090e-01,3.12057495e-01,3.41766357e-01,3.29917908e-01,  3.44184875e-01,3.95591736e-01,4.47082520e-01,4.82994080e-01,  4.98741150e-01,4.97138977e-01,4.88395691e-01,4.84596252e-01,  4.92904663e-01,4.91653442e-01,4.77584839e-01,4.73114014e-01,  4.78782654e-01,4.85069275e-01,4.84603882e-01,4.83283997e-01,  4.84420776e-01,4.82749939e-01,4.76409912e-01,4.69062805e-01,  4.58984375e-01,4.38911438e-01,4.02427673e-01,3.65142822e-01,  3.71162415e-01,4.05822754e-01,4.19425964e-01,4.30320740e-01,  4.35516357e-01,4.30465698e-01,4.18357849e-01,4.02862549e-01,  3.84323120e-01,3.67691040e-01,3.56697083e-01,3.37417603e-01,  2.97966003e-01,2.86376953e-01,2.66647339e-01,1.90910339e-01,  9.41295624e-02,3.51543427e-02,1.77373886e-02,1.53882504e-02,  1.71437263e-02,2.34684944e-02,2.06604004e-02,2.24804878e-02,  2.98643112e-02,3.06358337e-02,3.50255966e-02,3.93896103e-02,  5.19895554e-02,6.81915283e-02,8.41045380e-02,1.00490570e-01,  1.01671219e-01,1.01116180e-01,1.12850189e-01,1.29837036e-01,  1.51390076e-01,1.85943604e-01,1.92264557e-01,1.68159485e-01,  1.55815125e-01,1.69834137e-01,1.85443878e-01,2.04502106e-01,  2.33192444e-01,2.56546021e-01,2.63816833e-01,2.69439697e-01,  2.81288147e-01,2.84904480e-01,2.86796570e-01,2.89764404e-01,  2.90885925e-01,2.90374756e-01,2.90313721e-01,2.91427612e-01,  2.89985657e-01,2.86941528e-01,2.83706665e-01,2.74963379e-01,  2.57675171e-01,2.49179840e-01,2.56324768e-01,2.63961792e-01,  2.65983582e-01,2.62977600e-01,2.51167297e-01,2.46299744e-01,  2.49961853e-01,2.58201599e-01,2.63084412e-01,2.66426086e-01,  2.67265320e-01,2.66975403e-01,2.67013550e-01,2.67478943e-01,  2.67967224e-01,2.66433716e-01,2.61978149e-01,2.55386353e-01,  2.50541687e-01,2.45506287e-01,2.40295410e-01,2.35710144e-01,  2.31338501e-01,2.26055145e-01,2.24189758e-01],
        [3.67027283e-01,3.84414673e-01,4.04113770e-01,4.78775024e-01,  5.09460449e-01,5.24139404e-01,5.26947021e-01,5.26031494e-01,  5.77850342e-01,6.63223267e-01,8.87283325e-01,1.00753784e+00,  1.06491089e+00,1.11599731e+00,1.12719727e+00,1.10192871e+00,  1.08999634e+00,1.18496704e+00,1.29403687e+00,1.39733887e+00,  1.47689819e+00,1.51535034e+00,1.54449463e+00,1.54919434e+00,  1.55850220e+00,1.59167480e+00,1.60699463e+00,1.55325317e+00,  1.56991577e+00,1.59957886e+00,1.57888794e+00,1.58782959e+00,  1.58572388e+00,1.52966309e+00,1.53439331e+00,1.56863403e+00,  1.59271240e+00,1.59921265e+00,1.58807373e+00,1.59213257e+00,  1.59170532e+00,1.57711792e+00,1.55004883e+00,1.53704834e+00,  1.52536011e+00,1.52206421e+00,1.53796387e+00,1.53753662e+00,  1.51452637e+00,1.51834106e+00,1.50561523e+00,1.50329590e+00,  1.49075317e+00,1.47775269e+00,1.48049927e+00,1.45373535e+00,  1.44177246e+00,1.46554565e+00,1.47381592e+00,1.47149658e+00,  1.45620728e+00,1.42120361e+00,1.45541382e+00,1.48138428e+00,  1.47665405e+00,1.46807861e+00,1.44030762e+00,1.32400513e+00,  1.27966309e+00,1.35644531e+00,1.37432861e+00,1.37496948e+00,  1.36526489e+00,1.30535889e+00,1.24719238e+00,1.24615479e+00,  1.26486206e+00,1.28906250e+00,1.29122925e+00,1.28805542e+00,  1.27331543e+00,1.14837646e+00,7.72644043e-01,8.39599609e-01,  1.12619019e+00,1.19546509e+00,1.19799805e+00,1.18862915e+00,  1.17337036e+00,1.16680908e+00,1.15707397e+00,1.14538574e+00,  1.11495972e+00,1.03555298e+00,1.01007080e+00,1.02673340e+00,  1.02978516e+00,1.04467773e+00,1.04470825e+00,1.03167725e+00,  9.96643066e-01,9.76303101e-01,9.94400024e-01,9.72351074e-01,  9.69451904e-01,9.81979370e-01,9.77890015e-01,9.65484619e-01,  9.63226318e-01,8.92425537e-01,8.56872559e-01,8.58535767e-01,  8.10379028e-01,8.21090698e-01,8.33206177e-01,7.83569336e-01,  6.40869141e-01,5.10437012e-01,5.20889282e-01,5.31524658e-01,  5.44662476e-01,5.69885254e-01,6.10519409e-01,6.69082642e-01,  7.13577271e-01,7.25067139e-01,7.41455078e-01,7.62557983e-01,  7.70812988e-01,7.69210815e-01,7.58728027e-01,7.48062134e-01,  7.45422363e-01,7.42095947e-01,7.34634399e-01,7.26531982e-01,  7.17025757e-01,7.07183838e-01,6.99096680e-01,6.89605713e-01,  6.79519653e-01,6.66870117e-01,6.53182983e-01,6.40670776e-01,  6.34155273e-01,6.31927490e-01,6.24465942e-01,6.14196777e-01,  6.01776123e-01,5.84213257e-01,5.66848755e-01,5.45013428e-01,  4.89212036e-01,3.89579773e-01,3.04710388e-01,2.78800964e-01,  2.90237427e-01,3.16230774e-01,3.45954895e-01,3.34388733e-01,  3.48487854e-01,3.99108887e-01,4.49882507e-01,4.85275269e-01,  5.00701904e-01,4.98847961e-01,4.90074158e-01,4.86244202e-01,  4.94407654e-01,4.93072510e-01,4.79110718e-01,4.74578857e-01,  4.80010986e-01,4.86206055e-01,4.85610962e-01,4.84092712e-01,  4.85069275e-01,4.83390808e-01,4.77043152e-01,4.69779968e-01,  4.59815979e-01,4.40116882e-01,4.04251099e-01,3.67271423e-01,  3.73130798e-01,4.07287598e-01,4.20356750e-01,4.30900574e-01,  4.36119080e-01,4.31221008e-01,4.19319153e-01,4.03968811e-01,  3.85612488e-01,3.69155884e-01,3.58436584e-01,3.39408875e-01,  3.00598145e-01,2.88940430e-01,2.69264221e-01,1.93748474e-01,  9.66758728e-02,3.66210938e-02,1.85346603e-02,1.60346031e-02,  1.80015564e-02,2.47921944e-02,2.16879845e-02,2.36864090e-02,  3.15608978e-02,3.21741104e-02,3.66468430e-02,4.11214828e-02,  5.40513992e-02,7.05051422e-02,8.66355896e-02,1.03116989e-01,  1.04272842e-01,1.03851318e-01,1.15756989e-01,1.32728577e-01,  1.54094696e-01,1.88678741e-01,1.94885254e-01,1.70825958e-01,  1.58393860e-01,1.72382355e-01,1.87961578e-01,2.06737518e-01,  2.34912872e-01,2.57995605e-01,2.65037537e-01,2.70500183e-01,  2.82073975e-01,2.85461426e-01,2.87422180e-01,2.90245056e-01,  2.91366577e-01,2.90740967e-01,2.90588379e-01,2.91648865e-01,  2.90267944e-01,2.87300110e-01,2.84042358e-01,2.75344849e-01,  2.58338928e-01,2.49740601e-01,2.56744385e-01,2.64350891e-01,  2.66242981e-01,2.63313293e-01,2.51884460e-01,2.46978760e-01,  2.50617981e-01,2.58712769e-01,2.63412476e-01,2.66738892e-01,  2.67700195e-01,2.67585754e-01,2.67692566e-01,2.68218994e-01,  2.68676758e-01,2.67005920e-01,2.62947083e-01,2.57164001e-01,  2.52174377e-01,2.47055054e-01,2.42233276e-01,2.36831665e-01,  2.32341766e-01,2.27668762e-01,2.25326538e-01],
        [3.81172180e-01,3.92250061e-01,4.15489197e-01,4.89791870e-01,  5.20065308e-01,5.36773682e-01,5.37139893e-01,5.38925171e-01,  5.90209961e-01,6.77688599e-01,9.07791138e-01,1.02624512e+00,  1.08459473e+00,1.13409424e+00,1.14443970e+00,1.11785889e+00,  1.10516357e+00,1.20294189e+00,1.31130981e+00,1.41580200e+00,  1.49639893e+00,1.53277588e+00,1.56332397e+00,1.56625366e+00,  1.57540894e+00,1.61013794e+00,1.62344360e+00,1.56933594e+00,  1.58609009e+00,1.61480713e+00,1.59454346e+00,1.60226440e+00,  1.59991455e+00,1.54299927e+00,1.54724121e+00,1.58258057e+00,  1.60577393e+00,1.61193848e+00,1.60079956e+00,1.60354614e+00,  1.60449219e+00,1.58911133e+00,1.56170654e+00,1.54968262e+00,  1.53753662e+00,1.53512573e+00,1.55056763e+00,1.54733276e+00,  1.52667236e+00,1.52911377e+00,1.51623535e+00,1.51470947e+00,  1.50015259e+00,1.48864746e+00,1.49035645e+00,1.46215820e+00,  1.45208740e+00,1.47357178e+00,1.48327637e+00,1.47985840e+00,  1.46267700e+00,1.42864990e+00,1.46112061e+00,1.48669434e+00,  1.48315430e+00,1.47369385e+00,1.44680786e+00,1.32904053e+00,  1.28576660e+00,1.36340332e+00,1.37957764e+00,1.38098145e+00,  1.37252808e+00,1.30767822e+00,1.25082397e+00,1.25195312e+00,  1.26849365e+00,1.29565430e+00,1.29351807e+00,1.29049683e+00,  1.27850342e+00,1.14978027e+00,7.79693604e-01,8.49502563e-01,  1.13055420e+00,1.20220947e+00,1.20068359e+00,1.19082642e+00,  1.17715454e+00,1.16403198e+00,1.15954590e+00,1.15124512e+00,  1.11120605e+00,1.04638672e+00,1.01864624e+00,1.02243042e+00,  1.03421021e+00,1.04589844e+00,1.05004883e+00,1.03997803e+00,  9.94461060e-01,9.78424072e-01,9.92919922e-01,9.66491699e-01,  9.79095459e-01,9.87167358e-01,9.74044800e-01,9.84603882e-01,  9.63684082e-01,8.92318726e-01,8.46542358e-01,8.38272095e-01,  8.27239990e-01,8.35311890e-01,8.38989258e-01,8.17947388e-01,  6.55197144e-01,5.14709473e-01,5.24841309e-01,5.35430908e-01,  5.48400879e-01,5.73516846e-01,6.13723755e-01,6.71829224e-01,  7.15515137e-01,7.26928711e-01,7.43103027e-01,7.63793945e-01,  7.71667480e-01,7.69958496e-01,7.59704590e-01,7.48916626e-01,  7.46200562e-01,7.42736816e-01,7.35351562e-01,7.27294922e-01,  7.17681885e-01,7.07931519e-01,6.99905396e-01,6.90612793e-01,  6.80541992e-01,6.68106079e-01,6.54296875e-01,6.41769409e-01,  6.35131836e-01,6.33041382e-01,6.25457764e-01,6.15310669e-01,  6.02981567e-01,5.85418701e-01,5.68191528e-01,5.46691895e-01,  4.91287231e-01,3.92051697e-01,3.07601929e-01,2.81845093e-01,  2.93205261e-01,3.19000244e-01,3.48693848e-01,3.37165833e-01,  3.51226807e-01,4.01657104e-01,4.51881409e-01,4.86976624e-01,  5.02151489e-01,5.00213623e-01,4.91416931e-01,4.87594604e-01,  4.95536804e-01,4.94247437e-01,4.80468750e-01,4.75868225e-01,  4.81079102e-01,4.87228394e-01,4.86534119e-01,4.85015869e-01,  4.85824585e-01,4.84054565e-01,4.77844238e-01,4.70581055e-01,  4.60769653e-01,4.41322327e-01,4.05769348e-01,3.69277954e-01,  3.75274658e-01,4.08721924e-01,4.21241760e-01,4.31739807e-01,  4.36866760e-01,4.31968689e-01,4.20120239e-01,4.04914856e-01,  3.86741638e-01,3.70399475e-01,3.59748840e-01,3.40873718e-01,  3.02368164e-01,2.90580750e-01,2.71003723e-01,1.95556641e-01,  9.81197357e-02,3.73449326e-02,1.88527107e-02,1.62801743e-02,  1.83553696e-02,2.54921913e-02,2.22773552e-02,2.43330002e-02,  3.23963165e-02,3.30066681e-02,3.75299454e-02,4.20761108e-02,  5.52139282e-02,7.18441010e-02,8.81195068e-02,1.04742050e-01,  1.05890274e-01,1.05484009e-01,1.17515564e-01,1.34433746e-01,  1.55765533e-01,1.90399170e-01,1.96571350e-01,1.72462463e-01,  1.59946442e-01,1.73950195e-01,1.89353943e-01,2.08061218e-01,  2.36225128e-01,2.59048462e-01,2.65808105e-01,2.71240234e-01,  2.82684326e-01,2.85949707e-01,2.87849426e-01,2.90542603e-01,  2.91633606e-01,2.91076660e-01,2.90893555e-01,2.91984558e-01,  2.90405273e-01,2.87414551e-01,2.84187317e-01,2.75627136e-01,  2.58758545e-01,2.50228882e-01,2.57301331e-01,2.64556885e-01,  2.66319275e-01,2.63549805e-01,2.52235413e-01,2.47512817e-01,  2.51014709e-01,2.58956909e-01,2.63717651e-01,2.66952515e-01,  2.67921448e-01,2.67768860e-01,2.67959595e-01,2.68371582e-01,  2.68753052e-01,2.67395020e-01,2.63069153e-01,2.56896973e-01,  2.52433777e-01,2.47848511e-01,2.42958069e-01,2.37285614e-01,  2.32921600e-01,2.28168488e-01,2.24781036e-01],
        [3.91494751e-01,4.06173706e-01,4.26681519e-01,5.03677368e-01,  5.35690308e-01,5.51498413e-01,5.54031372e-01,5.53054810e-01,  6.05499268e-01,6.95465088e-01,9.29107666e-01,1.04870605e+00,  1.10617065e+00,1.15631104e+00,1.16567993e+00,1.13711548e+00,  1.12530518e+00,1.22277832e+00,1.33279419e+00,1.43804932e+00,  1.51828003e+00,1.55694580e+00,1.58453369e+00,1.58795166e+00,  1.59786987e+00,1.63073730e+00,1.64343262e+00,1.58767700e+00,  1.60580444e+00,1.63433838e+00,1.61160278e+00,1.62060547e+00,  1.61694336e+00,1.55834961e+00,1.56411743e+00,1.59909058e+00,  1.62326050e+00,1.62777710e+00,1.61511230e+00,1.61944580e+00,  1.61935425e+00,1.60369873e+00,1.57681274e+00,1.56344604e+00,  1.55303955e+00,1.54949951e+00,1.56439209e+00,1.56262207e+00,  1.53918457e+00,1.54321289e+00,1.53186035e+00,1.52786255e+00,  1.51391602e+00,1.50134277e+00,1.50485229e+00,1.47705078e+00,  1.46365356e+00,1.48706055e+00,1.49545288e+00,1.49020386e+00,  1.47381592e+00,1.43814087e+00,1.47259521e+00,1.49658203e+00,  1.49105835e+00,1.48342896e+00,1.45446777e+00,1.33685303e+00,  1.29779053e+00,1.37145996e+00,1.38812256e+00,1.38754272e+00,  1.37661743e+00,1.31637573e+00,1.25833130e+00,1.26013184e+00,  1.27859497e+00,1.30160522e+00,1.30206299e+00,1.29885864e+00,  1.28317261e+00,1.15463257e+00,7.80578613e-01,8.56460571e-01,  1.13958740e+00,1.20709229e+00,1.20797729e+00,1.19760132e+00,  1.17889404e+00,1.17486572e+00,1.16665649e+00,1.15444946e+00,  1.12042236e+00,1.04306030e+00,1.02020264e+00,1.03396606e+00,  1.03836060e+00,1.05563354e+00,1.05456543e+00,1.03976440e+00,  1.00439453e+00,9.82696533e-01,1.00289917e+00,9.76333618e-01,  9.79156494e-01,9.97253418e-01,9.84146118e-01,9.82452393e-01,  9.83703613e-01,9.08950806e-01,8.62396240e-01,8.57284546e-01,  8.15826416e-01,8.25576782e-01,8.42636108e-01,8.10104370e-01,  6.53472900e-01,5.18951416e-01,5.29357910e-01,5.39749146e-01,  5.52459717e-01,5.77484131e-01,6.17431641e-01,6.74682617e-01,  7.17895508e-01,7.28866577e-01,7.44720459e-01,7.65075684e-01,  7.72796631e-01,7.70889282e-01,7.60498047e-01,7.49664307e-01,  7.47085571e-01,7.43530273e-01,7.36068726e-01,7.28012085e-01,  7.18414307e-01,7.08770752e-01,7.00607300e-01,6.91329956e-01,  6.81411743e-01,6.68914795e-01,6.55120850e-01,6.42745972e-01,  6.36077881e-01,6.33850098e-01,6.26296997e-01,6.16104126e-01,  6.03866577e-01,5.86502075e-01,5.69503784e-01,5.48324585e-01,  4.93453979e-01,3.94691467e-01,3.10684204e-01,2.85041809e-01,  2.96249390e-01,3.21891785e-01,3.51539612e-01,3.40194702e-01,  3.54141235e-01,4.04090881e-01,4.53758240e-01,4.88571167e-01,  5.03479004e-01,5.01525879e-01,4.92675781e-01,4.88861084e-01,  4.96711731e-01,4.95292664e-01,4.81437683e-01,4.76936340e-01,  4.81941223e-01,4.88090515e-01,4.87380981e-01,4.85687256e-01,  4.86434937e-01,4.84733582e-01,4.78393555e-01,4.71275330e-01,  4.61532593e-01,4.42337036e-01,4.07333374e-01,3.71330261e-01,  3.76953125e-01,4.09980774e-01,4.22096252e-01,4.32205200e-01,  4.37362671e-01,4.32624817e-01,4.20845032e-01,4.05693054e-01,  3.87664795e-01,3.71429443e-01,3.61030579e-01,3.42269897e-01,  3.04054260e-01,2.92488098e-01,2.72781372e-01,1.97364807e-01,  9.97810364e-02,3.81660461e-02,1.92308426e-02,1.65758133e-02,  1.88612938e-02,2.63633728e-02,2.29182243e-02,2.49762535e-02,  3.34062576e-02,3.38335037e-02,3.83596420e-02,4.32643890e-02,  5.64985275e-02,7.34062195e-02,8.98075104e-02,1.06443405e-01,  1.07574463e-01,1.07276917e-01,1.19451523e-01,1.36276245e-01,  1.57501221e-01,1.92043304e-01,1.98246002e-01,1.74133301e-01,  1.61808014e-01,1.75613403e-01,1.90921783e-01,2.09499359e-01,  2.37335205e-01,2.60017395e-01,2.66685486e-01,2.71888733e-01,  2.83279419e-01,2.86392212e-01,2.88162231e-01,2.90893555e-01,  2.91862488e-01,2.91275024e-01,2.91076660e-01,2.92053223e-01,  2.90504456e-01,2.87445068e-01,2.84164429e-01,2.75665283e-01,  2.59071350e-01,2.50640869e-01,2.57469177e-01,2.64686584e-01,  2.66380310e-01,2.63595581e-01,2.52555847e-01,2.47753143e-01,  2.51136780e-01,2.59056091e-01,2.63687134e-01,2.67028809e-01,  2.67845154e-01,2.67784119e-01,2.67829895e-01,2.68341064e-01,  2.69042969e-01,2.67280579e-01,2.63229370e-01,2.57408142e-01,  2.52960205e-01,2.47993469e-01,2.42809296e-01,2.37018585e-01,  2.32780457e-01,2.28569031e-01,2.25093842e-01],
        [4.01268005e-01,4.15161133e-01,4.39323425e-01,5.13656616e-01,  5.49240112e-01,5.63674927e-01,5.63217163e-01,5.65261841e-01,  6.16104126e-01,7.09930420e-01,9.45297241e-01,1.06481934e+00,  1.12390137e+00,1.17138672e+00,1.18191528e+00,1.15182495e+00,  1.13931274e+00,1.24005127e+00,1.34906006e+00,1.45565796e+00,  1.53521729e+00,1.57235718e+00,1.60159302e+00,1.60287476e+00,  1.61401367e+00,1.64746094e+00,1.65893555e+00,1.60296631e+00,  1.61987305e+00,1.64791870e+00,1.62570190e+00,1.63369751e+00,  1.63079834e+00,1.57092285e+00,1.57656860e+00,1.61148071e+00,  1.63482666e+00,1.64031982e+00,1.62725830e+00,1.63095093e+00,  1.63137817e+00,1.61425781e+00,1.58880615e+00,1.57504272e+00,  1.56405640e+00,1.56256104e+00,1.57583618e+00,1.57366943e+00,  1.55187988e+00,1.55303955e+00,1.54232788e+00,1.53842163e+00,  1.52340698e+00,1.51284790e+00,1.51266479e+00,1.48593140e+00,  1.47445679e+00,1.49447632e+00,1.50338745e+00,1.49835205e+00,  1.48141479e+00,1.44674683e+00,1.47924805e+00,1.50497437e+00,  1.49923706e+00,1.48925781e+00,1.46304321e+00,1.34320068e+00,  1.30349731e+00,1.38012695e+00,1.39453125e+00,1.39752197e+00,  1.38449097e+00,1.32061768e+00,1.26898193e+00,1.26718140e+00,  1.28430176e+00,1.30804443e+00,1.30447388e+00,1.30398560e+00,  1.28866577e+00,1.15615845e+00,7.89916992e-01,8.64120483e-01,  1.14410400e+00,1.21087646e+00,1.20715332e+00,1.20715332e+00,  1.18859863e+00,1.17608643e+00,1.17550659e+00,1.15856934e+00,  1.11938477e+00,1.05548096e+00,1.02539062e+00,1.04205322e+00,  1.05218506e+00,1.05841064e+00,1.06658936e+00,1.04641724e+00,  1.00186157e+00,9.94445801e-01,1.00357056e+00,9.79812622e-01,  9.86419678e-01,9.82955933e-01,9.83993530e-01,9.87792969e-01,  9.65209961e-01,9.16015625e-01,8.59191895e-01,8.53149414e-01,  8.35372925e-01,8.29010010e-01,8.48220825e-01,8.32595825e-01,  6.64916992e-01,5.24734497e-01,5.34744263e-01,5.45272827e-01,  5.57693481e-01,5.82427979e-01,6.21780396e-01,6.78497314e-01,  7.20809937e-01,7.31445312e-01,7.46688843e-01,7.66342163e-01,  7.73513794e-01,7.71499634e-01,7.61154175e-01,7.50259399e-01,  7.47711182e-01,7.44293213e-01,7.36663818e-01,7.28561401e-01,  7.19009399e-01,7.09152222e-01,7.01126099e-01,6.91772461e-01,  6.82006836e-01,6.69692993e-01,6.55929565e-01,6.43554688e-01,  6.36993408e-01,6.34628296e-01,6.26983643e-01,6.16897583e-01,  6.04766846e-01,5.87554932e-01,5.71212769e-01,5.50216675e-01,  4.95712280e-01,3.97926331e-01,3.14598083e-01,2.89054871e-01,  3.00361633e-01,3.25630188e-01,3.55049133e-01,3.43841553e-01,  3.57841492e-01,4.07142639e-01,4.56275940e-01,4.90562439e-01,  5.05096436e-01,5.02853394e-01,4.94155884e-01,4.90295410e-01,  4.98062134e-01,4.96551514e-01,4.82780457e-01,4.78157043e-01,  4.83100891e-01,4.89021301e-01,4.88166809e-01,4.86579895e-01,  4.87022400e-01,4.85099792e-01,4.78897095e-01,4.71893311e-01,  4.62265015e-01,4.43321228e-01,4.08515930e-01,3.72955322e-01,  3.78562927e-01,4.11079407e-01,4.22760010e-01,4.32800293e-01,  4.37889099e-01,4.33158875e-01,4.21569824e-01,4.06723022e-01,  3.88885498e-01,3.72756958e-01,3.62442017e-01,3.43955994e-01,  3.06205750e-01,2.94570923e-01,2.75001526e-01,1.99756622e-01,  1.01730347e-01,3.92103195e-02,1.94740295e-02,1.66440010e-02,  1.91159248e-02,2.69799232e-02,2.33631134e-02,2.56018639e-02,  3.44095230e-02,3.49121094e-02,3.95336151e-02,4.44116592e-02,  5.80101013e-02,7.50885010e-02,9.16175842e-02,1.08507156e-01,  1.09577179e-01,1.09321594e-01,1.21576309e-01,1.38477325e-01,  1.59755707e-01,1.94271088e-01,2.00233459e-01,1.76185608e-01,  1.63791656e-01,1.77616119e-01,1.92863464e-01,2.11200714e-01,  2.38773346e-01,2.61024475e-01,2.67341614e-01,2.72377014e-01,  2.83515930e-01,2.86758423e-01,2.88276672e-01,2.90916443e-01,  2.91870117e-01,2.91107178e-01,2.90924072e-01,2.91824341e-01,  2.90290833e-01,2.87223816e-01,2.83859253e-01,2.75558472e-01,  2.59063721e-01,2.50679016e-01,2.57362366e-01,2.64282227e-01,  2.65975952e-01,2.63259888e-01,2.52281189e-01,2.47505188e-01,  2.50915527e-01,2.58644104e-01,2.63168335e-01,2.66487122e-01,  2.67204285e-01,2.67074585e-01,2.67036438e-01,2.67234802e-01,  2.67860413e-01,2.65754700e-01,2.61169434e-01,2.54631042e-01,  2.49851227e-01,2.45460510e-01,2.39696503e-01,2.34142303e-01,  2.30117798e-01,2.24559784e-01,2.21992493e-01],
        [4.08958435e-01,4.23561096e-01,4.48753357e-01,5.25451660e-01,  5.55282593e-01,5.72525024e-01,5.73333740e-01,5.73532104e-01,  6.25137329e-01,7.18551636e-01,9.56390381e-01,1.07693481e+00,  1.13586426e+00,1.18521118e+00,1.19357300e+00,1.16354370e+00,  1.15087891e+00,1.25012207e+00,1.36157227e+00,1.46801758e+00,  1.54718018e+00,1.58432007e+00,1.61273193e+00,1.61462402e+00,  1.62390137e+00,1.65744019e+00,1.66918945e+00,1.61209106e+00,  1.62872314e+00,1.65603638e+00,1.63391113e+00,1.64254761e+00,  1.63839722e+00,1.57867432e+00,1.58343506e+00,1.61853027e+00,  1.64273071e+00,1.64752197e+00,1.63519287e+00,1.63848877e+00,  1.63909912e+00,1.62280273e+00,1.59536743e+00,1.58303833e+00,  1.57232666e+00,1.56991577e+00,1.58444214e+00,1.57974243e+00,  1.55706787e+00,1.56002808e+00,1.54714966e+00,1.54461670e+00,  1.52920532e+00,1.51730347e+00,1.51931763e+00,1.49066162e+00,  1.47885132e+00,1.50021362e+00,1.50775146e+00,1.50332642e+00,  1.48648071e+00,1.44958496e+00,1.48248291e+00,1.50784302e+00,  1.50158691e+00,1.49160767e+00,1.46481323e+00,1.34768677e+00,  1.30722046e+00,1.38204956e+00,1.39694214e+00,1.39712524e+00,  1.38632202e+00,1.32446289e+00,1.26791382e+00,1.26983643e+00,  1.28543091e+00,1.30831909e+00,1.30584717e+00,1.30355835e+00,  1.28958130e+00,1.15988159e+00,7.92663574e-01,8.68087769e-01,  1.14581299e+00,1.21240234e+00,1.21133423e+00,1.20202637e+00,  1.18792725e+00,1.17654419e+00,1.16833496e+00,1.16076660e+00,  1.12304688e+00,1.05572510e+00,1.03079224e+00,1.03970337e+00,  1.04943848e+00,1.05783081e+00,1.05688477e+00,1.04705811e+00,  1.00408936e+00,9.88922119e-01,1.00552368e+00,9.77005005e-01,  9.84710693e-01,9.85488892e-01,9.75616455e-01,9.84863281e-01,  9.72671509e-01,9.07119751e-01,8.66149902e-01,8.64685059e-01,  8.36746216e-01,8.39477539e-01,8.30841064e-01,8.12057495e-01,  6.62277222e-01,5.26550293e-01,5.36437988e-01,5.46997070e-01,  5.59692383e-01,5.84350586e-01,6.23428345e-01,6.79397583e-01,  7.21267700e-01,7.32055664e-01,7.47283936e-01,7.66799927e-01,  7.73773193e-01,7.71591187e-01,7.61199951e-01,7.50488281e-01,  7.47756958e-01,7.44155884e-01,7.36480713e-01,7.28637695e-01,  7.19024658e-01,7.09350586e-01,7.01202393e-01,6.91940308e-01,  6.82144165e-01,6.69982910e-01,6.56219482e-01,6.43875122e-01,  6.37145996e-01,6.34826660e-01,6.27090454e-01,6.17034912e-01,  6.04812622e-01,5.87829590e-01,5.71594238e-01,5.50918579e-01,  4.96536255e-01,3.99047852e-01,3.15979004e-01,2.90634155e-01,  3.01818848e-01,3.26950073e-01,3.56407166e-01,3.45222473e-01,  3.59069824e-01,4.08302307e-01,4.57115173e-01,4.91127014e-01,  5.05813599e-01,5.03524780e-01,4.94598389e-01,4.90814209e-01,  4.98435974e-01,4.96826172e-01,4.83116150e-01,4.78645325e-01,  4.83566284e-01,4.89273071e-01,4.88441467e-01,4.86701965e-01,  4.87236023e-01,4.85351562e-01,4.78981018e-01,4.71992493e-01,  4.62539673e-01,4.43862915e-01,4.09622192e-01,3.74282837e-01,  3.79714966e-01,4.11788940e-01,4.23179626e-01,4.32952881e-01,  4.37950134e-01,4.33219910e-01,4.21768188e-01,4.06875610e-01,  3.89129639e-01,3.73207092e-01,3.62937927e-01,3.44566345e-01,  3.06938171e-01,2.95349121e-01,2.75672913e-01,2.00622559e-01,  1.02483749e-01,3.95002365e-02,1.95679665e-02,1.67708397e-02,  1.92894936e-02,2.73342133e-02,2.35347748e-02,2.57701874e-02,  3.48901749e-02,3.52792740e-02,3.99198532e-02,4.49581146e-02,  5.86338043e-02,7.57808685e-02,9.23748016e-02,1.09231949e-01,  1.10363007e-01,1.10181808e-01,1.22488022e-01,1.39396667e-01,  1.60507202e-01,1.95003510e-01,2.00965881e-01,1.76891327e-01,  1.64569855e-01,1.78283691e-01,1.93496704e-01,2.11784363e-01,  2.39105225e-01,2.61322021e-01,2.67517090e-01,2.72544861e-01,  2.83714294e-01,2.86827087e-01,2.88322449e-01,2.90847778e-01,  2.91816711e-01,2.91114807e-01,2.90908813e-01,2.91694641e-01,  2.90077209e-01,2.86911011e-01,2.83683777e-01,2.75405884e-01,  2.59185791e-01,2.50831604e-01,2.57324219e-01,2.64106750e-01,  2.65747070e-01,2.63137817e-01,2.52334595e-01,2.47600555e-01,  2.50793457e-01,2.58445740e-01,2.62962341e-01,2.66120911e-01,  2.67021179e-01,2.67028809e-01,2.67021179e-01,2.67158508e-01,  2.67379761e-01,2.65396118e-01,2.60757446e-01,2.54325867e-01,  2.49187469e-01,2.44014740e-01,2.38788605e-01,2.33623505e-01,  2.29133606e-01,2.24052429e-01,2.21397400e-01],
        [4.17587280e-01,4.31953430e-01,4.55154419e-01,5.33828735e-01,  5.69885254e-01,5.81832886e-01,5.81710815e-01,5.80474854e-01,  6.32949829e-01,7.26989746e-01,9.67071533e-01,1.09121704e+00,  1.15087891e+00,1.19995117e+00,1.20870972e+00,1.17724609e+00,  1.16235352e+00,1.26287842e+00,1.37429810e+00,1.48123169e+00,  1.56127930e+00,1.59805298e+00,1.62692261e+00,1.62762451e+00,  1.63665771e+00,1.66992188e+00,1.68179321e+00,1.62377930e+00,  1.63833618e+00,1.66653442e+00,1.64468384e+00,1.65246582e+00,  1.64831543e+00,1.58706665e+00,1.59121704e+00,1.62667847e+00,  1.65072632e+00,1.65673828e+00,1.64340210e+00,1.64697266e+00,  1.64727783e+00,1.63085938e+00,1.60449219e+00,1.59082031e+00,  1.57943726e+00,1.57736206e+00,1.59109497e+00,1.58850098e+00,  1.56579590e+00,1.56765747e+00,1.55578613e+00,1.55108643e+00,  1.53588867e+00,1.52331543e+00,1.52288818e+00,1.49642944e+00,  1.48495483e+00,1.50466919e+00,1.51217651e+00,1.50628662e+00,  1.48989868e+00,1.45349121e+00,1.48477173e+00,1.51062012e+00,  1.50521851e+00,1.49581909e+00,1.46817017e+00,1.35012817e+00,  1.30847168e+00,1.38226318e+00,1.39764404e+00,1.39941406e+00,  1.38751221e+00,1.32696533e+00,1.27273560e+00,1.27178955e+00,  1.28823853e+00,1.31134033e+00,1.30807495e+00,1.30764771e+00,  1.29302979e+00,1.16366577e+00,7.98233032e-01,8.68179321e-01,  1.14627075e+00,1.21472168e+00,1.21130371e+00,1.20330811e+00,  1.19003296e+00,1.17913818e+00,1.17486572e+00,1.16271973e+00,  1.12457275e+00,1.05364990e+00,1.02862549e+00,1.04281616e+00,  1.05001831e+00,1.05694580e+00,1.06195068e+00,1.04428101e+00,  9.97146606e-01,9.85992432e-01,1.00372314e+00,9.77630615e-01,  9.79888916e-01,9.89746094e-01,9.80117798e-01,9.80773926e-01,  9.71710205e-01,9.14489746e-01,8.62838745e-01,8.56872559e-01,  8.40881348e-01,8.34182739e-01,8.43338013e-01,8.21609497e-01,  6.65573120e-01,5.32791138e-01,5.42938232e-01,5.53161621e-01,  5.65597534e-01,5.89706421e-01,6.28448486e-01,6.83731079e-01,  7.24899292e-01,7.35046387e-01,7.49908447e-01,7.68615723e-01,  7.75253296e-01,7.73132324e-01,7.62619019e-01,7.51663208e-01,  7.49114990e-01,7.45391846e-01,7.37960815e-01,7.29949951e-01,  7.20260620e-01,7.10449219e-01,7.02468872e-01,6.93328857e-01,  6.83639526e-01,6.71218872e-01,6.57714844e-01,6.45446777e-01,  6.38763428e-01,6.36306763e-01,6.28509521e-01,6.18423462e-01,  6.06338501e-01,5.89614868e-01,5.73837280e-01,5.53436279e-01,  4.99725342e-01,4.03083801e-01,3.20518494e-01,2.95440674e-01,  3.06457520e-01,3.31237793e-01,3.60694885e-01,3.49662781e-01,  3.63273621e-01,4.12055969e-01,4.60128784e-01,4.93682861e-01,  5.07965088e-01,5.05630493e-01,4.96704102e-01,4.92813110e-01,  5.00152588e-01,4.98542786e-01,4.84901428e-01,4.80415344e-01,  4.85092163e-01,4.90776062e-01,4.89776611e-01,4.87937927e-01,  4.88357544e-01,4.86228943e-01,4.79934692e-01,4.73068237e-01,  4.63752747e-01,4.45259094e-01,4.11369324e-01,3.76319885e-01,  3.81828308e-01,4.13475037e-01,4.24339294e-01,4.33876038e-01,  4.39041138e-01,4.34440613e-01,4.23011780e-01,4.08363342e-01,  3.90754700e-01,3.75015259e-01,3.64936829e-01,3.46824646e-01,  3.09684753e-01,2.98141479e-01,2.78450012e-01,2.03567505e-01,  1.05205536e-01,4.12263870e-02,2.05812454e-02,1.75948143e-02,  2.03070641e-02,2.88949013e-02,2.49376297e-02,2.72564888e-02,  3.67240906e-02,3.70836258e-02,4.17976379e-02,4.68606949e-02,  6.08491898e-02,7.82890320e-02,9.50737000e-02,1.12049103e-01,  1.13077164e-01,1.13046646e-01,1.25526428e-01,1.42272949e-01,  1.63272858e-01,1.97677612e-01,2.03662872e-01,1.79748535e-01,  1.67388916e-01,1.80938721e-01,1.96083069e-01,2.14134216e-01,  2.41218567e-01,2.62969971e-01,2.68997192e-01,2.73887634e-01,  2.84828186e-01,2.87818909e-01,2.89268494e-01,2.91717529e-01,  2.92617798e-01,2.91816711e-01,2.91519165e-01,2.92442322e-01,  2.90878296e-01,2.87689209e-01,2.84370422e-01,2.76252747e-01,  2.60108948e-01,2.51754761e-01,2.58247375e-01,2.64938354e-01,  2.66601562e-01,2.63999939e-01,2.53379822e-01,2.48752594e-01,  2.51907349e-01,2.59346008e-01,2.63793945e-01,2.67181396e-01,  2.68272400e-01,2.68264771e-01,2.68341064e-01,2.68516541e-01,  2.68989563e-01,2.67005920e-01,2.62725830e-01,2.56797791e-01,  2.52235413e-01,2.47898102e-01,2.42481232e-01,2.37159729e-01,  2.32604980e-01,2.27638245e-01,2.25173950e-01],
        [4.30435181e-01,4.40902710e-01,4.66529846e-01,5.41259766e-01,  5.76080322e-01,5.91506958e-01,5.88363647e-01,5.86883545e-01,  6.38626099e-01,7.31597900e-01,9.75570679e-01,1.10098267e+00,  1.16116333e+00,1.21084595e+00,1.21838379e+00,1.18786621e+00,  1.17077637e+00,1.27108765e+00,1.38214111e+00,1.48919678e+00,  1.57037354e+00,1.60507202e+00,1.63351440e+00,1.63449097e+00,  1.64230347e+00,1.67654419e+00,1.68762207e+00,1.62957764e+00,  1.64315796e+00,1.67053223e+00,1.64910889e+00,1.65557861e+00,  1.65267944e+00,1.59207153e+00,1.59405518e+00,1.63070679e+00,  1.65423584e+00,1.65997314e+00,1.64773560e+00,1.65008545e+00,  1.65057373e+00,1.63406372e+00,1.60668945e+00,1.59533691e+00,  1.58386230e+00,1.58203125e+00,1.59423828e+00,1.58926392e+00,  1.56835938e+00,1.56918335e+00,1.55657959e+00,1.55319214e+00,  1.53607178e+00,1.52365112e+00,1.52432251e+00,1.49725342e+00,  1.48599243e+00,1.50332642e+00,1.51153564e+00,1.50646973e+00,  1.48754883e+00,1.45300293e+00,1.48397827e+00,1.50839233e+00,  1.50341797e+00,1.49209595e+00,1.46685791e+00,1.35247803e+00,  1.30899048e+00,1.38250732e+00,1.39642334e+00,1.39648438e+00,  1.38635254e+00,1.32519531e+00,1.27154541e+00,1.27182007e+00,  1.28530884e+00,1.30865479e+00,1.30349731e+00,1.30224609e+00,  1.28823853e+00,1.15969849e+00,8.01254272e-01,8.66348267e-01,  1.13983154e+00,1.21249390e+00,1.20574951e+00,1.19650269e+00,  1.18869019e+00,1.17639160e+00,1.17059326e+00,1.15997314e+00,  1.12103271e+00,1.05706787e+00,1.02429199e+00,1.03384399e+00,  1.04571533e+00,1.05053711e+00,1.05596924e+00,1.04464722e+00,  9.95025635e-01,9.89135742e-01,1.00415039e+00,9.68841553e-01,  9.78912354e-01,9.73297119e-01,9.73937988e-01,9.88220215e-01,  9.61380005e-01,9.05914307e-01,8.74176025e-01,8.54141235e-01,  8.29589844e-01,8.27072144e-01,8.32595825e-01,8.24661255e-01,  6.67999268e-01,5.33584595e-01,5.43518066e-01,5.53619385e-01,  5.66207886e-01,5.90393066e-01,6.28768921e-01,6.83258057e-01,  7.24105835e-01,7.34222412e-01,7.48855591e-01,7.67654419e-01,  7.74200439e-01,7.71728516e-01,7.61276245e-01,7.50579834e-01,  7.48092651e-01,7.44430542e-01,7.36755371e-01,7.28637695e-01,  7.19070435e-01,7.09365845e-01,7.01263428e-01,6.92123413e-01,  6.82510376e-01,6.70318604e-01,6.56890869e-01,6.44592285e-01,  6.37786865e-01,6.35238647e-01,6.27441406e-01,6.17462158e-01,  6.05453491e-01,5.88882446e-01,5.73120117e-01,5.52978516e-01,  4.99626160e-01,4.03236389e-01,3.20945740e-01,2.95944214e-01,  3.06915283e-01,3.31527710e-01,3.60992432e-01,3.50082397e-01,  3.63716125e-01,4.12208557e-01,4.59991455e-01,4.93354797e-01,  5.07492065e-01,5.04974365e-01,4.96047974e-01,4.92233276e-01,  4.99702454e-01,4.98008728e-01,4.84436035e-01,4.79957581e-01,  4.84642029e-01,4.90150452e-01,4.89128113e-01,4.87266541e-01,  4.87525940e-01,4.85481262e-01,4.79400635e-01,4.72587585e-01,  4.63294983e-01,4.44946289e-01,4.11476135e-01,3.76808167e-01,  3.82141113e-01,4.13284302e-01,4.23896790e-01,4.33403015e-01,  4.38415527e-01,4.33708191e-01,4.22401428e-01,4.07791138e-01,  3.90213013e-01,3.74565125e-01,3.64547729e-01,3.46473694e-01,  3.09638977e-01,2.98187256e-01,2.78518677e-01,2.03784943e-01,  1.05432510e-01,4.12912369e-02,2.04396248e-02,1.73983574e-02,  2.02355385e-02,2.90112495e-02,2.49137878e-02,2.72831917e-02,  3.69482040e-02,3.70969772e-02,4.18138504e-02,4.70151901e-02,  6.10313416e-02,7.85160065e-02,9.53292847e-02,1.12386703e-01,  1.13485336e-01,1.13439560e-01,1.25896454e-01,1.42688751e-01,  1.63543701e-01,1.97914124e-01,2.03800201e-01,1.79924011e-01,  1.67747498e-01,1.81159973e-01,1.96086884e-01,2.14096069e-01,  2.41024017e-01,2.62702942e-01,2.68554688e-01,2.73406982e-01,  2.84530640e-01,2.87361145e-01,2.88558960e-01,2.91046143e-01,  2.92045593e-01,2.91282654e-01,2.90992737e-01,2.91725159e-01,  2.90000916e-01,2.86842346e-01,2.83599854e-01,2.75413513e-01,  2.59506226e-01,2.51281738e-01,2.57606506e-01,2.64198303e-01,  2.65632629e-01,2.63229370e-01,2.52738953e-01,2.48142242e-01,  2.51281738e-01,2.58529663e-01,2.62855530e-01,2.66082764e-01,  2.67219543e-01,2.67372131e-01,2.67356873e-01,2.67425537e-01,  2.67623901e-01,2.65411377e-01,2.60559082e-01,2.54219055e-01,  2.49824524e-01,2.44773865e-01,2.39650726e-01,2.34233856e-01,  2.29652405e-01,2.24750519e-01,2.22351074e-01],
        [4.36019897e-01,4.53117371e-01,4.73052979e-01,5.56640625e-01,  5.90530396e-01,6.02386475e-01,6.04385376e-01,5.97976685e-01,  6.52862549e-01,7.44766235e-01,9.90875244e-01,1.12149048e+00,  1.17974854e+00,1.23144531e+00,1.23959351e+00,1.20675659e+00,  1.18957520e+00,1.28784180e+00,1.40173340e+00,1.50936890e+00,  1.58959961e+00,1.62683105e+00,1.65371704e+00,1.65487671e+00,  1.66183472e+00,1.69454956e+00,1.70758057e+00,1.64666748e+00,  1.66003418e+00,1.68865967e+00,1.66488647e+00,1.67242432e+00,  1.66836548e+00,1.60702515e+00,1.60943604e+00,1.64505005e+00,  1.67031860e+00,1.67596436e+00,1.66259766e+00,1.66629028e+00,  1.66506958e+00,1.64941406e+00,1.62185669e+00,1.60852051e+00,  1.59838867e+00,1.59432983e+00,1.60754395e+00,1.60494995e+00,  1.58026123e+00,1.58358765e+00,1.56915283e+00,1.56430054e+00,  1.54901123e+00,1.53311157e+00,1.53573608e+00,1.50952148e+00,  1.49481201e+00,1.51480103e+00,1.52001953e+00,1.51394653e+00,  1.49758911e+00,1.45953369e+00,1.49130249e+00,1.51608276e+00,  1.50973511e+00,1.50024414e+00,1.47225952e+00,1.35928345e+00,  1.31658936e+00,1.38687134e+00,1.40173340e+00,1.40188599e+00,  1.39074707e+00,1.33392334e+00,1.27728271e+00,1.27667236e+00,  1.29086304e+00,1.31079102e+00,1.31033325e+00,1.30581665e+00,  1.28933716e+00,1.17141724e+00,8.07815552e-01,8.71032715e-01,  1.14700317e+00,1.21133423e+00,1.21264648e+00,1.20309448e+00,  1.18844604e+00,1.18395996e+00,1.17315674e+00,1.16094971e+00,  1.12799072e+00,1.05947876e+00,1.03475952e+00,1.04345703e+00,  1.04452515e+00,1.05914307e+00,1.05740356e+00,1.03686523e+00,  1.00164795e+00,9.81872559e-01,9.96124268e-01,9.76577759e-01,  9.73937988e-01,9.89105225e-01,9.80438232e-01,9.78515625e-01,  9.82254028e-01,9.02847290e-01,8.58673096e-01,8.66134644e-01,  8.26324463e-01,8.41522217e-01,8.51974487e-01,7.96630859e-01,  6.61117554e-01,5.36682129e-01,5.46783447e-01,5.56854248e-01,  5.69076538e-01,5.93276978e-01,6.31790161e-01,6.86645508e-01,  7.27264404e-01,7.37182617e-01,7.51586914e-01,7.70065308e-01,  7.76596069e-01,7.74291992e-01,7.63641357e-01,7.52761841e-01,  7.50183105e-01,7.46582031e-01,7.38937378e-01,7.30941772e-01,  7.21435547e-01,7.11593628e-01,7.03445435e-01,6.94198608e-01,  6.84570312e-01,6.72515869e-01,6.59194946e-01,6.46896362e-01,  6.39999390e-01,6.37252808e-01,6.29455566e-01,6.19369507e-01,  6.07452393e-01,5.90850830e-01,5.75271606e-01,5.55068970e-01,  5.01708984e-01,4.05372620e-01,3.23020935e-01,2.98095703e-01,  3.09097290e-01,3.33633423e-01,3.63067627e-01,3.52218628e-01,  3.65837097e-01,4.14299011e-01,4.62013245e-01,4.95407104e-01,  5.09460449e-01,5.06988525e-01,4.98001099e-01,4.94064331e-01,  5.01388550e-01,4.99618530e-01,4.86129761e-01,4.81666565e-01,  4.86312866e-01,4.91851807e-01,4.90814209e-01,4.88731384e-01,  4.89120483e-01,4.87197876e-01,4.80926514e-01,4.74052429e-01,  4.64813232e-01,4.46670532e-01,4.13414001e-01,3.79089355e-01,  3.84330750e-01,4.15184021e-01,4.25491333e-01,4.34829712e-01,  4.39758301e-01,4.35112000e-01,4.23828125e-01,4.09164429e-01,  3.91624451e-01,3.75999451e-01,3.65997314e-01,3.48052979e-01,  3.11164856e-01,2.99652100e-01,2.79876709e-01,2.05085754e-01,  1.06353760e-01,4.17184830e-02,2.06117630e-02,1.74531937e-02,  2.04391479e-02,2.94442177e-02,2.52881050e-02,2.76107788e-02,  3.73268127e-02,3.75480652e-02,4.23173904e-02,4.76427078e-02,  6.17465973e-02,7.92808533e-02,9.62524414e-02,1.13311768e-01,  1.14406586e-01,1.14604950e-01,1.27067566e-01,1.43836975e-01,  1.64722443e-01,1.98993683e-01,2.05005646e-01,1.81079865e-01,  1.68704987e-01,1.82216644e-01,1.97181702e-01,2.15175629e-01,  2.42073059e-01,2.63648987e-01,2.69584656e-01,2.74353027e-01,  2.85240173e-01,2.88169861e-01,2.89443970e-01,2.91900635e-01,  2.92755127e-01,2.91992188e-01,2.91625977e-01,2.92297363e-01,  2.90657043e-01,2.87559509e-01,2.84339905e-01,2.76374817e-01,  2.60444641e-01,2.52090454e-01,2.58262634e-01,2.64762878e-01,  2.66311646e-01,2.63893127e-01,2.53463745e-01,2.48832703e-01,  2.51976013e-01,2.59246826e-01,2.63603210e-01,2.66883850e-01,  2.67784119e-01,2.67654419e-01,2.67623901e-01,2.67807007e-01,  2.68058777e-01,2.65632629e-01,2.60955811e-01,2.55027771e-01,  2.50183105e-01,2.45059967e-01,2.39807129e-01,2.34752655e-01,  2.30278015e-01,2.25185394e-01,2.22766876e-01],
        [4.44908142e-01,4.59449768e-01,4.81750488e-01,5.62850952e-01,  5.99792480e-01,6.10824585e-01,6.11785889e-01,6.08871460e-01,  6.60888672e-01,7.55004883e-01,1.00521851e+00,1.13589478e+00,  1.19528198e+00,1.24468994e+00,1.25247192e+00,1.21987915e+00,  1.20050049e+00,1.30157471e+00,1.41400146e+00,1.52325439e+00,  1.60382080e+00,1.63967896e+00,1.66744995e+00,1.66656494e+00,  1.67391968e+00,1.70703125e+00,1.71804810e+00,1.65682983e+00,  1.66979980e+00,1.69851685e+00,1.67520142e+00,1.68173218e+00,  1.67803955e+00,1.61569214e+00,1.61773682e+00,1.65420532e+00,  1.67794800e+00,1.68383789e+00,1.67007446e+00,1.67257690e+00,  1.67266846e+00,1.65536499e+00,1.62808228e+00,1.61535645e+00,  1.60516357e+00,1.60269165e+00,1.61389160e+00,1.61004639e+00,  1.58627319e+00,1.58798218e+00,1.57553101e+00,1.57000732e+00,  1.55249023e+00,1.53921509e+00,1.54003906e+00,1.51376343e+00,  1.49935913e+00,1.51748657e+00,1.52532959e+00,1.51751709e+00,  1.49978638e+00,1.46270752e+00,1.49395752e+00,1.51980591e+00,  1.51342773e+00,1.50155640e+00,1.47500610e+00,1.36080933e+00,  1.31854248e+00,1.38989258e+00,1.40197754e+00,1.40332031e+00,  1.39303589e+00,1.33435059e+00,1.27969360e+00,1.27807617e+00,  1.29302979e+00,1.31549072e+00,1.31051636e+00,1.30920410e+00,  1.29492188e+00,1.16906738e+00,8.10104370e-01,8.71673584e-01,  1.14401245e+00,1.21298218e+00,1.21032715e+00,1.20547485e+00,  1.18966675e+00,1.17926025e+00,1.17337036e+00,1.15921021e+00,  1.12646484e+00,1.06503296e+00,1.03097534e+00,1.04022217e+00,  1.05029297e+00,1.05718994e+00,1.06018066e+00,1.04220581e+00,  9.98138428e-01,9.82376099e-01,9.96063232e-01,9.78088379e-01,  9.90997314e-01,9.85153198e-01,9.81994629e-01,9.85946655e-01,  9.68551636e-01,9.15298462e-01,8.59817505e-01,8.53088379e-01,  8.28308105e-01,8.25958252e-01,8.44024658e-01,8.23989868e-01,  6.70043945e-01,5.39779663e-01,5.49728394e-01,5.59906006e-01,  5.72174072e-01,5.96099854e-01,6.34262085e-01,6.88247681e-01,  7.28179932e-01,7.37991333e-01,7.52212524e-01,7.70431519e-01,  7.76702881e-01,7.74185181e-01,7.63717651e-01,7.52731323e-01,  7.50030518e-01,7.46566772e-01,7.39044189e-01,7.30926514e-01,  7.21237183e-01,7.11578369e-01,7.03552246e-01,6.94503784e-01,  6.84753418e-01,6.72775269e-01,6.59393311e-01,6.47109985e-01,  6.40121460e-01,6.37420654e-01,6.29577637e-01,6.19430542e-01,  6.07620239e-01,5.91156006e-01,5.75744629e-01,5.55847168e-01,  5.02838135e-01,4.07104492e-01,3.25195312e-01,3.00430298e-01,  3.11332703e-01,3.35670471e-01,3.65089417e-01,3.54484558e-01,  3.67866516e-01,4.15863037e-01,4.63127136e-01,4.96116638e-01,  5.09979248e-01,5.07431030e-01,4.98374939e-01,4.94506836e-01,  5.01770020e-01,4.99961853e-01,4.86495972e-01,4.81994629e-01,  4.86557007e-01,4.91928101e-01,4.90921021e-01,4.88960266e-01,  4.89120483e-01,4.87182617e-01,4.80911255e-01,4.74098206e-01,  4.65065002e-01,4.47128296e-01,4.14237976e-01,3.80126953e-01,  3.85360718e-01,4.15733337e-01,4.25735474e-01,4.34906006e-01,  4.39842224e-01,4.35203552e-01,4.23942566e-01,4.09523010e-01,  3.92181396e-01,3.76594543e-01,3.66699219e-01,3.48869324e-01,  3.12347412e-01,3.00819397e-01,2.81127930e-01,2.06542969e-01,  1.07931137e-01,4.28705215e-02,2.14438438e-02,1.82981491e-02,  2.12941170e-02,3.05714607e-02,2.62489319e-02,2.85887718e-02,  3.86104584e-02,3.86428833e-02,4.35171127e-02,4.89273071e-02,  6.30989075e-02,8.07476044e-02,9.77478027e-02,1.14824295e-01,  1.15875244e-01,1.16065979e-01,1.28612518e-01,1.45336151e-01,  1.66229248e-01,2.00473785e-01,2.06329346e-01,1.82441711e-01,  1.70082092e-01,1.83582306e-01,1.98543549e-01,2.16312408e-01,  2.42954254e-01,2.64343262e-01,2.70042419e-01,2.74734497e-01,  2.85507202e-01,2.88452148e-01,2.89756775e-01,2.91999817e-01,  2.92884827e-01,2.92106628e-01,2.91763306e-01,2.92457581e-01,  2.90771484e-01,2.87620544e-01,2.84355164e-01,2.76374817e-01,  2.60765076e-01,2.52593994e-01,2.58811951e-01,2.65220642e-01,  2.66563416e-01,2.64167786e-01,2.53990173e-01,2.49347687e-01,  2.52403259e-01,2.59590149e-01,2.63732910e-01,2.67097473e-01,  2.68020630e-01,2.68089294e-01,2.68409729e-01,2.68753052e-01,  2.69104004e-01,2.67013550e-01,2.62748718e-01,2.57179260e-01,  2.52388000e-01,2.47428894e-01,2.42675781e-01,2.37678528e-01,  2.32604980e-01,2.27710724e-01,2.26066589e-01],
        [4.56314087e-01,4.68421936e-01,4.92355347e-01,5.73577881e-01,  6.09069824e-01,6.22283936e-01,6.21170044e-01,6.18133545e-01,  6.70776367e-01,7.67349243e-01,1.02111816e+00,1.14920044e+00,  1.20874023e+00,1.25802612e+00,1.26541138e+00,1.23086548e+00,  1.21087646e+00,1.31393433e+00,1.42687988e+00,1.53594971e+00,  1.61706543e+00,1.65228271e+00,1.68057251e+00,1.67892456e+00,  1.68649292e+00,1.72067261e+00,1.73062134e+00,1.66784668e+00,  1.68173218e+00,1.70980835e+00,1.68551636e+00,1.69180298e+00,  1.68722534e+00,1.62466431e+00,1.62756348e+00,1.66387939e+00,  1.68786621e+00,1.69256592e+00,1.67797852e+00,1.68075562e+00,  1.68078613e+00,1.66342163e+00,1.63629150e+00,1.62399292e+00,  1.61260986e+00,1.61044312e+00,1.62286377e+00,1.61785889e+00,  1.59341431e+00,1.59561157e+00,1.58255005e+00,1.57815552e+00,  1.55929565e+00,1.54608154e+00,1.54660034e+00,1.51843262e+00,  1.50625610e+00,1.52368164e+00,1.53146362e+00,1.52456665e+00,  1.50454712e+00,1.46795654e+00,1.49905396e+00,1.52420044e+00,  1.51766968e+00,1.50610352e+00,1.47952271e+00,1.36486816e+00,  1.32388306e+00,1.39611816e+00,1.40841675e+00,1.40823364e+00,  1.39624023e+00,1.33648682e+00,1.28436279e+00,1.28350830e+00,  1.29736328e+00,1.31671143e+00,1.31195068e+00,1.31008911e+00,  1.29513550e+00,1.16876221e+00,8.10211182e-01,8.80081177e-01,  1.15173340e+00,1.21859741e+00,1.21496582e+00,1.20791626e+00,  1.19314575e+00,1.18176270e+00,1.17730713e+00,1.16348267e+00,  1.12515259e+00,1.06448364e+00,1.03564453e+00,1.04382324e+00,  1.05410767e+00,1.05780029e+00,1.05972290e+00,1.04495239e+00,  9.97772217e-01,9.87915039e-01,1.00271606e+00,9.76516724e-01,  9.84466553e-01,9.84695435e-01,9.83596802e-01,9.88479614e-01,  9.69970703e-01,9.21600342e-01,8.74908447e-01,8.62091064e-01,  8.44467163e-01,8.40988159e-01,8.40438843e-01,8.32183838e-01,  6.75903320e-01,5.42144775e-01,5.51895142e-01,5.61965942e-01,  5.74096680e-01,5.98114014e-01,6.35986328e-01,6.89651489e-01,  7.29263306e-01,7.39089966e-01,7.53051758e-01,7.70904541e-01,  7.77023315e-01,7.74505615e-01,7.63809204e-01,7.53067017e-01,  7.50320435e-01,7.46810913e-01,7.39212036e-01,7.31124878e-01,  7.21420288e-01,7.11700439e-01,7.03735352e-01,6.94580078e-01,  6.85028076e-01,6.72744751e-01,6.59774780e-01,6.47567749e-01,  6.40579224e-01,6.37680054e-01,6.29821777e-01,6.19750977e-01,  6.07757568e-01,5.91491699e-01,5.76324463e-01,5.56701660e-01,  5.03921509e-01,4.08706665e-01,3.26980591e-01,3.02230835e-01,  3.13087463e-01,3.37265015e-01,3.66554260e-01,3.55934143e-01,  3.69346619e-01,4.17144775e-01,4.64294434e-01,4.96978760e-01,  5.10803223e-01,5.08071899e-01,4.99061584e-01,4.95162964e-01,  5.02319336e-01,5.00457764e-01,4.86938477e-01,4.82475281e-01,  4.86976624e-01,4.92477417e-01,4.91310120e-01,4.89181519e-01,  4.89341736e-01,4.87274170e-01,4.80979919e-01,4.74266052e-01,  4.65270996e-01,4.47494507e-01,4.14939880e-01,3.81050110e-01,  3.86169434e-01,4.16305542e-01,4.25979614e-01,4.35089111e-01,  4.39949036e-01,4.35272217e-01,4.24209595e-01,4.09797668e-01,  3.92471313e-01,3.76884460e-01,3.67172241e-01,3.49395752e-01,  3.13148499e-01,3.01628113e-01,2.81921387e-01,2.07473755e-01,  1.08655930e-01,4.30402756e-02,2.11915970e-02,1.79662704e-02,  2.10890770e-02,3.05743217e-02,2.61735916e-02,2.86383629e-02,  3.88765335e-02,3.89423370e-02,4.37288284e-02,4.92334366e-02,  6.35604858e-02,8.13808441e-02,9.84535217e-02,1.15667343e-01,  1.16842270e-01,1.17042542e-01,1.29497528e-01,1.46183014e-01,  1.66908264e-01,2.01072693e-01,2.06939697e-01,1.83109283e-01,  1.70845032e-01,1.84299469e-01,1.99119568e-01,2.16892242e-01,  2.43373871e-01,2.64617920e-01,2.70111084e-01,2.74757385e-01,  2.85537720e-01,2.88314819e-01,2.89428711e-01,2.91839600e-01,  2.92716980e-01,2.91831970e-01,2.91366577e-01,2.92015076e-01,  2.90298462e-01,2.87162781e-01,2.83943176e-01,2.76008606e-01,  2.60459900e-01,2.52326965e-01,2.58438110e-01,2.64686584e-01,  2.65960693e-01,2.63534546e-01,2.53356934e-01,2.48817444e-01,  2.51823425e-01,2.58911133e-01,2.63191223e-01,2.66479492e-01,  2.67395020e-01,2.67547607e-01,2.67486572e-01,2.67501831e-01,  2.67471313e-01,2.65090942e-01,2.60604858e-01,2.54745483e-01,  2.49450684e-01,2.43816376e-01,2.38445282e-01,2.33421326e-01,  2.29427338e-01,2.24246979e-01,2.20676422e-01],
        [4.63554382e-01,4.74250793e-01,5.00488281e-01,5.81848145e-01,  6.16027832e-01,6.30859375e-01,6.27059937e-01,6.27182007e-01,  6.79824829e-01,7.76840210e-01,1.03384399e+00,1.15930176e+00,  1.21890259e+00,1.26721191e+00,1.27304077e+00,1.23776245e+00,  1.21914673e+00,1.32232666e+00,1.43624878e+00,1.54486084e+00,  1.62631226e+00,1.66098022e+00,1.68905640e+00,1.68698120e+00,  1.69381714e+00,1.72827148e+00,1.73767090e+00,1.67456055e+00,  1.68902588e+00,1.71578979e+00,1.69229126e+00,1.69848633e+00,  1.69287109e+00,1.62927246e+00,1.63247681e+00,1.66940308e+00,  1.69305420e+00,1.69692993e+00,1.68298340e+00,1.68508911e+00,  1.68554688e+00,1.66760254e+00,1.63967896e+00,1.62844849e+00,  1.61715698e+00,1.61502075e+00,1.62722778e+00,1.62057495e+00,  1.59774780e+00,1.59918213e+00,1.58651733e+00,1.58206177e+00,  1.56262207e+00,1.55038452e+00,1.55053711e+00,1.52194214e+00,  1.51068115e+00,1.52694702e+00,1.53317261e+00,1.52664185e+00,  1.50686646e+00,1.47082520e+00,1.50213623e+00,1.52545166e+00,  1.52041626e+00,1.50881958e+00,1.48126221e+00,1.36694336e+00,  1.32650757e+00,1.39929199e+00,1.40942383e+00,1.40850830e+00,  1.39782715e+00,1.33630371e+00,1.28448486e+00,1.28457642e+00,  1.29727173e+00,1.32119751e+00,1.31378174e+00,1.30999756e+00,  1.29809570e+00,1.16732788e+00,8.10180664e-01,8.86383057e-01,  1.15142822e+00,1.21868896e+00,1.21371460e+00,1.20678711e+00,  1.19552612e+00,1.18145752e+00,1.17840576e+00,1.16488647e+00,  1.12387085e+00,1.06402588e+00,1.03677368e+00,1.04275513e+00,  1.05700684e+00,1.05935669e+00,1.06378174e+00,1.05038452e+00,  1.00009155e+00,9.88525391e-01,1.00582886e+00,9.71954346e-01,  9.88861084e-01,9.87182617e-01,9.79095459e-01,9.95574951e-01,  9.70153809e-01,9.20120239e-01,8.81118774e-01,8.63677979e-01,  8.50555420e-01,8.44177246e-01,8.39233398e-01,8.33816528e-01,  6.77413940e-01,5.44113159e-01,5.54000854e-01,5.63903809e-01,  5.75973511e-01,5.99533081e-01,6.37359619e-01,6.90444946e-01,  7.29766846e-01,7.39425659e-01,7.53311157e-01,7.70950317e-01,  7.76931763e-01,7.74230957e-01,7.63580322e-01,7.52807617e-01,  7.50045776e-01,7.46536255e-01,7.38861084e-01,7.30819702e-01,  7.21084595e-01,7.11410522e-01,7.03430176e-01,6.94320679e-01,  6.84661865e-01,6.72836304e-01,6.59698486e-01,6.47415161e-01,  6.40472412e-01,6.37557983e-01,6.29684448e-01,6.19644165e-01,  6.07879639e-01,5.91613770e-01,5.76492310e-01,5.57006836e-01,  5.04547119e-01,4.09690857e-01,3.28475952e-01,3.03863525e-01,  3.14651489e-01,3.38729858e-01,3.68034363e-01,3.57322693e-01,  3.70605469e-01,4.18113708e-01,4.64904785e-01,4.97398376e-01,  5.11032104e-01,5.08285522e-01,4.99336243e-01,4.95521545e-01,  5.02532959e-01,5.00640869e-01,4.87251282e-01,4.82795715e-01,  4.87167358e-01,4.92584229e-01,4.91416931e-01,4.89189148e-01,  4.89295959e-01,4.87289429e-01,4.81063843e-01,4.74334717e-01,  4.65454102e-01,4.47914124e-01,4.15512085e-01,3.81980896e-01,  3.87039185e-01,4.16725159e-01,4.26246643e-01,4.35112000e-01,  4.39956665e-01,4.35340881e-01,4.24171448e-01,4.09881592e-01,  3.92662048e-01,3.77365112e-01,3.67645264e-01,3.49960327e-01,  3.13941956e-01,3.02459717e-01,2.82783508e-01,2.08564758e-01,  1.09786987e-01,4.40073013e-02,2.20379829e-02,1.87592506e-02,  2.19197273e-02,3.16133499e-02,2.70848274e-02,2.95104980e-02,  3.99036407e-02,3.99827957e-02,4.48217392e-02,5.03063202e-02,  6.46667480e-02,8.25271606e-02,9.96551514e-02,1.16895676e-01,  1.17998123e-01,1.18291855e-01,1.30847931e-01,1.47445679e-01,  1.68140411e-01,2.02228546e-01,2.08049774e-01,1.84215546e-01,  1.71970367e-01,1.85352325e-01,2.00134277e-01,2.17739105e-01,  2.44091034e-01,2.65159607e-01,2.70637512e-01,2.75207520e-01,  2.85896301e-01,2.88635254e-01,2.89741516e-01,2.92053223e-01,  2.92984009e-01,2.92121887e-01,2.91763306e-01,2.92289734e-01,  2.90542603e-01,2.87399292e-01,2.84111023e-01,2.76359558e-01,  2.60902405e-01,2.52807617e-01,2.58880615e-01,2.64991760e-01,  2.66258240e-01,2.64007568e-01,2.54142761e-01,2.49713898e-01,  2.52593994e-01,2.59597778e-01,2.63725281e-01,2.66998291e-01,  2.68081665e-01,2.68226624e-01,2.68447876e-01,2.68737793e-01,  2.69012451e-01,2.67044067e-01,2.63053894e-01,2.57507324e-01,  2.52601624e-01,2.47901917e-01,2.43076324e-01,2.38353729e-01,  2.34058380e-01,2.29187012e-01,2.26570129e-01],
        [4.68811035e-01,4.79583740e-01,5.06088257e-01,5.91125488e-01,  6.24084473e-01,6.37802124e-01,6.33193970e-01,6.30722046e-01,  6.86004639e-01,7.85140991e-01,1.04098511e+00,1.16732788e+00,  1.22711182e+00,1.27459717e+00,1.27938843e+00,1.24319458e+00,  1.22509766e+00,1.32901001e+00,1.44314575e+00,1.55252075e+00,  1.63317871e+00,1.66806030e+00,1.69613647e+00,1.69381714e+00,  1.70062256e+00,1.73419189e+00,1.74264526e+00,1.67984009e+00,  1.69512939e+00,1.72155762e+00,1.69659424e+00,1.70388794e+00,  1.69772339e+00,1.63372803e+00,1.63778687e+00,1.67419434e+00,  1.69760132e+00,1.70138550e+00,1.68725586e+00,1.68969727e+00,  1.68939209e+00,1.67150879e+00,1.64410400e+00,1.63208008e+00,  1.62130737e+00,1.61981201e+00,1.63211060e+00,1.62530518e+00,  1.60150146e+00,1.60336304e+00,1.59115601e+00,1.58624268e+00,  1.56817627e+00,1.55471802e+00,1.55456543e+00,1.52630615e+00,  1.51467896e+00,1.53125000e+00,1.53610229e+00,1.53024292e+00,  1.51046753e+00,1.47311401e+00,1.50436401e+00,1.52783203e+00,  1.52175903e+00,1.51037598e+00,1.48257446e+00,1.36926270e+00,  1.33068848e+00,1.40106201e+00,1.41210938e+00,1.41110229e+00,  1.40100098e+00,1.34039307e+00,1.28823853e+00,1.28903198e+00,  1.30227661e+00,1.32125854e+00,1.31665039e+00,1.31341553e+00,  1.29702759e+00,1.16793823e+00,8.11279297e-01,8.88580322e-01,  1.15399170e+00,1.21978760e+00,1.21859741e+00,1.20846558e+00,  1.19500732e+00,1.18493652e+00,1.17861938e+00,1.16848755e+00,  1.12976074e+00,1.06689453e+00,1.04129028e+00,1.04959106e+00,  1.06097412e+00,1.06787109e+00,1.06729126e+00,1.05477905e+00,  1.00369263e+00,9.88555908e-01,1.00399780e+00,9.81796265e-01,  9.92797852e-01,9.91088867e-01,9.82574463e-01,9.89501953e-01,  9.72549438e-01,9.15832520e-01,8.80889893e-01,8.70559692e-01,  8.51669312e-01,8.49609375e-01,8.47274780e-01,8.29208374e-01,  6.77001953e-01,5.47485352e-01,5.57235718e-01,5.67031860e-01,  5.78857422e-01,6.02172852e-01,6.39450073e-01,6.91909790e-01,  7.30316162e-01,7.39669800e-01,7.53234863e-01,7.70370483e-01,  7.76000977e-01,7.73254395e-01,7.62649536e-01,7.51724243e-01,  7.49145508e-01,7.45391846e-01,7.37792969e-01,7.29812622e-01,  7.20291138e-01,7.10464478e-01,7.02575684e-01,6.93450928e-01,  6.83975220e-01,6.72058105e-01,6.58935547e-01,6.46759033e-01,  6.39633179e-01,6.36856079e-01,6.29043579e-01,6.19110107e-01,  6.07299805e-01,5.91171265e-01,5.76446533e-01,5.57388306e-01,  5.05416870e-01,4.11392212e-01,3.30848694e-01,3.06396484e-01,  3.17237854e-01,3.40957642e-01,3.70048523e-01,3.59558105e-01,  3.72787476e-01,4.19746399e-01,4.65820312e-01,4.97848511e-01,  5.11138916e-01,5.08407593e-01,4.99374390e-01,4.95574951e-01,  5.02532959e-01,5.00503540e-01,4.87182617e-01,4.82734680e-01,  4.87037659e-01,4.92256165e-01,4.91004944e-01,4.88777161e-01,  4.88800049e-01,4.86656189e-01,4.80506897e-01,4.73823547e-01,  4.65034485e-01,4.47624207e-01,4.15550232e-01,3.82293701e-01,  3.87275696e-01,4.16587830e-01,4.25842285e-01,4.34608459e-01,  4.39476013e-01,4.34906006e-01,4.23904419e-01,4.09721375e-01,  3.92730713e-01,3.77525330e-01,3.67927551e-01,3.50616455e-01,  3.14933777e-01,3.03588867e-01,2.84004211e-01,2.10075378e-01,  1.11385345e-01,4.50048447e-02,2.25605965e-02,1.91679001e-02,  2.24614143e-02,3.24993134e-02,2.78482437e-02,3.03969383e-02,  4.11396027e-02,4.11043167e-02,4.59327698e-02,5.15422821e-02,  6.61544800e-02,8.41445923e-02,1.01282120e-01,1.18530273e-01,  1.19590759e-01,1.19909286e-01,1.32572174e-01,1.49063110e-01,  1.69605255e-01,2.03609467e-01,2.09354401e-01,1.85607910e-01,  1.73427582e-01,1.86771393e-01,2.01351166e-01,2.18719482e-01,  2.44724274e-01,2.65495300e-01,2.70858765e-01,2.75390625e-01,  2.85842896e-01,2.88505554e-01,2.89527893e-01,2.91755676e-01,  2.92602539e-01,2.91755676e-01,2.91366577e-01,2.92037964e-01,  2.90161133e-01,2.86888123e-01,2.83561707e-01,2.75917053e-01,  2.60681152e-01,2.52578735e-01,2.58544922e-01,2.64648438e-01,  2.65869141e-01,2.63580322e-01,2.53860474e-01,2.49465942e-01,  2.52349854e-01,2.59223938e-01,2.63313293e-01,2.66540527e-01,  2.67555237e-01,2.67784119e-01,2.68051147e-01,2.68371582e-01,  2.68798828e-01,2.66746521e-01,2.62573242e-01,2.57102966e-01,  2.52616882e-01,2.48172760e-01,2.43011475e-01,2.38342285e-01,  2.33947754e-01,2.28996277e-01,2.26512909e-01],
        [4.73655701e-01,4.83787537e-01,5.10299683e-01,5.94497681e-01,  6.30096436e-01,6.43371582e-01,6.37207031e-01,6.36260986e-01,  6.90353394e-01,7.89932251e-01,1.04916382e+00,1.17532349e+00,  1.23498535e+00,1.28302002e+00,1.28765869e+00,1.25186157e+00,  1.23251343e+00,1.33706665e+00,1.45068359e+00,1.55938721e+00,  1.64126587e+00,1.67645264e+00,1.70413208e+00,1.70205688e+00,  1.70837402e+00,1.74246216e+00,1.75109863e+00,1.68765259e+00,  1.70236206e+00,1.72787476e+00,1.70394897e+00,1.71102905e+00,  1.70440674e+00,1.64031982e+00,1.64398193e+00,1.68066406e+00,  1.70437622e+00,1.70797729e+00,1.69430542e+00,1.69656372e+00,  1.69613647e+00,1.67849731e+00,1.65032959e+00,1.63903809e+00,  1.62878418e+00,1.62710571e+00,1.63903809e+00,1.63162231e+00,  1.60891724e+00,1.60995483e+00,1.59677124e+00,1.59249878e+00,  1.57333374e+00,1.56008911e+00,1.56036377e+00,1.53167725e+00,  1.52001953e+00,1.53686523e+00,1.54168701e+00,1.53656006e+00,  1.51553345e+00,1.47857666e+00,1.50988770e+00,1.53311157e+00,  1.52740479e+00,1.51535034e+00,1.48709106e+00,1.37423706e+00,  1.33453369e+00,1.40576172e+00,1.41534424e+00,1.41378784e+00,  1.40478516e+00,1.34375000e+00,1.29199219e+00,1.29272461e+00,  1.30435181e+00,1.32513428e+00,1.32040405e+00,1.31604004e+00,  1.30062866e+00,1.17092896e+00,8.16497803e-01,8.95431519e-01,  1.15881348e+00,1.22500610e+00,1.22225952e+00,1.21072388e+00,  1.20135498e+00,1.19012451e+00,1.18334961e+00,1.17098999e+00,  1.12963867e+00,1.06799316e+00,1.04287720e+00,1.04614258e+00,  1.05923462e+00,1.06564331e+00,1.06665039e+00,1.05255127e+00,  1.00149536e+00,9.94888306e-01,1.01254272e+00,9.78988647e-01,  9.99832153e-01,1.00421143e+00,9.86267090e-01,9.96734619e-01,  9.72015381e-01,9.14672852e-01,8.87557983e-01,8.66455078e-01,  8.48342896e-01,8.51119995e-01,8.43368530e-01,8.36975098e-01,  6.85638428e-01,5.50811768e-01,5.60348511e-01,5.70068359e-01,  5.81909180e-01,6.05133057e-01,6.42318726e-01,6.94381714e-01,  7.32513428e-01,7.41744995e-01,7.54974365e-01,7.71957397e-01,  7.77465820e-01,7.74612427e-01,7.63870239e-01,7.53051758e-01,  7.50198364e-01,7.46627808e-01,7.39044189e-01,7.30895996e-01,  7.21191406e-01,7.11425781e-01,7.03460693e-01,6.94488525e-01,  6.85165405e-01,6.73294067e-01,6.60110474e-01,6.48040771e-01,  6.41052246e-01,6.38076782e-01,6.30264282e-01,6.20239258e-01,  6.08413696e-01,5.92346191e-01,5.77865601e-01,5.58944702e-01,  5.07293701e-01,4.13581848e-01,3.33206177e-01,3.08944702e-01,  3.19587708e-01,3.43101501e-01,3.72138977e-01,3.61816406e-01,  3.75000000e-01,4.21760559e-01,4.67514038e-01,4.99328613e-01,  5.12619019e-01,5.09750366e-01,5.00747681e-01,4.96856689e-01,  5.03662109e-01,5.01663208e-01,4.88403320e-01,4.83917236e-01,  4.88143921e-01,4.93293762e-01,4.92019653e-01,4.89700317e-01,  4.89616394e-01,4.87533569e-01,4.81231689e-01,4.74563599e-01,  4.65858459e-01,4.48562622e-01,4.16831970e-01,3.83659363e-01,  3.88580322e-01,4.17694092e-01,4.26605225e-01,4.35310364e-01,  4.40155029e-01,4.35607910e-01,4.24804688e-01,4.10713196e-01,  3.93692017e-01,3.78608704e-01,3.69087219e-01,3.51791382e-01,  3.16383362e-01,3.05030823e-01,2.85377502e-01,2.11570740e-01,  1.12564087e-01,4.54893112e-02,2.24971771e-02,1.89843178e-02,  2.24566460e-02,3.28769684e-02,2.80032158e-02,3.05576324e-02,  4.16297913e-02,4.14981842e-02,4.62684631e-02,5.20191193e-02,  6.67762756e-02,8.49437714e-02,1.02268219e-01,1.19594574e-01,  1.20634079e-01,1.21028900e-01,1.33735657e-01,1.50222778e-01,  1.70696259e-01,2.04692841e-01,2.10342407e-01,1.86714172e-01,  1.74526215e-01,1.87767029e-01,2.02381134e-01,2.19703674e-01,  2.45582581e-01,2.66197205e-01,2.71324158e-01,2.75703430e-01,  2.86163330e-01,2.88749695e-01,2.89710999e-01,2.91908264e-01,  2.92678833e-01,2.91870117e-01,2.91412354e-01,2.91908264e-01,  2.90069580e-01,2.86903381e-01,2.83607483e-01,2.75909424e-01,  2.60749817e-01,2.52655029e-01,2.58438110e-01,2.64457703e-01,  2.65731812e-01,2.63519287e-01,2.53723145e-01,2.49225616e-01,  2.52021790e-01,2.58811951e-01,2.62817383e-01,2.66067505e-01,  2.67173767e-01,2.67211914e-01,2.67295837e-01,2.67280579e-01,  2.67265320e-01,2.64884949e-01,2.60528564e-01,2.54394531e-01,  2.49328613e-01,2.43957520e-01,2.38891602e-01,2.33669281e-01,  2.29175568e-01,2.24277496e-01,2.21370697e-01],
        [4.80064392e-01,4.92500305e-01,5.12695312e-01,6.00357056e-01,  6.33255005e-01,6.47079468e-01,6.45553589e-01,6.38229370e-01,  6.93267822e-01,7.92144775e-01,1.05126953e+00,1.18206787e+00,  1.24078369e+00,1.29006958e+00,1.29370117e+00,1.25671387e+00,  1.23745728e+00,1.33993530e+00,1.45639038e+00,1.56460571e+00,  1.64614868e+00,1.68167114e+00,1.70736694e+00,1.70654297e+00,  1.71234131e+00,1.74575806e+00,1.75567627e+00,1.69046021e+00,  1.70471191e+00,1.73086548e+00,1.70635986e+00,1.71347046e+00,  1.70639038e+00,1.64251709e+00,1.64562988e+00,1.68154907e+00,  1.70642090e+00,1.71026611e+00,1.69653320e+00,1.69934082e+00,  1.69747925e+00,1.68038940e+00,1.65219116e+00,1.64044189e+00,  1.63186646e+00,1.62866211e+00,1.64105225e+00,1.63378906e+00,  1.60867310e+00,1.61190796e+00,1.59841919e+00,1.59317017e+00,  1.57516479e+00,1.55895996e+00,1.56137085e+00,1.53280640e+00,  1.51904297e+00,1.53729248e+00,1.54107666e+00,1.53488159e+00,  1.51510620e+00,1.47674561e+00,1.50906372e+00,1.53280640e+00,  1.52569580e+00,1.51409912e+00,1.48455811e+00,1.37338257e+00,  1.33416748e+00,1.40234375e+00,1.41516113e+00,1.41351318e+00,  1.40213013e+00,1.34408569e+00,1.29022217e+00,1.29226685e+00,  1.30416870e+00,1.32110596e+00,1.31787109e+00,1.31265259e+00,  1.29953003e+00,1.17556763e+00,8.18344116e-01,8.91448975e-01,  1.15832520e+00,1.21905518e+00,1.21832275e+00,1.20922852e+00,  1.19519043e+00,1.18792725e+00,1.17675781e+00,1.16744995e+00,  1.13156128e+00,1.06341553e+00,1.04513550e+00,1.05197144e+00,  1.05499268e+00,1.06625366e+00,1.05950928e+00,1.04830933e+00,  1.01181030e+00,9.83947754e-01,1.00823975e+00,9.83688354e-01,  9.80270386e-01,9.98153687e-01,9.84725952e-01,9.88662720e-01,  9.83322144e-01,9.17526245e-01,8.80157471e-01,8.64303589e-01,  8.32397461e-01,8.47427368e-01,8.53515625e-01,8.23211670e-01,  6.79794312e-01,5.53390503e-01,5.62957764e-01,5.72570801e-01,  5.84350586e-01,6.07559204e-01,6.44271851e-01,6.95709229e-01,  7.33413696e-01,7.42492676e-01,7.55523682e-01,7.72033691e-01,  7.77267456e-01,7.74307251e-01,7.63671875e-01,7.52883911e-01,  7.50259399e-01,7.46582031e-01,7.38967896e-01,7.30865479e-01,  7.21191406e-01,7.11654663e-01,7.03643799e-01,6.94549561e-01,  6.85089111e-01,6.73233032e-01,6.60140991e-01,6.48010254e-01,  6.40914917e-01,6.37954712e-01,6.30142212e-01,6.20315552e-01,  6.08596802e-01,5.92559814e-01,5.78247070e-01,5.59600830e-01,  5.08316040e-01,4.15176392e-01,3.35174561e-01,3.11019897e-01,  3.21632385e-01,3.45008850e-01,3.73939514e-01,3.63647461e-01,  3.76663208e-01,4.23080444e-01,4.68452454e-01,5.00015259e-01,  5.13107300e-01,5.10253906e-01,5.01098633e-01,4.97215271e-01,  5.04013062e-01,5.01892090e-01,4.88616943e-01,4.84222412e-01,  4.88426208e-01,4.93476868e-01,4.92179871e-01,4.89730835e-01,  4.89517212e-01,4.87327576e-01,4.81224060e-01,4.74586487e-01,  4.65950012e-01,4.48738098e-01,4.17190552e-01,3.84231567e-01,  3.89129639e-01,4.17991638e-01,4.26788330e-01,4.35432434e-01,  4.40246582e-01,4.35768127e-01,4.24934387e-01,4.10980225e-01,  3.94058228e-01,3.79081726e-01,3.69667053e-01,3.52485657e-01,  3.17283630e-01,3.05976868e-01,2.86415100e-01,2.12738037e-01,  1.13761902e-01,4.62532043e-02,2.28400230e-02,1.92499161e-02,  2.28109360e-02,3.35245132e-02,2.85806656e-02,3.12366486e-02,  4.25243378e-02,4.22163010e-02,4.70256805e-02,5.29012680e-02,  6.79225922e-02,8.62083435e-02,1.03519440e-01,1.20891571e-01,  1.21849060e-01,1.22156143e-01,1.34891510e-01,1.51432037e-01,  1.71916962e-01,2.05814362e-01,2.11505890e-01,1.87801361e-01,  1.75662994e-01,1.88850403e-01,2.03334808e-01,2.20493317e-01,  2.46215820e-01,2.66616821e-01,2.71614075e-01,2.75886536e-01,  2.86308289e-01,2.88887024e-01,2.89733887e-01,2.91831970e-01,  2.92610168e-01,2.91748047e-01,2.91313171e-01,2.91786194e-01,  2.89886475e-01,2.86689758e-01,2.83386230e-01,2.75749207e-01,  2.60681152e-01,2.52639771e-01,2.58453369e-01,2.64259338e-01,  2.65411377e-01,2.63175964e-01,2.53593445e-01,2.49324799e-01,  2.52059937e-01,2.58758545e-01,2.62695312e-01,2.65983582e-01,  2.66906738e-01,2.67044067e-01,2.67005920e-01,2.67013550e-01,  2.66876221e-01,2.64526367e-01,2.59880066e-01,2.54035950e-01,  2.49347687e-01,2.43938446e-01,2.38849640e-01,2.33764648e-01,  2.28759766e-01,2.23552704e-01,2.20420837e-01],
        [4.85260010e-01,4.97001648e-01,5.17456055e-01,6.05361938e-01,  6.38351440e-01,6.50177002e-01,6.48437500e-01,6.42410278e-01,  6.99340820e-01,7.95669556e-01,1.05636597e+00,1.18884277e+00,  1.24722290e+00,1.29772949e+00,1.30187988e+00,1.26577759e+00,  1.24508667e+00,1.34597778e+00,1.46258545e+00,1.57064819e+00,  1.65255737e+00,1.68844604e+00,1.71359253e+00,1.71224976e+00,  1.71774292e+00,1.75103760e+00,1.76123047e+00,1.69583130e+00,  1.70874023e+00,1.73516846e+00,1.71054077e+00,1.71731567e+00,  1.71020508e+00,1.64660645e+00,1.64862061e+00,1.68463135e+00,  1.71032715e+00,1.71420288e+00,1.70028687e+00,1.70339966e+00,  1.70150757e+00,1.68469238e+00,1.65664673e+00,1.64434814e+00,  1.63491821e+00,1.63186646e+00,1.64385986e+00,1.63705444e+00,  1.61282349e+00,1.61502075e+00,1.59954834e+00,1.59564209e+00,  1.57717896e+00,1.56024170e+00,1.56222534e+00,1.53369141e+00,  1.51947021e+00,1.53704834e+00,1.54003906e+00,1.53527832e+00,  1.51550293e+00,1.47561646e+00,1.50729370e+00,1.53094482e+00,  1.52429199e+00,1.51318359e+00,1.48370361e+00,1.37435913e+00,  1.33215332e+00,1.39999390e+00,1.41491699e+00,1.41152954e+00,  1.40106201e+00,1.34506226e+00,1.29202271e+00,1.29312134e+00,  1.30300903e+00,1.31994629e+00,1.31680298e+00,1.31225586e+00,  1.29876709e+00,1.17507935e+00,8.19107056e-01,8.89297485e-01,  1.15274048e+00,1.21469116e+00,1.21795654e+00,1.20690918e+00,  1.19284058e+00,1.18643188e+00,1.17337036e+00,1.16775513e+00,  1.13320923e+00,1.06649780e+00,1.04132080e+00,1.04446411e+00,  1.05075073e+00,1.06372070e+00,1.05691528e+00,1.04367065e+00,  9.98458862e-01,9.77722168e-01,1.00335693e+00,9.74899292e-01,  9.75753784e-01,9.94125366e-01,9.78683472e-01,9.80606079e-01,  9.74258423e-01,9.08355713e-01,8.78982544e-01,8.77700806e-01,  8.34884644e-01,8.53576660e-01,8.40057373e-01,8.13095093e-01,  6.80664062e-01,5.55633545e-01,5.65231323e-01,5.74752808e-01,  5.86456299e-01,6.09329224e-01,6.45675659e-01,6.96533203e-01,  7.33612061e-01,7.42370605e-01,7.55340576e-01,7.71469116e-01,  7.76550293e-01,7.73712158e-01,7.62908936e-01,7.52044678e-01,  7.49481201e-01,7.45971680e-01,7.38128662e-01,7.30056763e-01,  7.20245361e-01,7.10632324e-01,7.02713013e-01,6.93725586e-01,  6.84326172e-01,6.72637939e-01,6.59576416e-01,6.47476196e-01,  6.40411377e-01,6.37466431e-01,6.29592896e-01,6.19659424e-01,  6.08047485e-01,5.92483521e-01,5.78353882e-01,5.59906006e-01,  5.08941650e-01,4.16404724e-01,3.36959839e-01,3.12919617e-01,  3.23432922e-01,3.46443176e-01,3.75282288e-01,3.65211487e-01,  3.78189087e-01,4.24186707e-01,4.69131470e-01,5.00305176e-01,  5.13259888e-01,5.10147095e-01,5.01007080e-01,4.97070312e-01,  5.03799438e-01,5.01770020e-01,4.88494873e-01,4.84115601e-01,  4.88121033e-01,4.93003845e-01,4.91706848e-01,4.89349365e-01,  4.89128113e-01,4.86877441e-01,4.80598450e-01,4.74029541e-01,  4.65461731e-01,4.48402405e-01,4.17106628e-01,3.84437561e-01,  3.89289856e-01,4.17816162e-01,4.26399231e-01,4.34921265e-01,  4.39720154e-01,4.35287476e-01,4.24713135e-01,4.10812378e-01,  3.93974304e-01,3.79119873e-01,3.69804382e-01,3.52844238e-01,  3.18069458e-01,3.06755066e-01,2.87269592e-01,2.13962555e-01,  1.14997864e-01,4.71258163e-02,2.33426094e-02,1.96762085e-02,  2.33244896e-02,3.43122482e-02,2.91571617e-02,3.19223404e-02,  4.36229706e-02,4.32786942e-02,4.80222702e-02,5.38864136e-02,  6.89258575e-02,8.73394012e-02,1.04728699e-01,1.22131348e-01,  1.22915268e-01,1.23357773e-01,1.36180878e-01,1.52706146e-01,  1.73069000e-01,2.06764221e-01,2.12371826e-01,1.88804626e-01,  1.76685333e-01,1.89762115e-01,2.04196930e-01,2.21084595e-01,  2.46509552e-01,2.66784668e-01,2.71690369e-01,2.75901794e-01,  2.86117554e-01,2.88558960e-01,2.89375305e-01,2.91473389e-01,  2.92213440e-01,2.91290283e-01,2.90748596e-01,2.91213989e-01,  2.89329529e-01,2.86163330e-01,2.82897949e-01,2.75382996e-01,  2.60375977e-01,2.52319336e-01,2.58071899e-01,2.63893127e-01,  2.65090942e-01,2.62908936e-01,2.53150940e-01,2.48878479e-01,  2.51609802e-01,2.58338928e-01,2.62306213e-01,2.65480042e-01,  2.66464233e-01,2.66464233e-01,2.66677856e-01,2.66662598e-01,  2.66548157e-01,2.63862610e-01,2.59407043e-01,2.53807068e-01,  2.48432159e-01,2.43103027e-01,2.37499237e-01,2.33619690e-01,  2.29537964e-01,2.24094391e-01,2.21401215e-01],
        [4.90409851e-01,5.04058838e-01,5.25985718e-01,6.09954834e-01,  6.45797729e-01,6.56402588e-01,6.52328491e-01,6.45767212e-01,  7.01690674e-01,7.98767090e-01,1.06134033e+00,1.19573975e+00,  1.25601196e+00,1.30450439e+00,1.30981445e+00,1.27340698e+00,  1.25106812e+00,1.35247803e+00,1.46826172e+00,1.57785034e+00,  1.65899658e+00,1.69314575e+00,1.72027588e+00,1.71762085e+00,  1.72253418e+00,1.75561523e+00,1.76629639e+00,1.70117188e+00,  1.71185303e+00,1.73953247e+00,1.71533203e+00,1.72100830e+00,  1.71560669e+00,1.65063477e+00,1.65142822e+00,1.68853760e+00,  1.71380615e+00,1.71871948e+00,1.70407104e+00,1.70639038e+00,  1.70571899e+00,1.68820190e+00,1.66009521e+00,1.64761353e+00,  1.63821411e+00,1.63656616e+00,1.64669800e+00,1.64044189e+00,  1.61596680e+00,1.61633301e+00,1.60260010e+00,1.59674072e+00,  1.57928467e+00,1.56411743e+00,1.56320190e+00,1.53613281e+00,  1.52206421e+00,1.53756714e+00,1.54290771e+00,1.53515625e+00,  1.51516724e+00,1.47695923e+00,1.50698853e+00,1.53149414e+00,  1.52410889e+00,1.51248169e+00,1.48556519e+00,1.37551880e+00,  1.33374023e+00,1.40155029e+00,1.41268921e+00,1.41152954e+00,  1.39984131e+00,1.34329224e+00,1.29086304e+00,1.28958130e+00,  1.30212402e+00,1.32037354e+00,1.31365967e+00,1.31207275e+00,  1.29650879e+00,1.17517090e+00,8.25241089e-01,8.88244629e-01,  1.15264893e+00,1.21786499e+00,1.21499634e+00,1.20748901e+00,  1.19436646e+00,1.18179321e+00,1.17517090e+00,1.16229248e+00,  1.13018799e+00,1.06570435e+00,1.03738403e+00,1.04785156e+00,  1.05419922e+00,1.06158447e+00,1.06265259e+00,1.04229736e+00,  9.96673584e-01,9.82177734e-01,1.00250244e+00,9.75677490e-01,  9.79019165e-01,9.85504150e-01,9.78027344e-01,9.85275269e-01,  9.70336914e-01,9.18502808e-01,8.71887207e-01,8.67324829e-01,  8.50158691e-01,8.47640991e-01,8.44238281e-01,8.27178955e-01,  6.83364868e-01,5.56869507e-01,5.66406250e-01,5.75927734e-01,  5.87585449e-01,6.10137939e-01,6.46682739e-01,6.97692871e-01,  7.35107422e-01,7.43789673e-01,7.56774902e-01,7.73101807e-01,  7.78259277e-01,7.75268555e-01,7.64617920e-01,7.53814697e-01,  7.51220703e-01,7.47497559e-01,7.39715576e-01,7.31460571e-01,  7.21786499e-01,7.12036133e-01,7.04147339e-01,6.95281982e-01,  6.85943604e-01,6.74255371e-01,6.61193848e-01,6.49002075e-01,  6.41830444e-01,6.38824463e-01,6.31011963e-01,6.20986938e-01,  6.09451294e-01,5.93566895e-01,5.79360962e-01,5.60882568e-01,  5.09902954e-01,4.17221069e-01,3.37600708e-01,3.13476562e-01,  3.24012756e-01,3.47190857e-01,3.76113892e-01,3.65974426e-01,  3.78982544e-01,4.25048828e-01,4.70077515e-01,5.01373291e-01,  5.14282227e-01,5.11184692e-01,5.02182007e-01,4.98291016e-01,  5.04974365e-01,5.02868652e-01,4.89585876e-01,4.85183716e-01,  4.89242554e-01,4.94232178e-01,4.92935181e-01,4.90539551e-01,  4.90303040e-01,4.87998962e-01,4.81773376e-01,4.75219727e-01,  4.66735840e-01,4.49836731e-01,4.18731689e-01,3.86222839e-01,  3.91014099e-01,4.19296265e-01,4.27772522e-01,4.36180115e-01,  4.40864563e-01,4.36408997e-01,4.25704956e-01,4.11788940e-01,  3.94981384e-01,3.80111694e-01,3.70788574e-01,3.53797913e-01,  3.18870544e-01,3.07525635e-01,2.88024902e-01,2.14607239e-01,  1.15657806e-01,4.79354858e-02,2.43415833e-02,2.06661224e-02,  2.43530273e-02,3.52287292e-02,3.01122665e-02,3.27978134e-02,  4.42657471e-02,4.39996719e-02,4.89187241e-02,5.48009872e-02,  6.98356628e-02,8.81443024e-02,1.05501175e-01,1.22926712e-01,  1.23926163e-01,1.24362946e-01,1.37104034e-01,1.53415680e-01,  1.73755646e-01,2.07515717e-01,2.13218689e-01,1.89617157e-01,  1.77528381e-01,1.90662384e-01,2.05097198e-01,2.22133636e-01,  2.47650146e-01,2.67898560e-01,2.72758484e-01,2.76969910e-01,  2.87345886e-01,2.89894104e-01,2.90687561e-01,2.92839050e-01,  2.93670654e-01,2.92770386e-01,2.92381287e-01,2.92739868e-01,  2.90802002e-01,2.87635803e-01,2.84393311e-01,2.76954651e-01,  2.62069702e-01,2.54066467e-01,2.59834290e-01,2.65640259e-01,  2.66784668e-01,2.64694214e-01,2.55187988e-01,2.50984192e-01,  2.53753662e-01,2.60421753e-01,2.64442444e-01,2.67829895e-01,  2.68905640e-01,2.69020081e-01,2.69515991e-01,2.69668579e-01,  2.70317078e-01,2.68875122e-01,2.65296936e-01,2.60665894e-01,  2.56004333e-01,2.52006531e-01,2.48115540e-01,2.43240356e-01,  2.38849640e-01,2.34321594e-01,2.30895996e-01],
        [4.92187500e-01,5.08041382e-01,5.31326294e-01,6.15585327e-01,  6.51229858e-01,6.62094116e-01,6.59759521e-01,6.53915405e-01,  7.09274292e-01,8.06365967e-01,1.06857300e+00,1.20355225e+00,  1.26312256e+00,1.31213379e+00,1.31695557e+00,1.27859497e+00,  1.25592041e+00,1.35949707e+00,1.47433472e+00,1.58413696e+00,  1.66534424e+00,1.70016479e+00,1.72726440e+00,1.72381592e+00,  1.72906494e+00,1.76190186e+00,1.77136230e+00,1.70669556e+00,  1.71759033e+00,1.74478149e+00,1.71978760e+00,1.72543335e+00,  1.71975708e+00,1.65435791e+00,1.65606689e+00,1.69302368e+00,  1.71752930e+00,1.72265625e+00,1.70782471e+00,1.71020508e+00,  1.70922852e+00,1.69143677e+00,1.66390991e+00,1.65179443e+00,  1.64187622e+00,1.63992310e+00,1.64974976e+00,1.64425659e+00,  1.61929321e+00,1.61993408e+00,1.60638428e+00,1.59997559e+00,  1.58203125e+00,1.56637573e+00,1.56530762e+00,1.53863525e+00,  1.52471924e+00,1.53942871e+00,1.54400635e+00,1.53625488e+00,  1.51666260e+00,1.47805786e+00,1.50845337e+00,1.53414917e+00,  1.52508545e+00,1.51312256e+00,1.48672485e+00,1.37518311e+00,  1.33364868e+00,1.40182495e+00,1.41412354e+00,1.41390991e+00,  1.40118408e+00,1.34509277e+00,1.29394531e+00,1.29107666e+00,  1.30368042e+00,1.32135010e+00,1.31552124e+00,1.31335449e+00,  1.29763794e+00,1.17578125e+00,8.26919556e-01,8.91998291e-01,  1.15533447e+00,1.21685791e+00,1.21301270e+00,1.20834351e+00,  1.19354248e+00,1.18493652e+00,1.17864990e+00,1.16259766e+00,  1.12896729e+00,1.06546021e+00,1.03970337e+00,1.05117798e+00,  1.05441284e+00,1.06204224e+00,1.06649780e+00,1.04373169e+00,  9.97528076e-01,9.89456177e-01,1.00128174e+00,9.76013184e-01,  9.81781006e-01,9.83474731e-01,9.84954834e-01,9.86419678e-01,  9.69757080e-01,9.15252686e-01,8.71719360e-01,8.71032715e-01,  8.54248047e-01,8.38333130e-01,8.44390869e-01,8.21899414e-01,  6.81869507e-01,5.57601929e-01,5.67260742e-01,5.76522827e-01,  5.88165283e-01,6.10946655e-01,6.47293091e-01,6.97906494e-01,  7.34680176e-01,7.42980957e-01,7.55798340e-01,7.72064209e-01,  7.77145386e-01,7.74047852e-01,7.63427734e-01,7.52410889e-01,  7.49771118e-01,7.46078491e-01,7.38540649e-01,7.30346680e-01,  7.20565796e-01,7.10815430e-01,7.02957153e-01,6.94000244e-01,  6.84844971e-01,6.73110962e-01,6.60202026e-01,6.48025513e-01,  6.40869141e-01,6.37908936e-01,6.29959106e-01,6.20040894e-01,  6.08459473e-01,5.92773438e-01,5.78765869e-01,5.60424805e-01,  5.09750366e-01,4.17518616e-01,3.38333130e-01,3.14308167e-01,  3.24722290e-01,3.47755432e-01,3.76640320e-01,3.66546631e-01,  3.79394531e-01,4.25300598e-01,4.70024109e-01,5.01007080e-01,  5.13793945e-01,5.10726929e-01,5.01770020e-01,4.97856140e-01,  5.04425049e-01,5.02227783e-01,4.89028931e-01,4.84664917e-01,  4.88662720e-01,4.93530273e-01,4.92080688e-01,4.89707947e-01,  4.89448547e-01,4.87266541e-01,4.81071472e-01,4.74624634e-01,  4.66072083e-01,4.49218750e-01,4.18251038e-01,3.85955811e-01,  3.90693665e-01,4.18807983e-01,4.27108765e-01,4.35447693e-01,  4.40307617e-01,4.35752869e-01,4.25048828e-01,4.11163330e-01,  3.94477844e-01,3.79722595e-01,3.70445251e-01,3.53485107e-01,  3.18832397e-01,3.07556152e-01,2.88055420e-01,2.14916229e-01,  1.15961075e-01,4.78982925e-02,2.39639282e-02,2.02317238e-02,  2.39763260e-02,3.50809097e-02,2.98132896e-02,3.25803757e-02,  4.43153381e-02,4.39300537e-02,4.88443375e-02,5.47180176e-02,  6.98795319e-02,8.83312225e-02,1.05756760e-01,1.23163223e-01,  1.24145508e-01,1.24679565e-01,1.37413025e-01,1.53663635e-01,  1.73957825e-01,2.07687378e-01,2.13325500e-01,1.89712524e-01,  1.77574158e-01,1.90704346e-01,2.05127716e-01,2.21973419e-01,  2.47249603e-01,2.67356873e-01,2.72186279e-01,2.76435852e-01,  2.86582947e-01,2.89024353e-01,2.89855957e-01,2.91893005e-01,  2.92617798e-01,2.91648865e-01,2.91076660e-01,2.91435242e-01,  2.89558411e-01,2.86460876e-01,2.83195496e-01,2.75749207e-01,  2.61016846e-01,2.52960205e-01,2.58552551e-01,2.64228821e-01,  2.65335083e-01,2.63336182e-01,2.53852844e-01,2.49691010e-01,  2.52380371e-01,2.58857727e-01,2.62733459e-01,2.66029358e-01,  2.67044067e-01,2.66998291e-01,2.67280579e-01,2.67532349e-01,  2.67761230e-01,2.65541077e-01,2.60803223e-01,2.55462646e-01,  2.50991821e-01,2.46009827e-01,2.41031647e-01,2.35740662e-01,  2.32250214e-01,2.27413177e-01,2.24449158e-01],
        [4.98664856e-01,5.13122559e-01,5.33157349e-01,6.21200562e-01,  6.53717041e-01,6.67022705e-01,6.64855957e-01,6.56997681e-01,  7.14065552e-01,8.11676025e-01,1.07620239e+00,1.21011353e+00,  1.26855469e+00,1.31756592e+00,1.32113647e+00,1.28256226e+00,  1.26089478e+00,1.36328125e+00,1.48019409e+00,1.58996582e+00,  1.67098999e+00,1.70568848e+00,1.73083496e+00,1.72851562e+00,  1.73406982e+00,1.76638794e+00,1.77639771e+00,1.70944214e+00,  1.72186279e+00,1.74896240e+00,1.72369385e+00,1.73016357e+00,  1.72265625e+00,1.65734863e+00,1.66027832e+00,1.69650269e+00,  1.72167969e+00,1.72531128e+00,1.71002197e+00,1.71286011e+00,  1.71142578e+00,1.69421387e+00,1.66622925e+00,1.65356445e+00,  1.64471436e+00,1.64157104e+00,1.65304565e+00,1.64669800e+00,  1.62051392e+00,1.62377930e+00,1.60888672e+00,1.60290527e+00,  1.58450317e+00,1.56753540e+00,1.56854248e+00,1.54089355e+00,  1.52679443e+00,1.54321289e+00,1.54556274e+00,1.53936768e+00,  1.52014160e+00,1.47866821e+00,1.50967407e+00,1.53344727e+00,  1.52661133e+00,1.51596069e+00,1.48629761e+00,1.37774658e+00,  1.33773804e+00,1.40206909e+00,1.41540527e+00,1.41418457e+00,  1.40246582e+00,1.34603882e+00,1.29080200e+00,1.29196167e+00,  1.30487061e+00,1.32119751e+00,1.31695557e+00,1.31143188e+00,  1.29556274e+00,1.17486572e+00,8.24798584e-01,8.94577026e-01,  1.15496826e+00,1.21511841e+00,1.21691895e+00,1.20816040e+00,  1.19104004e+00,1.18441772e+00,1.17480469e+00,1.16604614e+00,  1.13043213e+00,1.06304932e+00,1.04553223e+00,1.05081177e+00,  1.05294800e+00,1.06521606e+00,1.06204224e+00,1.04696655e+00,  1.00338745e+00,9.81933594e-01,1.01055908e+00,9.79904175e-01,  9.84802246e-01,9.98107910e-01,9.84741211e-01,9.86633301e-01,  9.79309082e-01,9.14566040e-01,8.81927490e-01,8.72283936e-01,  8.31924438e-01,8.48190308e-01,8.53759766e-01,8.16177368e-01,  6.80206299e-01,5.56701660e-01,5.66238403e-01,5.75622559e-01,  5.87371826e-01,6.10244751e-01,6.46423340e-01,6.96868896e-01,  7.33551025e-01,7.42202759e-01,7.54852295e-01,7.70980835e-01,  7.75878906e-01,7.72857666e-01,7.62145996e-01,7.51281738e-01,  7.48718262e-01,7.44964600e-01,7.37380981e-01,7.29339600e-01,  7.19665527e-01,7.09991455e-01,7.01934814e-01,6.93038940e-01,  6.83837891e-01,6.72225952e-01,6.59408569e-01,6.47125244e-01,  6.39892578e-01,6.36871338e-01,6.29135132e-01,6.19293213e-01,  6.07482910e-01,5.91903687e-01,5.77911377e-01,5.59707642e-01,  5.09017944e-01,4.16923523e-01,3.37760925e-01,3.13873291e-01,  3.24356079e-01,3.47328186e-01,3.75946045e-01,3.65875244e-01,  3.78929138e-01,4.24888611e-01,4.69467163e-01,5.00350952e-01,  5.13076782e-01,5.10025024e-01,5.00961304e-01,4.97116089e-01,  5.03692627e-01,5.01571655e-01,4.88395691e-01,4.83947754e-01,  4.87876892e-01,4.92782593e-01,4.91470337e-01,4.89067078e-01,  4.88807678e-01,4.86564636e-01,4.80407715e-01,4.73884583e-01,  4.65431213e-01,4.48738098e-01,4.17945862e-01,3.85864258e-01,  3.90670776e-01,4.18518066e-01,4.26651001e-01,4.34867859e-01,  4.39537048e-01,4.35050964e-01,4.24423218e-01,4.10667419e-01,  3.93951416e-01,3.79104614e-01,3.69979858e-01,3.53080750e-01,  3.18435669e-01,3.07250977e-01,2.87696838e-01,2.14591980e-01,  1.15829468e-01,4.78610992e-02,2.38809586e-02,2.01654434e-02,  2.39901543e-02,3.51438522e-02,2.98361778e-02,3.24945450e-02,  4.41493988e-02,4.38613892e-02,4.88023758e-02,5.47475815e-02,  6.98223114e-02,8.82892609e-02,1.05703354e-01,1.23073578e-01,  1.23996735e-01,1.24498367e-01,1.37271881e-01,1.53549194e-01,  1.73740387e-01,2.07336426e-01,2.12944031e-01,1.89414978e-01,  1.77368164e-01,1.90345764e-01,2.04658508e-01,2.21649170e-01,  2.47001648e-01,2.67005920e-01,2.71789551e-01,2.75947571e-01,  2.86148071e-01,2.88551331e-01,2.89314270e-01,2.91343689e-01,  2.92098999e-01,2.91152954e-01,2.90618896e-01,2.91023254e-01,  2.89253235e-01,2.85980225e-01,2.82707214e-01,2.75207520e-01,  2.60627747e-01,2.52601624e-01,2.58125305e-01,2.63847351e-01,  2.64968872e-01,2.62947083e-01,2.53524780e-01,2.49217987e-01,  2.51876831e-01,2.58369446e-01,2.62237549e-01,2.65632629e-01,  2.66563416e-01,2.66654968e-01,2.66807556e-01,2.66700745e-01,  2.66868591e-01,2.64656067e-01,2.60337830e-01,2.55065918e-01,  2.50350952e-01,2.45597839e-01,2.40390778e-01,2.35416412e-01,  2.32021332e-01,2.26966858e-01,2.24166870e-01],
        [5.07980347e-01,5.16845703e-01,5.38314819e-01,6.28768921e-01,  6.58828735e-01,6.73477173e-01,6.69128418e-01,6.62094116e-01,  7.18750000e-01,8.18374634e-01,1.08557129e+00,1.21621704e+00,  1.27557373e+00,1.32394409e+00,1.32568359e+00,1.28701782e+00,  1.26605225e+00,1.36917114e+00,1.48574829e+00,1.59518433e+00,  1.67587280e+00,1.71029663e+00,1.73565674e+00,1.73294067e+00,  1.73904419e+00,1.77270508e+00,1.78076172e+00,1.71362305e+00,  1.72717285e+00,1.75369263e+00,1.72821045e+00,1.73504639e+00,  1.72680664e+00,1.66156006e+00,1.66436768e+00,1.70053101e+00,  1.72579956e+00,1.72885132e+00,1.71371460e+00,1.71633911e+00,  1.71520996e+00,1.69754028e+00,1.66921997e+00,1.65759277e+00,  1.64828491e+00,1.64578247e+00,1.65747070e+00,1.64889526e+00,  1.62442017e+00,1.62747192e+00,1.61172485e+00,1.60665894e+00,  1.58743286e+00,1.57144165e+00,1.57315063e+00,1.54486084e+00,  1.53195190e+00,1.54754639e+00,1.54861450e+00,1.54244995e+00,  1.52249146e+00,1.48336792e+00,1.51516724e+00,1.53619385e+00,  1.53039551e+00,1.51846313e+00,1.48901367e+00,1.37902832e+00,  1.33996582e+00,1.40786743e+00,1.41940308e+00,1.41540527e+00,  1.40438843e+00,1.34765625e+00,1.29486084e+00,1.29678345e+00,  1.30914307e+00,1.32571411e+00,1.31967163e+00,1.31472778e+00,  1.30090332e+00,1.17440796e+00,8.24783325e-01,9.02374268e-01,  1.15832520e+00,1.21728516e+00,1.21691895e+00,1.20300293e+00,  1.19393921e+00,1.18487549e+00,1.17758179e+00,1.16943359e+00,  1.13104248e+00,1.06509399e+00,1.04489136e+00,1.04809570e+00,  1.05569458e+00,1.06689453e+00,1.05889893e+00,1.04867554e+00,  1.00146484e+00,9.89395142e-01,1.00427246e+00,9.72305298e-01,  9.83963013e-01,9.94827271e-01,9.82269287e-01,9.91928101e-01,  9.72198486e-01,9.09271240e-01,8.77685547e-01,8.65905762e-01,  8.56521606e-01,8.62075806e-01,8.44070435e-01,8.28002930e-01,  6.87530518e-01,5.60714722e-01,5.69931030e-01,5.79147339e-01,  5.90835571e-01,6.13357544e-01,6.49230957e-01,6.99066162e-01,  7.35336304e-01,7.43667603e-01,7.56164551e-01,7.71881104e-01,  7.76672363e-01,7.73605347e-01,7.62939453e-01,7.51846313e-01,  7.49130249e-01,7.45620728e-01,7.38128662e-01,7.29858398e-01,  7.20199585e-01,7.10647583e-01,7.02590942e-01,6.93771362e-01,  6.84387207e-01,6.72760010e-01,6.59957886e-01,6.47796631e-01,  6.40441895e-01,6.37420654e-01,6.29638672e-01,6.19781494e-01,  6.08200073e-01,5.92864990e-01,5.79116821e-01,5.61035156e-01,  5.10665894e-01,4.19136047e-01,3.40492249e-01,3.16825867e-01,  3.27064514e-01,3.49853516e-01,3.78631592e-01,3.68682861e-01,  3.81347656e-01,4.26795959e-01,4.70939636e-01,5.01617432e-01,  5.14160156e-01,5.11047363e-01,5.02059937e-01,4.98184204e-01,  5.04531860e-01,5.02182007e-01,4.89196777e-01,4.84825134e-01,  4.88769531e-01,4.93621826e-01,4.92149353e-01,4.89639282e-01,  4.89303589e-01,4.86976624e-01,4.80842590e-01,4.74555969e-01,  4.66110229e-01,4.49455261e-01,4.18846130e-01,3.86917114e-01,  3.91563416e-01,4.19258118e-01,4.27253723e-01,4.35470581e-01,  4.40055847e-01,4.35585022e-01,4.24980164e-01,4.11376953e-01,  3.94752502e-01,3.80218506e-01,3.71208191e-01,3.54339600e-01,  3.19984436e-01,3.08792114e-01,2.89382935e-01,2.16407776e-01,  1.17582321e-01,4.91409302e-02,2.48594284e-02,2.09465027e-02,  2.49533653e-02,3.65085602e-02,3.09600830e-02,3.37238312e-02,  4.57611084e-02,4.52938080e-02,5.01604080e-02,5.61361313e-02,  7.14626312e-02,9.00077820e-02,1.07385635e-01,1.24891281e-01,  1.25793457e-01,1.26361847e-01,1.39186859e-01,1.55460358e-01,  1.75590515e-01,2.09053040e-01,2.14611053e-01,1.91173553e-01,  1.79164886e-01,1.92085266e-01,2.06417084e-01,2.23102570e-01,  2.48107910e-01,2.68005371e-01,2.72712708e-01,2.76802063e-01,  2.86834717e-01,2.89230347e-01,2.90008545e-01,2.91862488e-01,  2.92686462e-01,2.91786194e-01,2.91252136e-01,2.91679382e-01,  2.89764404e-01,2.86476135e-01,2.83218384e-01,2.75917053e-01,  2.61413574e-01,2.53410339e-01,2.59025574e-01,2.64549255e-01,  2.65518188e-01,2.63519287e-01,2.54295349e-01,2.50160217e-01,  2.52883911e-01,2.59315491e-01,2.63099670e-01,2.66456604e-01,  2.67608643e-01,2.67707825e-01,2.68157959e-01,2.68241882e-01,  2.68432617e-01,2.66632080e-01,2.62847900e-01,2.58384705e-01,  2.54142761e-01,2.49500275e-01,2.44991302e-01,2.40119934e-01,  2.36175537e-01,2.32006073e-01,2.29393005e-01],
        [5.06057739e-01,5.18447876e-01,5.42800903e-01,6.30966187e-01,  6.66000366e-01,6.78314209e-01,6.72698975e-01,6.67648315e-01,  7.21374512e-01,8.23532104e-01,1.09048462e+00,1.22128296e+00,  1.27984619e+00,1.32720947e+00,1.33071899e+00,1.29071045e+00,  1.26937866e+00,1.37448120e+00,1.48989868e+00,1.60028076e+00,  1.68093872e+00,1.71569824e+00,1.74157715e+00,1.73724365e+00,  1.74383545e+00,1.77716064e+00,1.78479004e+00,1.71875000e+00,  1.73190308e+00,1.75784302e+00,1.73214722e+00,1.73934937e+00,  1.73159790e+00,1.66415405e+00,1.66751099e+00,1.70492554e+00,  1.72894287e+00,1.73193359e+00,1.71707153e+00,1.71917725e+00,  1.71838379e+00,1.70001221e+00,1.67263794e+00,1.66113281e+00,  1.65127563e+00,1.65078735e+00,1.66107178e+00,1.65277100e+00,  1.62890625e+00,1.62954712e+00,1.61593628e+00,1.60992432e+00,  1.59054565e+00,1.57525635e+00,1.57476807e+00,1.54705811e+00,  1.53320312e+00,1.54888916e+00,1.55224609e+00,1.54507446e+00,  1.52471924e+00,1.48663330e+00,1.51739502e+00,1.54010010e+00,  1.53237915e+00,1.52020264e+00,1.49261475e+00,1.38150024e+00,  1.34417725e+00,1.41107178e+00,1.41995239e+00,1.41897583e+00,  1.40747070e+00,1.34930420e+00,1.29931641e+00,1.29855347e+00,  1.31030273e+00,1.32708740e+00,1.31893921e+00,1.31704712e+00,  1.30160522e+00,1.17309570e+00,8.26965332e-01,9.04693604e-01,  1.16107178e+00,1.22164917e+00,1.21743774e+00,1.21081543e+00,  1.19805908e+00,1.18716431e+00,1.18167114e+00,1.16845703e+00,  1.13211060e+00,1.07357788e+00,1.04293823e+00,1.05084229e+00,  1.06146240e+00,1.06564331e+00,1.07019043e+00,1.05191040e+00,  1.00347900e+00,9.90341187e-01,1.00265503e+00,9.78042603e-01,  9.86892700e-01,9.88037109e-01,9.84329224e-01,9.88876343e-01,  9.76730347e-01,9.18899536e-01,8.84704590e-01,8.65417480e-01,  8.50616455e-01,8.51959229e-01,8.53439331e-01,8.40927124e-01,  6.88720703e-01,5.60028076e-01,5.69366455e-01,5.78430176e-01,  5.90042114e-01,6.12792969e-01,6.48803711e-01,6.98837280e-01,  7.34970093e-01,7.43209839e-01,7.55630493e-01,7.71331787e-01,  7.76000977e-01,7.72842407e-01,7.62237549e-01,7.51342773e-01,  7.48718262e-01,7.45208740e-01,7.37640381e-01,7.29293823e-01,  7.19619751e-01,7.09991455e-01,7.02087402e-01,6.93313599e-01,  6.84112549e-01,6.72302246e-01,6.59408569e-01,6.47323608e-01,  6.40106201e-01,6.37115479e-01,6.29226685e-01,6.19323730e-01,  6.07818604e-01,5.92330933e-01,5.78506470e-01,5.60684204e-01,  5.10299683e-01,4.18716431e-01,3.40011597e-01,3.16360474e-01,  3.26629639e-01,3.49357605e-01,3.78189087e-01,3.68179321e-01,  3.80912781e-01,4.26460266e-01,4.70581055e-01,5.01068115e-01,  5.13732910e-01,5.10696411e-01,5.01586914e-01,4.97665405e-01,  5.04211426e-01,5.01846313e-01,4.88868713e-01,4.84550476e-01,  4.88342285e-01,4.93095398e-01,4.91760254e-01,4.89212036e-01,  4.88914490e-01,4.86595154e-01,4.80552673e-01,4.74067688e-01,  4.65660095e-01,4.49119568e-01,4.18663025e-01,3.86795044e-01,  3.91502380e-01,4.18945312e-01,4.26872253e-01,4.34951782e-01,  4.39598083e-01,4.35157776e-01,4.24636841e-01,4.10873413e-01,  3.94371033e-01,3.79730225e-01,3.70628357e-01,3.53858948e-01,  3.19511414e-01,3.08219910e-01,2.88795471e-01,2.15892792e-01,  1.17088318e-01,4.85486984e-02,2.41355896e-02,2.02555656e-02,  2.42695808e-02,3.57465744e-02,3.02643776e-02,3.31068039e-02,  4.50038910e-02,4.46434021e-02,4.95357513e-02,5.54971695e-02,  7.08007812e-02,8.93821716e-02,1.06700897e-01,1.24225616e-01,  1.25259399e-01,1.25865936e-01,1.38721466e-01,1.54949188e-01,  1.74934387e-01,2.08511353e-01,2.13996887e-01,1.90528870e-01,  1.78600311e-01,1.91486359e-01,2.05718994e-01,2.22446442e-01,  2.47531891e-01,2.67478943e-01,2.72048950e-01,2.75947571e-01,  2.86132812e-01,2.88558960e-01,2.89245605e-01,2.91213989e-01,  2.91809082e-01,2.90908813e-01,2.90298462e-01,2.90702820e-01,  2.88818359e-01,2.85545349e-01,2.82302856e-01,2.74833679e-01,  2.60345459e-01,2.52479553e-01,2.57965088e-01,2.63442993e-01,  2.64366150e-01,2.62420654e-01,2.53189087e-01,2.48935699e-01,  2.51518250e-01,2.57820129e-01,2.61642456e-01,2.64945984e-01,  2.65869141e-01,2.65815735e-01,2.65876770e-01,2.65800476e-01,  2.65777588e-01,2.63389587e-01,2.58514404e-01,2.52990723e-01,  2.47974396e-01,2.43190765e-01,2.38403320e-01,2.33009338e-01,  2.28923798e-01,2.23621368e-01,2.21462250e-01],
        [5.03555298e-01,5.19989014e-01,5.45867920e-01,6.30371094e-01,  6.70440674e-01,6.79748535e-01,6.73599243e-01,6.69021606e-01,  7.22305298e-01,8.26232910e-01,1.09332275e+00,1.22512817e+00,  1.28500366e+00,1.33044434e+00,1.33477783e+00,1.29391479e+00,  1.27243042e+00,1.37716675e+00,1.49209595e+00,1.60354614e+00,  1.68408203e+00,1.71844482e+00,1.74472046e+00,1.73941040e+00,  1.74652100e+00,1.77941895e+00,1.78674316e+00,1.72079468e+00,  1.73242188e+00,1.75955200e+00,1.73394775e+00,1.73953247e+00,  1.73269653e+00,1.66552734e+00,1.66894531e+00,1.70550537e+00,  1.72918701e+00,1.73425293e+00,1.71853638e+00,1.72033691e+00,  1.71896362e+00,1.70053101e+00,1.67401123e+00,1.66149902e+00,  1.65213013e+00,1.65106201e+00,1.66036987e+00,1.65386963e+00,  1.62875366e+00,1.62957764e+00,1.61767578e+00,1.60980225e+00,  1.59042358e+00,1.57565308e+00,1.57321167e+00,1.54757690e+00,  1.53384399e+00,1.54870605e+00,1.55349731e+00,1.54418945e+00,  1.52465820e+00,1.48626709e+00,1.51531982e+00,1.54089355e+00,  1.53189087e+00,1.51879883e+00,1.49255371e+00,1.38113403e+00,  1.34423828e+00,1.41024780e+00,1.41842651e+00,1.41836548e+00,  1.40512085e+00,1.34759521e+00,1.29852295e+00,1.29452515e+00,  1.30865479e+00,1.32586670e+00,1.31762695e+00,1.31817627e+00,  1.30111694e+00,1.17489624e+00,8.31588745e-01,9.00726318e-01,  1.16009521e+00,1.22222900e+00,1.21548462e+00,1.21188354e+00,  1.19256592e+00,1.18313599e+00,1.18377686e+00,1.16400146e+00,  1.13211060e+00,1.07327271e+00,1.03860474e+00,1.05224609e+00,  1.05972290e+00,1.05899048e+00,1.06805420e+00,1.04476929e+00,  1.00100708e+00,9.95788574e-01,9.97741699e-01,9.81597900e-01,  9.88632202e-01,9.79995728e-01,9.86633301e-01,9.86129761e-01,  9.64660645e-01,9.23461914e-01,8.71231079e-01,8.68576050e-01,  8.58444214e-01,8.42971802e-01,8.62686157e-01,8.46969604e-01,  6.88430786e-01,5.62911987e-01,5.72357178e-01,5.81649780e-01,  5.93200684e-01,6.15615845e-01,6.51214600e-01,7.00820923e-01,  7.36816406e-01,7.45025635e-01,7.57400513e-01,7.73117065e-01,  7.77801514e-01,7.74536133e-01,7.63656616e-01,7.52655029e-01,  7.50122070e-01,7.46551514e-01,7.38845825e-01,7.30667114e-01,  7.21023560e-01,7.11334229e-01,7.03353882e-01,6.94610596e-01,  6.85241699e-01,6.73614502e-01,6.60766602e-01,6.48742676e-01,  6.41510010e-01,6.38290405e-01,6.30477905e-01,6.20605469e-01,  6.09054565e-01,5.93841553e-01,5.80047607e-01,5.62103271e-01,  5.12023926e-01,4.20562744e-01,3.42140198e-01,3.18664551e-01,  3.28872681e-01,3.51409912e-01,3.80157471e-01,3.70185852e-01,  3.83026123e-01,4.28253174e-01,4.72084045e-01,5.02670288e-01,  5.15197754e-01,5.12054443e-01,5.02960205e-01,4.99038696e-01,  5.05493164e-01,5.03143311e-01,4.90135193e-01,4.85626221e-01,  4.89440918e-01,4.94262695e-01,4.92881775e-01,4.90348816e-01,  4.89891052e-01,4.87510681e-01,4.81513977e-01,4.75090027e-01,  4.66766357e-01,4.50286865e-01,4.19769287e-01,3.88031006e-01,  3.92807007e-01,4.20127869e-01,4.27757263e-01,4.35806274e-01,  4.40505981e-01,4.36157227e-01,4.25697327e-01,4.12017822e-01,  3.95378113e-01,3.80950928e-01,3.71879578e-01,3.55171204e-01,  3.20953369e-01,3.09715271e-01,2.90260315e-01,2.17460632e-01,  1.18612289e-01,4.96902466e-02,2.51035690e-02,2.13007927e-02,  2.52103806e-02,3.69968414e-02,3.14207077e-02,3.42063904e-02,  4.63991165e-02,4.58650589e-02,5.08480072e-02,5.68962097e-02,  7.22694397e-02,9.08508301e-02,1.08303070e-01,1.25747681e-01,  1.26789093e-01,1.27429962e-01,1.40281677e-01,1.56509399e-01,  1.76506042e-01,2.09987640e-01,2.15526581e-01,1.92153931e-01,  1.80149078e-01,1.93042755e-01,2.07210541e-01,2.23850250e-01,  2.48832703e-01,2.68646240e-01,2.73147583e-01,2.77168274e-01,  2.87284851e-01,2.89573669e-01,2.90229797e-01,2.92228699e-01,  2.93037415e-01,2.92053223e-01,2.91313171e-01,2.91702271e-01,  2.89779663e-01,2.86621094e-01,2.83386230e-01,2.76046753e-01,  2.61535645e-01,2.53677368e-01,2.59162903e-01,2.64587402e-01,  2.65663147e-01,2.63740540e-01,2.54531860e-01,2.50358582e-01,  2.52922058e-01,2.59346008e-01,2.63259888e-01,2.66510010e-01,  2.67494202e-01,2.67555237e-01,2.68005371e-01,2.68074036e-01,  2.68043518e-01,2.66464233e-01,2.62908936e-01,2.58071899e-01,  2.53524780e-01,2.48874664e-01,2.44487762e-01,2.39830017e-01,  2.35851288e-01,2.31166840e-01,2.27664948e-01],
        [5.15258789e-01,5.26443481e-01,5.48065186e-01,6.39053345e-01,  6.71768188e-01,6.82540894e-01,6.80389404e-01,6.71890259e-01,  7.28668213e-01,8.28842163e-01,1.09844971e+00,1.23190308e+00,  1.29013062e+00,1.33941650e+00,1.34149170e+00,1.30133057e+00,  1.27926636e+00,1.38244629e+00,1.50106812e+00,1.61080933e+00,  1.69168091e+00,1.72714233e+00,1.75167847e+00,1.74819946e+00,  1.75247192e+00,1.78549194e+00,1.79428101e+00,1.72598267e+00,  1.73803711e+00,1.76406860e+00,1.73825073e+00,1.74581909e+00,  1.73709106e+00,1.67095947e+00,1.67388916e+00,1.70980835e+00,  1.73519897e+00,1.73825073e+00,1.72262573e+00,1.72500610e+00,  1.72354126e+00,1.70593262e+00,1.67794800e+00,1.66635132e+00,  1.65756226e+00,1.65402222e+00,1.66461182e+00,1.65682983e+00,  1.63204956e+00,1.63473511e+00,1.61849976e+00,1.61221313e+00,  1.59329224e+00,1.57650757e+00,1.57824707e+00,1.54980469e+00,  1.53598022e+00,1.55261230e+00,1.55456543e+00,1.54721069e+00,  1.52758789e+00,1.48733521e+00,1.51995850e+00,1.54135132e+00,  1.53390503e+00,1.52328491e+00,1.49261475e+00,1.38412476e+00,  1.34643555e+00,1.41015625e+00,1.42227173e+00,1.41894531e+00,  1.40658569e+00,1.35107422e+00,1.29949951e+00,1.30117798e+00,  1.31286621e+00,1.32653809e+00,1.32192993e+00,1.31771851e+00,  1.30276489e+00,1.17932129e+00,8.30322266e-01,9.04617310e-01,  1.16122437e+00,1.22036743e+00,1.22039795e+00,1.20794678e+00,  1.19454956e+00,1.18844604e+00,1.17684937e+00,1.16976929e+00,  1.13589478e+00,1.06875610e+00,1.04727173e+00,1.05184937e+00,  1.05563354e+00,1.06759644e+00,1.06082153e+00,1.04663086e+00,  1.00836182e+00,9.86038208e-01,1.00726318e+00,9.82467651e-01,  9.80407715e-01,9.95239258e-01,9.77142334e-01,9.81460571e-01,  9.72610474e-01,9.06677246e-01,8.81881714e-01,8.77441406e-01,  8.34762573e-01,8.47671509e-01,8.50082397e-01,8.10958862e-01,  6.83563232e-01,5.64834595e-01,5.74050903e-01,5.83206177e-01,  5.94665527e-01,6.16806030e-01,6.52404785e-01,7.01644897e-01,  7.37213135e-01,7.44781494e-01,7.57034302e-01,7.72338867e-01,  7.76855469e-01,7.73620605e-01,7.62893677e-01,7.51968384e-01,  7.49282837e-01,7.45559692e-01,7.37777710e-01,7.29934692e-01,  7.20108032e-01,7.10464478e-01,7.02499390e-01,6.93710327e-01,  6.84494019e-01,6.72836304e-01,6.60079956e-01,6.47933960e-01,  6.40731812e-01,6.37725830e-01,6.29943848e-01,6.19979858e-01,  6.08612061e-01,5.93246460e-01,5.79727173e-01,5.62149048e-01,  5.12359619e-01,4.21585083e-01,3.43490601e-01,3.20152283e-01,  3.30245972e-01,3.52577209e-01,3.81210327e-01,3.71444702e-01,  3.84025574e-01,4.29061890e-01,4.72625732e-01,5.02777100e-01,  5.15045166e-01,5.11871338e-01,5.02807617e-01,4.98908997e-01,  5.05310059e-01,5.02883911e-01,4.89936829e-01,4.85420227e-01,  4.89204407e-01,4.93888855e-01,4.92454529e-01,4.89784241e-01,  4.89372253e-01,4.87068176e-01,4.80987549e-01,4.74479675e-01,  4.66255188e-01,4.49821472e-01,4.19540405e-01,3.87916565e-01,  3.92593384e-01,4.19876099e-01,4.27391052e-01,4.35310364e-01,  4.40010071e-01,4.35707092e-01,4.25315857e-01,4.11766052e-01,  3.95263672e-01,3.80752563e-01,3.71833801e-01,3.55323792e-01,  3.21411133e-01,3.10173035e-01,2.90657043e-01,2.18051910e-01,  1.19277954e-01,5.01327515e-02,2.50902176e-02,2.11577415e-02,  2.53357887e-02,3.72810364e-02,3.15818787e-02,3.45096588e-02,  4.68091965e-02,4.62741852e-02,5.12084961e-02,5.73539734e-02,  7.28340149e-02,9.14344788e-02,1.08911514e-01,1.26415253e-01,  1.27380371e-01,1.28101349e-01,1.40968323e-01,1.57176971e-01,  1.77143097e-01,2.10586548e-01,2.16030121e-01,1.92619324e-01,  1.80683136e-01,1.93458557e-01,2.07607269e-01,2.24140167e-01,  2.48912811e-01,2.68455505e-01,2.72888184e-01,2.76855469e-01,  2.86857605e-01,2.89031982e-01,2.89558411e-01,2.91511536e-01,  2.92213440e-01,2.91305542e-01,2.90611267e-01,2.90901184e-01,  2.89085388e-01,2.85842896e-01,2.82501221e-01,2.75230408e-01,  2.60787964e-01,2.52876282e-01,2.58285522e-01,2.63702393e-01,  2.64640808e-01,2.62603760e-01,2.53623962e-01,2.49423981e-01,  2.51960754e-01,2.58270264e-01,2.61985779e-01,2.65228271e-01,  2.66189575e-01,2.66334534e-01,2.66464233e-01,2.66380310e-01,  2.66311646e-01,2.64015198e-01,2.60223389e-01,2.55241394e-01,  2.49725342e-01,2.45101929e-01,2.40085602e-01,2.35389709e-01,  2.31174469e-01,2.25765228e-01,2.22969055e-01],
        [5.16159058e-01,5.29373169e-01,5.50262451e-01,6.41098022e-01,  6.77276611e-01,6.86035156e-01,6.80450439e-01,6.73965454e-01,  7.30224609e-01,8.32061768e-01,1.10202026e+00,1.23532104e+00,  1.29321289e+00,1.34213257e+00,1.34487915e+00,1.30386353e+00,  1.28152466e+00,1.38552856e+00,1.50271606e+00,1.61306763e+00,  1.69406128e+00,1.72784424e+00,1.75299072e+00,1.74984741e+00,  1.75604248e+00,1.78863525e+00,1.79663086e+00,1.72830200e+00,  1.74005127e+00,1.76605225e+00,1.73999023e+00,1.74636841e+00,  1.73846436e+00,1.67214966e+00,1.67416382e+00,1.71102905e+00,  1.73593140e+00,1.73922729e+00,1.72406006e+00,1.72637939e+00,  1.72448730e+00,1.70660400e+00,1.67886353e+00,1.66711426e+00,  1.65744019e+00,1.65505981e+00,1.66598511e+00,1.65832520e+00,  1.63269043e+00,1.63455200e+00,1.61944580e+00,1.61340332e+00,  1.59487915e+00,1.57833862e+00,1.57885742e+00,1.55148315e+00,  1.53671265e+00,1.55163574e+00,1.55471802e+00,1.54702759e+00,  1.52642822e+00,1.48648071e+00,1.51779175e+00,1.54031372e+00,  1.53244019e+00,1.52127075e+00,1.49319458e+00,1.38397217e+00,  1.34518433e+00,1.41094971e+00,1.42108154e+00,1.41806030e+00,  1.40588379e+00,1.35079956e+00,1.30102539e+00,1.29989624e+00,  1.31091309e+00,1.32702637e+00,1.31991577e+00,1.31546021e+00,  1.30218506e+00,1.17822266e+00,8.33374023e-01,9.04998779e-01,  1.16125488e+00,1.22070312e+00,1.21774292e+00,1.20983887e+00,  1.19451904e+00,1.18841553e+00,1.18023682e+00,1.16732788e+00,  1.13308716e+00,1.06594849e+00,1.04379272e+00,1.05706787e+00,  1.05999756e+00,1.06646729e+00,1.06051636e+00,1.04425049e+00,  1.00204468e+00,9.84329224e-01,1.00524902e+00,9.76821899e-01,  9.85733032e-01,9.91134644e-01,9.85443115e-01,9.94216919e-01,  9.80117798e-01,9.18624878e-01,8.77670288e-01,8.69125366e-01,  8.47549438e-01,8.47869873e-01,8.51043701e-01,8.34411621e-01,  6.88735962e-01,5.64819336e-01,5.74172974e-01,5.83297729e-01,  5.94528198e-01,6.16775513e-01,6.52343750e-01,7.01080322e-01,  7.36495972e-01,7.44384766e-01,7.56576538e-01,7.71835327e-01,  7.76184082e-01,7.72994995e-01,7.62039185e-01,7.51190186e-01,  7.48565674e-01,7.44888306e-01,7.37350464e-01,7.29156494e-01,  7.19421387e-01,7.09594727e-01,7.01873779e-01,6.93054199e-01,  6.83837891e-01,6.72225952e-01,6.59378052e-01,6.47369385e-01,  6.40151978e-01,6.37176514e-01,6.29135132e-01,6.19354248e-01,  6.07894897e-01,5.92758179e-01,5.79391479e-01,5.61630249e-01,  5.11932373e-01,4.21409607e-01,3.43635559e-01,3.20243835e-01,  3.30436707e-01,3.52630615e-01,3.81225586e-01,3.71490479e-01,  3.84094238e-01,4.28939819e-01,4.72290039e-01,5.02365112e-01,  5.14663696e-01,5.11459351e-01,5.02334595e-01,4.98443604e-01,  5.04760742e-01,5.02487183e-01,4.89524841e-01,4.85099792e-01,  4.88777161e-01,4.93354797e-01,4.91950989e-01,4.89265442e-01,  4.88838196e-01,4.86526489e-01,4.80339050e-01,4.73976135e-01,  4.65728760e-01,4.49409485e-01,4.19334412e-01,3.87771606e-01,  3.92501831e-01,4.19555664e-01,4.26933289e-01,4.34844971e-01,  4.39537048e-01,4.35264587e-01,4.24728394e-01,4.11300659e-01,  3.94859314e-01,3.80416870e-01,3.71520996e-01,3.54972839e-01,  3.21197510e-01,3.10005188e-01,2.90550232e-01,2.18067169e-01,  1.19323730e-01,5.00211716e-02,2.49333382e-02,2.10046768e-02,  2.51793861e-02,3.72877121e-02,3.14826965e-02,3.43208313e-02,  4.68034744e-02,4.62055206e-02,5.11169434e-02,5.73215485e-02,  7.27710724e-02,9.15088654e-02,1.08982086e-01,1.26480103e-01,  1.27414703e-01,1.28059387e-01,1.40983582e-01,1.57218933e-01,  1.77192688e-01,2.10456848e-01,2.15957642e-01,1.92550659e-01,  1.80625916e-01,1.93485260e-01,2.07637787e-01,2.23937988e-01,  2.48538971e-01,2.68112183e-01,2.72529602e-01,2.76428223e-01,  2.86376953e-01,2.88665771e-01,2.89215088e-01,2.91114807e-01,  2.91732788e-01,2.90763855e-01,2.90054321e-01,2.90397644e-01,  2.88528442e-01,2.85285950e-01,2.81967163e-01,2.74703979e-01,  2.60330200e-01,2.52349854e-01,2.57705688e-01,2.63061523e-01,  2.64053345e-01,2.62145996e-01,2.52967834e-01,2.48733521e-01,  2.51403809e-01,2.57553101e-01,2.61337280e-01,2.64511108e-01,  2.65602112e-01,2.65533447e-01,2.65449524e-01,2.65319824e-01,  2.65144348e-01,2.62428284e-01,2.57934570e-01,2.52662659e-01,  2.47779846e-01,2.42809296e-01,2.37625122e-01,2.32620239e-01,  2.29320526e-01,2.23915100e-01,2.21138000e-01],
        [5.16769409e-01,5.32287598e-01,5.54428101e-01,6.46102905e-01,  6.81106567e-01,6.89285278e-01,6.85287476e-01,6.76284790e-01,  7.33734131e-01,8.35067749e-01,1.10348511e+00,1.23825073e+00,  1.29580688e+00,1.34451294e+00,1.34796143e+00,1.30636597e+00,  1.28482056e+00,1.38754272e+00,1.50518799e+00,1.61621094e+00,  1.69662476e+00,1.73110962e+00,1.75585938e+00,1.75149536e+00,  1.75695801e+00,1.78942871e+00,1.79827881e+00,1.72952271e+00,  1.74154663e+00,1.76846313e+00,1.74157715e+00,1.74792480e+00,  1.73983765e+00,1.67294312e+00,1.67672729e+00,1.71261597e+00,  1.73745728e+00,1.74084473e+00,1.72479248e+00,1.72793579e+00,  1.72576904e+00,1.70794678e+00,1.68057251e+00,1.66854858e+00,  1.65927124e+00,1.65682983e+00,1.66702271e+00,1.65951538e+00,  1.63265991e+00,1.63534546e+00,1.62182617e+00,1.61489868e+00,  1.59594727e+00,1.57891846e+00,1.57846069e+00,1.55194092e+00,  1.53671265e+00,1.55133057e+00,1.55609131e+00,1.54754639e+00,  1.52789307e+00,1.48681641e+00,1.51782227e+00,1.54251099e+00,  1.53372192e+00,1.52270508e+00,1.49246216e+00,1.38360596e+00,  1.34716797e+00,1.41094971e+00,1.42166138e+00,1.41949463e+00,  1.40579224e+00,1.35153198e+00,1.30047607e+00,1.29885864e+00,  1.31161499e+00,1.32580566e+00,1.31942749e+00,1.31655884e+00,  1.30050659e+00,1.17910767e+00,8.33114624e-01,9.02847290e-01,  1.16113281e+00,1.21820068e+00,1.21771240e+00,1.21343994e+00,  1.19345093e+00,1.18655396e+00,1.17944336e+00,1.16531372e+00,  1.13485718e+00,1.07089233e+00,1.04339600e+00,1.05307007e+00,  1.05624390e+00,1.06768799e+00,1.06832886e+00,1.04315186e+00,  1.00247192e+00,9.86572266e-01,1.00015259e+00,9.86679077e-01,  9.83505249e-01,9.87426758e-01,9.87960815e-01,9.83566284e-01,  9.81613159e-01,9.25994873e-01,8.76480103e-01,8.90060425e-01,  8.50585938e-01,8.44985962e-01,8.70895386e-01,8.26446533e-01,  6.83181763e-01,5.64636230e-01,5.74295044e-01,5.83190918e-01,  5.94604492e-01,6.16821289e-01,6.52465820e-01,7.01339722e-01,  7.36785889e-01,7.44674683e-01,7.56652832e-01,7.71957397e-01,  7.76535034e-01,7.73269653e-01,7.62496948e-01,7.51586914e-01,  7.49038696e-01,7.45254517e-01,7.37686157e-01,7.29507446e-01,  7.19726562e-01,7.10144043e-01,7.02285767e-01,6.93542480e-01,  6.84295654e-01,6.72805786e-01,6.59744263e-01,6.47811890e-01,  6.40594482e-01,6.37420654e-01,6.29501343e-01,6.19537354e-01,  6.08261108e-01,5.92941284e-01,5.79559326e-01,5.61874390e-01,  5.12069702e-01,4.21417236e-01,3.43467712e-01,3.20098877e-01,  3.30383301e-01,3.52668762e-01,3.81149292e-01,3.71406555e-01,  3.84040833e-01,4.28962708e-01,4.72366333e-01,5.02502441e-01,  5.14907837e-01,5.11672974e-01,5.02593994e-01,4.98664856e-01,  5.05172729e-01,5.02639771e-01,4.89700317e-01,4.85321045e-01,  4.89059448e-01,4.93682861e-01,4.92233276e-01,4.89646912e-01,  4.89196777e-01,4.86854553e-01,4.80651855e-01,4.74426270e-01,  4.66148376e-01,4.49890137e-01,4.19830322e-01,3.88458252e-01,  3.92982483e-01,4.19876099e-01,4.27322388e-01,4.35226440e-01,  4.39834595e-01,4.35562134e-01,4.25094604e-01,4.11560059e-01,  3.95072937e-01,3.80577087e-01,3.71749878e-01,3.55201721e-01,  3.21418762e-01,3.10104370e-01,2.90618896e-01,2.18128204e-01,  1.19407654e-01,5.01556396e-02,2.51665115e-02,2.12435722e-02,  2.53577232e-02,3.74088287e-02,3.17544937e-02,3.45468521e-02,  4.69598770e-02,4.63294983e-02,5.13191223e-02,5.75189590e-02,  7.29942322e-02,9.15355682e-02,1.09075546e-01,1.26621246e-01,  1.27628326e-01,1.28273010e-01,1.41094208e-01,1.57272339e-01,  1.77204132e-01,2.10517883e-01,2.15950012e-01,1.92623138e-01,  1.80763245e-01,1.93481445e-01,2.07550049e-01,2.24056244e-01,  2.48825073e-01,2.68371582e-01,2.72811890e-01,2.76611328e-01,  2.86613464e-01,2.88925171e-01,2.89459229e-01,2.91397095e-01,  2.92015076e-01,2.91069031e-01,2.90443420e-01,2.90748596e-01,  2.88795471e-01,2.85560608e-01,2.82218933e-01,2.75039673e-01,  2.60719299e-01,2.52792358e-01,2.58163452e-01,2.63549805e-01,  2.64488220e-01,2.62535095e-01,2.53501892e-01,2.49412537e-01,  2.51869202e-01,2.58148193e-01,2.61787415e-01,2.65090942e-01,  2.66075134e-01,2.66265869e-01,2.66136169e-01,2.66151428e-01,  2.66151428e-01,2.63603210e-01,2.59742737e-01,2.54432678e-01,  2.49706268e-01,2.44857788e-01,2.40425110e-01,2.35996246e-01,  2.31552124e-01,2.27390289e-01,2.24304199e-01],
        [5.19515991e-01,5.34210205e-01,5.53009033e-01,6.42410278e-01,  6.80084229e-01,6.91284180e-01,6.87194824e-01,6.76345825e-01,  7.33322144e-01,8.35571289e-01,1.10443115e+00,1.23989868e+00,  1.29736328e+00,1.34616089e+00,1.34878540e+00,1.30691528e+00,  1.28454590e+00,1.38745117e+00,1.50479126e+00,1.61581421e+00,  1.69613647e+00,1.73123169e+00,1.75555420e+00,1.75186157e+00,  1.75723267e+00,1.78872681e+00,1.79824829e+00,1.72961426e+00,  1.74142456e+00,1.76828003e+00,1.74090576e+00,1.74771118e+00,  1.74017334e+00,1.67263794e+00,1.67593384e+00,1.71170044e+00,  1.73696899e+00,1.74069214e+00,1.72531128e+00,1.72778320e+00,  1.72509766e+00,1.70800781e+00,1.68057251e+00,1.66775513e+00,  1.66000366e+00,1.65716553e+00,1.66656494e+00,1.65948486e+00,  1.63244629e+00,1.63494873e+00,1.62121582e+00,1.61425781e+00,  1.59622192e+00,1.57864380e+00,1.57800293e+00,1.55197144e+00,  1.53640747e+00,1.55093384e+00,1.55383301e+00,1.54580688e+00,  1.52731323e+00,1.48571777e+00,1.51544189e+00,1.53842163e+00,  1.53189087e+00,1.52127075e+00,1.49194336e+00,1.38241577e+00,  1.34487915e+00,1.40841675e+00,1.42019653e+00,1.41683960e+00,  1.40301514e+00,1.34884644e+00,1.29763794e+00,1.29638672e+00,  1.31091309e+00,1.32424927e+00,1.31826782e+00,1.31546021e+00,  1.29815674e+00,1.17626953e+00,8.31726074e-01,9.02023315e-01,  1.16052246e+00,1.21847534e+00,1.21765137e+00,1.21206665e+00,  1.19329834e+00,1.18429565e+00,1.17590332e+00,1.16162109e+00,  1.13485718e+00,1.06497192e+00,1.04074097e+00,1.05395508e+00,  1.04809570e+00,1.06372070e+00,1.07049561e+00,1.04531860e+00,  1.00506592e+00,9.86907959e-01,9.99404907e-01,9.82238770e-01,  9.80514526e-01,9.84497070e-01,9.85137939e-01,9.72366333e-01,  9.70977783e-01,9.23599243e-01,8.69567871e-01,8.83132935e-01,  8.49716187e-01,8.31008911e-01,8.55239868e-01,8.28155518e-01,  6.84906006e-01,5.61325073e-01,5.70922852e-01,5.80032349e-01,  5.91522217e-01,6.13952637e-01,6.49688721e-01,6.99325562e-01,  7.35229492e-01,7.43530273e-01,7.55874634e-01,7.71438599e-01,  7.76275635e-01,7.73086548e-01,7.62313843e-01,7.51296997e-01,  7.48596191e-01,7.45025635e-01,7.37442017e-01,7.29339600e-01,  7.19497681e-01,7.09869385e-01,7.02072144e-01,6.93252563e-01,  6.84036255e-01,6.72409058e-01,6.59774780e-01,6.47644043e-01,  6.40472412e-01,6.37252808e-01,6.29180908e-01,6.19247437e-01,  6.07681274e-01,5.92376709e-01,5.78720093e-01,5.60836792e-01,  5.10620117e-01,4.19380188e-01,3.41072083e-01,3.17497253e-01,  3.27781677e-01,3.50395203e-01,3.79135132e-01,3.69224548e-01,  3.81996155e-01,4.27223206e-01,4.71099854e-01,5.01602173e-01,  5.14083862e-01,5.10910034e-01,5.01785278e-01,4.97962952e-01,  5.04470825e-01,5.02090454e-01,4.89166260e-01,4.84725952e-01,  4.88517761e-01,4.93316650e-01,4.91928101e-01,4.89349365e-01,  4.88929749e-01,4.86724854e-01,4.80583191e-01,4.74182129e-01,  4.66011047e-01,4.49829102e-01,4.19845581e-01,3.88481140e-01,  3.93096924e-01,4.19952393e-01,4.27299500e-01,4.35195923e-01,  4.39697266e-01,4.35287476e-01,4.24804688e-01,4.11140442e-01,  3.94554138e-01,3.79882812e-01,3.70895386e-01,3.54194641e-01,  3.20045471e-01,3.08822632e-01,2.89337158e-01,2.16609955e-01,  1.17876053e-01,4.91466522e-02,2.46162415e-02,2.08845139e-02,  2.48646736e-02,3.64885330e-02,3.08985710e-02,3.37305069e-02,  4.58059311e-02,4.54111099e-02,5.02185822e-02,5.62534332e-02,  7.16381073e-02,9.02614594e-02,1.07635498e-01,1.25030518e-01,  1.26205444e-01,1.26937866e-01,1.39705658e-01,1.55757904e-01,  1.75720215e-01,2.09190369e-01,2.14641571e-01,1.91261292e-01,  1.79298401e-01,1.92169189e-01,2.06317902e-01,2.22972870e-01,  2.47833252e-01,2.67578125e-01,2.72178650e-01,2.76222229e-01,  2.86254883e-01,2.88551331e-01,2.89222717e-01,2.91152954e-01,  2.91854858e-01,2.90962219e-01,2.90405273e-01,2.90725708e-01,  2.88719177e-01,2.85499573e-01,2.82302856e-01,2.74993896e-01,  2.60704041e-01,2.52845764e-01,2.58255005e-01,2.63435364e-01,  2.64297485e-01,2.62397766e-01,2.53433228e-01,2.49355316e-01,  2.51876831e-01,2.58010864e-01,2.61718750e-01,2.64961243e-01,  2.66029358e-01,2.66021729e-01,2.66090393e-01,2.65922546e-01,  2.65708923e-01,2.63633728e-01,2.59613037e-01,2.54653931e-01,  2.49523163e-01,2.44972229e-01,2.40234375e-01,2.35485077e-01,  2.31346130e-01,2.26295471e-01,2.23995209e-01],
        [5.21209717e-01,5.32791138e-01,5.57403564e-01,6.46652222e-01,  6.82098389e-01,6.92245483e-01,6.85333252e-01,6.78588867e-01,  7.33001709e-01,8.35708618e-01,1.10772705e+00,1.24130249e+00,  1.30014038e+00,1.34741211e+00,1.34951782e+00,1.30831909e+00,  1.28533936e+00,1.39028931e+00,1.50692749e+00,1.61709595e+00,  1.69799805e+00,1.73147583e+00,1.75735474e+00,1.75283813e+00,  1.75796509e+00,1.79104614e+00,1.79858398e+00,1.73110962e+00,  1.74316406e+00,1.76919556e+00,1.74349976e+00,1.74896240e+00,  1.74118042e+00,1.67434692e+00,1.67626953e+00,1.71322632e+00,  1.73779297e+00,1.74169922e+00,1.72647095e+00,1.72805786e+00,  1.72738647e+00,1.70867920e+00,1.68078613e+00,1.66952515e+00,  1.65960693e+00,1.65844727e+00,1.66900635e+00,1.65917969e+00,  1.63418579e+00,1.63653564e+00,1.62261963e+00,1.61578369e+00,  1.59515381e+00,1.57962036e+00,1.57833862e+00,1.55114746e+00,  1.53860474e+00,1.55166626e+00,1.55429077e+00,1.54675293e+00,  1.52703857e+00,1.48770142e+00,1.51705933e+00,1.54022217e+00,  1.53356934e+00,1.52023315e+00,1.49221802e+00,1.38369751e+00,  1.34481812e+00,1.41024780e+00,1.41943359e+00,1.41683960e+00,  1.40640259e+00,1.34716797e+00,1.29696655e+00,1.29711914e+00,  1.30761719e+00,1.32507324e+00,1.31860352e+00,1.31420898e+00,  1.29995728e+00,1.17449951e+00,8.32748413e-01,9.05181885e-01,  1.15890503e+00,1.22061157e+00,1.21795654e+00,1.20904541e+00,  1.19360352e+00,1.18145752e+00,1.17651367e+00,1.16360474e+00,  1.12948608e+00,1.07128906e+00,1.04589844e+00,1.04971313e+00,  1.05734253e+00,1.06207275e+00,1.06060791e+00,1.05111694e+00,  1.00045776e+00,9.90921021e-01,1.00949097e+00,9.74639893e-01,  9.87960815e-01,9.95101929e-01,9.78668213e-01,9.86145020e-01,  9.70382690e-01,9.19830322e-01,8.84292603e-01,8.64379883e-01,  8.48159790e-01,8.50006104e-01,8.50036621e-01,8.20983887e-01,  6.82998657e-01,5.59951782e-01,5.69396973e-01,5.78567505e-01,  5.90118408e-01,6.12564087e-01,6.48498535e-01,6.98242188e-01,  7.34512329e-01,7.42614746e-01,7.55081177e-01,7.70935059e-01,  7.75619507e-01,7.72537231e-01,7.61749268e-01,7.50915527e-01,  7.48321533e-01,7.44598389e-01,7.36984253e-01,7.28744507e-01,  7.19223022e-01,7.09518433e-01,7.01675415e-01,6.92977905e-01,  6.83715820e-01,6.72012329e-01,6.59271240e-01,6.47247314e-01,  6.39968872e-01,6.36718750e-01,6.28860474e-01,6.18789673e-01,  6.07299805e-01,5.91827393e-01,5.78033447e-01,5.60043335e-01,  5.09780884e-01,4.18373108e-01,3.39965820e-01,3.16497803e-01,  3.26766968e-01,3.49395752e-01,3.78166199e-01,3.68171692e-01,  3.80958557e-01,4.26361084e-01,4.70329285e-01,5.00961304e-01,  5.13565063e-01,5.10284424e-01,5.01312256e-01,4.97474670e-01,  5.03952026e-01,5.01632690e-01,4.88594055e-01,4.84268188e-01,  4.88067627e-01,4.93034363e-01,4.91546631e-01,4.89028931e-01,  4.88639832e-01,4.86335754e-01,4.80300903e-01,4.73968506e-01,  4.65812683e-01,4.49554443e-01,4.19906616e-01,3.88572693e-01,  3.93157959e-01,4.19860840e-01,4.27116394e-01,4.34875488e-01,  4.39323425e-01,4.34951782e-01,4.24301147e-01,4.10659790e-01,  3.94042969e-01,3.79463196e-01,3.70399475e-01,3.53668213e-01,  3.19427490e-01,3.08265686e-01,2.88688660e-01,2.15854645e-01,  1.17242813e-01,4.89101410e-02,2.46410370e-02,2.09794044e-02,  2.49371529e-02,3.63407135e-02,3.07731628e-02,3.36542130e-02,  4.55751419e-02,4.50868607e-02,5.00364304e-02,5.60770035e-02,  7.13176727e-02,8.97922516e-02,1.07168198e-01,1.24603271e-01,  1.25705719e-01,1.26392365e-01,1.39274597e-01,1.55303955e-01,  1.75270081e-01,2.08641052e-01,2.14080811e-01,1.90711975e-01,  1.78760529e-01,1.91669464e-01,2.05913544e-01,2.22564697e-01,  2.47570038e-01,2.67333984e-01,2.71972656e-01,2.75985718e-01,  2.86094666e-01,2.88391113e-01,2.89070129e-01,2.91007996e-01,  2.91877747e-01,2.90916443e-01,2.90290833e-01,2.90649414e-01,  2.88635254e-01,2.85316467e-01,2.82135010e-01,2.74986267e-01,  2.60841370e-01,2.52922058e-01,2.58262634e-01,2.63580322e-01,  2.64457703e-01,2.62542725e-01,2.53501892e-01,2.49485016e-01,  2.51922607e-01,2.58155823e-01,2.61940002e-01,2.65205383e-01,  2.66128540e-01,2.66304016e-01,2.66372681e-01,2.66204834e-01,  2.66525269e-01,2.64625549e-01,2.60665894e-01,2.55569458e-01,  2.51365662e-01,2.46456146e-01,2.42359161e-01,2.37762451e-01,  2.34149933e-01,2.29274750e-01,2.26413727e-01],
        [5.19989014e-01,5.34667969e-01,5.56045532e-01,6.47171021e-01,  6.78878784e-01,6.91650391e-01,6.86050415e-01,6.77215576e-01,  7.34893799e-01,8.36303711e-01,1.10653687e+00,1.24017334e+00,  1.29806519e+00,1.34716797e+00,1.34906006e+00,1.30657959e+00,  1.28448486e+00,1.38760376e+00,1.50564575e+00,1.61578369e+00,  1.69607544e+00,1.73132324e+00,1.75567627e+00,1.75167847e+00,  1.75698853e+00,1.78903198e+00,1.79803467e+00,1.72894287e+00,  1.74035645e+00,1.76684570e+00,1.74066162e+00,1.74804688e+00,  1.74005127e+00,1.67361450e+00,1.67654419e+00,1.71200562e+00,  1.73727417e+00,1.74075317e+00,1.72503662e+00,1.72760010e+00,  1.72525024e+00,1.70819092e+00,1.68032837e+00,1.66799927e+00,  1.65988159e+00,1.65664673e+00,1.66723633e+00,1.65914917e+00,  1.63247681e+00,1.63607788e+00,1.62142944e+00,1.61422729e+00,  1.59634399e+00,1.57843018e+00,1.57855225e+00,1.55264282e+00,  1.53713989e+00,1.55236816e+00,1.55349731e+00,1.54550171e+00,  1.52737427e+00,1.48666382e+00,1.51736450e+00,1.53933716e+00,  1.53027344e+00,1.52062988e+00,1.49142456e+00,1.38211060e+00,  1.34545898e+00,1.40762329e+00,1.41839600e+00,1.41629028e+00,  1.40295410e+00,1.34744263e+00,1.29556274e+00,1.29394531e+00,  1.30865479e+00,1.32284546e+00,1.31808472e+00,1.31372070e+00,  1.29742432e+00,1.17779541e+00,8.33343506e-01,9.04220581e-01,  1.16223145e+00,1.21691895e+00,1.21704102e+00,1.21084595e+00,  1.19061279e+00,1.18426514e+00,1.17669678e+00,1.16125488e+00,  1.13427734e+00,1.06481934e+00,1.03775024e+00,1.05386353e+00,  1.05368042e+00,1.06411743e+00,1.06420898e+00,1.03723145e+00,  1.00219727e+00,9.86129761e-01,9.97528076e-01,9.82360840e-01,  9.82971191e-01,9.90936279e-01,9.86740112e-01,9.78591919e-01,  9.74487305e-01,9.11727905e-01,8.70452881e-01,8.79486084e-01,  8.51287842e-01,8.45092773e-01,8.64242554e-01,8.23806763e-01,  6.79824829e-01,5.57708740e-01,5.67398071e-01,5.76553345e-01,  5.88027954e-01,6.10748291e-01,6.46957397e-01,6.96975708e-01,  7.33398438e-01,7.41668701e-01,7.54486084e-01,7.70385742e-01,  7.75283813e-01,7.72369385e-01,7.61474609e-01,7.50564575e-01,  7.47985840e-01,7.44308472e-01,7.36694336e-01,7.28485107e-01,  7.18856812e-01,7.09213257e-01,7.01492310e-01,6.92718506e-01,  6.83441162e-01,6.72088623e-01,6.59408569e-01,6.47186279e-01,  6.39862061e-01,6.36749268e-01,6.28616333e-01,6.18637085e-01,  6.07055664e-01,5.91491699e-01,5.77438354e-01,5.59356689e-01,  5.08850098e-01,4.17121887e-01,3.38447571e-01,3.14781189e-01,  3.25218201e-01,3.48068237e-01,3.76739502e-01,3.66752625e-01,  3.79722595e-01,4.25346375e-01,4.69627380e-01,5.00350952e-01,  5.13000488e-01,5.09887695e-01,5.00732422e-01,4.96871948e-01,  5.03448486e-01,5.01296997e-01,4.88227844e-01,4.83955383e-01,  4.87800598e-01,4.92500305e-01,4.91294861e-01,4.88792419e-01,  4.88464355e-01,4.86259460e-01,4.80216980e-01,4.73899841e-01,  4.65682983e-01,4.49523926e-01,4.19708252e-01,3.88534546e-01,  3.93112183e-01,4.19708252e-01,4.26986694e-01,4.34684753e-01,  4.39140320e-01,4.34600830e-01,4.23942566e-01,4.10224915e-01,  3.93600464e-01,3.78952026e-01,3.69659424e-01,3.52905273e-01,  3.18466187e-01,3.07281494e-01,2.87803650e-01,2.14836121e-01,  1.16212845e-01,4.80699539e-02,2.40707397e-02,2.02827454e-02,  2.42476463e-02,3.55033875e-02,3.01876068e-02,3.28893661e-02,  4.45919037e-02,4.42848206e-02,4.91561890e-02,5.51195145e-02,  7.02590942e-02,8.87660980e-02,1.06071472e-01,1.23527527e-01,  1.24803543e-01,1.25545502e-01,1.38183594e-01,1.54170990e-01,  1.74209595e-01,2.07622528e-01,2.13165283e-01,1.89765930e-01,  1.77833557e-01,1.90795898e-01,2.04994202e-01,2.21588135e-01,  2.46761322e-01,2.66777039e-01,2.71476746e-01,2.75512695e-01,  2.85629272e-01,2.87986755e-01,2.88703918e-01,2.90725708e-01,  2.91481018e-01,2.90519714e-01,2.89985657e-01,2.90267944e-01,  2.88276672e-01,2.85034180e-01,2.81791687e-01,2.74559021e-01,  2.60223389e-01,2.52418518e-01,2.57743835e-01,2.62931824e-01,  2.63839722e-01,2.61909485e-01,2.53021240e-01,2.48981476e-01,  2.51342773e-01,2.57629395e-01,2.61253357e-01,2.64366150e-01,  2.65449524e-01,2.65373230e-01,2.65426636e-01,2.65289307e-01,  2.64930725e-01,2.62985229e-01,2.58865356e-01,2.53791809e-01,  2.48664856e-01,2.44354248e-01,2.39685059e-01,2.35073090e-01,  2.30701447e-01,2.25833893e-01,2.22564697e-01],
        [5.25741577e-01,5.38269043e-01,5.62469482e-01,6.50405884e-01,  6.86782837e-01,6.97326660e-01,6.90261841e-01,6.83807373e-01,  7.39593506e-01,8.43872070e-01,1.11544800e+00,1.24917603e+00,  1.30804443e+00,1.35595703e+00,1.35842896e+00,1.31701660e+00,  1.29376221e+00,1.39813232e+00,1.51522827e+00,1.62567139e+00,  1.70672607e+00,1.74072266e+00,1.76593018e+00,1.76132202e+00,  1.76644897e+00,1.79992676e+00,1.80755615e+00,1.73898315e+00,  1.75021362e+00,1.77618408e+00,1.75018311e+00,1.75631714e+00,  1.74826050e+00,1.68048096e+00,1.68304443e+00,1.71975708e+00,  1.74426270e+00,1.74789429e+00,1.73175049e+00,1.73345947e+00,  1.73263550e+00,1.71469116e+00,1.68646240e+00,1.67437744e+00,  1.66516113e+00,1.66351318e+00,1.67257690e+00,1.66403198e+00,  1.63897705e+00,1.64007568e+00,1.62643433e+00,1.62069702e+00,  1.60095215e+00,1.58444214e+00,1.58325195e+00,1.55618286e+00,  1.54296875e+00,1.55746460e+00,1.55984497e+00,1.55184937e+00,  1.53115845e+00,1.49084473e+00,1.52099609e+00,1.54330444e+00,  1.53601074e+00,1.52377319e+00,1.49697876e+00,1.38833618e+00,  1.34841919e+00,1.41360474e+00,1.42288208e+00,1.42114258e+00,  1.40829468e+00,1.35128784e+00,1.29995728e+00,1.29882812e+00,  1.31066895e+00,1.32830811e+00,1.32202148e+00,1.31799316e+00,  1.30255127e+00,1.17855835e+00,8.37341309e-01,9.08782959e-01,  1.16333008e+00,1.22271729e+00,1.22030640e+00,1.21224976e+00,  1.19567871e+00,1.18667603e+00,1.18048096e+00,1.16687012e+00,  1.13119507e+00,1.06903076e+00,1.04037476e+00,1.05194092e+00,  1.06066895e+00,1.06582642e+00,1.06271362e+00,1.04476929e+00,  1.00109863e+00,9.89273071e-01,1.00662231e+00,9.81109619e-01,  9.87792969e-01,9.91271973e-01,9.89059448e-01,9.89837646e-01,  9.72061157e-01,9.17999268e-01,8.73794556e-01,8.70361328e-01,  8.49044800e-01,8.58367920e-01,8.55102539e-01,8.24676514e-01,  6.83578491e-01,5.60043335e-01,5.69686890e-01,5.78796387e-01,  5.90332031e-01,6.12762451e-01,6.48880005e-01,6.98516846e-01,  7.34756470e-01,7.42889404e-01,7.55462646e-01,7.71301270e-01,  7.76123047e-01,7.72811890e-01,7.62161255e-01,7.51190186e-01,  7.48550415e-01,7.44964600e-01,7.37304688e-01,7.29156494e-01,  7.19436646e-01,7.09884644e-01,7.02117920e-01,6.93222046e-01,  6.84112549e-01,6.72531128e-01,6.59606934e-01,6.47644043e-01,  6.40319824e-01,6.37069702e-01,6.29119873e-01,6.19003296e-01,  6.07528687e-01,5.91979980e-01,5.78201294e-01,5.60241699e-01,  5.09826660e-01,4.18342590e-01,3.39904785e-01,3.16452026e-01,  3.26858521e-01,3.49388123e-01,3.78250122e-01,3.68164062e-01,  3.81034851e-01,4.26521301e-01,4.70535278e-01,5.01190186e-01,  5.13702393e-01,5.10528564e-01,5.01312256e-01,4.97497559e-01,  5.04180908e-01,5.01724243e-01,4.88769531e-01,4.84519958e-01,  4.88197327e-01,4.93026733e-01,4.91752625e-01,4.89242554e-01,  4.88845825e-01,4.86663818e-01,4.80575562e-01,4.74243164e-01,  4.66133118e-01,4.49996948e-01,4.20196533e-01,3.89030457e-01,  3.93531799e-01,4.20257568e-01,4.27299500e-01,4.35005188e-01,  4.39537048e-01,4.35028076e-01,4.24446106e-01,4.10690308e-01,  3.94165039e-01,3.79585266e-01,3.70475769e-01,3.53652954e-01,  3.19320679e-01,3.08105469e-01,2.88581848e-01,2.15816498e-01,  1.17137909e-01,4.87527847e-02,2.44507790e-02,2.05488205e-02,  2.45423317e-02,3.61061096e-02,3.05576324e-02,3.33042145e-02,  4.53567505e-02,4.48722839e-02,4.97856140e-02,5.58900833e-02,  7.10315704e-02,8.95309448e-02,1.06948853e-01,1.24435425e-01,  1.25640869e-01,1.26380920e-01,1.39122009e-01,1.55105591e-01,  1.75014496e-01,2.08469391e-01,2.13928223e-01,1.90494537e-01,  1.78668976e-01,1.91581726e-01,2.05718994e-01,2.22339630e-01,  2.47322083e-01,2.67181396e-01,2.71766663e-01,2.75772095e-01,  2.85911560e-01,2.88261414e-01,2.88841248e-01,2.90847778e-01,  2.91603088e-01,2.90786743e-01,2.90115356e-01,2.90328979e-01,  2.88375854e-01,2.85148621e-01,2.81921387e-01,2.74642944e-01,  2.60414124e-01,2.52571106e-01,2.57865906e-01,2.63092041e-01,  2.63908386e-01,2.62031555e-01,2.53234863e-01,2.49046326e-01,  2.51525879e-01,2.57629395e-01,2.61299133e-01,2.64434814e-01,  2.65472412e-01,2.65495300e-01,2.65388489e-01,2.65167236e-01,  2.64747620e-01,2.62107849e-01,2.58522034e-01,2.53036499e-01,  2.48664856e-01,2.43835449e-01,2.39337921e-01,2.33974457e-01,  2.30289459e-01,2.25875854e-01,2.22011566e-01],
        [5.29586792e-01,5.40115356e-01,5.61614990e-01,6.54922485e-01,  6.87057495e-01,6.98303223e-01,6.92245483e-01,6.83792114e-01,  7.40005493e-01,8.41598511e-01,1.11398315e+00,1.24835205e+00,  1.30664062e+00,1.35522461e+00,1.35635376e+00,1.31481934e+00,  1.29241943e+00,1.39602661e+00,1.51339722e+00,1.62377930e+00,  1.70611572e+00,1.74017334e+00,1.76406860e+00,1.75955200e+00,  1.76467896e+00,1.79776001e+00,1.80612183e+00,1.73681641e+00,  1.74862671e+00,1.77438354e+00,1.74807739e+00,1.75494385e+00,  1.74697876e+00,1.68011475e+00,1.68188477e+00,1.71798706e+00,  1.74356079e+00,1.74639893e+00,1.73056030e+00,1.73342896e+00,  1.73104858e+00,1.71365356e+00,1.68634033e+00,1.67376709e+00,  1.66476440e+00,1.66204834e+00,1.67266846e+00,1.66333008e+00,  1.63705444e+00,1.64007568e+00,1.62490845e+00,1.61898804e+00,  1.60040283e+00,1.58380127e+00,1.58291626e+00,1.55603027e+00,  1.54071045e+00,1.55569458e+00,1.55737305e+00,1.54956055e+00,  1.53018188e+00,1.48959351e+00,1.52059937e+00,1.54351807e+00,  1.53533936e+00,1.52386475e+00,1.49508667e+00,1.38671875e+00,  1.34948730e+00,1.41195679e+00,1.42279053e+00,1.42129517e+00,  1.40725708e+00,1.35083008e+00,1.29843140e+00,1.29943848e+00,  1.31210327e+00,1.32617188e+00,1.32192993e+00,1.31890869e+00,  1.30151367e+00,1.18084717e+00,8.38333130e-01,9.08889771e-01,  1.16390991e+00,1.22058105e+00,1.22006226e+00,1.21240234e+00,  1.19479370e+00,1.18783569e+00,1.17880249e+00,1.16683960e+00,  1.13488770e+00,1.07052612e+00,1.04586792e+00,1.05419922e+00,  1.05630493e+00,1.07031250e+00,1.06854248e+00,1.04434204e+00,  1.00537109e+00,9.89807129e-01,1.00347900e+00,9.86267090e-01,  9.86450195e-01,9.97375488e-01,9.84603882e-01,9.81140137e-01,  9.85549927e-01,9.19433594e-01,8.78036499e-01,8.80371094e-01,  8.50936890e-01,8.55255127e-01,8.59695435e-01,8.24539185e-01,  6.83746338e-01,5.61538696e-01,5.71105957e-01,5.80230713e-01,  5.91827393e-01,6.13952637e-01,6.49749756e-01,6.99508667e-01,  7.35626221e-01,7.43728638e-01,7.56088257e-01,7.71743774e-01,  7.76535034e-01,7.73254395e-01,7.62588501e-01,7.51556396e-01,  7.48825073e-01,7.45452881e-01,7.37869263e-01,7.29507446e-01,  7.20001221e-01,7.10403442e-01,7.02423096e-01,6.93786621e-01,  6.84539795e-01,6.72958374e-01,6.60186768e-01,6.47994995e-01,  6.40762329e-01,6.37481689e-01,6.29440308e-01,6.19628906e-01,  6.08047485e-01,5.92727661e-01,5.78857422e-01,5.60791016e-01,  5.10787964e-01,4.19509888e-01,3.41148376e-01,3.17848206e-01,  3.28079224e-01,3.50646973e-01,3.79386902e-01,3.69430542e-01,  3.82240295e-01,4.27452087e-01,4.71206665e-01,5.01708984e-01,  5.14221191e-01,5.11047363e-01,5.01922607e-01,4.98123169e-01,  5.04562378e-01,5.02182007e-01,4.89265442e-01,4.84985352e-01,  4.88670349e-01,4.93453979e-01,4.92179871e-01,4.89524841e-01,  4.89227295e-01,4.86900330e-01,4.80804443e-01,4.74548340e-01,  4.66377258e-01,4.50340271e-01,4.20616150e-01,3.89465332e-01,  3.94065857e-01,4.20631409e-01,4.27703857e-01,4.35348511e-01,  4.39758301e-01,4.35386658e-01,4.24819946e-01,4.11216736e-01,  3.94554138e-01,3.80004883e-01,3.70964050e-01,3.54202271e-01,  3.20014954e-01,3.08784485e-01,2.89360046e-01,2.16594696e-01,  1.17935181e-01,4.94155884e-02,2.49819756e-02,2.11248398e-02,  2.51436234e-02,3.67946625e-02,3.11937332e-02,3.40738297e-02,  4.61359024e-02,4.56190109e-02,5.04989624e-02,5.65452576e-02,  7.18841553e-02,9.04064178e-02,1.07889175e-01,1.25320435e-01,  1.26564026e-01,1.27155304e-01,1.39888763e-01,1.55956268e-01,  1.75884247e-01,2.09220886e-01,2.14637756e-01,1.91333771e-01,  1.79409027e-01,1.92333221e-01,2.06409454e-01,2.22980499e-01,  2.47970581e-01,2.67776489e-01,2.72270203e-01,2.76260376e-01,  2.86300659e-01,2.88604736e-01,2.89276123e-01,2.91229248e-01,  2.91984558e-01,2.91099548e-01,2.90443420e-01,2.90756226e-01,  2.88726807e-01,2.85575867e-01,2.82348633e-01,2.75077820e-01,  2.60986328e-01,2.52983093e-01,2.58285522e-01,2.63595581e-01,  2.64465332e-01,2.62580872e-01,2.53669739e-01,2.49473572e-01,  2.52067566e-01,2.58140564e-01,2.61917114e-01,2.65090942e-01,  2.66128540e-01,2.66014099e-01,2.66136169e-01,2.66128540e-01,  2.65892029e-01,2.63839722e-01,2.59925842e-01,2.55088806e-01,  2.51075745e-01,2.46242523e-01,2.42221832e-01,2.37934113e-01,  2.33978271e-01,2.29362488e-01,2.26150513e-01],
        [5.22689819e-01,5.38619995e-01,5.57861328e-01,6.46667480e-01,  6.84234619e-01,6.93496704e-01,6.89773560e-01,6.79641724e-01,  7.36846924e-01,8.37524414e-01,1.10717773e+00,1.24273682e+00,  1.30041504e+00,1.34848022e+00,1.35073853e+00,1.30929565e+00,  1.28646851e+00,1.38977051e+00,1.50573730e+00,1.61663818e+00,  1.69708252e+00,1.73171997e+00,1.75552368e+00,1.75106812e+00,  1.75665283e+00,1.78845215e+00,1.79757690e+00,1.72888184e+00,  1.74005127e+00,1.76748657e+00,1.74041748e+00,1.74682617e+00,  1.73822021e+00,1.67092896e+00,1.67401123e+00,1.71002197e+00,  1.73504639e+00,1.73864746e+00,1.72244263e+00,1.72525024e+00,  1.72314453e+00,1.70516968e+00,1.67807007e+00,1.66476440e+00,  1.65686035e+00,1.65444946e+00,1.66415405e+00,1.65682983e+00,  1.62911987e+00,1.63150024e+00,1.61810303e+00,1.61041260e+00,  1.59182739e+00,1.57455444e+00,1.57427979e+00,1.54971313e+00,  1.53350830e+00,1.54803467e+00,1.55108643e+00,1.54180908e+00,  1.52261353e+00,1.48220825e+00,1.51229858e+00,1.53616333e+00,  1.52694702e+00,1.51632690e+00,1.48849487e+00,1.37939453e+00,  1.34194946e+00,1.40551758e+00,1.41604614e+00,1.41427612e+00,  1.39971924e+00,1.34475708e+00,1.29428101e+00,1.29049683e+00,  1.30438232e+00,1.32067871e+00,1.31423950e+00,1.31170654e+00,  1.29434204e+00,1.17453003e+00,8.33786011e-01,9.01046753e-01,  1.15838623e+00,1.21508789e+00,1.21316528e+00,1.20932007e+00,  1.18872070e+00,1.17962646e+00,1.17468262e+00,1.15640259e+00,  1.12960815e+00,1.06747437e+00,1.03753662e+00,1.05184937e+00,  1.05136108e+00,1.05648804e+00,1.06463623e+00,1.03915405e+00,  1.00054932e+00,9.86465454e-01,9.94918823e-01,9.78042603e-01,  9.77081299e-01,9.84680176e-01,9.94110107e-01,9.76638794e-01,  9.63745117e-01,9.21356201e-01,8.65097046e-01,8.68515015e-01,  8.45565796e-01,8.33633423e-01,8.58642578e-01,8.26156616e-01,  6.79855347e-01,5.57052612e-01,5.66802979e-01,5.75973511e-01,  5.87326050e-01,6.09863281e-01,6.45996094e-01,6.95922852e-01,  7.32147217e-01,7.40493774e-01,7.53280640e-01,7.69058228e-01,  7.74124146e-01,7.71179199e-01,7.60192871e-01,7.49191284e-01,  7.46582031e-01,7.42980957e-01,7.35427856e-01,7.27218628e-01,  7.17697144e-01,7.07962036e-01,7.00195312e-01,6.91574097e-01,  6.82266235e-01,6.70806885e-01,6.58294678e-01,6.46102905e-01,  6.38748169e-01,6.35498047e-01,6.27319336e-01,6.17538452e-01,  6.05957031e-01,5.90301514e-01,5.76400757e-01,5.58364868e-01,  5.07965088e-01,4.16435242e-01,3.38050842e-01,3.14521790e-01,  3.24867249e-01,3.47679138e-01,3.76380920e-01,3.66264343e-01,  3.79341125e-01,4.24766541e-01,4.68887329e-01,4.99572754e-01,  5.12054443e-01,5.08941650e-01,4.99748230e-01,4.95834351e-01,  5.02532959e-01,5.00259399e-01,4.87289429e-01,4.83078003e-01,  4.86869812e-01,4.91729736e-01,4.90455627e-01,4.87922668e-01,  4.87655640e-01,4.85382080e-01,4.79438782e-01,4.73106384e-01,  4.65049744e-01,4.48959351e-01,4.19265747e-01,3.88435364e-01,  3.92883301e-01,4.19281006e-01,4.26361084e-01,4.34043884e-01,  4.38407898e-01,4.33921814e-01,4.23294067e-01,4.09637451e-01,  3.93020630e-01,3.78517151e-01,3.69224548e-01,3.52432251e-01,  3.18084717e-01,3.06854248e-01,2.87528992e-01,2.14694977e-01,  1.16418839e-01,4.85954285e-02,2.46024132e-02,2.09846497e-02,  2.48141289e-02,3.60517502e-02,3.06901932e-02,3.34510803e-02,  4.51993942e-02,4.47654724e-02,4.97388840e-02,5.55753708e-02,  7.07511902e-02,8.90789032e-02,1.06348038e-01,1.23847961e-01,  1.25114441e-01,1.25820160e-01,1.38389587e-01,1.54365540e-01,  1.74274445e-01,2.07561493e-01,2.13096619e-01,1.89765930e-01,  1.77860260e-01,1.90780640e-01,2.04963684e-01,2.21530914e-01,  2.46513367e-01,2.66418457e-01,2.71194458e-01,2.75199890e-01,  2.85247803e-01,2.87719727e-01,2.88436890e-01,2.90557861e-01,  2.91244507e-01,2.90237427e-01,2.89695740e-01,2.89970398e-01,  2.87956238e-01,2.84744263e-01,2.81494141e-01,2.74353027e-01,  2.60162354e-01,2.52426147e-01,2.57804871e-01,2.63000488e-01,  2.63908386e-01,2.61962891e-01,2.53135681e-01,2.49080658e-01,  2.51480103e-01,2.57568359e-01,2.61230469e-01,2.64411926e-01,  2.65487671e-01,2.65396118e-01,2.65747070e-01,2.65708923e-01,  2.65556335e-01,2.63969421e-01,2.60444641e-01,2.56134033e-01,  2.51686096e-01,2.47543335e-01,2.43049622e-01,2.38521576e-01,  2.34512329e-01,2.29854584e-01,2.27291107e-01],
        [5.26962280e-01,5.41900635e-01,5.63262939e-01,6.54357910e-01,  6.91146851e-01,6.97616577e-01,6.93649292e-01,6.86172485e-01,  7.42034912e-01,8.42498779e-01,1.11437988e+00,1.25186157e+00,  1.31036377e+00,1.35903931e+00,1.36132812e+00,1.31875610e+00,  1.29550171e+00,1.39831543e+00,1.51535034e+00,1.62744141e+00,  1.70776367e+00,1.74182129e+00,1.76657104e+00,1.76174927e+00,  1.76699829e+00,1.79885864e+00,1.80755615e+00,1.73870850e+00,  1.74911499e+00,1.77578735e+00,1.74899292e+00,1.75497437e+00,  1.74688721e+00,1.67932129e+00,1.68127441e+00,1.71768188e+00,  1.74230957e+00,1.74615479e+00,1.73007202e+00,1.73260498e+00,  1.73028564e+00,1.71237183e+00,1.68490601e+00,1.67163086e+00,  1.66268921e+00,1.66076660e+00,1.67004395e+00,1.66223145e+00,  1.63528442e+00,1.63711548e+00,1.62393188e+00,1.61520386e+00,  1.59735107e+00,1.58029175e+00,1.57952881e+00,1.55358887e+00,  1.53784180e+00,1.55215454e+00,1.55529785e+00,1.54608154e+00,  1.52783203e+00,1.48806763e+00,1.51742554e+00,1.54144287e+00,  1.53259277e+00,1.52038574e+00,1.49209595e+00,1.38278198e+00,  1.34652710e+00,1.40911865e+00,1.41799927e+00,1.41735840e+00,  1.40411377e+00,1.34814453e+00,1.29669189e+00,1.29492188e+00,  1.30831909e+00,1.32391357e+00,1.31716919e+00,1.31542969e+00,  1.29864502e+00,1.17864990e+00,8.38104248e-01,9.04525757e-01,  1.16040039e+00,1.21936035e+00,1.21548462e+00,1.21130371e+00,  1.19259644e+00,1.18255615e+00,1.17825317e+00,1.15933228e+00,  1.12747192e+00,1.06802368e+00,1.03762817e+00,1.05032349e+00,  1.05422974e+00,1.06085205e+00,1.06866455e+00,1.04141235e+00,  9.97756958e-01,9.87014771e-01,9.93774414e-01,9.77691650e-01,  9.81109619e-01,9.84802246e-01,9.90890503e-01,9.81277466e-01,  9.65103149e-01,9.16702271e-01,8.73535156e-01,8.77792358e-01,  8.54385376e-01,8.40682983e-01,8.57131958e-01,8.36395264e-01,  6.84753418e-01,5.59844971e-01,5.69412231e-01,5.78643799e-01,  5.90179443e-01,6.12609863e-01,6.48513794e-01,6.98211670e-01,  7.34191895e-01,7.42156982e-01,7.54821777e-01,7.70553589e-01,  7.75222778e-01,7.72171021e-01,7.61322021e-01,7.50350952e-01,  7.47802734e-01,7.44171143e-01,7.36450195e-01,7.28454590e-01,  7.18872070e-01,7.09075928e-01,7.01278687e-01,6.92550659e-01,  6.83410645e-01,6.71875000e-01,6.59210205e-01,6.47232056e-01,  6.39831543e-01,6.36596680e-01,6.28601074e-01,6.18591309e-01,  6.06979370e-01,5.91445923e-01,5.77667236e-01,5.59692383e-01,  5.09490967e-01,4.18167114e-01,3.39996338e-01,3.16596985e-01,  3.26927185e-01,3.49563599e-01,3.78181458e-01,3.68247986e-01,  3.81126404e-01,4.26338196e-01,4.70260620e-01,5.00701904e-01,  5.13183594e-01,5.10055542e-01,5.01007080e-01,4.97070312e-01,  5.03540039e-01,5.01235962e-01,4.88380432e-01,4.84039307e-01,  4.87762451e-01,4.92523193e-01,4.91226196e-01,4.88777161e-01,  4.88319397e-01,4.86000061e-01,4.80125427e-01,4.73739624e-01,  4.65614319e-01,4.49508667e-01,4.19776917e-01,3.88809204e-01,  3.93386841e-01,4.19830322e-01,4.26879883e-01,4.34547424e-01,  4.39041138e-01,4.34547424e-01,4.23934937e-01,4.10293579e-01,  3.93646240e-01,3.79104614e-01,3.70018005e-01,3.53309631e-01,  3.19129944e-01,3.07907104e-01,2.88421631e-01,2.15599060e-01,  1.17084503e-01,4.86545563e-02,2.43000984e-02,2.04815865e-02,  2.44622231e-02,3.60946655e-02,3.04818153e-02,3.33776474e-02,  4.54187393e-02,4.48694229e-02,4.98971939e-02,5.58519363e-02,  7.11669922e-02,8.96148682e-02,1.06958389e-01,1.24502182e-01,  1.25587463e-01,1.26396179e-01,1.39186859e-01,1.55090332e-01,  1.74980164e-01,2.08381653e-01,2.13802338e-01,1.90406799e-01,  1.78627014e-01,1.91459656e-01,2.05543518e-01,2.22126007e-01,  2.46971130e-01,2.66738892e-01,2.71255493e-01,2.75329590e-01,  2.85232544e-01,2.87635803e-01,2.88391113e-01,2.90252686e-01,  2.90969849e-01,2.89901733e-01,2.89398193e-01,2.89649963e-01,  2.87620544e-01,2.84446716e-01,2.81120300e-01,2.74002075e-01,  2.59788513e-01,2.51953125e-01,2.57148743e-01,2.62298584e-01,  2.63046265e-01,2.61276245e-01,2.52304077e-01,2.48275757e-01,  2.50595093e-01,2.56774902e-01,2.60406494e-01,2.63320923e-01,  2.64396667e-01,2.64236450e-01,2.64175415e-01,2.63748169e-01,  2.63397217e-01,2.60726929e-01,2.56477356e-01,2.51144409e-01,  2.46520996e-01,2.41508484e-01,2.36782074e-01,2.31769562e-01,  2.28164673e-01,2.23148346e-01,2.20417023e-01],
        [5.24948120e-01,5.37796021e-01,5.63369751e-01,6.53274536e-01,  6.87194824e-01,6.98928833e-01,6.92901611e-01,6.84997559e-01,  7.41119385e-01,8.43719482e-01,1.11608887e+00,1.25143433e+00,  1.31021118e+00,1.35739136e+00,1.36007690e+00,1.31900024e+00,  1.29486084e+00,1.39883423e+00,1.51525879e+00,1.62661743e+00,  1.70855713e+00,1.74182129e+00,1.76651001e+00,1.76116943e+00,  1.76654053e+00,1.79925537e+00,1.80636597e+00,1.73736572e+00,  1.74829102e+00,1.77465820e+00,1.74929810e+00,1.75442505e+00,  1.74603271e+00,1.67916870e+00,1.68093872e+00,1.71813965e+00,  1.74212646e+00,1.74517822e+00,1.72952271e+00,1.73129272e+00,  1.72973633e+00,1.71163940e+00,1.68371582e+00,1.67214966e+00,  1.66262817e+00,1.66067505e+00,1.67034912e+00,1.66088867e+00,  1.63577271e+00,1.63726807e+00,1.62353516e+00,1.61798096e+00,  1.59725952e+00,1.58117676e+00,1.58105469e+00,1.55358887e+00,  1.53985596e+00,1.55282593e+00,1.55529785e+00,1.54705811e+00,  1.52612305e+00,1.48745728e+00,1.51824951e+00,1.54104614e+00,  1.53323364e+00,1.52087402e+00,1.49304199e+00,1.38525391e+00,  1.34472656e+00,1.40869141e+00,1.41882324e+00,1.41735840e+00,  1.40573120e+00,1.34841919e+00,1.29632568e+00,1.29501343e+00,  1.30673218e+00,1.32366943e+00,1.31658936e+00,1.31301880e+00,  1.30004883e+00,1.17919922e+00,8.36975098e-01,9.04815674e-01,  1.15704346e+00,1.21618652e+00,1.21673584e+00,1.20831299e+00,  1.19296265e+00,1.18316650e+00,1.17462158e+00,1.16491699e+00,  1.13021851e+00,1.06649780e+00,1.03921509e+00,1.04666138e+00,  1.05349731e+00,1.06069946e+00,1.05590820e+00,1.04641724e+00,  1.00250244e+00,9.82803345e-01,1.00750732e+00,9.81918335e-01,  9.88327026e-01,9.91394043e-01,9.72976685e-01,9.76089478e-01,  9.72778320e-01,9.14169312e-01,8.68057251e-01,8.61984253e-01,  8.46939087e-01,8.57238770e-01,8.48037720e-01,8.28094482e-01,  6.86157227e-01,5.59768677e-01,5.69458008e-01,5.78445435e-01,  5.89996338e-01,6.12594604e-01,6.48513794e-01,6.98440552e-01,  7.34771729e-01,7.42797852e-01,7.55477905e-01,7.71423340e-01,  7.76168823e-01,7.73162842e-01,7.62298584e-01,7.51327515e-01,  7.48825073e-01,7.45117188e-01,7.37442017e-01,7.29293823e-01,  7.19543457e-01,7.09991455e-01,7.02224731e-01,6.93450928e-01,  6.84341431e-01,6.72851562e-01,6.60034180e-01,6.47872925e-01,  6.40701294e-01,6.37344360e-01,6.29211426e-01,6.19400024e-01,  6.07803345e-01,5.92117310e-01,5.78308105e-01,5.60211182e-01,  5.09674072e-01,4.18174744e-01,3.39782715e-01,3.16345215e-01,  3.26759338e-01,3.49494934e-01,3.78166199e-01,3.68095398e-01,  3.81088257e-01,4.26391602e-01,4.70573425e-01,5.01129150e-01,  5.13580322e-01,5.10589600e-01,5.01419067e-01,4.97566223e-01,  5.04104614e-01,5.01693726e-01,4.88777161e-01,4.84512329e-01,  4.88273621e-01,4.93095398e-01,4.91821289e-01,4.89288330e-01,  4.88967896e-01,4.86671448e-01,4.80644226e-01,4.74411011e-01,  4.66293335e-01,4.50233459e-01,4.20478821e-01,3.89373779e-01,  3.94134521e-01,4.20547485e-01,4.27452087e-01,4.35180664e-01,  4.39559937e-01,4.35119629e-01,4.24484253e-01,4.10728455e-01,  3.94096375e-01,3.79516602e-01,3.70315552e-01,3.53523254e-01,  3.19221497e-01,3.07952881e-01,2.88528442e-01,2.15641022e-01,  1.17086411e-01,4.88052368e-02,2.45294571e-02,2.08592415e-02,  2.48231888e-02,3.63006592e-02,3.08828354e-02,3.35693359e-02,  4.55350876e-02,4.51545715e-02,4.99334335e-02,5.60483932e-02,  7.12547302e-02,8.95996094e-02,1.07074738e-01,1.24494553e-01,  1.25762939e-01,1.26544952e-01,1.39137268e-01,1.55132294e-01,  1.75086975e-01,2.08496094e-01,2.13951111e-01,1.90544128e-01,  1.78611755e-01,1.91516876e-01,2.05665588e-01,2.22282410e-01,  2.47287750e-01,2.67196655e-01,2.71781921e-01,2.75817871e-01,  2.85888672e-01,2.88269043e-01,2.88970947e-01,2.90855408e-01,  2.91618347e-01,2.90626526e-01,2.90077209e-01,2.90367126e-01,  2.88375854e-01,2.85133362e-01,2.81921387e-01,2.74703979e-01,  2.60543823e-01,2.52731323e-01,2.58041382e-01,2.63191223e-01,  2.64045715e-01,2.62054443e-01,2.53143311e-01,2.49259949e-01,  2.51579285e-01,2.57843018e-01,2.61367798e-01,2.64503479e-01,  2.65441895e-01,2.65388489e-01,2.65518188e-01,2.65014648e-01,  2.64801025e-01,2.62489319e-01,2.58705139e-01,2.54158020e-01,  2.49923706e-01,2.45498657e-01,2.41245270e-01,2.36557007e-01,  2.32315063e-01,2.27882385e-01,2.25776672e-01],
        [5.28121948e-01,5.43151855e-01,5.65200806e-01,6.55807495e-01,  6.91162109e-01,7.00653076e-01,6.97998047e-01,6.87530518e-01,  7.44415283e-01,8.47122192e-01,1.12014771e+00,1.25711060e+00,  1.31546021e+00,1.36416626e+00,1.36611938e+00,1.32321167e+00,  1.29983521e+00,1.40368652e+00,1.52197266e+00,1.63397217e+00,  1.71395874e+00,1.74865723e+00,1.77297974e+00,1.76785278e+00,  1.77337646e+00,1.80541992e+00,1.81326294e+00,1.74340820e+00,  1.75445557e+00,1.78137207e+00,1.75427246e+00,1.76043701e+00,  1.75201416e+00,1.68414307e+00,1.68679810e+00,1.72250366e+00,  1.74728394e+00,1.75088501e+00,1.73449707e+00,1.73706055e+00,  1.73513794e+00,1.71725464e+00,1.68884277e+00,1.67498779e+00,  1.66677856e+00,1.66510010e+00,1.67373657e+00,1.66656494e+00,  1.64019775e+00,1.64114380e+00,1.62677002e+00,1.61874390e+00,  1.60107422e+00,1.58435059e+00,1.58386230e+00,1.55828857e+00,  1.54220581e+00,1.55679321e+00,1.55990601e+00,1.54962158e+00,  1.53186035e+00,1.49081421e+00,1.52078247e+00,1.54614258e+00,  1.53723145e+00,1.52566528e+00,1.49719238e+00,1.38693237e+00,  1.34936523e+00,1.41348267e+00,1.42245483e+00,1.42187500e+00,  1.40756226e+00,1.35125732e+00,1.30105591e+00,1.29718018e+00,  1.31106567e+00,1.32745361e+00,1.32040405e+00,1.31997681e+00,  1.30227661e+00,1.17938232e+00,8.39080811e-01,9.05075073e-01,  1.16171265e+00,1.22146606e+00,1.21557617e+00,1.21435547e+00,  1.19726562e+00,1.18499756e+00,1.18273926e+00,1.16268921e+00,  1.12911987e+00,1.06964111e+00,1.04000854e+00,1.05654907e+00,  1.05615234e+00,1.05609131e+00,1.06808472e+00,1.04595947e+00,  1.00396729e+00,9.93957520e-01,9.97528076e-01,9.81094360e-01,  9.89395142e-01,9.86953735e-01,9.91134644e-01,9.84405518e-01,  9.71588135e-01,9.32006836e-01,8.73428345e-01,8.63128662e-01,  8.53103638e-01,8.36700439e-01,8.52523804e-01,8.39141846e-01,  6.88858032e-01,5.62484741e-01,5.72357178e-01,5.81344604e-01,  5.92529297e-01,6.15005493e-01,6.51245117e-01,7.00851440e-01,  7.37045288e-01,7.45254517e-01,7.57568359e-01,7.73315430e-01,  7.78259277e-01,7.74963379e-01,7.64114380e-01,7.53326416e-01,  7.50793457e-01,7.46994019e-01,7.39364624e-01,7.31170654e-01,  7.21343994e-01,7.11700439e-01,7.03872681e-01,6.95159912e-01,  6.85928345e-01,6.74392700e-01,6.61651611e-01,6.49505615e-01,  6.42227173e-01,6.38839722e-01,6.30722046e-01,6.20849609e-01,  6.09207153e-01,5.93795776e-01,5.79971313e-01,5.61813354e-01,  5.11505127e-01,4.20066833e-01,3.41697693e-01,3.18298340e-01,  3.28651428e-01,3.51295471e-01,3.80058289e-01,3.70033264e-01,  3.82911682e-01,4.28184509e-01,4.72145081e-01,5.02639771e-01,  5.15075684e-01,5.11947632e-01,5.02868652e-01,4.98893738e-01,  5.05462646e-01,5.03128052e-01,4.89860535e-01,4.85801697e-01,  4.89616394e-01,4.94331360e-01,4.93164062e-01,4.90478516e-01,  4.90127563e-01,4.87937927e-01,4.81788635e-01,4.75486755e-01,  4.67407227e-01,4.51278687e-01,4.21600342e-01,3.90495300e-01,  3.95019531e-01,4.21554565e-01,4.28489685e-01,4.36225891e-01,  4.40795898e-01,4.36248779e-01,4.25613403e-01,4.11895752e-01,  3.95233154e-01,3.80714417e-01,3.71658325e-01,3.54797363e-01,  3.20510864e-01,3.09318542e-01,2.89825439e-01,2.16918945e-01,  1.18061066e-01,4.95147705e-02,2.51135826e-02,2.12464333e-02,  2.53438950e-02,3.70368958e-02,3.14245224e-02,3.42521667e-02,  4.63638306e-02,4.59089279e-02,5.07879257e-02,5.68828583e-02,  7.21778870e-02,9.05990601e-02,1.08098984e-01,1.25640869e-01,  1.26796722e-01,1.27620697e-01,1.40270233e-01,1.56314850e-01,  1.76265717e-01,2.09594727e-01,2.15152740e-01,1.91650391e-01,  1.79763794e-01,1.92813873e-01,2.06802368e-01,2.23407745e-01,  2.48317719e-01,2.68081665e-01,2.72773743e-01,2.76649475e-01,  2.86811829e-01,2.89131165e-01,2.89741516e-01,2.91717529e-01,  2.92449951e-01,2.91549683e-01,2.90924072e-01,2.91229248e-01,  2.89215088e-01,2.85919189e-01,2.82714844e-01,2.75520325e-01,  2.61474609e-01,2.53494263e-01,2.58804321e-01,2.63999939e-01,  2.64900208e-01,2.62985229e-01,2.54043579e-01,2.50076294e-01,  2.52487183e-01,2.58499146e-01,2.62260437e-01,2.65495300e-01,  2.66532898e-01,2.66448975e-01,2.66525269e-01,2.66372681e-01,  2.66281128e-01,2.64274597e-01,2.60772705e-01,2.56843567e-01,  2.52197266e-01,2.48741150e-01,2.43774414e-01,2.40051270e-01,  2.35828400e-01,2.31216431e-01,2.29122162e-01],
        [5.27359009e-01,5.42755127e-01,5.63156128e-01,6.55960083e-01,  6.91864014e-01,6.98669434e-01,6.96502686e-01,6.87988281e-01,  7.44491577e-01,8.46939087e-01,1.11877441e+00,1.25537109e+00,  1.31378174e+00,1.36264038e+00,1.36523438e+00,1.32186890e+00,  1.29940796e+00,1.40252686e+00,1.52044678e+00,1.63262939e+00,  1.71328735e+00,1.74774170e+00,1.77069092e+00,1.76580811e+00,  1.77185059e+00,1.80355835e+00,1.81176758e+00,1.74279785e+00,  1.75350952e+00,1.78048706e+00,1.75292969e+00,1.75933838e+00,  1.75109863e+00,1.68231201e+00,1.68551636e+00,1.72204590e+00,  1.74606323e+00,1.75000000e+00,1.73315430e+00,1.73571777e+00,  1.73406982e+00,1.71548462e+00,1.68774414e+00,1.67395020e+00,  1.66586304e+00,1.66436768e+00,1.67260742e+00,1.66583252e+00,  1.63922119e+00,1.63970947e+00,1.62612915e+00,1.61825562e+00,  1.59976196e+00,1.58294678e+00,1.58270264e+00,1.55770874e+00,  1.54107666e+00,1.55514526e+00,1.55929565e+00,1.54928589e+00,  1.53073120e+00,1.49105835e+00,1.51980591e+00,1.54531860e+00,  1.53567505e+00,1.52349854e+00,1.49584961e+00,1.38604736e+00,  1.34875488e+00,1.41183472e+00,1.42047119e+00,1.42062378e+00,  1.40679932e+00,1.35025024e+00,1.29928589e+00,1.29479980e+00,  1.30908203e+00,1.32589722e+00,1.31814575e+00,1.31826782e+00,  1.30075073e+00,1.18011475e+00,8.40820312e-01,9.06158447e-01,  1.16314697e+00,1.22204590e+00,1.21490479e+00,1.21224976e+00,  1.19659424e+00,1.18566895e+00,1.18423462e+00,1.16467285e+00,  1.12994385e+00,1.07186890e+00,1.04040527e+00,1.05480957e+00,  1.05554199e+00,1.05484009e+00,1.06512451e+00,1.04385376e+00,  9.95849609e-01,9.86831665e-01,9.97268677e-01,9.81170654e-01,  9.85900879e-01,9.80682373e-01,9.88235474e-01,9.86557007e-01,  9.66690063e-01,9.20379639e-01,8.65737915e-01,8.65234375e-01,  8.64273071e-01,8.36853027e-01,8.51104736e-01,8.52127075e-01,  6.91802979e-01,5.60287476e-01,5.70114136e-01,5.79208374e-01,  5.90591431e-01,6.13250732e-01,6.49414062e-01,6.99569702e-01,  7.35900879e-01,7.44110107e-01,7.56774902e-01,7.72644043e-01,  7.77618408e-01,7.74505615e-01,7.63580322e-01,7.52868652e-01,  7.50106812e-01,7.46337891e-01,7.38845825e-01,7.30697632e-01,  7.20855713e-01,7.11318970e-01,7.03414917e-01,6.94595337e-01,  6.85440063e-01,6.73797607e-01,6.61148071e-01,6.49093628e-01,  6.41708374e-01,6.38397217e-01,6.30416870e-01,6.20315552e-01,  6.08566284e-01,5.93109131e-01,5.79040527e-01,5.60806274e-01,  5.10330200e-01,4.18479919e-01,3.39942932e-01,3.16635132e-01,  3.26873779e-01,3.49548340e-01,3.78540039e-01,3.68431091e-01,  3.81340027e-01,4.27024841e-01,4.71214294e-01,5.01770020e-01,  5.14480591e-01,5.11352539e-01,5.02166748e-01,4.98313904e-01,  5.04791260e-01,5.02532959e-01,4.89547729e-01,4.85214233e-01,  4.89089966e-01,4.93904114e-01,4.92698669e-01,4.90211487e-01,  4.89784241e-01,4.87617493e-01,4.81582642e-01,4.75128174e-01,  4.67102051e-01,4.50996399e-01,4.21180725e-01,3.90136719e-01,  3.94683838e-01,4.21142578e-01,4.28146362e-01,4.35882568e-01,  4.40269470e-01,4.35768127e-01,4.25018311e-01,4.11254883e-01,  3.94630432e-01,3.79875183e-01,3.70758057e-01,3.53935242e-01,  3.19419861e-01,3.08334351e-01,2.88780212e-01,2.15606689e-01,  1.17042542e-01,4.87813950e-02,2.45327950e-02,2.09240913e-02,  2.48670578e-02,3.63111496e-02,3.08237076e-02,3.35712433e-02,  4.54778671e-02,4.50372696e-02,4.98619080e-02,5.58376312e-02,  7.10639954e-02,8.95767212e-02,1.06994629e-01,1.24420166e-01,  1.25778198e-01,1.26499176e-01,1.39209747e-01,1.55208588e-01,  1.75109863e-01,2.08625793e-01,2.14035034e-01,1.90605164e-01,  1.78726196e-01,1.91707611e-01,2.05883026e-01,2.22557068e-01,  2.47550964e-01,2.67417908e-01,2.72087097e-01,2.76176453e-01,  2.86247253e-01,2.88627625e-01,2.89329529e-01,2.91320801e-01,  2.92167664e-01,2.91236877e-01,2.90603638e-01,2.90863037e-01,  2.88825989e-01,2.85575867e-01,2.82310486e-01,2.75039673e-01,  2.60848999e-01,2.53059387e-01,2.58216858e-01,2.63504028e-01,  2.64320374e-01,2.62382507e-01,2.53501892e-01,2.49298096e-01,  2.51800537e-01,2.58041382e-01,2.61634827e-01,2.64900208e-01,  2.65907288e-01,2.65777588e-01,2.65800476e-01,2.65556335e-01,  2.65266418e-01,2.63061523e-01,2.59315491e-01,2.54524231e-01,  2.50518799e-01,2.45876312e-01,2.41836548e-01,2.37106323e-01,  2.33207703e-01,2.28458405e-01,2.26150513e-01],
        [5.27542114e-01,5.44799805e-01,5.63827515e-01,6.52984619e-01,  6.92703247e-01,7.00775146e-01,6.97189331e-01,6.88354492e-01,  7.42034912e-01,8.45993042e-01,1.11880493e+00,1.25518799e+00,  1.31457520e+00,1.36273193e+00,1.36544800e+00,1.32238770e+00,  1.29910278e+00,1.40277100e+00,1.51953125e+00,1.63208008e+00,  1.71228027e+00,1.74673462e+00,1.77203369e+00,1.76614380e+00,  1.77157593e+00,1.80331421e+00,1.81091309e+00,1.74224854e+00,  1.75277710e+00,1.77932739e+00,1.75210571e+00,1.75854492e+00,  1.75082397e+00,1.68212891e+00,1.68453979e+00,1.72082520e+00,  1.74429321e+00,1.74868774e+00,1.73229980e+00,1.73464966e+00,  1.73260498e+00,1.71282959e+00,1.68563843e+00,1.67330933e+00,  1.66390991e+00,1.66247559e+00,1.67071533e+00,1.66339111e+00,  1.63778687e+00,1.63821411e+00,1.62573242e+00,1.61703491e+00,  1.59783936e+00,1.58212280e+00,1.58047485e+00,1.55520630e+00,  1.54089355e+00,1.55361938e+00,1.55834961e+00,1.54794312e+00,  1.52832031e+00,1.48971558e+00,1.51782227e+00,1.54196167e+00,  1.53366089e+00,1.52096558e+00,1.49484253e+00,1.38418579e+00,  1.34622192e+00,1.41235352e+00,1.41955566e+00,1.41903687e+00,  1.40679932e+00,1.34921265e+00,1.29989624e+00,1.29605103e+00,  1.30914307e+00,1.32696533e+00,1.31732178e+00,1.31704712e+00,  1.30136108e+00,1.17767334e+00,8.39111328e-01,9.05670166e-01,  1.16036987e+00,1.22225952e+00,1.21264648e+00,1.20727539e+00,  1.19464111e+00,1.18225098e+00,1.17907715e+00,1.16552734e+00,  1.13165283e+00,1.07482910e+00,1.04238892e+00,1.05136108e+00,  1.05795288e+00,1.05804443e+00,1.06430054e+00,1.04598999e+00,  9.99526978e-01,9.85992432e-01,1.00064087e+00,9.74792480e-01,  9.88922119e-01,9.85305786e-01,9.86328125e-01,9.89791870e-01,  9.62249756e-01,9.15832520e-01,8.78936768e-01,8.56628418e-01,  8.47824097e-01,8.45275879e-01,8.42300415e-01,8.37783813e-01,  6.90719604e-01,5.64071655e-01,5.73837280e-01,5.82870483e-01,  5.94024658e-01,6.16470337e-01,6.52160645e-01,7.01110840e-01,  7.36648560e-01,7.44796753e-01,7.57034302e-01,7.72277832e-01,  7.76992798e-01,7.73849487e-01,7.62878418e-01,7.51968384e-01,  7.49343872e-01,7.45605469e-01,7.38021851e-01,7.29965210e-01,  7.20031738e-01,7.10479736e-01,7.02697754e-01,6.93878174e-01,  6.84600830e-01,6.73034668e-01,6.60491943e-01,6.48361206e-01,  6.41052246e-01,6.37741089e-01,6.29684448e-01,6.19949341e-01,  6.08306885e-01,5.92941284e-01,5.79345703e-01,5.61492920e-01,  5.11489868e-01,4.20753479e-01,3.42933655e-01,3.19686890e-01,  3.29986572e-01,3.52386475e-01,3.80966187e-01,3.71025085e-01,  3.83842468e-01,4.28756714e-01,4.72167969e-01,5.02197266e-01,  5.14541626e-01,5.11474609e-01,5.02410889e-01,4.98497009e-01,  5.04928589e-01,5.02593994e-01,4.89570618e-01,4.85221863e-01,  4.88952637e-01,4.93644714e-01,4.92317200e-01,4.89570618e-01,  4.89166260e-01,4.86862183e-01,4.80781555e-01,4.74502563e-01,  4.66476440e-01,4.50363159e-01,4.20547485e-01,3.89457703e-01,  3.94111633e-01,4.20669556e-01,4.27642822e-01,4.35371399e-01,  4.39910889e-01,4.35394287e-01,4.24896240e-01,4.11338806e-01,  3.94844055e-01,3.80355835e-01,3.71437073e-01,3.54766846e-01,  3.20755005e-01,3.09631348e-01,2.90084839e-01,2.17456818e-01,  1.18846893e-01,4.97026443e-02,2.48613358e-02,2.09922791e-02,  2.51274109e-02,3.72228622e-02,3.14149857e-02,3.41863632e-02,  4.65698242e-02,4.60472107e-02,5.09538651e-02,5.71622849e-02,  7.25517273e-02,9.11846161e-02,1.08682632e-01,1.26178741e-01,  1.27254486e-01,1.27998352e-01,1.40701294e-01,1.56799316e-01,  1.76704407e-01,2.10006714e-01,2.15419769e-01,1.92123413e-01,  1.80286407e-01,1.93084717e-01,2.06974030e-01,2.23506927e-01,  2.48199463e-01,2.67707825e-01,2.72171021e-01,2.76069641e-01,  2.86018372e-01,2.88360596e-01,2.88764954e-01,2.90710449e-01,  2.91336060e-01,2.90382385e-01,2.89710999e-01,2.89909363e-01,  2.88063049e-01,2.84736633e-01,2.81372070e-01,2.74253845e-01,  2.59925842e-01,2.52044678e-01,2.57362366e-01,2.62496948e-01,  2.63366699e-01,2.61268616e-01,2.52532959e-01,2.48378754e-01,  2.50747681e-01,2.57003784e-01,2.60498047e-01,2.63580322e-01,  2.64373779e-01,2.64205933e-01,2.64068604e-01,2.63313293e-01,  2.62992859e-01,2.60116577e-01,2.55729675e-01,2.50999451e-01,  2.46204376e-01,2.41905212e-01,2.36621857e-01,2.32215881e-01,  2.27359772e-01,2.22366333e-01,2.21378326e-01],
        [5.26947021e-01,5.42327881e-01,5.65582275e-01,6.56860352e-01,  6.90704346e-01,6.99630737e-01,6.94580078e-01,6.86599731e-01,  7.43164062e-01,8.44284058e-01,1.11886597e+00,1.25521851e+00,  1.31408691e+00,1.36297607e+00,1.36532593e+00,1.32284546e+00,  1.29882812e+00,1.40295410e+00,1.52011108e+00,1.63153076e+00,  1.71325684e+00,1.74713135e+00,1.77124023e+00,1.76678467e+00,  1.77172852e+00,1.80416870e+00,1.81182861e+00,1.74291992e+00,  1.75448608e+00,1.78036499e+00,1.75314331e+00,1.75909424e+00,  1.75122070e+00,1.68386841e+00,1.68533325e+00,1.72146606e+00,  1.74594116e+00,1.74951172e+00,1.73336792e+00,1.73513794e+00,  1.73333740e+00,1.71551514e+00,1.68725586e+00,1.67504883e+00,  1.66583252e+00,1.66339111e+00,1.67269897e+00,1.66415405e+00,  1.63861084e+00,1.64071655e+00,1.62622070e+00,1.61868286e+00,  1.59954834e+00,1.58288574e+00,1.58294678e+00,1.55523682e+00,  1.54077148e+00,1.55596924e+00,1.55883789e+00,1.55102539e+00,  1.52984619e+00,1.48977661e+00,1.52017212e+00,1.54370117e+00,  1.53561401e+00,1.52420044e+00,1.49560547e+00,1.38641357e+00,  1.34799194e+00,1.41262817e+00,1.42163086e+00,1.42022705e+00,  1.40713501e+00,1.35244751e+00,1.30096436e+00,1.29772949e+00,  1.30987549e+00,1.32775879e+00,1.32095337e+00,1.31625366e+00,  1.30059814e+00,1.18112183e+00,8.36807251e-01,9.04586792e-01,  1.16177368e+00,1.22018433e+00,1.21691895e+00,1.21282959e+00,  1.19805908e+00,1.18569946e+00,1.17770386e+00,1.16336060e+00,  1.13192749e+00,1.06942749e+00,1.04074097e+00,1.05606079e+00,  1.05804443e+00,1.06390381e+00,1.06759644e+00,1.04458618e+00,  1.00283813e+00,9.89761353e-01,1.00155640e+00,9.79476929e-01,  9.84863281e-01,9.92492676e-01,9.90264893e-01,9.87548828e-01,  9.70993042e-01,9.20181274e-01,8.80538940e-01,8.72207642e-01,  8.57498169e-01,8.47473145e-01,8.60595703e-01,8.37875366e-01,  6.90170288e-01,5.65780640e-01,5.75485229e-01,5.84228516e-01,  5.95596313e-01,6.17858887e-01,6.53442383e-01,7.02346802e-01,  7.37945557e-01,7.46017456e-01,7.58117676e-01,7.73330688e-01,  7.77908325e-01,7.74642944e-01,7.63702393e-01,7.52944946e-01,  7.50564575e-01,7.46658325e-01,7.39013672e-01,7.31018066e-01,  7.21130371e-01,7.11349487e-01,7.03506470e-01,6.94686890e-01,  6.85455322e-01,6.74087524e-01,6.61331177e-01,6.49063110e-01,  6.41769409e-01,6.38732910e-01,6.30661011e-01,6.20727539e-01,  6.09222412e-01,5.93688965e-01,5.80123901e-01,5.62530518e-01,  5.12466431e-01,4.21737671e-01,3.44123840e-01,3.20915222e-01,  3.30993652e-01,3.53424072e-01,3.81988525e-01,3.71994019e-01,  3.84872437e-01,4.29840088e-01,4.73030090e-01,5.03158569e-01,  5.15487671e-01,5.12191772e-01,5.03189087e-01,4.99397278e-01,  5.05691528e-01,5.03234863e-01,4.90257263e-01,4.85954285e-01,  4.89715576e-01,4.94415283e-01,4.92958069e-01,4.90333557e-01,  4.89837646e-01,4.87503052e-01,4.81445312e-01,4.75082397e-01,  4.67086792e-01,4.50843811e-01,4.21066284e-01,3.89991760e-01,  3.94577026e-01,4.21112061e-01,4.28268433e-01,4.36019897e-01,  4.40414429e-01,4.36119080e-01,4.25552368e-01,4.11972046e-01,  3.95553589e-01,3.81027222e-01,3.72055054e-01,3.55552673e-01,  3.21563721e-01,3.10348511e-01,2.90931702e-01,2.18231201e-01,  1.19493484e-01,5.04226685e-02,2.55379677e-02,2.15730667e-02,  2.58784294e-02,3.78942490e-02,3.20940018e-02,3.50065231e-02,  4.74138260e-02,4.68244553e-02,5.17225266e-02,5.78937531e-02,  7.33928680e-02,9.20448303e-02,1.09390259e-01,1.26983643e-01,  1.28063202e-01,1.28799438e-01,1.41597748e-01,1.57634735e-01,  1.77516937e-01,2.10922241e-01,2.16297150e-01,1.92974091e-01,  1.81034088e-01,1.93817139e-01,2.07847595e-01,2.24349976e-01,  2.48962402e-01,2.68470764e-01,2.72827148e-01,2.76741028e-01,  2.86682129e-01,2.88948059e-01,2.89497375e-01,2.91511536e-01,  2.92167664e-01,2.91213989e-01,2.90481567e-01,2.90756226e-01,  2.88734436e-01,2.85514832e-01,2.82150269e-01,2.74902344e-01,  2.60749817e-01,2.52838135e-01,2.58209229e-01,2.63481140e-01,  2.64274597e-01,2.62306213e-01,2.53425598e-01,2.49328613e-01,  2.51785278e-01,2.57949829e-01,2.61734009e-01,2.64816284e-01,  2.65670776e-01,2.65625000e-01,2.65556335e-01,2.65281677e-01,  2.65022278e-01,2.62962341e-01,2.59315491e-01,2.54936218e-01,  2.50274658e-01,2.45906830e-01,2.41489410e-01,2.37350464e-01,  2.33932495e-01,2.28698730e-01,2.25849152e-01],
        [5.26397705e-01,5.39215088e-01,5.59829712e-01,6.52786255e-01,  6.89407349e-01,6.97265625e-01,6.93420410e-01,6.84509277e-01,  7.40402222e-01,8.43200684e-01,1.11444092e+00,1.25152588e+00,  1.31088257e+00,1.35949707e+00,1.36245728e+00,1.31948853e+00,  1.29541016e+00,1.39926147e+00,1.51635742e+00,1.62933350e+00,  1.70965576e+00,1.74340820e+00,1.76788330e+00,1.76275635e+00,  1.76821899e+00,1.79959106e+00,1.80767822e+00,1.73898315e+00,  1.74932861e+00,1.77648926e+00,1.74960327e+00,1.75537109e+00,  1.74795532e+00,1.67996216e+00,1.68289185e+00,1.71896362e+00,  1.74310303e+00,1.74652100e+00,1.72985840e+00,1.73239136e+00,  1.73049927e+00,1.71163940e+00,1.68426514e+00,1.67117310e+00,  1.66189575e+00,1.66033936e+00,1.66943359e+00,1.66271973e+00,  1.63595581e+00,1.63714600e+00,1.62432861e+00,1.61669922e+00,  1.59686279e+00,1.58053589e+00,1.57946777e+00,1.55407715e+00,  1.53906250e+00,1.55264282e+00,1.55737305e+00,1.54714966e+00,  1.52819824e+00,1.48840332e+00,1.51712036e+00,1.54220581e+00,  1.53341675e+00,1.52047729e+00,1.49472046e+00,1.38510132e+00,  1.34582520e+00,1.41113281e+00,1.42001343e+00,1.41940308e+00,  1.40631104e+00,1.34930420e+00,1.29989624e+00,1.29638672e+00,  1.30859375e+00,1.32546997e+00,1.31765747e+00,1.31616211e+00,  1.29989624e+00,1.17700195e+00,8.36776733e-01,9.03244019e-01,  1.15945435e+00,1.22183228e+00,1.21667480e+00,1.21197510e+00,  1.19534302e+00,1.18182373e+00,1.17749023e+00,1.16262817e+00,  1.12814331e+00,1.07238770e+00,1.03939819e+00,1.04846191e+00,  1.05722046e+00,1.06130981e+00,1.06265259e+00,1.04656982e+00,  9.99649048e-01,9.93301392e-01,1.00320435e+00,9.77996826e-01,  9.88204956e-01,9.78713989e-01,9.79873657e-01,9.87258911e-01,  9.67086792e-01,9.18594360e-01,8.75183105e-01,8.62640381e-01,  8.49899292e-01,8.38592529e-01,8.35479736e-01,8.34533691e-01,  6.89483643e-01,5.62255859e-01,5.72052002e-01,5.81268311e-01,  5.92544556e-01,6.14654541e-01,6.50588989e-01,7.00378418e-01,  7.36038208e-01,7.44003296e-01,7.56652832e-01,7.72109985e-01,  7.76672363e-01,7.73620605e-01,7.62847900e-01,7.51785278e-01,  7.49206543e-01,7.45712280e-01,7.38052368e-01,7.29827881e-01,  7.20077515e-01,7.10494995e-01,7.02468872e-01,6.93679810e-01,  6.84600830e-01,6.73049927e-01,6.60293579e-01,6.48223877e-01,  6.41067505e-01,6.37649536e-01,6.29501343e-01,6.19674683e-01,  6.08261108e-01,5.92636108e-01,5.78857422e-01,5.60989380e-01,  5.10726929e-01,4.19441223e-01,3.41606140e-01,3.18542480e-01,  3.28598022e-01,3.51165771e-01,3.80004883e-01,3.69888306e-01,  3.82659912e-01,4.27803040e-01,4.71435547e-01,5.01724243e-01,  5.14358521e-01,5.11093140e-01,5.01922607e-01,4.98199463e-01,  5.04760742e-01,5.02197266e-01,4.89112854e-01,4.84901428e-01,  4.88723755e-01,4.93392944e-01,4.92065430e-01,4.89532471e-01,  4.89051819e-01,4.86732483e-01,4.80758667e-01,4.74426270e-01,  4.66209412e-01,4.50157166e-01,4.20288086e-01,3.88999939e-01,  3.93707275e-01,4.20402527e-01,4.27360535e-01,4.35264587e-01,  4.39750671e-01,4.35226440e-01,4.24629211e-01,4.11094666e-01,  3.94363403e-01,3.79837036e-01,3.70864868e-01,3.54095459e-01,  3.19931030e-01,3.08830261e-01,2.89337158e-01,2.16464996e-01,  1.17860794e-01,4.91647720e-02,2.45456696e-02,2.07009315e-02,  2.49099731e-02,3.67374420e-02,3.10273170e-02,3.38964462e-02,  4.59718704e-02,4.55083847e-02,5.05113602e-02,5.64908981e-02,  7.18574524e-02,9.04178619e-02,1.07696533e-01,1.25259399e-01,  1.26483917e-01,1.27212524e-01,1.39984131e-01,1.56024933e-01,  1.75994873e-01,2.09323883e-01,2.14622498e-01,1.91307068e-01,  1.79412842e-01,1.92314148e-01,2.06459045e-01,2.22869873e-01,  2.47661591e-01,2.67288208e-01,2.71736145e-01,2.75756836e-01,  2.85842896e-01,2.88055420e-01,2.88696289e-01,2.90603638e-01,  2.91213989e-01,2.90298462e-01,2.89649963e-01,2.89901733e-01,  2.87956238e-01,2.84675598e-01,2.81280518e-01,2.74032593e-01,  2.59780884e-01,2.51983643e-01,2.57293701e-01,2.62390137e-01,  2.63320923e-01,2.61314392e-01,2.52273560e-01,2.48279572e-01,  2.50663757e-01,2.56729126e-01,2.60353088e-01,2.63481140e-01,  2.64450073e-01,2.64183044e-01,2.63679504e-01,2.63130188e-01,  2.62733459e-01,2.60261536e-01,2.55813599e-01,2.50587463e-01,  2.46150970e-01,2.41584778e-01,2.37075806e-01,2.32093811e-01,  2.28755951e-01,2.22995758e-01,2.19627380e-01],
        [5.23071289e-01,5.37567139e-01,5.59448242e-01,6.48254395e-01,  6.86706543e-01,6.94396973e-01,6.91909790e-01,6.84768677e-01,  7.38967896e-01,8.41110229e-01,1.11245728e+00,1.24926758e+00,  1.30941772e+00,1.35641479e+00,1.35989380e+00,1.31741333e+00,  1.29296875e+00,1.39785767e+00,1.51449585e+00,1.62731934e+00,  1.70822144e+00,1.74151611e+00,1.76785278e+00,1.76205444e+00,  1.76760864e+00,1.80004883e+00,1.80792236e+00,1.73977661e+00,  1.74938965e+00,1.77645874e+00,1.74978638e+00,1.75561523e+00,  1.74862671e+00,1.68023682e+00,1.68209839e+00,1.71926880e+00,  1.74340820e+00,1.74722290e+00,1.73062134e+00,1.73254395e+00,  1.73150635e+00,1.71249390e+00,1.68557739e+00,1.67300415e+00,  1.66333008e+00,1.66207886e+00,1.67077637e+00,1.66156006e+00,  1.63714600e+00,1.63726807e+00,1.62432861e+00,1.61709595e+00,  1.59634399e+00,1.58142090e+00,1.58065796e+00,1.55484009e+00,  1.54043579e+00,1.55319214e+00,1.55780029e+00,1.54846191e+00,  1.52844238e+00,1.49020386e+00,1.51791382e+00,1.54165649e+00,  1.53543091e+00,1.52182007e+00,1.49572754e+00,1.38528442e+00,  1.34536743e+00,1.41244507e+00,1.42126465e+00,1.42047119e+00,  1.40811157e+00,1.34854126e+00,1.30072021e+00,1.29708862e+00,  1.30700684e+00,1.32620239e+00,1.31835938e+00,1.31460571e+00,  1.30142212e+00,1.17898560e+00,8.38409424e-01,9.05044556e-01,  1.15814209e+00,1.22210693e+00,1.21621704e+00,1.20895386e+00,  1.19918823e+00,1.18347168e+00,1.17703247e+00,1.16790771e+00,  1.12814331e+00,1.07153320e+00,1.04162598e+00,1.04702759e+00,  1.06097412e+00,1.06198120e+00,1.06045532e+00,1.05029297e+00,  1.00338745e+00,9.91760254e-01,1.01034546e+00,9.72839355e-01,  9.91897583e-01,9.99893188e-01,9.78576660e-01,9.85443115e-01,  9.63745117e-01,9.09576416e-01,8.78387451e-01,8.67202759e-01,  8.54034424e-01,8.57406616e-01,8.39172363e-01,8.26385498e-01,  6.86386108e-01,5.61203003e-01,5.71090698e-01,5.80291748e-01,  5.91857910e-01,6.14181519e-01,6.50314331e-01,7.00256348e-01,  7.36389160e-01,7.44476318e-01,7.57202148e-01,7.73132324e-01,  7.77862549e-01,7.74734497e-01,7.63854980e-01,7.53097534e-01,  7.50503540e-01,7.46734619e-01,7.39105225e-01,7.30880737e-01,  7.21237183e-01,7.11517334e-01,7.03674316e-01,6.94854736e-01,  6.85607910e-01,6.74041748e-01,6.61453247e-01,6.49200439e-01,  6.41815186e-01,6.38458252e-01,6.30462646e-01,6.20559692e-01,  6.08932495e-01,5.93338013e-01,5.79284668e-01,5.61172485e-01,  5.10772705e-01,4.19067383e-01,3.40774536e-01,3.17443848e-01,  3.27796936e-01,3.50517273e-01,3.79287720e-01,3.69171143e-01,  3.82110596e-01,4.27452087e-01,4.71496582e-01,5.02151489e-01,  5.14724731e-01,5.11566162e-01,5.02395630e-01,4.98619080e-01,  5.05065918e-01,5.02670288e-01,4.89593506e-01,4.85343933e-01,  4.89158630e-01,4.93980408e-01,4.92721558e-01,4.90242004e-01,  4.89845276e-01,4.87556458e-01,4.81483459e-01,4.75105286e-01,  4.66964722e-01,4.50828552e-01,4.20867920e-01,3.89640808e-01,  3.94386292e-01,4.21157837e-01,4.28131104e-01,4.35913086e-01,  4.40467834e-01,4.35882568e-01,4.25071716e-01,4.11437988e-01,  3.94851685e-01,3.80172729e-01,3.71124268e-01,3.54370117e-01,  3.19915771e-01,3.08776855e-01,2.89321899e-01,2.16239929e-01,  1.17416382e-01,4.90636826e-02,2.48069763e-02,2.09512711e-02,  2.49195099e-02,3.66163254e-02,3.09853554e-02,3.38544846e-02,  4.59556580e-02,4.54483032e-02,5.03244400e-02,5.64479828e-02,  7.16362000e-02,9.00707245e-02,1.07545853e-01,1.25007629e-01,  1.26129150e-01,1.26865387e-01,1.39602661e-01,1.55632019e-01,  1.75682068e-01,2.09053040e-01,2.14374542e-01,1.91047668e-01,  1.79161072e-01,1.92066193e-01,2.06207275e-01,2.22888947e-01,  2.47756958e-01,2.67608643e-01,2.72224426e-01,2.76161194e-01,  2.86285400e-01,2.88711548e-01,2.89344788e-01,2.91305542e-01,  2.92098999e-01,2.91046143e-01,2.90367126e-01,2.90641785e-01,  2.88635254e-01,2.85453796e-01,2.82196045e-01,2.74925232e-01,  2.60711670e-01,2.52746582e-01,2.57980347e-01,2.63313293e-01,  2.64167786e-01,2.62191772e-01,2.53364563e-01,2.49210358e-01,  2.51609802e-01,2.57850647e-01,2.61390686e-01,2.64511108e-01,  2.65518188e-01,2.65190125e-01,2.65129089e-01,2.64907837e-01,  2.64541626e-01,2.62443542e-01,2.58514404e-01,2.54005432e-01,  2.50289917e-01,2.45960236e-01,2.41199493e-01,2.37369537e-01,  2.33268738e-01,2.28374481e-01,2.26512909e-01],
        [5.23193359e-01,5.37063599e-01,5.60760498e-01,6.48239136e-01,  6.84402466e-01,6.94946289e-01,6.89147949e-01,6.82388306e-01,  7.38616943e-01,8.39736938e-01,1.11209106e+00,1.24734497e+00,  1.30709839e+00,1.35577393e+00,1.35781860e+00,1.31732178e+00,  1.29214478e+00,1.39727783e+00,1.51486206e+00,1.62594604e+00,  1.70751953e+00,1.74038696e+00,1.76547241e+00,1.76101685e+00,  1.76553345e+00,1.79843140e+00,1.80603027e+00,1.73800659e+00,  1.74887085e+00,1.77487183e+00,1.74893188e+00,1.75445557e+00,  1.74700928e+00,1.68029785e+00,1.68124390e+00,1.71862793e+00,  1.74316406e+00,1.74633789e+00,1.73086548e+00,1.73245239e+00,  1.73083496e+00,1.71356201e+00,1.68511963e+00,1.67236328e+00,  1.66204834e+00,1.65960693e+00,1.66931152e+00,1.66067505e+00,  1.63525391e+00,1.63739014e+00,1.62268066e+00,1.61666870e+00,  1.59695435e+00,1.58041382e+00,1.58062744e+00,1.55206299e+00,  1.53820801e+00,1.55349731e+00,1.55529785e+00,1.54751587e+00,  1.52862549e+00,1.48825073e+00,1.51879883e+00,1.54241943e+00,  1.53594971e+00,1.52398682e+00,1.49468994e+00,1.38684082e+00,  1.34616089e+00,1.41067505e+00,1.42120361e+00,1.41860962e+00,  1.40789795e+00,1.35028076e+00,1.29626465e+00,1.29629517e+00,  1.30923462e+00,1.32623291e+00,1.32138062e+00,1.31558228e+00,  1.30035400e+00,1.18096924e+00,8.37127686e-01,9.04235840e-01,  1.15975952e+00,1.22119141e+00,1.22018433e+00,1.21090698e+00,  1.19470215e+00,1.18634033e+00,1.17758179e+00,1.16696167e+00,  1.13357544e+00,1.06884766e+00,1.04187012e+00,1.04953003e+00,  1.05569458e+00,1.06225586e+00,1.06320190e+00,1.04431152e+00,  9.97375488e-01,9.83184814e-01,1.00408936e+00,9.77966309e-01,  9.77478027e-01,9.89974976e-01,9.86343384e-01,9.86221313e-01,  9.73068237e-01,9.13284302e-01,8.66928101e-01,8.69842529e-01,  8.47198486e-01,8.50143433e-01,8.46405029e-01,8.24340820e-01,  6.84219360e-01,5.61248779e-01,5.71365356e-01,5.80307007e-01,  5.91690063e-01,6.14349365e-01,6.50604248e-01,7.00988770e-01,  7.37503052e-01,7.45620728e-01,7.58529663e-01,7.74597168e-01,  7.79220581e-01,7.76321411e-01,7.65594482e-01,7.54470825e-01,  7.51815796e-01,7.48229980e-01,7.40463257e-01,7.32406616e-01,  7.22671509e-01,7.12982178e-01,7.04910278e-01,6.96090698e-01,  6.87057495e-01,6.75430298e-01,6.62612915e-01,6.50512695e-01,  6.43142700e-01,6.39755249e-01,6.31683350e-01,6.21810913e-01,  6.10031128e-01,5.94299316e-01,5.80200195e-01,5.61996460e-01,  5.11184692e-01,4.19158936e-01,3.40660095e-01,3.17237854e-01,  3.27560425e-01,3.50387573e-01,3.79417419e-01,3.69171143e-01,  3.82095337e-01,4.27764893e-01,4.72076416e-01,5.02609253e-01,  5.15350342e-01,5.12374878e-01,5.03128052e-01,4.99130249e-01,  5.05935669e-01,5.03555298e-01,4.90455627e-01,4.86175537e-01,  4.90081787e-01,4.94834900e-01,4.93614197e-01,4.91165161e-01,  4.90829468e-01,4.88487244e-01,4.82482910e-01,4.76150513e-01,  4.67895508e-01,4.51644897e-01,4.21661377e-01,3.90289307e-01,  3.94920349e-01,4.21882629e-01,4.29016113e-01,4.36729431e-01,  4.41299438e-01,4.36798096e-01,4.25956726e-01,4.12239075e-01,  3.95538330e-01,3.80790710e-01,3.71574402e-01,3.54644775e-01,  3.20075989e-01,3.08906555e-01,2.89360046e-01,2.16175079e-01,  1.17261887e-01,4.89244461e-02,2.48446465e-02,2.10518837e-02,  2.49433517e-02,3.65209579e-02,3.09238434e-02,3.37409973e-02,  4.57649231e-02,4.52785492e-02,5.01480103e-02,5.61513901e-02,  7.13367462e-02,8.98189545e-02,1.07328415e-01,1.24868393e-01,  1.26010895e-01,1.26701355e-01,1.39362335e-01,1.55445099e-01,  1.75491333e-01,2.09026337e-01,2.14454651e-01,1.90937042e-01,  1.79058075e-01,1.92077637e-01,2.06272125e-01,2.22911835e-01,  2.47978210e-01,2.68020630e-01,2.72811890e-01,2.76763916e-01,  2.86949158e-01,2.89230347e-01,2.89947510e-01,2.91969299e-01,  2.92663574e-01,2.91793823e-01,2.91160583e-01,2.91496277e-01,  2.89489746e-01,2.86132812e-01,2.82859802e-01,2.75558472e-01,  2.61230469e-01,2.53372192e-01,2.58728027e-01,2.63992310e-01,  2.64877319e-01,2.62901306e-01,2.53936768e-01,2.49851227e-01,  2.52365112e-01,2.58438110e-01,2.62176514e-01,2.65296936e-01,  2.66143799e-01,2.66197205e-01,2.66128540e-01,2.65846252e-01,  2.65586853e-01,2.63313293e-01,2.60459900e-01,2.55767822e-01,  2.51365662e-01,2.48043060e-01,2.43587494e-01,2.39505768e-01,  2.36255646e-01,2.30838776e-01,2.28351593e-01],
        [5.19226074e-01,5.35842896e-01,5.57235718e-01,6.47766113e-01,  6.84539795e-01,6.92626953e-01,6.89590454e-01,6.80343628e-01,  7.36511230e-01,8.37112427e-01,1.10842896e+00,1.24621582e+00,  1.30590820e+00,1.35476685e+00,1.35836792e+00,1.31713867e+00,  1.29275513e+00,1.39532471e+00,1.51333618e+00,1.62515259e+00,  1.70629883e+00,1.74075317e+00,1.76550293e+00,1.76040649e+00,  1.76525879e+00,1.79736328e+00,1.80661011e+00,1.73843384e+00,  1.74865723e+00,1.77560425e+00,1.74884033e+00,1.75473022e+00,  1.74777222e+00,1.67980957e+00,1.68136597e+00,1.71813965e+00,  1.74224854e+00,1.74667358e+00,1.73043823e+00,1.73248291e+00,  1.73147583e+00,1.71322632e+00,1.68536377e+00,1.67230225e+00,  1.66271973e+00,1.66101074e+00,1.66995239e+00,1.66259766e+00,  1.63644409e+00,1.63735962e+00,1.62359619e+00,1.61618042e+00,  1.59771729e+00,1.58172607e+00,1.58096313e+00,1.55404663e+00,  1.53909302e+00,1.55319214e+00,1.55722046e+00,1.54858398e+00,  1.52972412e+00,1.48971558e+00,1.51937866e+00,1.54318237e+00,  1.53530884e+00,1.52404785e+00,1.49639893e+00,1.38601685e+00,  1.34561157e+00,1.41082764e+00,1.42071533e+00,1.42065430e+00,  1.40795898e+00,1.35098267e+00,1.29818726e+00,1.29629517e+00,  1.30932617e+00,1.32812500e+00,1.32098389e+00,1.31851196e+00,  1.30157471e+00,1.18133545e+00,8.36013794e-01,8.99642944e-01,  1.16131592e+00,1.22311401e+00,1.21722412e+00,1.21218872e+00,  1.19613647e+00,1.18511963e+00,1.18194580e+00,1.16720581e+00,  1.13320923e+00,1.07379150e+00,1.04141235e+00,1.04837036e+00,  1.05612183e+00,1.06442261e+00,1.06668091e+00,1.04266357e+00,  9.98306274e-01,9.85000610e-01,9.99664307e-01,9.85671997e-01,  9.97146606e-01,9.89944458e-01,9.79339600e-01,9.80148315e-01,  9.65942383e-01,9.28848267e-01,8.79165649e-01,8.72985840e-01,  8.53134155e-01,8.41094971e-01,8.50204468e-01,8.32061768e-01,  6.88583374e-01,5.60821533e-01,5.70938110e-01,5.79895020e-01,  5.91308594e-01,6.13998413e-01,6.50238037e-01,7.00363159e-01,  7.36892700e-01,7.45239258e-01,7.57858276e-01,7.73864746e-01,  7.78945923e-01,7.75833130e-01,7.64938354e-01,7.54013062e-01,  7.51403809e-01,7.47772217e-01,7.40112305e-01,7.31918335e-01,  7.22259521e-01,7.12600708e-01,7.04513550e-01,6.95739746e-01,  6.86538696e-01,6.74942017e-01,6.62094116e-01,6.50009155e-01,  6.42578125e-01,6.39251709e-01,6.31134033e-01,6.21276855e-01,  6.09558105e-01,5.93841553e-01,5.79757690e-01,5.61492920e-01,  5.10833740e-01,4.18884277e-01,3.40209961e-01,3.16925049e-01,  3.27308655e-01,3.50090027e-01,3.78974915e-01,3.68843079e-01,  3.81813049e-01,4.27383423e-01,4.71710205e-01,5.02380371e-01,  5.14999390e-01,5.11901855e-01,5.02761841e-01,4.98970032e-01,  5.05554199e-01,5.03067017e-01,4.90081787e-01,4.85870361e-01,  4.89639282e-01,4.94415283e-01,4.93301392e-01,4.90776062e-01,  4.90425110e-01,4.88113403e-01,4.82131958e-01,4.75814819e-01,  4.67475891e-01,4.51164246e-01,4.21112061e-01,3.89701843e-01,  3.94348145e-01,4.21356201e-01,4.28611755e-01,4.36355591e-01,  4.40803528e-01,4.36363220e-01,4.25636292e-01,4.11880493e-01,  3.95126343e-01,3.80462646e-01,3.71391296e-01,3.54309082e-01,  3.19740295e-01,3.08593750e-01,2.88993835e-01,2.15774536e-01,  1.17063522e-01,4.87174988e-02,2.45966911e-02,2.09035873e-02,  2.48041153e-02,3.63035202e-02,3.07545662e-02,3.36160660e-02,  4.56314087e-02,4.50706482e-02,4.99076843e-02,5.60140610e-02,  7.12242126e-02,8.96873474e-02,1.07124329e-01,1.24555588e-01,  1.25736237e-01,1.26419067e-01,1.39102936e-01,1.55193329e-01,  1.75231934e-01,2.08778381e-01,2.14214325e-01,1.90753937e-01,  1.78836823e-01,1.91890717e-01,2.06047058e-01,2.22728729e-01,  2.47722626e-01,2.67623901e-01,2.72331238e-01,2.76435852e-01,  2.86590576e-01,2.89009094e-01,2.89573669e-01,2.91458130e-01,  2.92228699e-01,2.91313171e-01,2.90664673e-01,2.90969849e-01,  2.88932800e-01,2.85652161e-01,2.82386780e-01,2.75070190e-01,  2.60742188e-01,2.52998352e-01,2.58270264e-01,2.63542175e-01,  2.64579773e-01,2.62489319e-01,2.53379822e-01,2.49248505e-01,  2.51762390e-01,2.57820129e-01,2.61573792e-01,2.64778137e-01,  2.65495300e-01,2.65495300e-01,2.65205383e-01,2.64595032e-01,  2.64633179e-01,2.62435913e-01,2.58453369e-01,2.54318237e-01,  2.50221252e-01,2.45285034e-01,2.41481781e-01,2.37537384e-01,  2.33329773e-01,2.28553772e-01,2.25990295e-01],
        [5.18768311e-01,5.33279419e-01,5.54672241e-01,6.46270752e-01,  6.79916382e-01,6.90032959e-01,6.86584473e-01,6.77444458e-01,  7.35580444e-01,8.35418701e-01,1.10635376e+00,1.24423218e+00,  1.30258179e+00,1.35260010e+00,1.35610962e+00,1.31466675e+00,  1.29095459e+00,1.39376831e+00,1.51110840e+00,1.62203979e+00,  1.70339966e+00,1.73846436e+00,1.76333618e+00,1.75973511e+00,  1.76483154e+00,1.79635620e+00,1.80499268e+00,1.73678589e+00,  1.74740601e+00,1.77401733e+00,1.74703979e+00,1.75335693e+00,  1.74557495e+00,1.67846680e+00,1.68099976e+00,1.71731567e+00,  1.74142456e+00,1.74539185e+00,1.72937012e+00,1.73168945e+00,  1.72970581e+00,1.71197510e+00,1.68359375e+00,1.67016602e+00,  1.66156006e+00,1.65905762e+00,1.66815186e+00,1.66162109e+00,  1.63424683e+00,1.63699341e+00,1.62329102e+00,1.61584473e+00,  1.59768677e+00,1.57958984e+00,1.57946777e+00,1.55419922e+00,  1.53851318e+00,1.55407715e+00,1.55728149e+00,1.54800415e+00,  1.52941895e+00,1.48834229e+00,1.51748657e+00,1.54312134e+00,  1.53472900e+00,1.52328491e+00,1.49560547e+00,1.38562012e+00,  1.34689331e+00,1.41061401e+00,1.42080688e+00,1.42135620e+00,  1.40762329e+00,1.35134888e+00,1.29934692e+00,1.29574585e+00,  1.30938721e+00,1.32540894e+00,1.31970215e+00,1.31866455e+00,  1.30191040e+00,1.18338013e+00,8.40209961e-01,9.01184082e-01,  1.16088867e+00,1.22140503e+00,1.21923828e+00,1.21432495e+00,  1.19589233e+00,1.18615723e+00,1.18344116e+00,1.16546631e+00,  1.13275146e+00,1.06787109e+00,1.03738403e+00,1.05038452e+00,  1.05130005e+00,1.05798340e+00,1.06933594e+00,1.04458618e+00,  1.00357056e+00,9.88571167e-01,9.97756958e-01,9.79598999e-01,  9.80773926e-01,9.84405518e-01,9.94857788e-01,9.85671997e-01,  9.72061157e-01,9.29641724e-01,8.75976562e-01,8.75869751e-01,  8.53607178e-01,8.35952759e-01,8.56384277e-01,8.31039429e-01,  6.82754517e-01,5.58242798e-01,5.68679810e-01,5.77682495e-01,  5.89019775e-01,6.11785889e-01,6.48574829e-01,6.99188232e-01,  7.35748291e-01,7.44064331e-01,7.57247925e-01,7.73376465e-01,  7.78411865e-01,7.75527954e-01,7.64755249e-01,7.53646851e-01,  7.50991821e-01,7.47299194e-01,7.39746094e-01,7.31597900e-01,  7.21710205e-01,7.12142944e-01,7.04391479e-01,6.95495605e-01,  6.86096191e-01,6.74591064e-01,6.61804199e-01,6.49566650e-01,  6.42196655e-01,6.38977051e-01,6.30981445e-01,6.20986938e-01,  6.09085083e-01,5.93246460e-01,5.79132080e-01,5.60684204e-01,  5.09567261e-01,4.17335510e-01,3.38539124e-01,3.14910889e-01,  3.25294495e-01,3.48442078e-01,3.77410889e-01,3.67042542e-01,  3.80195618e-01,4.26124573e-01,4.70695496e-01,5.01510620e-01,  5.14251709e-01,5.11276245e-01,5.02029419e-01,4.98146057e-01,  5.04898071e-01,5.02593994e-01,4.89402771e-01,4.85191345e-01,  4.89242554e-01,4.94056702e-01,4.92835999e-01,4.90341187e-01,  4.90127563e-01,4.87892151e-01,4.81750488e-01,4.75395203e-01,  4.67254639e-01,4.50820923e-01,4.20516968e-01,3.89144897e-01,  3.93951416e-01,4.20890808e-01,4.28245544e-01,4.36141968e-01,  4.40582275e-01,4.35905457e-01,4.25102234e-01,4.11338806e-01,  3.94599915e-01,3.79730225e-01,3.70559692e-01,3.53530884e-01,  3.18656921e-01,3.07502747e-01,2.88009644e-01,2.14477539e-01,  1.15640640e-01,4.77409363e-02,2.38580704e-02,2.01444626e-02,  2.40082741e-02,3.53021622e-02,2.98976898e-02,3.25975418e-02,  4.43010330e-02,4.40454483e-02,4.89025116e-02,5.48915863e-02,  7.00893402e-02,8.83693695e-02,1.05747223e-01,1.23289108e-01,  1.24500275e-01,1.25133514e-01,1.37786865e-01,1.53907776e-01,  1.74022675e-01,2.07515717e-01,2.13050842e-01,1.89491272e-01,  1.77467346e-01,1.90628052e-01,2.04845428e-01,2.21553802e-01,  2.46837616e-01,2.66883850e-01,2.71568298e-01,2.75688171e-01,  2.85934448e-01,2.88414001e-01,2.89062500e-01,2.91175842e-01,  2.91748047e-01,2.90779114e-01,2.90168762e-01,2.90451050e-01,  2.88452148e-01,2.85186768e-01,2.81883240e-01,2.74467468e-01,  2.60047913e-01,2.52235413e-01,2.57560730e-01,2.62733459e-01,  2.63702393e-01,2.61672974e-01,2.52487183e-01,2.48252869e-01,  2.50747681e-01,2.57011414e-01,2.60551453e-01,2.63679504e-01,  2.64518738e-01,2.64152527e-01,2.63854980e-01,2.63084412e-01,  2.62451172e-01,2.59719849e-01,2.55409241e-01,2.50244141e-01,  2.45834351e-01,2.41447449e-01,2.36576080e-01,2.32151031e-01,  2.28179932e-01,2.22904205e-01,2.21054077e-01],
        [5.13977051e-01,5.29891968e-01,5.54901123e-01,6.42333984e-01,  6.80191040e-01,6.88110352e-01,6.82861328e-01,6.76696777e-01,  7.31750488e-01,8.33877563e-01,1.10385132e+00,1.23928833e+00,  1.30023193e+00,1.34790039e+00,1.35134888e+00,1.31085205e+00,  1.28659058e+00,1.39175415e+00,1.50784302e+00,1.61975098e+00,  1.70068359e+00,1.73391724e+00,1.75994873e+00,1.75555420e+00,  1.76110840e+00,1.79342651e+00,1.80065918e+00,1.73364258e+00,  1.74423218e+00,1.77026367e+00,1.74511719e+00,1.74978638e+00,  1.74365234e+00,1.67620850e+00,1.67669678e+00,1.71423340e+00,  1.73785400e+00,1.74169922e+00,1.72637939e+00,1.72769165e+00,  1.72705078e+00,1.70822144e+00,1.68051147e+00,1.66854858e+00,  1.65701294e+00,1.65591431e+00,1.66534424e+00,1.65695190e+00,  1.63327026e+00,1.63293457e+00,1.61950684e+00,1.61343384e+00,  1.59347534e+00,1.57821655e+00,1.57736206e+00,1.54968262e+00,  1.53707886e+00,1.55020142e+00,1.55456543e+00,1.54598999e+00,  1.52536011e+00,1.48773193e+00,1.51623535e+00,1.54089355e+00,  1.53506470e+00,1.52130127e+00,1.49411011e+00,1.38507080e+00,  1.34295654e+00,1.41012573e+00,1.41854858e+00,1.41751099e+00,  1.40917969e+00,1.34906006e+00,1.29754639e+00,1.29589844e+00,  1.30691528e+00,1.32653809e+00,1.32034302e+00,1.31613159e+00,  1.30282593e+00,1.17913818e+00,8.35220337e-01,9.01809692e-01,  1.15615845e+00,1.22183228e+00,1.21817017e+00,1.20858765e+00,  1.19802856e+00,1.18261719e+00,1.17645264e+00,1.16967773e+00,  1.13000488e+00,1.07150269e+00,1.04043579e+00,1.04592896e+00,  1.05783081e+00,1.06246948e+00,1.06060791e+00,1.05102539e+00,  9.99465942e-01,9.83398438e-01,1.00384521e+00,9.71740723e-01,  9.83703613e-01,9.89181519e-01,9.82192993e-01,9.96551514e-01,  9.79843140e-01,9.18014526e-01,8.79837036e-01,8.58078003e-01,  8.48388672e-01,8.56643677e-01,8.42239380e-01,8.25012207e-01,  6.85363770e-01,5.59677124e-01,5.69580078e-01,5.78643799e-01,  5.90148926e-01,6.12670898e-01,6.48681641e-01,6.99157715e-01,  7.35626221e-01,7.43713379e-01,7.56561279e-01,7.72598267e-01,  7.77526855e-01,7.74490356e-01,7.63610840e-01,7.52609253e-01,  7.50045776e-01,7.46200562e-01,7.38708496e-01,7.30758667e-01,  7.21038818e-01,7.11135864e-01,7.03231812e-01,6.94656372e-01,  6.85379028e-01,6.73492432e-01,6.60614014e-01,6.48498535e-01,  6.41281128e-01,6.38015747e-01,6.30081177e-01,6.20178223e-01,  6.08566284e-01,5.92742920e-01,5.78552246e-01,5.60394287e-01,  5.09719849e-01,4.17739868e-01,3.39302063e-01,3.16238403e-01,  3.26629639e-01,3.49365234e-01,3.78150940e-01,3.67874146e-01,  3.80943298e-01,4.26521301e-01,4.70527649e-01,5.01235962e-01,  5.13946533e-01,5.10848999e-01,5.01708984e-01,4.97924805e-01,  5.04531860e-01,5.02212524e-01,4.89212036e-01,4.84832764e-01,  4.88571167e-01,4.93446350e-01,4.92218018e-01,4.89738464e-01,  4.89425659e-01,4.87182617e-01,4.81117249e-01,4.74723816e-01,  4.66362000e-01,4.50012207e-01,4.19799805e-01,3.88160706e-01,  3.92974854e-01,4.20143127e-01,4.27497864e-01,4.35379028e-01,  4.39888000e-01,4.35424805e-01,4.24690247e-01,4.10903931e-01,  3.94210815e-01,3.79661560e-01,3.70498657e-01,3.53462219e-01,  3.19015503e-01,3.07823181e-01,2.88162231e-01,2.14962006e-01,  1.16325378e-01,4.82110977e-02,2.40502357e-02,2.03251839e-02,  2.43525505e-02,3.58200073e-02,3.01804543e-02,3.30705643e-02,  4.50124741e-02,4.44774628e-02,4.94880676e-02,5.54561615e-02,  7.06214905e-02,8.90789032e-02,1.06370926e-01,1.23859406e-01,  1.24988556e-01,1.25587463e-01,1.38393402e-01,1.54552460e-01,  1.74602509e-01,2.08114624e-01,2.13485718e-01,1.90063477e-01,  1.78203583e-01,1.91131592e-01,2.05272675e-01,2.21881866e-01,  2.46990204e-01,2.66792297e-01,2.71446228e-01,2.75550842e-01,  2.85606384e-01,2.87933350e-01,2.88696289e-01,2.90565491e-01,  2.91191101e-01,2.90367126e-01,2.89718628e-01,2.89962769e-01,  2.87948608e-01,2.84683228e-01,2.81303406e-01,2.73948669e-01,  2.59628296e-01,2.51602173e-01,2.56988525e-01,2.62397766e-01,  2.63122559e-01,2.61215210e-01,2.52258301e-01,2.47909546e-01,  2.50236511e-01,2.56629944e-01,2.60368347e-01,2.63168335e-01,  2.64114380e-01,2.63893127e-01,2.63145447e-01,2.62405396e-01,  2.61840820e-01,2.59071350e-01,2.54798889e-01,2.49938965e-01,  2.45376587e-01,2.40619659e-01,2.36633301e-01,2.31372833e-01,  2.27729797e-01,2.23548889e-01,2.19692230e-01],
        [5.12390137e-01,5.29876709e-01,5.50842285e-01,6.40121460e-01,  6.75872803e-01,6.84356689e-01,6.83395386e-01,6.73171997e-01,  7.29507446e-01,8.31314087e-01,1.10025024e+00,1.23712158e+00,  1.29595947e+00,1.34542847e+00,1.34866333e+00,1.30712891e+00,  1.28454590e+00,1.38720703e+00,1.50448608e+00,1.61727905e+00,  1.69723511e+00,1.73248291e+00,1.75701904e+00,1.75271606e+00,  1.75830078e+00,1.79003906e+00,1.79962158e+00,1.73175049e+00,  1.74188232e+00,1.76943970e+00,1.74273682e+00,1.74801636e+00,  1.74124146e+00,1.67382812e+00,1.67593384e+00,1.71228027e+00,  1.73684692e+00,1.74060059e+00,1.72418213e+00,1.72708130e+00,  1.72485352e+00,1.70635986e+00,1.67916870e+00,1.66546631e+00,  1.65658569e+00,1.65426636e+00,1.66336060e+00,1.65817261e+00,  1.63076782e+00,1.63275146e+00,1.61877441e+00,1.61004639e+00,  1.59207153e+00,1.57604980e+00,1.57553101e+00,1.55001831e+00,  1.53451538e+00,1.54925537e+00,1.55364990e+00,1.54452515e+00,  1.52624512e+00,1.48623657e+00,1.51635742e+00,1.54220581e+00,  1.53268433e+00,1.52145386e+00,1.49484253e+00,1.38421631e+00,  1.34503174e+00,1.40890503e+00,1.42004395e+00,1.42025757e+00,  1.40545654e+00,1.34890747e+00,1.29968262e+00,1.29580688e+00,  1.30950928e+00,1.32598877e+00,1.31814575e+00,1.31774902e+00,  1.30209351e+00,1.18029785e+00,8.35006714e-01,8.98559570e-01,  1.16094971e+00,1.22094727e+00,1.21530151e+00,1.21066284e+00,  1.19396973e+00,1.18286133e+00,1.18057251e+00,1.16113281e+00,  1.13140869e+00,1.07278442e+00,1.03775024e+00,1.05477905e+00,  1.05712891e+00,1.05899048e+00,1.06713867e+00,1.04324341e+00,  1.00195312e+00,9.94720459e-01,9.97314453e-01,9.82299805e-01,  9.84313965e-01,9.80545044e-01,9.94644165e-01,9.85549927e-01,  9.63882446e-01,9.23278809e-01,8.63494873e-01,8.64212036e-01,  8.56231689e-01,8.38333130e-01,8.55712891e-01,8.35968018e-01,  6.84921265e-01,5.60028076e-01,5.70373535e-01,5.79391479e-01,  5.90667725e-01,6.13174438e-01,6.49444580e-01,6.99478149e-01,  7.35809326e-01,7.44018555e-01,7.56805420e-01,7.72674561e-01,  7.77755737e-01,7.74719238e-01,7.63671875e-01,7.52685547e-01,  7.50137329e-01,7.46444702e-01,7.38983154e-01,7.30926514e-01,  7.21023560e-01,7.11212158e-01,7.03491211e-01,6.94671631e-01,  6.85333252e-01,6.73522949e-01,6.60781860e-01,6.48849487e-01,  6.41510010e-01,6.38214111e-01,6.30111694e-01,6.20239258e-01,  6.08642578e-01,5.92849731e-01,5.78857422e-01,5.60607910e-01,  5.09857178e-01,4.18037415e-01,3.39683533e-01,3.16467285e-01,  3.26957703e-01,3.49639893e-01,3.78517151e-01,3.68385315e-01,  3.81210327e-01,4.26544189e-01,4.70802307e-01,5.01541138e-01,  5.14114380e-01,5.11077881e-01,5.01922607e-01,4.98130798e-01,  5.04806519e-01,5.02410889e-01,4.89242554e-01,4.84985352e-01,  4.88845825e-01,4.93568420e-01,4.92294312e-01,4.89784241e-01,  4.89501953e-01,4.87274170e-01,4.81124878e-01,4.74784851e-01,  4.66484070e-01,4.49897766e-01,4.19471741e-01,3.88015747e-01,  3.92875671e-01,4.20066833e-01,4.27505493e-01,4.35485840e-01,  4.40078735e-01,4.35623169e-01,4.24835205e-01,4.11079407e-01,  3.94485474e-01,3.79745483e-01,3.70590210e-01,3.53706360e-01,  3.19190979e-01,3.08036804e-01,2.88314819e-01,2.15019226e-01,  1.16302490e-01,4.81452942e-02,2.40855217e-02,2.04501152e-02,  2.42829323e-02,3.57923508e-02,3.03821564e-02,3.31058502e-02,  4.49886322e-02,4.45394516e-02,4.95262146e-02,5.55467606e-02,  7.06272125e-02,8.89911652e-02,1.06462479e-01,1.23855591e-01,  1.24897003e-01,1.25812531e-01,1.38607025e-01,1.54747009e-01,  1.74797058e-01,2.08435059e-01,2.13768005e-01,1.90219879e-01,  1.78318024e-01,1.91368103e-01,2.05406189e-01,2.22034454e-01,  2.46997833e-01,2.66891479e-01,2.71545410e-01,2.75566101e-01,  2.85736084e-01,2.88108826e-01,2.88696289e-01,2.90687561e-01,  2.91252136e-01,2.90290833e-01,2.89726257e-01,2.89932251e-01,  2.87948608e-01,2.84744263e-01,2.81417847e-01,2.74017334e-01,  2.59353638e-01,2.51403809e-01,2.56927490e-01,2.62222290e-01,  2.63107300e-01,2.61077881e-01,2.51968384e-01,2.47829437e-01,  2.50350952e-01,2.56553650e-01,2.60101318e-01,2.63160706e-01,  2.63786316e-01,2.63450623e-01,2.63122559e-01,2.62191772e-01,  2.61390686e-01,2.58758545e-01,2.54882812e-01,2.49656677e-01,  2.45079041e-01,2.40787506e-01,2.35572815e-01,2.31807709e-01,  2.28092194e-01,2.22671509e-01,2.19360352e-01],
        [5.12985229e-01,5.25924683e-01,5.48797607e-01,6.37390137e-01,  6.73400879e-01,6.83319092e-01,6.80969238e-01,6.72256470e-01,  7.29522705e-01,8.30017090e-01,1.09906006e+00,1.23458862e+00,  1.29479980e+00,1.34396362e+00,1.34704590e+00,1.30645752e+00,  1.28356934e+00,1.38757324e+00,1.50433350e+00,1.61602783e+00,  1.69784546e+00,1.73178101e+00,1.75735474e+00,1.75289917e+00,  1.75854492e+00,1.79104614e+00,1.79913330e+00,1.73150635e+00,  1.74291992e+00,1.76934814e+00,1.74270630e+00,1.74819946e+00,  1.74096680e+00,1.67456055e+00,1.67639160e+00,1.71286011e+00,  1.73687744e+00,1.74081421e+00,1.72497559e+00,1.72735596e+00,  1.72540283e+00,1.70663452e+00,1.67837524e+00,1.66555786e+00,  1.65600586e+00,1.65338135e+00,1.66314697e+00,1.65628052e+00,  1.63092041e+00,1.63235474e+00,1.61791992e+00,1.61074829e+00,  1.59262085e+00,1.57696533e+00,1.57662964e+00,1.54983521e+00,  1.53552246e+00,1.55068970e+00,1.55523682e+00,1.54724121e+00,  1.52709961e+00,1.48733521e+00,1.51855469e+00,1.54266357e+00,  1.53417969e+00,1.52276611e+00,1.49530029e+00,1.38516235e+00,  1.34494019e+00,1.41165161e+00,1.42205811e+00,1.42047119e+00,  1.40734863e+00,1.35104370e+00,1.30029297e+00,1.29888916e+00,  1.31225586e+00,1.32971191e+00,1.32144165e+00,1.31771851e+00,  1.30221558e+00,1.18060303e+00,8.33618164e-01,9.00527954e-01,  1.16131592e+00,1.22216797e+00,1.21987915e+00,1.21206665e+00,  1.19915771e+00,1.18963623e+00,1.18209839e+00,1.16656494e+00,  1.13403320e+00,1.07141113e+00,1.04058838e+00,1.05404663e+00,  1.05902100e+00,1.06735229e+00,1.06814575e+00,1.04367065e+00,  1.00479126e+00,9.87686157e-01,1.00082397e+00,9.76776123e-01,  9.86938477e-01,9.91867065e-01,9.84130859e-01,9.79705811e-01,  9.73037720e-01,9.23034668e-01,8.65875244e-01,8.67019653e-01,  8.56536865e-01,8.47274780e-01,8.64151001e-01,8.34976196e-01,  6.86080933e-01,5.64300537e-01,5.74722290e-01,5.83374023e-01,  5.94528198e-01,6.16744995e-01,6.52481079e-01,7.02117920e-01,  7.37930298e-01,7.45773315e-01,7.57904053e-01,7.73468018e-01,  7.78182983e-01,7.75054932e-01,7.64205933e-01,7.53189087e-01,  7.50534058e-01,7.46871948e-01,7.39334106e-01,7.31094360e-01,  7.21466064e-01,7.11715698e-01,7.03628540e-01,6.94747925e-01,  6.85592651e-01,6.73919678e-01,6.61102295e-01,6.48880005e-01,  6.41616821e-01,6.38549805e-01,6.30691528e-01,6.20651245e-01,  6.09115601e-01,5.93643188e-01,5.80062866e-01,5.62133789e-01,  5.11734009e-01,4.20501709e-01,3.42880249e-01,3.20106506e-01,  3.30375671e-01,3.52561951e-01,3.81217957e-01,3.71231079e-01,  3.83987427e-01,4.28924561e-01,4.72358704e-01,5.02548218e-01,  5.15106201e-01,5.12100220e-01,5.03021240e-01,4.99191284e-01,  5.05630493e-01,5.03128052e-01,4.90097046e-01,4.85710144e-01,  4.89425659e-01,4.94102478e-01,4.92706299e-01,4.90142822e-01,  4.89830017e-01,4.87495422e-01,4.81246948e-01,4.74830627e-01,  4.66575623e-01,4.49996948e-01,4.19517517e-01,3.87718201e-01,  3.92593384e-01,4.20104980e-01,4.27688599e-01,4.35752869e-01,  4.40315247e-01,4.35928345e-01,4.25315857e-01,4.11781311e-01,  3.95195007e-01,3.80775452e-01,3.71871948e-01,3.55010986e-01,  3.20915222e-01,3.09837341e-01,2.90069580e-01,2.16979980e-01,  1.18373871e-01,4.96597290e-02,2.51746178e-02,2.12612152e-02,  2.52656937e-02,3.72915268e-02,3.15542221e-02,3.43875885e-02,  4.66985703e-02,4.61816788e-02,5.11875153e-02,5.72471619e-02,  7.24201202e-02,9.09671783e-02,1.08482361e-01,1.25946045e-01,  1.26853943e-01,1.27506256e-01,1.40506744e-01,1.56810760e-01,  1.76788330e-01,2.10147858e-01,2.15595245e-01,1.92188263e-01,  1.80282593e-01,1.93191528e-01,2.07153320e-01,2.23598480e-01,  2.48271942e-01,2.67936707e-01,2.72483826e-01,2.76390076e-01,  2.86338806e-01,2.88558960e-01,2.89100647e-01,2.90977478e-01,  2.91473389e-01,2.90649414e-01,2.89993286e-01,2.90153503e-01,  2.88162231e-01,2.84927368e-01,2.81730652e-01,2.74299622e-01,  2.59765625e-01,2.51869202e-01,2.57179260e-01,2.62451172e-01,  2.63412476e-01,2.61497498e-01,2.52281189e-01,2.47970581e-01,  2.50526428e-01,2.56813049e-01,2.60482788e-01,2.63626099e-01,  2.64518738e-01,2.64221191e-01,2.63504028e-01,2.62748718e-01,  2.62130737e-01,2.59719849e-01,2.56141663e-01,2.51686096e-01,  2.47215271e-01,2.42465973e-01,2.38590240e-01,2.34283447e-01,  2.30133057e-01,2.25692749e-01,2.23541260e-01],
        [5.11108398e-01,5.22171021e-01,5.43151855e-01,6.34368896e-01,  6.66687012e-01,6.79016113e-01,6.75781250e-01,6.67739868e-01,  7.25387573e-01,8.24478149e-01,1.09399414e+00,1.22991943e+00,  1.28820801e+00,1.33807373e+00,1.34088135e+00,1.30126953e+00,  1.27905273e+00,1.38171387e+00,1.49862671e+00,1.60940552e+00,  1.69186401e+00,1.72592163e+00,1.75076294e+00,1.74777222e+00,  1.75244141e+00,1.78518677e+00,1.79418945e+00,1.72583008e+00,  1.73797607e+00,1.76470947e+00,1.73837280e+00,1.74481201e+00,  1.73709106e+00,1.67123413e+00,1.67251587e+00,1.70797729e+00,  1.73312378e+00,1.73641968e+00,1.72088623e+00,1.72320557e+00,  1.72097778e+00,1.70422363e+00,1.67504883e+00,1.66146851e+00,  1.65197754e+00,1.64965820e+00,1.66055298e+00,1.65350342e+00,  1.62738037e+00,1.62973022e+00,1.61459351e+00,1.60842896e+00,  1.59060669e+00,1.57388306e+00,1.57498169e+00,1.54763794e+00,  1.53298950e+00,1.54858398e+00,1.55200195e+00,1.54531860e+00,  1.52655029e+00,1.48526001e+00,1.51657104e+00,1.53979492e+00,  1.53286743e+00,1.52325439e+00,1.49407959e+00,1.38449097e+00,  1.34420776e+00,1.40856934e+00,1.42126465e+00,1.41818237e+00,  1.40554810e+00,1.35165405e+00,1.29882812e+00,1.29833984e+00,  1.31076050e+00,1.32501221e+00,1.32153320e+00,1.31851196e+00,  1.30053711e+00,1.18118286e+00,8.31954956e-01,8.98925781e-01,  1.16040039e+00,1.21847534e+00,1.21987915e+00,1.21307373e+00,  1.19454956e+00,1.19073486e+00,1.17965698e+00,1.16317749e+00,  1.13430786e+00,1.06814575e+00,1.04577637e+00,1.05734253e+00,  1.05389404e+00,1.06393433e+00,1.06082153e+00,1.04205322e+00,  1.00759888e+00,9.86114502e-01,1.00531006e+00,9.88525391e-01,  9.79202271e-01,9.93103027e-01,9.93469238e-01,9.77676392e-01,  9.80545044e-01,9.16061401e-01,8.65097046e-01,8.78356934e-01,  8.40621948e-01,8.46191406e-01,8.65905762e-01,8.22174072e-01,  6.83303833e-01,5.62942505e-01,5.73043823e-01,5.81848145e-01,  5.93429565e-01,6.15936279e-01,6.51855469e-01,7.01751709e-01,  7.37792969e-01,7.45925903e-01,7.58529663e-01,7.74337769e-01,  7.78869629e-01,7.75619507e-01,7.64968872e-01,7.54089355e-01,  7.51525879e-01,7.47833252e-01,7.40158081e-01,7.32009888e-01,  7.22366333e-01,7.12722778e-01,7.04559326e-01,6.95571899e-01,  6.86355591e-01,6.74682617e-01,6.61911011e-01,6.49658203e-01,  6.42196655e-01,6.39083862e-01,6.31378174e-01,6.21490479e-01,  6.09771729e-01,5.94131470e-01,5.80261230e-01,5.62149048e-01,  5.11550903e-01,4.19815063e-01,3.41644287e-01,3.18588257e-01,  3.29010010e-01,3.51699829e-01,3.80470276e-01,3.70155334e-01,  3.82904053e-01,4.28222656e-01,4.72122192e-01,5.02548218e-01,  5.15228271e-01,5.12176514e-01,5.03128052e-01,4.99412537e-01,  5.05905151e-01,5.03402710e-01,4.90264893e-01,4.85923767e-01,  4.89852905e-01,4.94567871e-01,4.93194580e-01,4.90615845e-01,  4.90318298e-01,4.88014221e-01,4.81773376e-01,4.75402832e-01,  4.67048645e-01,4.50424194e-01,4.19731140e-01,3.87840271e-01,  3.92761230e-01,4.20341492e-01,4.28039551e-01,4.36149597e-01,  4.40811157e-01,4.36325073e-01,4.25643921e-01,4.12025452e-01,  3.95446777e-01,3.80798340e-01,3.71711731e-01,3.54850769e-01,  3.20564270e-01,3.09387207e-01,2.89520264e-01,2.16331482e-01,  1.17557526e-01,4.90808487e-02,2.48808861e-02,2.11157799e-02,  2.51450539e-02,3.68671417e-02,3.12075615e-02,3.40824127e-02,  4.62427139e-02,4.56962585e-02,5.05847931e-02,5.65357208e-02,  7.18345642e-02,9.03129578e-02,1.07637405e-01,1.25125885e-01,  1.26235962e-01,1.26838684e-01,1.39698029e-01,1.55975342e-01,  1.76124573e-01,2.09705353e-01,2.15137482e-01,1.91596985e-01,  1.79622650e-01,1.92588806e-01,2.06745148e-01,2.23243713e-01,  2.48058319e-01,2.67822266e-01,2.72598267e-01,2.76611328e-01,  2.86552429e-01,2.88887024e-01,2.89596558e-01,2.91389465e-01,  2.91877747e-01,2.91015625e-01,2.90428162e-01,2.90794373e-01,  2.88726807e-01,2.85369873e-01,2.82119751e-01,2.74665833e-01,  2.60116577e-01,2.52052307e-01,2.57514954e-01,2.63153076e-01,  2.63992310e-01,2.61802673e-01,2.52464294e-01,2.48371124e-01,  2.51007080e-01,2.57316589e-01,2.61146545e-01,2.64328003e-01,  2.65045166e-01,2.64617920e-01,2.64152527e-01,2.63626099e-01,  2.63282776e-01,2.61062622e-01,2.57293701e-01,2.53227234e-01,  2.49725342e-01,2.45029449e-01,2.40390778e-01,2.36957550e-01,  2.33482361e-01,2.28084564e-01,2.25234985e-01],
        [4.98641968e-01,5.16159058e-01,5.39962769e-01,6.24633789e-01,  6.62124634e-01,6.71020508e-01,6.69876099e-01,6.62628174e-01,  7.16949463e-01,8.18603516e-01,1.08273315e+00,1.21743774e+00,  1.27667236e+00,1.32446289e+00,1.32995605e+00,1.28933716e+00,  1.26776123e+00,1.37167358e+00,1.48718262e+00,1.59884644e+00,  1.67892456e+00,1.71395874e+00,1.73983765e+00,1.73538208e+00,  1.74243164e+00,1.77438354e+00,1.78298950e+00,1.71704102e+00,  1.72766113e+00,1.75512695e+00,1.72882080e+00,1.73486328e+00,  1.72839355e+00,1.66122437e+00,1.66424561e+00,1.70068359e+00,  1.72412109e+00,1.72845459e+00,1.71200562e+00,1.71429443e+00,  1.71298218e+00,1.69436646e+00,1.66708374e+00,1.65365601e+00,  1.64367676e+00,1.64242554e+00,1.65093994e+00,1.64559937e+00,  1.62008667e+00,1.62017822e+00,1.60830688e+00,1.60058594e+00,  1.58255005e+00,1.56793213e+00,1.56646729e+00,1.54129028e+00,  1.52688599e+00,1.54171753e+00,1.54745483e+00,1.53939819e+00,  1.51983643e+00,1.48068237e+00,1.50982666e+00,1.53594971e+00,  1.52798462e+00,1.51541138e+00,1.48898315e+00,1.37689209e+00,  1.33938599e+00,1.40539551e+00,1.41372681e+00,1.41546631e+00,  1.40319824e+00,1.34454346e+00,1.29501343e+00,1.29101562e+00,  1.30432129e+00,1.32269287e+00,1.31521606e+00,1.31558228e+00,  1.29769897e+00,1.17276001e+00,8.28247070e-01,8.93081665e-01,  1.15490723e+00,1.21942139e+00,1.21218872e+00,1.21057129e+00,  1.19464111e+00,1.18206787e+00,1.18136597e+00,1.16232300e+00,  1.12753296e+00,1.07095337e+00,1.03747559e+00,1.04681396e+00,  1.05630493e+00,1.05798340e+00,1.06375122e+00,1.04367065e+00,  9.99603271e-01,9.94888306e-01,9.96978760e-01,9.73114014e-01,  9.86465454e-01,9.79507446e-01,9.82772827e-01,9.83032227e-01,  9.62890625e-01,9.24896240e-01,8.73306274e-01,8.67828369e-01,  8.64379883e-01,8.39874268e-01,8.50036621e-01,8.44360352e-01,  6.85485840e-01,5.56655884e-01,5.67031860e-01,5.75973511e-01,  5.87707520e-01,6.10122681e-01,6.46438599e-01,6.97158813e-01,  7.33932495e-01,7.42401123e-01,7.55203247e-01,7.71255493e-01,  7.76504517e-01,7.73529053e-01,7.62832642e-01,7.51770020e-01,  7.49237061e-01,7.45483398e-01,7.37915039e-01,7.29766846e-01,  7.20138550e-01,7.10449219e-01,7.02453613e-01,6.93557739e-01,  6.84265137e-01,6.72622681e-01,6.59683228e-01,6.47460938e-01,  6.40274048e-01,6.37176514e-01,6.29257202e-01,6.19400024e-01,  6.07574463e-01,5.91796875e-01,5.77713013e-01,5.59204102e-01,  5.08071899e-01,4.15748596e-01,3.37211609e-01,3.14064026e-01,  3.24676514e-01,3.47625732e-01,3.76373291e-01,3.65982056e-01,  3.78952026e-01,4.24613953e-01,4.69047546e-01,4.99969482e-01,  5.12695312e-01,5.09750366e-01,5.00747681e-01,4.97093201e-01,  5.03692627e-01,5.01296997e-01,4.88197327e-01,4.83901978e-01,  4.87739563e-01,4.92523193e-01,4.91432190e-01,4.88983154e-01,  4.88685608e-01,4.86389160e-01,4.80178833e-01,4.73800659e-01,  4.65507507e-01,4.48829651e-01,4.18098450e-01,3.86146545e-01,  3.91082764e-01,4.18869019e-01,4.26689148e-01,4.34692383e-01,  4.39208984e-01,4.34745789e-01,4.24095154e-01,4.10270691e-01,  3.93478394e-01,3.78723145e-01,3.69659424e-01,3.52638245e-01,  3.17840576e-01,3.06625366e-01,2.86781311e-01,2.13386536e-01,  1.15020752e-01,4.75158691e-02,2.40020752e-02,2.04944611e-02,  2.43654251e-02,3.55405807e-02,3.00970078e-02,3.28102112e-02,  4.45203781e-02,4.41856384e-02,4.89301682e-02,5.48171997e-02,  6.98223114e-02,8.80069733e-02,1.05319977e-01,1.22612000e-01,  1.23676300e-01,1.24349594e-01,1.37191772e-01,1.53423309e-01,  1.73603058e-01,2.07172394e-01,2.12772369e-01,1.89170837e-01,  1.77112579e-01,1.90204620e-01,2.04376221e-01,2.21096039e-01,  2.46181488e-01,2.66174316e-01,2.71011353e-01,2.75192261e-01,  2.85308838e-01,2.87628174e-01,2.88360596e-01,2.90451050e-01,  2.91046143e-01,2.90107727e-01,2.89619446e-01,2.89932251e-01,  2.87910461e-01,2.84576416e-01,2.81272888e-01,2.73841858e-01,  2.59162903e-01,2.51243591e-01,2.56668091e-01,2.62107849e-01,  2.63107300e-01,2.61016846e-01,2.51731873e-01,2.47547150e-01,  2.50091553e-01,2.56523132e-01,2.60238647e-01,2.63259888e-01,  2.63961792e-01,2.63626099e-01,2.63084412e-01,2.62283325e-01,  2.61924744e-01,2.59643555e-01,2.56019592e-01,2.51991272e-01,  2.47932434e-01,2.43839264e-01,2.39555359e-01,2.35446930e-01,  2.31807709e-01,2.27451324e-01,2.24735260e-01],
        [4.99755859e-01,5.13320923e-01,5.32791138e-01,6.22009277e-01,  6.57684326e-01,6.68197632e-01,6.66839600e-01,6.59011841e-01,  7.14797974e-01,8.14544678e-01,1.07894897e+00,1.21261597e+00,  1.27069092e+00,1.31988525e+00,1.32418823e+00,1.28527832e+00,  1.26489258e+00,1.36657715e+00,1.48348999e+00,1.59411621e+00,  1.67437744e+00,1.71029663e+00,1.73532104e+00,1.73287964e+00,  1.73846436e+00,1.77014160e+00,1.77999878e+00,1.71313477e+00,  1.72546387e+00,1.75268555e+00,1.72564697e+00,1.73257446e+00,  1.72525024e+00,1.65887451e+00,1.66143799e+00,1.69723511e+00,  1.72171021e+00,1.72515869e+00,1.70947266e+00,1.71240234e+00,  1.71072388e+00,1.69317627e+00,1.66448975e+00,1.65069580e+00,  1.64187622e+00,1.63885498e+00,1.64947510e+00,1.64395142e+00,  1.61767578e+00,1.61944580e+00,1.60556030e+00,1.59786987e+00,  1.58218384e+00,1.56646729e+00,1.56600952e+00,1.53918457e+00,  1.52407837e+00,1.54135132e+00,1.54632568e+00,1.53833008e+00,  1.51983643e+00,1.47921753e+00,1.51083374e+00,1.53457642e+00,  1.52575684e+00,1.51702881e+00,1.48907471e+00,1.37652588e+00,  1.33837891e+00,1.40438843e+00,1.41702271e+00,1.41641235e+00,  1.40173340e+00,1.34613037e+00,1.29443359e+00,1.29174805e+00,  1.30654907e+00,1.32058716e+00,1.31570435e+00,1.31518555e+00,  1.29833984e+00,1.17684937e+00,8.26950073e-01,8.93585205e-01,  1.15939331e+00,1.21774292e+00,1.21380615e+00,1.21011353e+00,  1.18881226e+00,1.18634033e+00,1.18057251e+00,1.15884399e+00,  1.13244629e+00,1.06893921e+00,1.04040527e+00,1.05297852e+00,  1.05230713e+00,1.06170654e+00,1.06921387e+00,1.04299927e+00,  1.00238037e+00,9.85397339e-01,9.98809814e-01,9.89501953e-01,  9.85443115e-01,9.82086182e-01,9.87274170e-01,9.82192993e-01,  9.76181030e-01,9.23355103e-01,8.59451294e-01,8.66928101e-01,  8.44757080e-01,8.37539673e-01,8.59786987e-01,8.34655762e-01,  6.82769775e-01,5.56808472e-01,5.67321777e-01,5.75988770e-01,  5.87371826e-01,6.10153198e-01,6.46713257e-01,6.97311401e-01,  7.33703613e-01,7.41989136e-01,7.54943848e-01,7.71133423e-01,  7.76382446e-01,7.73147583e-01,7.62252808e-01,7.51358032e-01,  7.48962402e-01,7.45300293e-01,7.37655640e-01,7.29537964e-01,  7.19741821e-01,7.10037231e-01,7.02163696e-01,6.93267822e-01,  6.83853149e-01,6.71981812e-01,6.59088135e-01,6.47064209e-01,  6.40029907e-01,6.36825562e-01,6.28799438e-01,6.19033813e-01,  6.07482910e-01,5.91781616e-01,5.77529907e-01,5.58975220e-01,  5.07812500e-01,4.15588379e-01,3.37173462e-01,3.14102173e-01,  3.24714661e-01,3.47595215e-01,3.76426697e-01,3.66004944e-01,  3.78959656e-01,4.24468994e-01,4.68742371e-01,4.99641418e-01,  5.12512207e-01,5.09643555e-01,5.00625610e-01,4.96871948e-01,  5.03524780e-01,5.01174927e-01,4.88014221e-01,4.83604431e-01,  4.87579346e-01,4.92462158e-01,4.91264343e-01,4.88822937e-01,  4.88479614e-01,4.86221313e-01,4.80064392e-01,4.73625183e-01,  4.65087891e-01,4.48348999e-01,4.17488098e-01,3.85429382e-01,  3.90365601e-01,4.18243408e-01,4.26254272e-01,4.34440613e-01,  4.39010620e-01,4.34501648e-01,4.23828125e-01,4.10041809e-01,  3.93363953e-01,3.78616333e-01,3.69415283e-01,3.52348328e-01,  3.17771912e-01,3.06587219e-01,2.86590576e-01,2.13050842e-01,  1.14706039e-01,4.73527908e-02,2.37832069e-02,2.01931000e-02,  2.40702629e-02,3.53279114e-02,2.99463272e-02,3.26433182e-02,  4.43296432e-02,4.39796448e-02,4.87718582e-02,5.46665192e-02,  6.96849823e-02,8.78467560e-02,1.05052948e-01,1.22444153e-01,  1.23563766e-01,1.24164581e-01,1.36947632e-01,1.53270721e-01,  1.73542023e-01,2.07145691e-01,2.12539673e-01,1.89025879e-01,  1.77070618e-01,1.90093994e-01,2.04204559e-01,2.20848083e-01,  2.45998383e-01,2.66059875e-01,2.70843506e-01,2.74894714e-01,  2.85003662e-01,2.87414551e-01,2.88162231e-01,2.90168762e-01,  2.90725708e-01,2.89649963e-01,2.89108276e-01,2.89451599e-01,  2.87483215e-01,2.84263611e-01,2.81066895e-01,2.73490906e-01,  2.58750916e-01,2.50717163e-01,2.56134033e-01,2.61711121e-01,  2.62771606e-01,2.60498047e-01,2.51007080e-01,2.46978760e-01,  2.49549866e-01,2.55790710e-01,2.59475708e-01,2.62649536e-01,  2.63694763e-01,2.63183594e-01,2.62191772e-01,2.61230469e-01,  2.60971069e-01,2.58430481e-01,2.54211426e-01,2.49912262e-01,  2.46280670e-01,2.42263794e-01,2.37342834e-01,2.32585907e-01,  2.29324341e-01,2.24689484e-01,2.21721649e-01],
        [4.94651794e-01,5.07492065e-01,5.30029297e-01,6.18331909e-01,  6.54556274e-01,6.64398193e-01,6.61254883e-01,6.56097412e-01,  7.10937500e-01,8.10958862e-01,1.07629395e+00,1.20867920e+00,  1.26812744e+00,1.31698608e+00,1.32147217e+00,1.28317261e+00,  1.26159668e+00,1.36514282e+00,1.48056030e+00,1.59094238e+00,  1.67251587e+00,1.70770264e+00,1.73324585e+00,1.72979736e+00,  1.73605347e+00,1.76889038e+00,1.77725220e+00,1.71133423e+00,  1.72357178e+00,1.74984741e+00,1.72442627e+00,1.73019409e+00,  1.72286987e+00,1.65765381e+00,1.66021729e+00,1.69647217e+00,  1.71945190e+00,1.72293091e+00,1.70770264e+00,1.71008301e+00,  1.70916748e+00,1.69119263e+00,1.66308594e+00,1.65057373e+00,  1.63937378e+00,1.63681030e+00,1.64788818e+00,1.64111328e+00,  1.61730957e+00,1.61856079e+00,1.60351562e+00,1.59802246e+00,  1.58038330e+00,1.56539917e+00,1.56567383e+00,1.53802490e+00,  1.52398682e+00,1.54003906e+00,1.54495239e+00,1.53845215e+00,  1.51889038e+00,1.48025513e+00,1.51110840e+00,1.53396606e+00,  1.52719116e+00,1.51565552e+00,1.48849487e+00,1.37753296e+00,  1.33700562e+00,1.40524292e+00,1.41662598e+00,1.41598511e+00,  1.40432739e+00,1.34747314e+00,1.29623413e+00,1.29666138e+00,  1.30810547e+00,1.32446289e+00,1.31692505e+00,1.31164551e+00,  1.29852295e+00,1.17498779e+00,8.24111938e-01,8.93157959e-01,  1.15588379e+00,1.21920776e+00,1.21728516e+00,1.20913696e+00,  1.19641113e+00,1.18551636e+00,1.17831421e+00,1.16625977e+00,  1.13110352e+00,1.06597900e+00,1.04293823e+00,1.04940796e+00,  1.05221558e+00,1.06106567e+00,1.06112671e+00,1.04779053e+00,  1.00534058e+00,9.84954834e-01,1.00549316e+00,9.80651855e-01,  9.85885620e-01,9.91897583e-01,9.87243652e-01,9.94674683e-01,  9.75326538e-01,9.13467407e-01,8.73626709e-01,8.69216919e-01,  8.51547241e-01,8.55545044e-01,8.48831177e-01,8.14208984e-01,  6.80908203e-01,5.62835693e-01,5.73226929e-01,5.81863403e-01,  5.93231201e-01,6.15570068e-01,6.51565552e-01,7.01171875e-01,  7.36907959e-01,7.44857788e-01,7.57507324e-01,7.73101807e-01,  7.77877808e-01,7.74749756e-01,7.63656616e-01,7.52563477e-01,  7.50000000e-01,7.46490479e-01,7.39044189e-01,7.30789185e-01,  7.20916748e-01,7.11074829e-01,7.03201294e-01,6.94366455e-01,  6.85104370e-01,6.73217773e-01,6.60079956e-01,6.47903442e-01,  6.40838623e-01,6.37954712e-01,6.30172729e-01,6.20346069e-01,  6.08718872e-01,5.93185425e-01,5.79498291e-01,5.61325073e-01,  5.10787964e-01,4.19364929e-01,3.41583252e-01,3.18824768e-01,  3.29223633e-01,3.51776123e-01,3.80432129e-01,3.70185852e-01,  3.82881165e-01,4.27757263e-01,4.71374512e-01,5.01739502e-01,  5.14373779e-01,5.11413574e-01,5.02426147e-01,4.98641968e-01,  5.05065918e-01,5.02609253e-01,4.89578247e-01,4.85176086e-01,  4.88929749e-01,4.93583679e-01,4.92202759e-01,4.89715576e-01,  4.89395142e-01,4.87075806e-01,4.80796814e-01,4.74205017e-01,  4.65644836e-01,4.48822021e-01,4.17823792e-01,3.85520935e-01,  3.90518188e-01,4.18601990e-01,4.26849365e-01,4.35272217e-01,  4.39956665e-01,4.35562134e-01,4.24949646e-01,4.11338806e-01,  3.94836426e-01,3.80325317e-01,3.71322632e-01,3.54499817e-01,  3.20304871e-01,3.09165955e-01,2.89245605e-01,2.16018677e-01,  1.17418289e-01,4.91342545e-02,2.48765945e-02,2.10723877e-02,  2.52251625e-02,3.71103287e-02,3.14073563e-02,3.42578888e-02,  4.63905334e-02,4.58602905e-02,5.07059097e-02,5.67350388e-02,  7.19337463e-02,9.02919769e-02,1.07608795e-01,1.25091553e-01,  1.25988007e-01,1.26522064e-01,1.39450073e-01,1.55921936e-01,  1.76311493e-01,2.09789276e-01,2.15072632e-01,1.91619873e-01,  1.79695129e-01,1.92684174e-01,2.06638336e-01,2.23037720e-01,  2.47901917e-01,2.67593384e-01,2.72148132e-01,2.76100159e-01,  2.86224365e-01,2.88467407e-01,2.88879395e-01,2.90847778e-01,  2.91488647e-01,2.90473938e-01,2.89901733e-01,2.90161133e-01,  2.88108826e-01,2.84820557e-01,2.81661987e-01,2.74017334e-01,  2.59071350e-01,2.51174927e-01,2.56858826e-01,2.62443542e-01,  2.63343811e-01,2.61192322e-01,2.51823425e-01,2.47577667e-01,  2.50198364e-01,2.56622314e-01,2.60398865e-01,2.63587952e-01,  2.64137268e-01,2.63656616e-01,2.63206482e-01,2.62619019e-01,  2.62229919e-01,2.59895325e-01,2.56439209e-01,2.51861572e-01,  2.48062134e-01,2.43991852e-01,2.39704132e-01,2.35851288e-01,  2.31616974e-01,2.27172852e-01,2.25105286e-01],
        [4.87342834e-01,5.00961304e-01,5.27374268e-01,6.09649658e-01,  6.47003174e-01,6.61529541e-01,6.54541016e-01,6.50817871e-01,  7.04284668e-01,8.03695679e-01,1.06777954e+00,1.19839478e+00,  1.25881958e+00,1.30697632e+00,1.31106567e+00,1.27401733e+00,  1.25228882e+00,1.35617065e+00,1.47125244e+00,1.58151245e+00,  1.66348267e+00,1.69696045e+00,1.72415161e+00,1.72100830e+00,  1.72671509e+00,1.75973511e+00,1.76858521e+00,1.70385742e+00,  1.71569824e+00,1.74130249e+00,1.71673584e+00,1.72198486e+00,  1.71600342e+00,1.65090942e+00,1.65286255e+00,1.68942261e+00,  1.71246338e+00,1.71578979e+00,1.70153809e+00,1.70294189e+00,  1.70233154e+00,1.68469238e+00,1.65582275e+00,1.64389038e+00,  1.63250732e+00,1.63073730e+00,1.64294434e+00,1.63525391e+00,  1.61166382e+00,1.61273193e+00,1.59832764e+00,1.59262085e+00,  1.57327271e+00,1.55963135e+00,1.56024170e+00,1.53189087e+00,  1.52014160e+00,1.53567505e+00,1.54043579e+00,1.53442383e+00,  1.51425171e+00,1.47650146e+00,1.50738525e+00,1.53012085e+00,  1.52441406e+00,1.51193237e+00,1.48370361e+00,1.37460327e+00,  1.33355713e+00,1.40377808e+00,1.41497803e+00,1.41168213e+00,  1.40209961e+00,1.34497070e+00,1.29272461e+00,1.29260254e+00,  1.30380249e+00,1.32156372e+00,1.31665039e+00,1.31195068e+00,  1.29809570e+00,1.17221069e+00,8.21411133e-01,8.93371582e-01,  1.15267944e+00,1.21554565e+00,1.21636963e+00,1.20556641e+00,  1.19451904e+00,1.18356323e+00,1.17498779e+00,1.16394043e+00,  1.12753296e+00,1.06536865e+00,1.04049683e+00,1.04244995e+00,  1.05059814e+00,1.06329346e+00,1.05584717e+00,1.04666138e+00,  9.99664307e-01,9.88296509e-01,1.01040649e+00,9.78179932e-01,  9.82116699e-01,9.87915039e-01,9.73922729e-01,9.88159180e-01,  9.70489502e-01,9.05426025e-01,8.81866455e-01,8.70971680e-01,  8.42407227e-01,8.54965210e-01,8.45275879e-01,8.19564819e-01,  6.81976318e-01,5.59509277e-01,5.69992065e-01,5.78628540e-01,  5.89874268e-01,6.12365723e-01,6.48895264e-01,6.99295044e-01,  7.35519409e-01,7.43453979e-01,7.56149292e-01,7.72125244e-01,  7.77221680e-01,7.74261475e-01,7.63214111e-01,7.52197266e-01,  7.49664307e-01,7.46109009e-01,7.38662720e-01,7.30453491e-01,  7.20611572e-01,7.10800171e-01,7.02850342e-01,6.94000244e-01,  6.84600830e-01,6.72790527e-01,6.59744263e-01,6.47567749e-01,  6.40472412e-01,6.37451172e-01,6.29577637e-01,6.19796753e-01,  6.08230591e-01,5.92620850e-01,5.78689575e-01,5.60287476e-01,  5.09170532e-01,4.17243958e-01,3.39134216e-01,3.16177368e-01,  3.26667786e-01,3.49441528e-01,3.78234863e-01,3.67866516e-01,  3.80783081e-01,4.25872803e-01,4.69932556e-01,5.00671387e-01,  5.13458252e-01,5.10559082e-01,5.01586914e-01,4.97825623e-01,  5.04409790e-01,5.01983643e-01,4.88800049e-01,4.84390259e-01,  4.88220215e-01,4.93019104e-01,4.91806030e-01,4.89280701e-01,  4.88975525e-01,4.86648560e-01,4.80361938e-01,4.73693848e-01,  4.65332031e-01,4.48425293e-01,4.17076111e-01,3.84651184e-01,  3.89747620e-01,4.18075562e-01,4.26437378e-01,4.34928894e-01,  4.39567566e-01,4.35104370e-01,4.24453735e-01,4.10728455e-01,  3.94119263e-01,3.79432678e-01,3.70323181e-01,3.53500366e-01,  3.19053650e-01,3.07823181e-01,2.87796021e-01,2.14294434e-01,  1.15867615e-01,4.81557846e-02,2.43363380e-02,2.06465721e-02,  2.45804787e-02,3.60641479e-02,3.06119919e-02,3.34501266e-02,  4.52880859e-02,4.48732376e-02,4.96921539e-02,5.55963516e-02,  7.06558228e-02,8.87775421e-02,1.06122971e-01,1.23544693e-01,  1.24502182e-01,1.25061035e-01,1.37897491e-01,1.54445648e-01,  1.74831390e-01,2.08263397e-01,2.13645935e-01,1.90204620e-01,  1.78302765e-01,1.91230774e-01,2.05284119e-01,2.21904755e-01,  2.46955872e-01,2.66784668e-01,2.71438599e-01,2.75527954e-01,  2.85644531e-01,2.87963867e-01,2.88558960e-01,2.90557861e-01,  2.91198730e-01,2.90168762e-01,2.89642334e-01,2.90008545e-01,  2.88017273e-01,2.84744263e-01,2.81349182e-01,2.73811340e-01,  2.58895874e-01,2.50946045e-01,2.56599426e-01,2.62130737e-01,  2.63183594e-01,2.60986328e-01,2.51434326e-01,2.47257233e-01,  2.49866486e-01,2.56515503e-01,2.60406494e-01,2.63526917e-01,  2.64038086e-01,2.63267517e-01,2.63031006e-01,2.62069702e-01,  2.61703491e-01,2.59529114e-01,2.56034851e-01,2.51731873e-01,  2.47398376e-01,2.43358612e-01,2.39395142e-01,2.36122131e-01,  2.32204437e-01,2.26631165e-01,2.24632263e-01],
        [4.83947754e-01,4.97772217e-01,5.21270752e-01,6.07055664e-01,  6.41204834e-01,6.54815674e-01,6.51672363e-01,6.45965576e-01,  7.00515747e-01,7.99118042e-01,1.05963135e+00,1.19125366e+00,  1.24990845e+00,1.29931641e+00,1.30389404e+00,1.26696777e+00,  1.24700928e+00,1.34927368e+00,1.46441650e+00,1.57394409e+00,  1.65542603e+00,1.69015503e+00,1.71649170e+00,1.71371460e+00,  1.72015381e+00,1.75305176e+00,1.76242065e+00,1.69757080e+00,  1.70977783e+00,1.73629761e+00,1.71170044e+00,1.71783447e+00,  1.71127319e+00,1.64654541e+00,1.64895630e+00,1.68423462e+00,  1.70867920e+00,1.71224976e+00,1.69708252e+00,1.69982910e+00,  1.69836426e+00,1.68017578e+00,1.65188599e+00,1.63928223e+00,  1.62899780e+00,1.62609863e+00,1.63699341e+00,1.63073730e+00,  1.60647583e+00,1.60815430e+00,1.59384155e+00,1.58847046e+00,  1.57067871e+00,1.55587769e+00,1.55731201e+00,1.53039551e+00,  1.51617432e+00,1.53381348e+00,1.53781128e+00,1.53060913e+00,  1.51232910e+00,1.47415161e+00,1.50546265e+00,1.52947998e+00,  1.52267456e+00,1.51223755e+00,1.48437500e+00,1.37246704e+00,  1.33251953e+00,1.40176392e+00,1.41305542e+00,1.41262817e+00,  1.40145874e+00,1.34320068e+00,1.29248047e+00,1.29122925e+00,  1.30261230e+00,1.31985474e+00,1.31533813e+00,1.31143188e+00,  1.29678345e+00,1.17245483e+00,8.21395874e-01,8.91647339e-01,  1.15698242e+00,1.21868896e+00,1.21475220e+00,1.20529175e+00,  1.19097900e+00,1.18243408e+00,1.17620850e+00,1.16119385e+00,  1.12799072e+00,1.06335449e+00,1.03671265e+00,1.04473877e+00,  1.04986572e+00,1.06399536e+00,1.06256104e+00,1.04510498e+00,  1.00015259e+00,9.79568481e-01,1.00250244e+00,9.80667114e-01,  9.84146118e-01,9.87915039e-01,9.80926514e-01,9.85885620e-01,  9.75677490e-01,9.13619995e-01,8.66165161e-01,8.72467041e-01,  8.46893311e-01,8.45581055e-01,8.50418091e-01,8.20068359e-01,  6.78634644e-01,5.55801392e-01,5.66574097e-01,5.75408936e-01,  5.86791992e-01,6.09329224e-01,6.45843506e-01,6.96823120e-01,  7.33871460e-01,7.41989136e-01,7.54913330e-01,7.71179199e-01,  7.76290894e-01,7.73437500e-01,7.62603760e-01,7.51602173e-01,  7.49084473e-01,7.45376587e-01,7.37777710e-01,7.29675293e-01,  7.20016479e-01,7.10357666e-01,7.02316284e-01,6.93511963e-01,  6.84005737e-01,6.72027588e-01,6.59042358e-01,6.46896362e-01,  6.39770508e-01,6.36886597e-01,6.29043579e-01,6.19140625e-01,  6.07345581e-01,5.91690063e-01,5.77484131e-01,5.58883667e-01,  5.07492065e-01,4.14932251e-01,3.36441040e-01,3.13598633e-01,  3.24127197e-01,3.47068787e-01,3.75877380e-01,3.65356445e-01,  3.78326416e-01,4.23889160e-01,4.68376160e-01,4.99412537e-01,  5.12374878e-01,5.09490967e-01,5.00534058e-01,4.96864319e-01,  5.03540039e-01,5.01220703e-01,4.87960815e-01,4.83489990e-01,  4.87457275e-01,4.92309570e-01,4.91119385e-01,4.88731384e-01,  4.88456726e-01,4.86228943e-01,4.79927063e-01,4.73327637e-01,  4.64782715e-01,4.47677612e-01,4.16145325e-01,3.83567810e-01,  3.88740540e-01,4.17366028e-01,4.25964355e-01,4.34387207e-01,  4.38995361e-01,4.34501648e-01,4.23706055e-01,4.10049438e-01,  3.93318176e-01,3.78501892e-01,3.69354248e-01,3.52241516e-01,  3.17451477e-01,3.06243896e-01,2.86216736e-01,2.12535858e-01,  1.14135742e-01,4.68721390e-02,2.35710144e-02,2.00996399e-02,  2.38866806e-02,3.50818634e-02,2.96578407e-02,3.23486328e-02,  4.40082550e-02,4.36306000e-02,4.84447479e-02,5.41906357e-02,  6.90135956e-02,8.73203278e-02,1.04572296e-01,1.21854782e-01,  1.22776031e-01,1.23285294e-01,1.36291504e-01,1.52866364e-01,  1.73252106e-01,2.06764221e-01,2.12112427e-01,1.88652039e-01,  1.76692963e-01,1.89628601e-01,2.03792572e-01,2.20531464e-01,  2.45750427e-01,2.65884399e-01,2.70698547e-01,2.74841309e-01,  2.84980774e-01,2.87315369e-01,2.88078308e-01,2.90100098e-01,  2.90702820e-01,2.89665222e-01,2.89093018e-01,2.89543152e-01,  2.87689209e-01,2.84370422e-01,2.80876160e-01,2.73162842e-01,  2.58285522e-01,2.50312805e-01,2.55905151e-01,2.61535645e-01,  2.62565613e-01,2.60414124e-01,2.50930786e-01,2.46528625e-01,  2.49111176e-01,2.55737305e-01,2.59658813e-01,2.62611389e-01,  2.63160706e-01,2.62542725e-01,2.61772156e-01,2.61047363e-01,  2.60375977e-01,2.58064270e-01,2.53616333e-01,2.49134064e-01,  2.45784760e-01,2.41287231e-01,2.36499786e-01,2.31357574e-01,  2.28351593e-01,2.24140167e-01,2.20859528e-01],
        [4.76440430e-01,4.88227844e-01,5.13656616e-01,6.00479126e-01,  6.32583618e-01,6.44470215e-01,6.42776489e-01,6.38351440e-01,  6.92947388e-01,7.90863037e-01,1.05023193e+00,1.18005371e+00,  1.23937988e+00,1.28915405e+00,1.29409790e+00,1.25741577e+00,  1.23779297e+00,1.34017944e+00,1.45550537e+00,1.56448364e+00,  1.64514160e+00,1.68063354e+00,1.70639038e+00,1.70440674e+00,  1.71112061e+00,1.74328613e+00,1.75286865e+00,1.68777466e+00,  1.70065308e+00,1.72848511e+00,1.70294189e+00,1.70913696e+00,  1.70266724e+00,1.63864136e+00,1.64083862e+00,1.67593384e+00,  1.69958496e+00,1.70355225e+00,1.68847656e+00,1.69094849e+00,  1.68997192e+00,1.67233276e+00,1.64376831e+00,1.63052368e+00,  1.62060547e+00,1.61761475e+00,1.62878418e+00,1.62301636e+00,  1.59799194e+00,1.60079956e+00,1.58749390e+00,1.58169556e+00,  1.56457520e+00,1.54882812e+00,1.54916382e+00,1.52282715e+00,  1.50939941e+00,1.52725220e+00,1.53298950e+00,1.52639771e+00,  1.50723267e+00,1.46820068e+00,1.49996948e+00,1.52490234e+00,  1.51751709e+00,1.50717163e+00,1.47891235e+00,1.36657715e+00,  1.32821655e+00,1.39532471e+00,1.40817261e+00,1.40759277e+00,  1.39614868e+00,1.34024048e+00,1.28656006e+00,1.28512573e+00,  1.29876709e+00,1.31610107e+00,1.31225586e+00,1.30767822e+00,  1.29125977e+00,1.16772461e+00,8.14971924e-01,8.84140015e-01,  1.15158081e+00,1.21609497e+00,1.21154785e+00,1.20675659e+00,  1.18792725e+00,1.17971802e+00,1.17340088e+00,1.15737915e+00,  1.12417603e+00,1.06143188e+00,1.03643799e+00,1.04867554e+00,  1.04681396e+00,1.05615234e+00,1.05426025e+00,1.03436279e+00,  9.98291016e-01,9.82833862e-01,1.00070190e+00,9.82482910e-01,  9.83261108e-01,9.89669800e-01,9.78622437e-01,9.77203369e-01,  9.73266602e-01,9.18258667e-01,8.70391846e-01,8.77304077e-01,  8.46084595e-01,8.42773438e-01,8.51455688e-01,8.22204590e-01,  6.76544189e-01,5.53100586e-01,5.63903809e-01,5.72662354e-01,  5.84228516e-01,6.06948853e-01,6.43432617e-01,6.94763184e-01,  7.32025146e-01,7.40524292e-01,7.53692627e-01,7.70095825e-01,  7.75192261e-01,7.72338867e-01,7.61627197e-01,7.50869751e-01,  7.48275757e-01,7.44491577e-01,7.36816406e-01,7.28576660e-01,  7.18826294e-01,7.09350586e-01,7.01492310e-01,6.92535400e-01,  6.82983398e-01,6.70944214e-01,6.57897949e-01,6.45782471e-01,  6.38854980e-01,6.35971069e-01,6.28173828e-01,6.18270874e-01,  6.06460571e-01,5.90728760e-01,5.76385498e-01,5.57647705e-01,  5.06179810e-01,4.13269043e-01,3.34518433e-01,3.11439514e-01,  3.22074890e-01,3.45329285e-01,3.74351501e-01,3.63601685e-01,  3.76403809e-01,4.22187805e-01,4.66819763e-01,4.98100281e-01,  5.11230469e-01,5.08422852e-01,4.99389648e-01,4.95689392e-01,  5.02441406e-01,5.00228882e-01,4.87037659e-01,4.82528687e-01,  4.86495972e-01,4.91394043e-01,4.90196228e-01,4.87831116e-01,  4.87709045e-01,4.85542297e-01,4.79248047e-01,4.72602844e-01,  4.63859558e-01,4.46662903e-01,4.15115356e-01,3.82347107e-01,  3.87535095e-01,4.16397095e-01,4.24995422e-01,4.33586121e-01,  4.38362122e-01,4.33837891e-01,4.23042297e-01,4.09187317e-01,  3.92364502e-01,3.77510071e-01,3.68331909e-01,3.51234436e-01,  3.16322327e-01,3.05000305e-01,2.84881592e-01,2.11120605e-01,  1.12884521e-01,4.60634232e-02,2.30317116e-02,1.95326805e-02,  2.32238770e-02,3.42292786e-02,2.89897919e-02,3.16390991e-02,  4.30622101e-02,4.27331924e-02,4.75988388e-02,5.33170700e-02,  6.80561066e-02,8.61587524e-02,1.03292465e-01,1.20561600e-01,  1.21564865e-01,1.22053146e-01,1.34880066e-01,1.51397705e-01,  1.71878815e-01,2.05623627e-01,2.11040497e-01,1.87419891e-01,  1.75418854e-01,1.88426971e-01,2.02632904e-01,2.19493866e-01,  2.44812012e-01,2.64900208e-01,2.69783020e-01,2.74032593e-01,  2.84286499e-01,2.86697388e-01,2.87361145e-01,2.89321899e-01,  2.89916992e-01,2.89085388e-01,2.88536072e-01,2.88818359e-01,  2.86888123e-01,2.83599854e-01,2.80296326e-01,2.72636414e-01,  2.57606506e-01,2.49652863e-01,2.55157471e-01,2.60955811e-01,  2.62046814e-01,2.59796143e-01,2.49996185e-01,2.45510101e-01,  2.48268127e-01,2.55027771e-01,2.58857727e-01,2.61848450e-01,  2.62275696e-01,2.61573792e-01,2.60795593e-01,2.59826660e-01,  2.59117126e-01,2.56294250e-01,2.51655579e-01,2.46753693e-01,  2.43503571e-01,2.39025116e-01,2.33726501e-01,2.29225159e-01,  2.24662781e-01,2.20626831e-01,2.18433380e-01],
        [4.74647522e-01,4.85229492e-01,5.06271362e-01,5.93521118e-01,  6.28906250e-01,6.41921997e-01,6.39785767e-01,6.32995605e-01,  6.88705444e-01,7.86376953e-01,1.04400635e+00,1.17504883e+00,  1.23342896e+00,1.28356934e+00,1.28823853e+00,1.25225830e+00,  1.23348999e+00,1.33486938e+00,1.45034790e+00,1.55960083e+00,  1.64047241e+00,1.67544556e+00,1.70159912e+00,1.70022583e+00,  1.70730591e+00,1.73959351e+00,1.74920654e+00,1.68414307e+00,  1.69808960e+00,1.72528076e+00,1.69955444e+00,1.70654297e+00,  1.70034790e+00,1.63580322e+00,1.63848877e+00,1.67324829e+00,  1.69711304e+00,1.70108032e+00,1.68676758e+00,1.68951416e+00,  1.68753052e+00,1.66961670e+00,1.64154053e+00,1.62820435e+00,  1.61840820e+00,1.61495972e+00,1.62652588e+00,1.62112427e+00,  1.59579468e+00,1.59863281e+00,1.58480835e+00,1.57904053e+00,  1.56280518e+00,1.54739380e+00,1.54833984e+00,1.52243042e+00,  1.50833130e+00,1.52590942e+00,1.53137207e+00,1.52478027e+00,  1.50799561e+00,1.46905518e+00,1.50030518e+00,1.52471924e+00,  1.51800537e+00,1.50833130e+00,1.48019409e+00,1.36672974e+00,  1.32675171e+00,1.39651489e+00,1.40963745e+00,1.40850830e+00,  1.39605713e+00,1.34088135e+00,1.28903198e+00,1.28738403e+00,  1.30120850e+00,1.31719971e+00,1.31271362e+00,1.31066895e+00,  1.29293823e+00,1.16918945e+00,8.15612793e-01,8.85238647e-01,  1.15460205e+00,1.21484375e+00,1.21368408e+00,1.20675659e+00,  1.18917847e+00,1.18212891e+00,1.17468262e+00,1.15954590e+00,  1.12960815e+00,1.06765747e+00,1.03823853e+00,1.05126953e+00,  1.05279541e+00,1.05953979e+00,1.06182861e+00,1.03762817e+00,  1.00335693e+00,9.84420776e-01,9.99008179e-01,9.83154297e-01,  9.80422974e-01,9.86312866e-01,9.84573364e-01,9.77813721e-01,  9.68078613e-01,9.16564941e-01,8.68682861e-01,8.66287231e-01,  8.44131470e-01,8.45458984e-01,8.62503052e-01,8.24066162e-01,  6.76773071e-01,5.56640625e-01,5.67321777e-01,5.76293945e-01,  5.87661743e-01,6.09939575e-01,6.46545410e-01,6.97586060e-01,  7.34252930e-01,7.42416382e-01,7.55371094e-01,7.71530151e-01,  7.76550293e-01,7.73696899e-01,7.62893677e-01,7.51815796e-01,  7.49359131e-01,7.45666504e-01,7.38006592e-01,7.30117798e-01,  7.20413208e-01,7.10403442e-01,7.02346802e-01,6.93557739e-01,  6.84066772e-01,6.72149658e-01,6.59088135e-01,6.46667480e-01,  6.39602661e-01,6.36917114e-01,6.29074097e-01,6.19323730e-01,  6.07818604e-01,5.92025757e-01,5.77758789e-01,5.59280396e-01,  5.07904053e-01,4.15397644e-01,3.37173462e-01,3.14254761e-01,  3.24745178e-01,3.47640991e-01,3.76457214e-01,3.65867615e-01,  3.78784180e-01,4.24217224e-01,4.68460083e-01,4.99542236e-01,  5.12512207e-01,5.09597778e-01,5.00625610e-01,4.97024536e-01,  5.03646851e-01,5.01342773e-01,4.88075256e-01,4.83459473e-01,  4.87419128e-01,4.92370605e-01,4.91096497e-01,4.88708496e-01,  4.88403320e-01,4.86068726e-01,4.79858398e-01,4.73266602e-01,  4.64462280e-01,4.47097778e-01,4.15176392e-01,3.82148743e-01,  3.87428284e-01,4.16633606e-01,4.25575256e-01,4.34349060e-01,  4.39010620e-01,4.34539795e-01,4.23896790e-01,4.10255432e-01,  3.93501282e-01,3.78807068e-01,3.69674683e-01,3.52493286e-01,  3.17779541e-01,3.06610107e-01,2.86537170e-01,2.12871552e-01,  1.14406586e-01,4.70685959e-02,2.36477852e-02,2.01244354e-02,  2.40225792e-02,3.51352692e-02,2.96998024e-02,3.25889587e-02,  4.43248749e-02,4.39090729e-02,4.87413406e-02,5.44109344e-02,  6.93817139e-02,8.76674652e-02,1.04883194e-01,1.22213364e-01,  1.22884750e-01,1.23300552e-01,1.36348724e-01,1.53068542e-01,  1.73576355e-01,2.07126617e-01,2.12406158e-01,1.89002991e-01,  1.77074432e-01,1.90017700e-01,2.04116821e-01,2.20733643e-01,  2.45960236e-01,2.66036987e-01,2.70774841e-01,2.74971008e-01,  2.85003662e-01,2.87391663e-01,2.88101196e-01,2.90168762e-01,  2.90679932e-01,2.89497375e-01,2.89016724e-01,2.89527893e-01,  2.87620544e-01,2.84400940e-01,2.80960083e-01,2.73239136e-01,  2.58140564e-01,2.50030518e-01,2.55752563e-01,2.61505127e-01,  2.62588501e-01,2.60337830e-01,2.50640869e-01,2.46215820e-01,  2.48867035e-01,2.55584717e-01,2.59552002e-01,2.62550354e-01,  2.63084412e-01,2.62367249e-01,2.61512756e-01,2.60765076e-01,  2.59834290e-01,2.57164001e-01,2.53273010e-01,2.48947144e-01,  2.44968414e-01,2.40489960e-01,2.36152649e-01,2.31403351e-01,  2.28500366e-01,2.24067688e-01,2.21492767e-01],
        [4.66773987e-01,4.77951050e-01,5.01174927e-01,5.86776733e-01,  6.18606567e-01,6.33010864e-01,6.29974365e-01,6.25259399e-01,  6.81625366e-01,7.76992798e-01,1.03271484e+00,1.16137695e+00,  1.21987915e+00,1.27008057e+00,1.27481079e+00,1.24008179e+00,  1.22131348e+00,1.32287598e+00,1.43817139e+00,1.54565430e+00,  1.62673950e+00,1.66232300e+00,1.68814087e+00,1.68804932e+00,  1.69403076e+00,1.72714233e+00,1.73742676e+00,1.67367554e+00,  1.68786621e+00,1.71472168e+00,1.68957520e+00,1.69650269e+00,  1.69018555e+00,1.62759399e+00,1.62951660e+00,1.66448975e+00,  1.68884277e+00,1.69213867e+00,1.67819214e+00,1.67984009e+00,  1.67849731e+00,1.66223145e+00,1.63305664e+00,1.62005615e+00,  1.60937500e+00,1.60665894e+00,1.61975098e+00,1.61410522e+00,  1.58923340e+00,1.59228516e+00,1.57687378e+00,1.57278442e+00,  1.55532837e+00,1.53979492e+00,1.54287720e+00,1.51449585e+00,  1.50100708e+00,1.52029419e+00,1.52484131e+00,1.51968384e+00,  1.50225830e+00,1.46353149e+00,1.49618530e+00,1.52032471e+00,  1.51391602e+00,1.50402832e+00,1.47494507e+00,1.36282349e+00,  1.32266235e+00,1.39135742e+00,1.40576172e+00,1.40432739e+00,  1.39257812e+00,1.33639526e+00,1.28201294e+00,1.28216553e+00,  1.29663086e+00,1.31304932e+00,1.30972290e+00,1.30615234e+00,  1.28997803e+00,1.16845703e+00,8.10195923e-01,8.80874634e-01,  1.14859009e+00,1.21057129e+00,1.21389771e+00,1.20599365e+00,  1.18765259e+00,1.18203735e+00,1.17028809e+00,1.15832520e+00,  1.12734985e+00,1.05981445e+00,1.03570557e+00,1.04244995e+00,  1.04263306e+00,1.06021118e+00,1.06152344e+00,1.03897095e+00,  1.00213623e+00,9.78225708e-01,1.00180054e+00,9.84664917e-01,  9.74578857e-01,9.87854004e-01,9.82772827e-01,9.77676392e-01,  9.82223511e-01,9.15435791e-01,8.67507935e-01,8.73901367e-01,  8.35708618e-01,8.36868286e-01,8.54339600e-01,8.13903809e-01,  6.74102783e-01,5.51483154e-01,5.62271118e-01,5.71090698e-01,  5.82794189e-01,6.05422974e-01,6.42395020e-01,6.94076538e-01,  7.31552124e-01,7.40158081e-01,7.53265381e-01,7.69683838e-01,  7.75207520e-01,7.72445679e-01,7.61672974e-01,7.50701904e-01,  7.48184204e-01,7.44552612e-01,7.36938477e-01,7.28942871e-01,  7.19161987e-01,7.09381104e-01,7.01400757e-01,6.92474365e-01,  6.82891846e-01,6.70898438e-01,6.57836914e-01,6.45538330e-01,  6.38565063e-01,6.35818481e-01,6.28067017e-01,6.18209839e-01,  6.06597900e-01,5.90759277e-01,5.76202393e-01,5.57220459e-01,  5.05310059e-01,4.12139893e-01,3.33473206e-01,3.10417175e-01,  3.21075439e-01,3.44314575e-01,3.73199463e-01,3.62434387e-01,  3.75404358e-01,4.21333313e-01,4.66156006e-01,4.97612000e-01,  5.10925293e-01,5.08193970e-01,4.99145508e-01,4.95429993e-01,  5.02243042e-01,5.00030518e-01,4.86763000e-01,4.82223511e-01,  4.86389160e-01,4.91378784e-01,4.90036011e-01,4.87785339e-01,  4.87632751e-01,4.85374451e-01,4.79095459e-01,4.72434998e-01,  4.63623047e-01,4.46128845e-01,4.14001465e-01,3.80821228e-01,  3.86268616e-01,4.15733337e-01,4.24667358e-01,4.33456421e-01,  4.38293457e-01,4.33799744e-01,4.22988892e-01,4.09133911e-01,  3.92173767e-01,3.77235413e-01,3.68080139e-01,3.50952148e-01,  3.15826416e-01,3.04504395e-01,2.84317017e-01,2.10430145e-01,  1.12228394e-01,4.59241867e-02,2.32329369e-02,1.98245049e-02,  2.34375000e-02,3.40766907e-02,2.89139748e-02,3.16905975e-02,  4.30383682e-02,4.28113937e-02,4.75225449e-02,5.29794693e-02,  6.77013397e-02,8.57295990e-02,1.02895737e-01,1.20220184e-01,  1.21019363e-01,1.21295929e-01,1.34170532e-01,1.50882721e-01,  1.71520233e-01,2.05257416e-01,2.10678101e-01,1.87065125e-01,  1.75022125e-01,1.88087463e-01,2.02377319e-01,2.19215393e-01,  2.44586945e-01,2.64823914e-01,2.69851685e-01,2.74299622e-01,  2.84667969e-01,2.87040710e-01,2.87651062e-01,2.89596558e-01,  2.90336609e-01,2.89459229e-01,2.88948059e-01,2.89398193e-01,  2.87368774e-01,2.84065247e-01,2.80761719e-01,2.73033142e-01,  2.57766724e-01,2.49626160e-01,2.55378723e-01,2.61360168e-01,  2.62672424e-01,2.60459900e-01,2.50335693e-01,2.45948792e-01,  2.48748779e-01,2.55638123e-01,2.59689331e-01,2.62779236e-01,  2.63267517e-01,2.62405396e-01,2.61665344e-01,2.61146545e-01,  2.60536194e-01,2.58369446e-01,2.54348755e-01,2.49824524e-01,  2.46330261e-01,2.43400574e-01,2.38922119e-01,2.33825684e-01,  2.30331421e-01,2.25948334e-01,2.23339081e-01],
        [4.52522278e-01,4.68765259e-01,4.92729187e-01,5.74462891e-01,  6.11709595e-01,6.24404907e-01,6.22177124e-01,6.17187500e-01,  6.69769287e-01,7.67105103e-01,1.01959229e+00,1.14801025e+00,  1.20785522e+00,1.25656128e+00,1.26318359e+00,1.22842407e+00,  1.21011353e+00,1.31143188e+00,1.42358398e+00,1.53222656e+00,  1.61328125e+00,1.64901733e+00,1.67660522e+00,1.67584229e+00,  1.68255615e+00,1.71517944e+00,1.72564697e+00,1.66375732e+00,  1.67660522e+00,1.70471191e+00,1.68063354e+00,1.68630981e+00,  1.68136597e+00,1.61798096e+00,1.62051392e+00,1.65618896e+00,  1.67938232e+00,1.68408203e+00,1.66979980e+00,1.67196655e+00,  1.67156982e+00,1.65350342e+00,1.62567139e+00,1.61209106e+00,  1.60131836e+00,1.60006714e+00,1.61141968e+00,1.60717773e+00,  1.58328247e+00,1.58377075e+00,1.57000732e+00,1.56561279e+00,  1.54904175e+00,1.53591919e+00,1.53573608e+00,1.50955200e+00,  1.49688721e+00,1.51379395e+00,1.51977539e+00,1.51458740e+00,  1.49813843e+00,1.46041870e+00,1.48999023e+00,1.51550293e+00,  1.51055908e+00,1.49896240e+00,1.47201538e+00,1.35864258e+00,  1.31777954e+00,1.38916016e+00,1.40048218e+00,1.40060425e+00,  1.38983154e+00,1.33178711e+00,1.28070068e+00,1.27923584e+00,  1.29257202e+00,1.31240845e+00,1.30487061e+00,1.30285645e+00,  1.28811646e+00,1.16406250e+00,8.08151245e-01,8.76342773e-01,  1.14624023e+00,1.21102905e+00,1.20770264e+00,1.20117188e+00,  1.18576050e+00,1.17294312e+00,1.16732788e+00,1.15496826e+00,  1.11941528e+00,1.05575562e+00,1.02914429e+00,1.03756714e+00,  1.04733276e+00,1.05636597e+00,1.05517578e+00,1.04019165e+00,  9.91989136e-01,9.78485107e-01,9.98031616e-01,9.73693848e-01,  9.86740112e-01,9.86236572e-01,9.80697632e-01,9.92385864e-01,  9.66629028e-01,9.11773682e-01,8.70056152e-01,8.51287842e-01,  8.39950562e-01,8.44070435e-01,8.40087891e-01,8.25485229e-01,  6.76269531e-01,5.47927856e-01,5.58929443e-01,5.67932129e-01,  5.79559326e-01,6.02661133e-01,6.39968872e-01,6.92108154e-01,  7.29888916e-01,7.38586426e-01,7.52212524e-01,7.69165039e-01,  7.74642944e-01,7.72018433e-01,7.61352539e-01,7.50244141e-01,  7.47848511e-01,7.44140625e-01,7.36511230e-01,7.28485107e-01,  7.18795776e-01,7.09030151e-01,7.00973511e-01,6.91986084e-01,  6.82357788e-01,6.70379639e-01,6.57226562e-01,6.44943237e-01,  6.38015747e-01,6.35528564e-01,6.27716064e-01,6.17752075e-01,  6.06018066e-01,5.89920044e-01,5.75134277e-01,5.56076050e-01,  5.03784180e-01,4.10095215e-01,3.30993652e-01,3.07861328e-01,  3.18496704e-01,3.41995239e-01,3.71055603e-01,3.60214233e-01,  3.73336792e-01,4.19532776e-01,4.64698792e-01,4.96315002e-01,  5.09689331e-01,5.07110596e-01,4.98207092e-01,4.94712830e-01,  5.01663208e-01,4.99389648e-01,4.85969543e-01,4.81338501e-01,  4.85618591e-01,4.90768433e-01,4.89692688e-01,4.87411499e-01,  4.87304688e-01,4.85000610e-01,4.78782654e-01,4.72068787e-01,  4.63134766e-01,4.45594788e-01,4.13162231e-01,3.79615784e-01,  3.85093689e-01,4.14802551e-01,4.24148560e-01,4.33166504e-01,  4.37927246e-01,4.33357239e-01,4.22485352e-01,4.08424377e-01,  3.91532898e-01,3.76579285e-01,3.67164612e-01,3.49761963e-01,  3.14361572e-01,3.03031921e-01,2.82867432e-01,2.08721161e-01,  1.10609055e-01,4.47807312e-02,2.24099159e-02,1.90768242e-02,  2.26898193e-02,3.31716537e-02,2.82406807e-02,3.08103561e-02,  4.17203903e-02,4.15439606e-02,4.62951660e-02,5.18960953e-02,  6.64272308e-02,8.43238831e-02,1.01261139e-01,1.18442535e-01,  1.19291306e-01,1.19638443e-01,1.32591248e-01,1.49307251e-01,  1.69883728e-01,2.03712463e-01,2.09121704e-01,1.85565948e-01,  1.73576355e-01,1.86588287e-01,2.00942993e-01,2.17952728e-01,  2.43598938e-01,2.64053345e-01,2.69126892e-01,2.73567200e-01,  2.83897400e-01,2.86392212e-01,2.87223816e-01,2.89390564e-01,  2.89962769e-01,2.88986206e-01,2.88429260e-01,2.88948059e-01,  2.87223816e-01,2.83897400e-01,2.80471802e-01,2.72460938e-01,  2.57133484e-01,2.49137878e-01,2.55081177e-01,2.61032104e-01,  2.62161255e-01,2.59727478e-01,2.49588013e-01,2.45292664e-01,  2.48218536e-01,2.54928589e-01,2.58872986e-01,2.61863708e-01,  2.62596130e-01,2.61833191e-01,2.60879517e-01,2.59902954e-01,  2.59040833e-01,2.56820679e-01,2.52769470e-01,2.48332977e-01,  2.44655609e-01,2.40627289e-01,2.36335754e-01,2.31269836e-01,  2.27287292e-01,2.22808838e-01,2.20554352e-01],
        [4.51805115e-01,4.62028503e-01,4.86511230e-01,5.67367554e-01,  6.00387573e-01,6.12686157e-01,6.11358643e-01,6.08810425e-01,  6.62490845e-01,7.57446289e-01,1.00677490e+00,1.13418579e+00,  1.19314575e+00,1.24212646e+00,1.24917603e+00,1.21569824e+00,  1.19680786e+00,1.29818726e+00,1.41076660e+00,1.51864624e+00,  1.59951782e+00,1.63452148e+00,1.66165161e+00,1.66146851e+00,  1.66870117e+00,1.70196533e+00,1.71252441e+00,1.65139771e+00,  1.66540527e+00,1.69213867e+00,1.66802979e+00,1.67437744e+00,  1.66940308e+00,1.60736084e+00,1.60949707e+00,1.64520264e+00,  1.66842651e+00,1.67312622e+00,1.65948486e+00,1.66174316e+00,  1.66104126e+00,1.64392090e+00,1.61541748e+00,1.60253906e+00,  1.59124756e+00,1.58908081e+00,1.60089111e+00,1.59594727e+00,  1.57278442e+00,1.57510376e+00,1.56146240e+00,1.55642700e+00,  1.54022217e+00,1.52639771e+00,1.52822876e+00,1.50115967e+00,  1.48876953e+00,1.50683594e+00,1.51370239e+00,1.50827026e+00,  1.48889160e+00,1.45260620e+00,1.48492432e+00,1.50958252e+00,  1.50326538e+00,1.49301147e+00,1.46603394e+00,1.35339355e+00,  1.31158447e+00,1.38336182e+00,1.39633179e+00,1.39477539e+00,  1.38488770e+00,1.32586670e+00,1.27279663e+00,1.27285767e+00,  1.28500366e+00,1.30587769e+00,1.30310059e+00,1.29937744e+00,  1.28536987e+00,1.16030884e+00,8.03756714e-01,8.73092651e-01,  1.14077759e+00,1.20727539e+00,1.20855713e+00,1.19866943e+00,  1.18444824e+00,1.17480469e+00,1.16744995e+00,1.15542603e+00,  1.11862183e+00,1.05133057e+00,1.02835083e+00,1.03524780e+00,  1.04272461e+00,1.05181885e+00,1.05200195e+00,1.04418945e+00,  9.99389648e-01,9.79064941e-01,9.97482300e-01,9.76562500e-01,  9.82315063e-01,9.89715576e-01,9.78988647e-01,9.79476929e-01,  9.71298218e-01,9.04067993e-01,8.57391357e-01,8.51928711e-01,  8.25149536e-01,8.33206177e-01,8.34915161e-01,8.15460205e-01,  6.69891357e-01,5.41290283e-01,5.52307129e-01,5.61187744e-01,  5.73242188e-01,5.96649170e-01,6.34338379e-01,6.87728882e-01,  7.26837158e-01,7.35809326e-01,7.49740601e-01,7.67486572e-01,  7.73437500e-01,7.70950317e-01,7.60269165e-01,7.49313354e-01,  7.46826172e-01,7.43240356e-01,7.35610962e-01,7.27508545e-01,  7.17834473e-01,7.08251953e-01,7.00088501e-01,6.91192627e-01,  6.81610107e-01,6.69418335e-01,6.56280518e-01,6.44042969e-01,  6.36932373e-01,6.34124756e-01,6.26617432e-01,6.16729736e-01,  6.04782104e-01,5.88439941e-01,5.73272705e-01,5.53405762e-01,  5.00427246e-01,4.05807495e-01,3.25965881e-01,3.02734375e-01,  3.13545227e-01,3.37417603e-01,3.66714478e-01,3.55545044e-01,  3.68766785e-01,4.15802002e-01,4.61853027e-01,4.93942261e-01,  5.07720947e-01,5.05340576e-01,4.96429443e-01,4.92927551e-01,  4.99946594e-01,4.97825623e-01,4.84420776e-01,4.79972839e-01,  4.84245300e-01,4.89662170e-01,4.88616943e-01,4.86328125e-01,  4.86381531e-01,4.84298706e-01,4.78134155e-01,4.71450806e-01,  4.62379456e-01,4.44564819e-01,4.11933899e-01,3.78234863e-01,  3.83842468e-01,4.13887024e-01,4.23530579e-01,4.32571411e-01,  4.37309265e-01,4.32418823e-01,4.21249390e-01,4.07287598e-01,  3.90144348e-01,3.74931335e-01,3.65379333e-01,3.47579956e-01,  3.11645508e-01,3.00392151e-01,2.80090332e-01,2.05692291e-01,  1.07873917e-01,4.32405472e-02,2.17895508e-02,1.86204910e-02,  2.19593048e-02,3.19185257e-02,2.72278786e-02,2.95491219e-02,  3.99370193e-02,3.98893356e-02,4.47874069e-02,5.00621796e-02,  6.42528534e-02,8.17298889e-02,9.85126495e-02,1.15518570e-01,  1.16718292e-01,1.17067337e-01,1.29642487e-01,1.46305084e-01,  1.67034149e-01,2.01004028e-01,2.06592560e-01,1.82865143e-01,  1.70806885e-01,1.83841705e-01,1.98398590e-01,2.15732574e-01,  2.41752625e-01,2.62763977e-01,2.68104553e-01,2.72460938e-01,  2.83035278e-01,2.85682678e-01,2.86674500e-01,2.88986206e-01,  2.89726257e-01,2.88742065e-01,2.88177490e-01,2.88833618e-01,  2.87109375e-01,2.83859253e-01,2.80494690e-01,2.72323608e-01,  2.56736755e-01,2.48710632e-01,2.54905701e-01,2.61047363e-01,  2.62062073e-01,2.59613037e-01,2.49599457e-01,2.45117188e-01,  2.48138428e-01,2.55081177e-01,2.58918762e-01,2.61833191e-01,  2.62535095e-01,2.61802673e-01,2.60963440e-01,2.59834290e-01,  2.58842468e-01,2.56980896e-01,2.53936768e-01,2.49614716e-01,  2.45891571e-01,2.41386414e-01,2.37224579e-01,2.33295441e-01,  2.29576111e-01,2.24601746e-01,2.21900940e-01],
        [4.43519592e-01,4.53125000e-01,4.78057861e-01,5.58715820e-01,  5.91461182e-01,6.07391357e-01,6.04339600e-01,6.01699829e-01,  6.56433105e-01,7.48825073e-01,9.97482300e-01,1.12356567e+00,  1.18237305e+00,1.23269653e+00,1.23892212e+00,1.20648193e+00,  1.18811035e+00,1.28836060e+00,1.40243530e+00,1.50875854e+00,  1.59094238e+00,1.62576294e+00,1.65313721e+00,1.65338135e+00,  1.66049194e+00,1.69384766e+00,1.70468140e+00,1.64355469e+00,  1.65872192e+00,1.68493652e+00,1.66119385e+00,1.66754150e+00,  1.66207886e+00,1.60162354e+00,1.60369873e+00,1.63876343e+00,  1.66213989e+00,1.66610718e+00,1.65322876e+00,1.65518188e+00,  1.65426636e+00,1.63796997e+00,1.60919189e+00,1.59680176e+00,  1.58526611e+00,1.58126831e+00,1.59478760e+00,1.58978271e+00,  1.56665039e+00,1.56890869e+00,1.55487061e+00,1.55133057e+00,  1.53442383e+00,1.52124023e+00,1.52450562e+00,1.49548340e+00,  1.48263550e+00,1.50302124e+00,1.50970459e+00,1.50527954e+00,  1.48715210e+00,1.44955444e+00,1.48275757e+00,1.50579834e+00,  1.50091553e+00,1.49105835e+00,1.46292114e+00,1.35150146e+00,  1.31063843e+00,1.38101196e+00,1.39566040e+00,1.39309692e+00,  1.38265991e+00,1.32568359e+00,1.27133179e+00,1.27249146e+00,  1.28457642e+00,1.30361938e+00,1.30209351e+00,1.29785156e+00,  1.28332520e+00,1.15957642e+00,7.97790527e-01,8.68682861e-01,  1.14047241e+00,1.20437622e+00,1.20745850e+00,1.19546509e+00,  1.18273926e+00,1.17642212e+00,1.16354370e+00,1.15396118e+00,  1.12149048e+00,1.05001831e+00,1.02685547e+00,1.03652954e+00,  1.04168701e+00,1.05578613e+00,1.05081177e+00,1.03656006e+00,  9.96414185e-01,9.77203369e-01,9.98809814e-01,9.77554321e-01,  9.72854614e-01,9.88754272e-01,9.76409912e-01,9.77371216e-01,  9.73419189e-01,8.91921997e-01,8.57894897e-01,8.68240356e-01,  8.26446533e-01,8.37387085e-01,8.46237183e-01,8.02642822e-01,  6.63375854e-01,5.40527344e-01,5.51986694e-01,5.61111450e-01,  5.72967529e-01,5.96191406e-01,6.33926392e-01,6.87149048e-01,  7.25967407e-01,7.34954834e-01,7.48977661e-01,7.66433716e-01,  7.72552490e-01,7.70111084e-01,7.59399414e-01,7.48489380e-01,  7.46002197e-01,7.42370605e-01,7.34893799e-01,7.26745605e-01,  7.17025757e-01,7.07260132e-01,6.99386597e-01,6.90429688e-01,  6.80816650e-01,6.68792725e-01,6.55303955e-01,6.42929077e-01,  6.35894775e-01,6.33331299e-01,6.25701904e-01,6.15890503e-01,  6.04156494e-01,5.87844849e-01,5.72723389e-01,5.53009033e-01,  5.00015259e-01,4.05288696e-01,3.25592041e-01,3.02383423e-01,  3.13377380e-01,3.37303162e-01,3.66355896e-01,3.55194092e-01,  3.68362427e-01,4.15191650e-01,4.61151123e-01,4.93362427e-01,  5.07034302e-01,5.04653931e-01,4.95841980e-01,4.92240906e-01,  4.99313354e-01,4.97123718e-01,4.83604431e-01,4.79110718e-01,  4.83558655e-01,4.88868713e-01,4.87815857e-01,4.85733032e-01,  4.85832214e-01,4.83619690e-01,4.77279663e-01,4.70497131e-01,  4.61402893e-01,4.43519592e-01,4.10568237e-01,3.76678467e-01,  3.82400513e-01,4.12742615e-01,4.22645569e-01,4.31686401e-01,  4.36470032e-01,4.31831360e-01,4.20814514e-01,4.06723022e-01,  3.89678955e-01,3.74458313e-01,3.64830017e-01,3.47206116e-01,  3.11225891e-01,2.99819946e-01,2.79525757e-01,2.05329895e-01,  1.07669830e-01,4.29534912e-02,2.14171410e-02,1.82104111e-02,  2.13923454e-02,3.12075615e-02,2.64801979e-02,2.90884972e-02,  3.95278931e-02,3.96442413e-02,4.43515778e-02,4.96139526e-02,  6.37645721e-02,8.13198090e-02,9.81941223e-02,1.15346909e-01,  1.16291046e-01,1.16626740e-01,1.29314423e-01,1.45950317e-01,  1.66687012e-01,2.00572968e-01,2.06123352e-01,1.82556152e-01,  1.70459747e-01,1.83635712e-01,1.98101044e-01,2.15274811e-01,  2.41214752e-01,2.61955261e-01,2.67379761e-01,2.71965027e-01,  2.82531738e-01,2.85087585e-01,2.86064148e-01,2.88162231e-01,  2.88879395e-01,2.87887573e-01,2.87437439e-01,2.87971497e-01,  2.86224365e-01,2.82966614e-01,2.79556274e-01,2.71385193e-01,  2.55699158e-01,2.47543335e-01,2.53585815e-01,2.59834290e-01,  2.61161804e-01,2.58666992e-01,2.48233795e-01,2.43762970e-01,  2.46608734e-01,2.53547668e-01,2.57484436e-01,2.60574341e-01,  2.60986328e-01,2.60139465e-01,2.58918762e-01,2.57713318e-01,  2.56759644e-01,2.54272461e-01,2.50244141e-01,2.45342255e-01,  2.41191864e-01,2.36873627e-01,2.32421875e-01,2.27794647e-01,  2.23815918e-01,2.18902588e-01,2.17464447e-01],
        [4.28359985e-01,4.45411682e-01,4.70283508e-01,5.46295166e-01,  5.84182739e-01,5.96572876e-01,5.93765259e-01,5.93719482e-01,  6.43844604e-01,7.39761353e-01,9.84451294e-01,1.10943604e+00,  1.16937256e+00,1.21682739e+00,1.22561646e+00,1.19317627e+00,  1.17620850e+00,1.27716064e+00,1.38800049e+00,1.49563599e+00,  1.57568359e+00,1.61108398e+00,1.64047241e+00,1.63964844e+00,  1.64822388e+00,1.68099976e+00,1.69116211e+00,1.63287354e+00,  1.64627075e+00,1.67379761e+00,1.65112305e+00,1.65658569e+00,  1.65307617e+00,1.59158325e+00,1.59414673e+00,1.62927246e+00,  1.65136719e+00,1.65670776e+00,1.64318848e+00,1.64498901e+00,  1.64572144e+00,1.62805176e+00,1.60064697e+00,1.58752441e+00,  1.57440186e+00,1.57333374e+00,1.58599854e+00,1.58148193e+00,  1.56005859e+00,1.55972290e+00,1.54748535e+00,1.54360962e+00,  1.52688599e+00,1.51568604e+00,1.51638794e+00,1.48983765e+00,  1.47775269e+00,1.49594116e+00,1.50491333e+00,1.49969482e+00,  1.48141479e+00,1.44647217e+00,1.47650146e+00,1.50155640e+00,  1.49697876e+00,1.48526001e+00,1.46014404e+00,1.34606934e+00,  1.30480957e+00,1.37887573e+00,1.39089966e+00,1.39105225e+00,  1.38220215e+00,1.32257080e+00,1.27069092e+00,1.26907349e+00,  1.28076172e+00,1.30291748e+00,1.29797363e+00,1.29632568e+00,  1.28527832e+00,1.15576172e+00,7.95913696e-01,8.66043091e-01,  1.13607788e+00,1.20532227e+00,1.20202637e+00,1.19357300e+00,  1.18280029e+00,1.16726685e+00,1.16244507e+00,1.15377808e+00,  1.11209106e+00,1.05032349e+00,1.02023315e+00,1.02886963e+00,  1.04241943e+00,1.04794312e+00,1.04922485e+00,1.04086304e+00,  9.89166260e-01,9.82742310e-01,1.00259399e+00,9.63088989e-01,  9.71237183e-01,9.75296021e-01,9.62539673e-01,9.81384277e-01,  9.58190918e-01,9.03549194e-01,8.77288818e-01,8.50234985e-01,  8.28811646e-01,8.33648682e-01,8.26599121e-01,8.15124512e-01,  6.70913696e-01,5.41549683e-01,5.52780151e-01,5.61859131e-01,  5.73638916e-01,5.96832275e-01,6.34307861e-01,6.87438965e-01,  7.26348877e-01,7.35336304e-01,7.49282837e-01,7.66921997e-01,  7.72979736e-01,7.70477295e-01,7.59704590e-01,7.48703003e-01,  7.46246338e-01,7.42645264e-01,7.35046387e-01,7.27005005e-01,  7.17376709e-01,7.07580566e-01,6.99508667e-01,6.90444946e-01,  6.80877686e-01,6.68457031e-01,6.55227661e-01,6.42944336e-01,  6.35986328e-01,6.33560181e-01,6.25915527e-01,6.16210938e-01,  6.04324341e-01,5.88012695e-01,5.72845459e-01,5.53115845e-01,  5.00320435e-01,4.05776978e-01,3.26278687e-01,3.03131104e-01,  3.13949585e-01,3.37852478e-01,3.66859436e-01,3.55842590e-01,  3.68949890e-01,4.15679932e-01,4.61479187e-01,4.93560791e-01,  5.07354736e-01,5.04882812e-01,4.96032715e-01,4.92416382e-01,  4.99458313e-01,4.97352600e-01,4.83886719e-01,4.79309082e-01,  4.83757019e-01,4.89044189e-01,4.87922668e-01,4.85763550e-01,  4.85939026e-01,4.83772278e-01,4.77394104e-01,4.70588684e-01,  4.61341858e-01,4.43244934e-01,4.09980774e-01,3.75801086e-01,  3.81553650e-01,4.12300110e-01,4.22401428e-01,4.31709290e-01,  4.36714172e-01,4.32113647e-01,4.21165466e-01,4.07073975e-01,  3.89877319e-01,3.74664307e-01,3.65089417e-01,3.47450256e-01,  3.11553955e-01,3.00285339e-01,2.80052185e-01,2.05738068e-01,  1.08045578e-01,4.31509018e-02,2.16555595e-02,1.84879303e-02,  2.17337608e-02,3.16400528e-02,2.69284248e-02,2.95538902e-02,  4.01821136e-02,4.00772095e-02,4.47120667e-02,5.00106812e-02,  6.42395020e-02,8.17604065e-02,9.85870361e-02,1.15692139e-01,  1.16704941e-01,1.16861343e-01,1.29585266e-01,1.46362305e-01,  1.67129517e-01,2.01015472e-01,2.06535339e-01,1.82922363e-01,  1.70883179e-01,1.84078217e-01,1.98516846e-01,2.15724945e-01,  2.41592407e-01,2.62336731e-01,2.67669678e-01,2.72293091e-01,  2.82852173e-01,2.85430908e-01,2.86346436e-01,2.88513184e-01,  2.89108276e-01,2.88185120e-01,2.87590027e-01,2.88246155e-01,  2.86537170e-01,2.83409119e-01,2.80097961e-01,2.72033691e-01,  2.56019592e-01,2.47695923e-01,2.53799438e-01,2.60162354e-01,  2.61581421e-01,2.59078979e-01,2.48683929e-01,2.44167328e-01,  2.47001648e-01,2.53890991e-01,2.57957458e-01,2.61009216e-01,  2.61573792e-01,2.60925293e-01,2.59803772e-01,2.58529663e-01,  2.57469177e-01,2.55111694e-01,2.51159668e-01,2.46452332e-01,  2.43087769e-01,2.39349365e-01,2.34889984e-01,2.30274200e-01,  2.25666046e-01,2.21210480e-01,2.18315125e-01],
        [4.26338196e-01,4.36309814e-01,4.61463928e-01,5.43228149e-01,  5.74050903e-01,5.88409424e-01,5.87890625e-01,5.85937500e-01,  6.39221191e-01,7.30438232e-01,9.75097656e-01,1.09811401e+00,  1.15737915e+00,1.20755005e+00,1.21438599e+00,1.18438721e+00,  1.16860962e+00,1.26776123e+00,1.37994385e+00,1.48645020e+00,  1.56729126e+00,1.60308838e+00,1.63046265e+00,1.63217163e+00,  1.63955688e+00,1.67294312e+00,1.68435669e+00,1.62521362e+00,  1.64050293e+00,1.66738892e+00,1.64419556e+00,1.65148926e+00,  1.64639282e+00,1.58654785e+00,1.58938599e+00,1.62380981e+00,  1.64724731e+00,1.65066528e+00,1.63772583e+00,1.64086914e+00,  1.64016724e+00,1.62368774e+00,1.59524536e+00,1.58203125e+00,  1.57098389e+00,1.56701660e+00,1.58215332e+00,1.57751465e+00,  1.55386353e+00,1.55706787e+00,1.54244995e+00,1.53887939e+00,  1.52481079e+00,1.51165771e+00,1.51449585e+00,1.48715210e+00,  1.47424316e+00,1.49542236e+00,1.50213623e+00,1.49813843e+00,  1.48092651e+00,1.44375610e+00,1.47824097e+00,1.50183105e+00,  1.49645996e+00,1.48773193e+00,1.45764160e+00,1.34460449e+00,  1.30441284e+00,1.37548828e+00,1.39215088e+00,1.39166260e+00,  1.38095093e+00,1.32302856e+00,1.26791382e+00,1.26876831e+00,  1.28280640e+00,1.30236816e+00,1.30120850e+00,1.29562378e+00,  1.28088379e+00,1.15771484e+00,7.92739868e-01,8.64715576e-01,  1.14080811e+00,1.20346069e+00,1.20666504e+00,1.19873047e+00,  1.18011475e+00,1.17333984e+00,1.16497803e+00,1.15490723e+00,  1.12115479e+00,1.04598999e+00,1.02590942e+00,1.03350830e+00,  1.03558350e+00,1.05480957e+00,1.05334473e+00,1.03656006e+00,  1.00079346e+00,9.77371216e-01,9.99877930e-01,9.78286743e-01,  9.78530884e-01,9.93316650e-01,9.88250732e-01,9.76669312e-01,  9.72290039e-01,9.04190063e-01,8.60321045e-01,8.67279053e-01,  8.21197510e-01,8.32809448e-01,8.44467163e-01,8.11767578e-01,  6.65161133e-01,5.37811279e-01,5.49087524e-01,5.58090210e-01,  5.69976807e-01,5.93017578e-01,6.31011963e-01,6.84967041e-01,  7.24334717e-01,7.33627319e-01,7.48123169e-01,7.65975952e-01,  7.71987915e-01,7.69592285e-01,7.58865356e-01,7.47848511e-01,  7.45407104e-01,7.41912842e-01,7.34436035e-01,7.26348877e-01,  7.16812134e-01,7.07015991e-01,6.98913574e-01,6.89926147e-01,  6.80068970e-01,6.67892456e-01,6.54388428e-01,6.41983032e-01,  6.35284424e-01,6.32904053e-01,6.25366211e-01,6.15585327e-01,  6.03683472e-01,5.87188721e-01,5.71716309e-01,5.51788330e-01,  4.98512268e-01,4.03434753e-01,3.23562622e-01,3.00277710e-01,  3.11248779e-01,3.35380554e-01,3.64501953e-01,3.53172302e-01,  3.66455078e-01,4.13452148e-01,4.59686279e-01,4.92179871e-01,  5.06027222e-01,5.03662109e-01,4.94842529e-01,4.91302490e-01,  4.98580933e-01,4.96482849e-01,4.82963562e-01,4.78317261e-01,  4.82879639e-01,4.88189697e-01,4.87106323e-01,4.85176086e-01,  4.85313416e-01,4.83154297e-01,4.76890564e-01,4.70008850e-01,  4.60716248e-01,4.42413330e-01,4.08683777e-01,3.74275208e-01,  3.80134583e-01,4.11254883e-01,4.21699524e-01,4.31259155e-01,  4.36111450e-01,4.31518555e-01,4.20494080e-01,4.06234741e-01,  3.89045715e-01,3.73733521e-01,3.63967896e-01,3.46176147e-01,  3.10035706e-01,2.98622131e-01,2.78457642e-01,2.03964233e-01,  1.06401443e-01,4.22029495e-02,2.11110115e-02,1.79691315e-02,  2.11853981e-02,3.07478905e-02,2.62007713e-02,2.87160873e-02,  3.89642715e-02,3.90329361e-02,4.37126160e-02,4.88615036e-02,  6.29158020e-02,8.03203583e-02,9.71374512e-02,1.14173889e-01,  1.15064621e-01,1.15274429e-01,1.27891541e-01,1.44660950e-01,  1.65542603e-01,1.99535370e-01,2.04975128e-01,1.81423187e-01,  1.69357300e-01,1.82537079e-01,1.97093964e-01,2.14359283e-01,  2.40581512e-01,2.61604309e-01,2.66952515e-01,2.71629333e-01,  2.82264709e-01,2.84820557e-01,2.85766602e-01,2.87963867e-01,  2.88803101e-01,2.88002014e-01,2.87490845e-01,2.88078308e-01,  2.86148071e-01,2.82913208e-01,2.79594421e-01,2.71347046e-01,  2.55538940e-01,2.47344971e-01,2.53562927e-01,2.59933472e-01,  2.61100769e-01,2.58499146e-01,2.48081207e-01,2.43507385e-01,  2.46585846e-01,2.53768921e-01,2.57804871e-01,2.60772705e-01,  2.61230469e-01,2.60139465e-01,2.58972168e-01,2.57850647e-01,  2.56683350e-01,2.54463196e-01,2.50534058e-01,2.46353149e-01,  2.42885590e-01,2.38098145e-01,2.33726501e-01,2.28706360e-01,  2.24979401e-01,2.20893860e-01,2.18074799e-01],
        [4.13665771e-01,4.29222107e-01,4.50920105e-01,5.27496338e-01,  5.62789917e-01,5.74386597e-01,5.76232910e-01,5.73791504e-01,  6.24923706e-01,7.18719482e-01,9.56649780e-01,1.07946777e+00,  1.13806152e+00,1.18753052e+00,1.19696045e+00,1.16622925e+00,  1.15078735e+00,1.24932861e+00,1.36013794e+00,1.46694946e+00,  1.54678345e+00,1.58358765e+00,1.61093140e+00,1.61175537e+00,  1.62158203e+00,1.65350342e+00,1.66537476e+00,1.60830688e+00,  1.62274170e+00,1.65075684e+00,1.62762451e+00,1.63470459e+00,  1.63119507e+00,1.57070923e+00,1.57379150e+00,1.60845947e+00,  1.63171387e+00,1.63662720e+00,1.62292480e+00,1.62603760e+00,  1.62548828e+00,1.60910034e+00,1.58193970e+00,1.56741333e+00,  1.55621338e+00,1.55407715e+00,1.56628418e+00,1.56372070e+00,  1.54067993e+00,1.54238892e+00,1.53109741e+00,1.52697754e+00,  1.51214600e+00,1.49993896e+00,1.50103760e+00,1.47463989e+00,  1.46334839e+00,1.48385620e+00,1.49227905e+00,1.48663330e+00,  1.47006226e+00,1.43472290e+00,1.46624756e+00,1.49304199e+00,  1.48736572e+00,1.47695923e+00,1.45117188e+00,1.33453369e+00,  1.29461670e+00,1.36807251e+00,1.38275146e+00,1.38555908e+00,  1.37335205e+00,1.31277466e+00,1.26013184e+00,1.25885010e+00,  1.27416992e+00,1.29760742e+00,1.29290771e+00,1.29156494e+00,  1.27630615e+00,1.14935303e+00,7.87597656e-01,8.54278564e-01,  1.13204956e+00,1.20114136e+00,1.19622803e+00,1.18969727e+00,  1.17526245e+00,1.16638184e+00,1.16131592e+00,1.14694214e+00,  1.11291504e+00,1.04840088e+00,1.01525879e+00,1.02990723e+00,  1.04040527e+00,1.04446411e+00,1.04931641e+00,1.03067017e+00,  9.85763550e-01,9.77264404e-01,9.91104126e-01,9.67803955e-01,  9.72457886e-01,9.75616455e-01,9.83032227e-01,9.80880737e-01,  9.53903198e-01,9.05715942e-01,8.54476929e-01,8.45855713e-01,  8.27270508e-01,8.25576782e-01,8.25698853e-01,8.16909790e-01,  6.62704468e-01,5.31799316e-01,5.43243408e-01,5.52551270e-01,  5.64682007e-01,5.88272095e-01,6.26831055e-01,6.81564331e-01,  7.21466064e-01,7.30834961e-01,7.45758057e-01,7.64114380e-01,  7.70782471e-01,7.68630981e-01,7.58041382e-01,7.46948242e-01,  7.44552612e-01,7.40997314e-01,7.33352661e-01,7.25341797e-01,  7.15713501e-01,7.05795288e-01,6.97860718e-01,6.88888550e-01,  6.79183960e-01,6.66870117e-01,6.53549194e-01,6.41036987e-01,  6.34216309e-01,6.31881714e-01,6.24084473e-01,6.14166260e-01,  6.02371216e-01,5.85617065e-01,5.69854736e-01,5.49652100e-01,  4.95834351e-01,3.99864197e-01,3.19473267e-01,2.95936584e-01,  3.07106018e-01,3.31474304e-01,3.60557556e-01,3.49037170e-01,  3.62640381e-01,4.10232544e-01,4.57374573e-01,4.90295410e-01,  5.04333496e-01,5.02029419e-01,4.93194580e-01,4.89669800e-01,  4.96986389e-01,4.95079041e-01,4.81521606e-01,4.76943970e-01,  4.81643677e-01,4.87274170e-01,4.86289978e-01,4.84313965e-01,  4.84542847e-01,4.82368469e-01,4.76112366e-01,4.69230652e-01,  4.59754944e-01,4.41368103e-01,4.07341003e-01,3.72573853e-01,  3.78601074e-01,4.10125732e-01,4.20913696e-01,4.30572510e-01,  4.35409546e-01,4.30740356e-01,4.19525146e-01,4.05143738e-01,  3.87733459e-01,3.72360229e-01,3.62442017e-01,3.44345093e-01,  3.07701111e-01,2.96295166e-01,2.76039124e-01,2.01324463e-01,  1.04028702e-01,4.08506393e-02,2.04644203e-02,1.75395012e-02,  2.05421448e-02,2.95233727e-02,2.51965523e-02,2.77185440e-02,  3.74441147e-02,3.75909805e-02,4.22697067e-02,4.72822189e-02,  6.11124039e-02,7.82585144e-02,9.49249268e-02,1.11831665e-01,  1.12731934e-01,1.12821579e-01,1.25411987e-01,1.42257690e-01,  1.63211823e-01,1.97288513e-01,2.02766418e-01,1.79107666e-01,  1.67030334e-01,1.80358887e-01,1.95030212e-01,2.12574005e-01,  2.38929749e-01,2.60231018e-01,2.65838623e-01,2.70668030e-01,  2.81440735e-01,2.84271240e-01,2.85514832e-01,2.87689209e-01,  2.88505554e-01,2.87590027e-01,2.87017822e-01,2.87635803e-01,  2.85873413e-01,2.82661438e-01,2.79365540e-01,2.71217346e-01,  2.55104065e-01,2.46757507e-01,2.53067017e-01,2.59475708e-01,  2.60810852e-01,2.58255005e-01,2.47619629e-01,2.42980957e-01,  2.46162415e-01,2.53440857e-01,2.57514954e-01,2.60177612e-01,  2.60627747e-01,2.59757996e-01,2.58499146e-01,2.57232666e-01,  2.56469727e-01,2.54684448e-01,2.50457764e-01,2.45403290e-01,  2.41962433e-01,2.37789154e-01,2.33909607e-01,2.29095459e-01,  2.26013184e-01,2.21168518e-01,2.18219757e-01],
        [4.00413513e-01,4.19219971e-01,4.40162659e-01,5.17745972e-01,  5.52871704e-01,5.63110352e-01,5.66848755e-01,5.64315796e-01,  6.16348267e-01,7.07656860e-01,9.42169189e-01,1.06533813e+00,  1.12213135e+00,1.17175293e+00,1.18191528e+00,1.15155029e+00,  1.13882446e+00,1.23571777e+00,1.34616089e+00,1.45254517e+00,  1.53045654e+00,1.56863403e+00,1.59661865e+00,1.59826660e+00,  1.60769653e+00,1.63903809e+00,1.65243530e+00,1.59533691e+00,  1.60995483e+00,1.63931274e+00,1.61614990e+00,1.62322998e+00,  1.61993408e+00,1.55999756e+00,1.56408691e+00,1.59786987e+00,  1.62106323e+00,1.62646484e+00,1.61297607e+00,1.61700439e+00,  1.61547852e+00,1.59866333e+00,1.57232666e+00,1.55749512e+00,  1.54660034e+00,1.54418945e+00,1.55776978e+00,1.55657959e+00,  1.53088379e+00,1.53329468e+00,1.52261353e+00,1.51669312e+00,  1.50396729e+00,1.49282837e+00,1.49343872e+00,1.46829224e+00,  1.45465088e+00,1.47711182e+00,1.48675537e+00,1.47998047e+00,  1.46463013e+00,1.42938232e+00,1.46121216e+00,1.48837280e+00,  1.48184204e+00,1.47341919e+00,1.44747925e+00,1.33053589e+00,  1.29016113e+00,1.36401367e+00,1.37915039e+00,1.38201904e+00,  1.37011719e+00,1.31118774e+00,1.25671387e+00,1.25231934e+00,  1.27154541e+00,1.29406738e+00,1.28866577e+00,1.28952026e+00,  1.27362061e+00,1.14743042e+00,7.84515381e-01,8.50936890e-01,  1.13153076e+00,1.19937134e+00,1.19427490e+00,1.19171143e+00,  1.17163086e+00,1.16326904e+00,1.16424561e+00,1.14239502e+00,  1.10916138e+00,1.04476929e+00,1.01226807e+00,1.02972412e+00,  1.03414917e+00,1.04214478e+00,1.05505371e+00,1.02661133e+00,  9.88555908e-01,9.82864380e-01,9.83108521e-01,9.67071533e-01,  9.76318359e-01,9.72824097e-01,9.74700928e-01,9.69711304e-01,  9.56848145e-01,9.09088135e-01,8.46450806e-01,8.46206665e-01,  8.39828491e-01,8.18115234e-01,8.32870483e-01,8.28186035e-01,  6.60354614e-01,5.26199341e-01,5.37719727e-01,5.47103882e-01,  5.59387207e-01,5.83404541e-01,6.22390747e-01,6.78039551e-01,  7.18963623e-01,7.28805542e-01,7.43865967e-01,7.62802124e-01,  7.69668579e-01,7.67456055e-01,7.56958008e-01,7.46200562e-01,  7.43759155e-01,7.40280151e-01,7.32803345e-01,7.24731445e-01,  7.15057373e-01,7.05215454e-01,6.97143555e-01,6.87988281e-01,  6.78176880e-01,6.65893555e-01,6.52420044e-01,6.40060425e-01,  6.33438110e-01,6.31103516e-01,6.23489380e-01,6.13479614e-01,  6.01379395e-01,5.84335327e-01,5.68069458e-01,5.47393799e-01,  4.93064880e-01,3.96492004e-01,3.15574646e-01,2.91908264e-01,  3.03138733e-01,3.27850342e-01,3.57002258e-01,3.45275879e-01,  3.58924866e-01,4.07104492e-01,4.54780579e-01,4.88243103e-01,  5.02670288e-01,5.00625610e-01,4.91844177e-01,4.88265991e-01,  4.95742798e-01,4.93843079e-01,4.80094910e-01,4.75624084e-01,  4.80453491e-01,4.86114502e-01,4.85260010e-01,4.83459473e-01,  4.83963013e-01,4.81887817e-01,4.75585938e-01,4.68513489e-01,  4.58946228e-01,4.40193176e-01,4.05723572e-01,3.70620728e-01,  3.76815796e-01,4.08882141e-01,4.20097351e-01,4.29870605e-01,  4.34753418e-01,4.29939270e-01,4.18525696e-01,4.03930664e-01,  3.86390686e-01,3.70765686e-01,3.60839844e-01,3.42582703e-01,  3.05473328e-01,2.94181824e-01,2.73765564e-01,1.98657990e-01,  1.01612091e-01,3.92913818e-02,1.95274353e-02,1.67684555e-02,  1.95369720e-02,2.81014442e-02,2.40163803e-02,2.64320374e-02,  3.57036591e-02,3.59144211e-02,4.05120850e-02,4.53834534e-02,  5.89981079e-02,7.60231018e-02,9.25254822e-02,1.09325409e-01,  1.10254288e-01,1.10248566e-01,1.22732162e-01,1.39659882e-01,  1.60739899e-01,1.94820404e-01,2.00542450e-01,1.76811218e-01,  1.64550781e-01,1.77970886e-01,1.92623138e-01,2.10464478e-01,  2.37289429e-01,2.58789062e-01,2.64770508e-01,2.69699097e-01,  2.80609131e-01,2.83393860e-01,2.84515381e-01,2.86903381e-01,  2.87765503e-01,2.87048340e-01,2.86636353e-01,2.87239075e-01,  2.85522461e-01,2.82257080e-01,2.78884888e-01,2.70370483e-01,  2.53974915e-01,2.45773315e-01,2.52304077e-01,2.59071350e-01,  2.60482788e-01,2.57736206e-01,2.46738434e-01,2.41958618e-01,  2.45094299e-01,2.52532959e-01,2.56706238e-01,2.59742737e-01,  2.60192871e-01,2.59101868e-01,2.57644653e-01,2.56057739e-01,  2.54737854e-01,2.51998901e-01,2.48111725e-01,2.42988586e-01,  2.39189148e-01,2.35595703e-01,2.31182098e-01,2.26207733e-01,  2.21561432e-01,2.16514587e-01,2.13554382e-01],
        [3.95309448e-01,4.05715942e-01,4.31320190e-01,5.05615234e-01,  5.38269043e-01,5.54458618e-01,5.52429199e-01,5.54244995e-01,  6.03881836e-01,6.91986084e-01,9.25949097e-01,1.04550171e+00,  1.10461426e+00,1.15429688e+00,1.16357422e+00,1.13632202e+00,  1.12283325e+00,1.22021484e+00,1.32965088e+00,1.43359375e+00,  1.51419067e+00,1.55081177e+00,1.57888794e+00,1.58123779e+00,  1.59002686e+00,1.62368774e+00,1.63598633e+00,1.58032227e+00,  1.59634399e+00,1.62374878e+00,1.60235596e+00,1.60968018e+00,  1.60601807e+00,1.54821777e+00,1.55133057e+00,1.58544922e+00,  1.60830688e+00,1.61340332e+00,1.60137939e+00,1.60357666e+00,  1.60369873e+00,1.58776855e+00,1.55950928e+00,1.54678345e+00,  1.53384399e+00,1.53079224e+00,1.54580688e+00,1.54354858e+00,  1.52175903e+00,1.52307129e+00,1.51046753e+00,1.50854492e+00,  1.49353027e+00,1.48287964e+00,1.48678589e+00,1.45834351e+00,  1.44781494e+00,1.47021484e+00,1.47894287e+00,1.47543335e+00,  1.45864868e+00,1.42352295e+00,1.45724487e+00,1.48171997e+00,  1.47836304e+00,1.46862793e+00,1.44186401e+00,1.32699585e+00,  1.28390503e+00,1.35928345e+00,1.37683105e+00,1.37756348e+00,  1.36672974e+00,1.30648804e+00,1.25122070e+00,1.25283813e+00,  1.26788330e+00,1.29110718e+00,1.28970337e+00,1.28573608e+00,  1.27218628e+00,1.14630127e+00,7.78335571e-01,8.48937988e-01,  1.12612915e+00,1.19479370e+00,1.19631958e+00,1.18356323e+00,  1.17184448e+00,1.16601562e+00,1.15518188e+00,1.14694214e+00,  1.10928345e+00,1.03906250e+00,1.01882935e+00,1.02877808e+00,  1.03421021e+00,1.04727173e+00,1.04083252e+00,1.02996826e+00,  9.88693237e-01,9.66598511e-01,9.91485596e-01,9.66445923e-01,  9.67681885e-01,9.83322144e-01,9.71466064e-01,9.64202881e-01,  9.63195801e-01,8.92196655e-01,8.56475830e-01,8.52859497e-01,  8.27743530e-01,8.41583252e-01,8.35922241e-01,8.01040649e-01,  6.53366089e-01,5.24658203e-01,5.36544800e-01,5.46005249e-01,  5.58303833e-01,5.82321167e-01,6.21261597e-01,6.77078247e-01,  7.18048096e-01,7.28134155e-01,7.43408203e-01,7.62557983e-01,  7.69638062e-01,7.67608643e-01,7.57034302e-01,7.46154785e-01,  7.43865967e-01,7.40234375e-01,7.32696533e-01,7.24563599e-01,  7.14996338e-01,7.05154419e-01,6.97067261e-01,6.88064575e-01,  6.78192139e-01,6.65832520e-01,6.52160645e-01,6.39678955e-01,  6.33041382e-01,6.30737305e-01,6.23184204e-01,6.13281250e-01,  6.01348877e-01,5.84381104e-01,5.67962646e-01,5.47088623e-01,  4.92630005e-01,3.95698547e-01,3.14498901e-01,2.90779114e-01,  3.01986694e-01,3.26820374e-01,3.56147766e-01,3.44345093e-01,  3.58078003e-01,4.06394958e-01,4.54162598e-01,4.87823486e-01,  5.02243042e-01,5.00106812e-01,4.91416931e-01,4.87953186e-01,  4.95445251e-01,4.93637085e-01,4.79888916e-01,4.75395203e-01,  4.80247498e-01,4.86045837e-01,4.85084534e-01,4.83283997e-01,  4.83787537e-01,4.81704712e-01,4.75410461e-01,4.68399048e-01,  4.58732605e-01,4.39826965e-01,4.04968262e-01,3.69499207e-01,  3.75686646e-01,4.08142090e-01,4.19822693e-01,4.29847717e-01,  4.34806824e-01,4.29969788e-01,4.18556213e-01,4.03900146e-01,  3.86299133e-01,3.70689392e-01,3.60649109e-01,3.42338562e-01,  3.05145264e-01,2.93708801e-01,2.73307800e-01,1.98226929e-01,  1.01282120e-01,3.93667221e-02,1.97701454e-02,1.70111656e-02,  1.98264122e-02,2.82273293e-02,2.43425369e-02,2.66213417e-02,  3.57618332e-02,3.60193253e-02,4.06293869e-02,4.54320908e-02,  5.88722229e-02,7.58609772e-02,9.23271179e-02,1.09085083e-01,  1.09987259e-01,1.09918594e-01,1.22266769e-01,1.39225006e-01,  1.60400391e-01,1.94488525e-01,2.00145721e-01,1.76467896e-01,  1.64333344e-01,1.77772522e-01,1.92611694e-01,2.10338593e-01,  2.37171173e-01,2.58796692e-01,2.64801025e-01,2.69828796e-01,  2.80914307e-01,2.83782959e-01,2.85026550e-01,2.87437439e-01,  2.88200378e-01,2.87322998e-01,2.86834717e-01,2.87658691e-01,  2.86033630e-01,2.82928467e-01,2.79502869e-01,2.71026611e-01,  2.54631042e-01,2.46143341e-01,2.52639771e-01,2.59460449e-01,  2.60986328e-01,2.58224487e-01,2.47303009e-01,2.42584229e-01,  2.45918274e-01,2.53295898e-01,2.57484436e-01,2.60360718e-01,  2.60772705e-01,2.59765625e-01,2.58552551e-01,2.57553101e-01,  2.56767273e-01,2.54798889e-01,2.51091003e-01,2.46032715e-01,  2.42450714e-01,2.38227844e-01,2.34367371e-01,2.30407715e-01,  2.27329254e-01,2.22091675e-01,2.19139099e-01],
        [3.82392883e-01,3.97163391e-01,4.19418335e-01,4.92172241e-01,  5.25344849e-01,5.40344238e-01,5.41366577e-01,5.41656494e-01,  5.92544556e-01,6.80908203e-01,9.10293579e-01,1.02832031e+00,  1.08572388e+00,1.13568115e+00,1.14581299e+00,1.11856079e+00,  1.10665894e+00,1.20391846e+00,1.31274414e+00,1.41668701e+00,  1.49487305e+00,1.53216553e+00,1.56176758e+00,1.56466675e+00,  1.57485962e+00,1.60763550e+00,1.62106323e+00,1.56643677e+00,  1.58230591e+00,1.61047363e+00,1.58889771e+00,1.59667969e+00,  1.59378052e+00,1.53582764e+00,1.54034424e+00,1.57476807e+00,  1.59677124e+00,1.60186768e+00,1.58941650e+00,1.59301758e+00,  1.59310913e+00,1.57635498e+00,1.54937744e+00,1.53628540e+00,  1.52386475e+00,1.52026367e+00,1.53506470e+00,1.53375244e+00,  1.51217651e+00,1.51437378e+00,1.50219727e+00,1.49884033e+00,  1.48477173e+00,1.47500610e+00,1.47857666e+00,1.45153809e+00,  1.44036865e+00,1.46304321e+00,1.47283936e+00,1.46884155e+00,  1.45303345e+00,1.41918945e+00,1.45211792e+00,1.47860718e+00,  1.47348022e+00,1.46368408e+00,1.43734741e+00,1.32144165e+00,  1.27987671e+00,1.35690308e+00,1.37298584e+00,1.37319946e+00,  1.36288452e+00,1.30377197e+00,1.24826050e+00,1.24789429e+00,  1.26431274e+00,1.28707886e+00,1.28509521e+00,1.28536987e+00,  1.27069092e+00,1.14239502e+00,7.74948120e-01,8.44863892e-01,  1.12759399e+00,1.19577026e+00,1.19369507e+00,1.18820190e+00,  1.17227173e+00,1.16348267e+00,1.15713501e+00,1.14337158e+00,  1.10501099e+00,1.03707886e+00,1.00976562e+00,1.02386475e+00,  1.03091431e+00,1.04232788e+00,1.04553223e+00,1.02850342e+00,  9.87899780e-01,9.72686768e-01,9.87701416e-01,9.66705322e-01,  9.74914551e-01,9.77172852e-01,9.75463867e-01,9.73297119e-01,  9.52774048e-01,8.92196655e-01,8.52706909e-01,8.49960327e-01,  8.28536987e-01,8.22921753e-01,8.21685791e-01,8.03680420e-01,  6.51290894e-01,5.18066406e-01,5.29983521e-01,5.39489746e-01,  5.51803589e-01,5.76065063e-01,6.15585327e-01,6.72317505e-01,  7.14538574e-01,7.24731445e-01,7.40447998e-01,7.60299683e-01,  7.67562866e-01,7.65563965e-01,7.55249023e-01,7.44384766e-01,  7.42050171e-01,7.38586426e-01,7.31048584e-01,7.23022461e-01,  7.13485718e-01,7.03567505e-01,6.95526123e-01,6.86355591e-01,  6.76452637e-01,6.64047241e-01,6.50375366e-01,6.37893677e-01,  6.31408691e-01,6.29272461e-01,6.21734619e-01,6.11816406e-01,  5.99624634e-01,5.82305908e-01,5.65490723e-01,5.44082642e-01,  4.89135742e-01,3.91410828e-01,3.09745789e-01,2.85964966e-01,  2.97271729e-01,3.22425842e-01,3.51829529e-01,3.39813232e-01,  3.53515625e-01,4.02282715e-01,4.50744629e-01,4.84931946e-01,  4.99855042e-01,4.97985840e-01,4.89288330e-01,4.85885620e-01,  4.93553162e-01,4.91706848e-01,4.77912903e-01,4.73396301e-01,  4.78378296e-01,4.84222412e-01,4.83497620e-01,4.82017517e-01,  4.82727051e-01,4.80628967e-01,4.74372864e-01,4.67193604e-01,  4.57344055e-01,4.38087463e-01,4.02770996e-01,3.66859436e-01,  3.73222351e-01,4.06372070e-01,4.18479919e-01,4.28726196e-01,  4.33631897e-01,4.28833008e-01,4.17167664e-01,4.02366638e-01,  3.84559631e-01,3.68537903e-01,3.58444214e-01,3.39920044e-01,  3.02314758e-01,2.90863037e-01,2.70408630e-01,1.95072174e-01,  9.84725952e-02,3.75394821e-02,1.86629295e-02,1.60398483e-02,  1.86028481e-02,2.64720917e-02,2.28190422e-02,2.50535011e-02,  3.38001251e-02,3.41548920e-02,3.86514664e-02,4.32949066e-02,  5.64603806e-02,7.30667114e-02,8.93096924e-02,1.05957031e-01,  1.07089996e-01,1.06927872e-01,1.19199753e-01,1.36157990e-01,  1.57444000e-01,1.91661835e-01,1.97284698e-01,1.73446655e-01,  1.61262512e-01,1.74808502e-01,1.89731598e-01,2.07885742e-01,  2.35115051e-01,2.57049561e-01,2.63298035e-01,2.68310547e-01,  2.79380798e-01,2.82333374e-01,2.83775330e-01,2.86369324e-01,  2.87216187e-01,2.86491394e-01,2.86094666e-01,2.86819458e-01,  2.85156250e-01,2.81883240e-01,2.78488159e-01,2.69927979e-01,  2.53341675e-01,2.44945526e-01,2.51632690e-01,2.58651733e-01,  2.60154724e-01,2.57041931e-01,2.45811462e-01,2.41100311e-01,  2.44422913e-01,2.52128601e-01,2.56484985e-01,2.59407043e-01,  2.59643555e-01,2.58407593e-01,2.56515503e-01,2.55043030e-01,  2.54028320e-01,2.51686096e-01,2.47718811e-01,2.42580414e-01,  2.39158630e-01,2.34531403e-01,2.29755402e-01,2.24430084e-01,  2.20848083e-01,2.16197968e-01,2.14538574e-01],
        [3.69819641e-01,3.82202148e-01,4.08035278e-01,4.80018616e-01,  5.12496948e-01,5.29800415e-01,5.30151367e-01,5.31509399e-01,  5.79299927e-01,6.64794922e-01,8.91174316e-01,1.00833130e+00,  1.06610107e+00,1.11459351e+00,1.12524414e+00,1.10046387e+00,  1.08853149e+00,1.18511963e+00,1.29238892e+00,1.39532471e+00,  1.47442627e+00,1.51132202e+00,1.54055786e+00,1.54421997e+00,  1.55383301e+00,1.58761597e+00,1.60052490e+00,1.54788208e+00,  1.56488037e+00,1.59283447e+00,1.57202148e+00,1.57922363e+00,  1.57696533e+00,1.52056885e+00,1.52447510e+00,1.55844116e+00,  1.58081055e+00,1.58590698e+00,1.57415771e+00,1.57702637e+00,  1.57730103e+00,1.56170654e+00,1.53479004e+00,1.52102661e+00,  1.50808716e+00,1.50421143e+00,1.52038574e+00,1.51925659e+00,  1.49707031e+00,1.49981689e+00,1.48724365e+00,1.48544312e+00,  1.47271729e+00,1.46313477e+00,1.46661377e+00,1.43911743e+00,  1.42800903e+00,1.45065308e+00,1.46157837e+00,1.45892334e+00,  1.44271851e+00,1.40982056e+00,1.44409180e+00,1.46893311e+00,  1.46484375e+00,1.45675659e+00,1.42984009e+00,1.31338501e+00,  1.27038574e+00,1.34722900e+00,1.36471558e+00,1.36471558e+00,  1.35668945e+00,1.29577637e+00,1.23818970e+00,1.23947144e+00,  1.25634766e+00,1.27957153e+00,1.28082275e+00,1.27801514e+00,  1.26324463e+00,1.13595581e+00,7.65106201e-01,8.39263916e-01,  1.12121582e+00,1.19079590e+00,1.19161987e+00,1.18228149e+00,  1.16726685e+00,1.15826416e+00,1.15036011e+00,1.13964844e+00,  1.10192871e+00,1.02896118e+00,1.00720215e+00,1.01541138e+00,  1.02325439e+00,1.03890991e+00,1.03921509e+00,1.02496338e+00,  9.84817505e-01,9.64477539e-01,9.90737915e-01,9.67453003e-01,  9.64813232e-01,9.79354858e-01,9.73281860e-01,9.67010498e-01,  9.56176758e-01,8.89434814e-01,8.40881348e-01,8.39004517e-01,  8.11813354e-01,8.32656860e-01,8.36059570e-01,7.89291382e-01,  6.42730713e-01,5.13488770e-01,5.25711060e-01,5.35095215e-01,  5.47317505e-01,5.72143555e-01,6.12228394e-01,6.69708252e-01,  7.12890625e-01,7.23602295e-01,7.39562988e-01,7.59704590e-01,  7.67471313e-01,7.65441895e-01,7.54989624e-01,7.44216919e-01,  7.42034912e-01,7.38632202e-01,7.31338501e-01,7.23358154e-01,  7.13592529e-01,7.03781128e-01,6.95556641e-01,6.86279297e-01,  6.76437378e-01,6.63894653e-01,6.50131226e-01,6.37710571e-01,  6.31393433e-01,6.29135132e-01,6.21643066e-01,6.11526489e-01,  5.99243164e-01,5.81802368e-01,5.64788818e-01,5.42984009e-01,  4.87213135e-01,3.88648987e-01,3.06510925e-01,2.82524109e-01,  2.94075012e-01,3.19473267e-01,3.48922729e-01,3.36952209e-01,  3.50769043e-01,4.00039673e-01,4.49211121e-01,4.83665466e-01,  4.98680115e-01,4.96955872e-01,4.88388062e-01,4.84954834e-01,  4.92897034e-01,4.91218567e-01,4.77188110e-01,4.72793579e-01,  4.77867126e-01,4.83779907e-01,4.83261108e-01,4.81712341e-01,  4.82498169e-01,4.80651855e-01,4.74205017e-01,4.67086792e-01,  4.57252502e-01,4.37644958e-01,4.01908875e-01,3.65653992e-01,  3.72131348e-01,4.05708313e-01,4.18167114e-01,4.28527832e-01,  4.33464050e-01,4.28581238e-01,4.16885376e-01,4.02046204e-01,  3.84124756e-01,3.67958069e-01,3.57681274e-01,3.38775635e-01,  3.00628662e-01,2.89161682e-01,2.68753052e-01,1.93164825e-01,  9.69028473e-02,3.69682312e-02,1.86724663e-02,1.61819458e-02,  1.84898376e-02,2.59628296e-02,2.25543976e-02,2.47292519e-02,  3.30533981e-02,3.35197449e-02,3.79295349e-02,4.24432755e-02,  5.54094315e-02,7.18784332e-02,8.80088806e-02,1.04444504e-01,  1.05497360e-01,1.05205536e-01,1.17275238e-01,1.34265900e-01,  1.55761719e-01,1.90093994e-01,1.95915222e-01,1.72031403e-01,  1.59774780e-01,1.73366547e-01,1.88297272e-01,2.06562042e-01,  2.34230042e-01,2.56446838e-01,2.62969971e-01,2.68249512e-01,  2.79487610e-01,2.82501221e-01,2.84004211e-01,2.86720276e-01,  2.87666321e-01,2.86865234e-01,2.86590576e-01,2.87399292e-01,  2.85697937e-01,2.82546997e-01,2.79205322e-01,2.70729065e-01,  2.53814697e-01,2.45292664e-01,2.51922607e-01,2.59033203e-01,  2.60833740e-01,2.57934570e-01,2.46696472e-01,2.41847992e-01,  2.45101929e-01,2.52799988e-01,2.57232666e-01,2.60345459e-01,  2.60658264e-01,2.59681702e-01,2.58201599e-01,2.56744385e-01,  2.56019592e-01,2.54127502e-01,2.50740051e-01,2.46261597e-01,  2.42721558e-01,2.39200592e-01,2.33818054e-01,2.29972839e-01,  2.26284027e-01,2.21981049e-01,2.20863342e-01],
        [3.54850769e-01,3.73283386e-01,3.92211914e-01,4.63134766e-01,  4.95895386e-01,5.07934570e-01,5.14205933e-01,5.14328003e-01,  5.64666748e-01,6.50573730e-01,8.67996216e-01,9.85580444e-01,  1.04306030e+00,1.09268188e+00,1.10498047e+00,1.07916260e+00,  1.06964111e+00,1.16366577e+00,1.26965332e+00,1.37356567e+00,  1.45095825e+00,1.48959351e+00,1.51849365e+00,1.52221680e+00,  1.53399658e+00,1.56628418e+00,1.58078003e+00,1.52951050e+00,  1.54531860e+00,1.57479858e+00,1.55383301e+00,1.56143188e+00,  1.56045532e+00,1.50439453e+00,1.50964355e+00,1.54272461e+00,  1.56457520e+00,1.57092285e+00,1.55877686e+00,1.56280518e+00,  1.56246948e+00,1.54693604e+00,1.52096558e+00,1.50622559e+00,  1.49389648e+00,1.49154663e+00,1.50515747e+00,1.50677490e+00,  1.48538208e+00,1.48648071e+00,1.47601318e+00,1.47225952e+00,  1.46166992e+00,1.45230103e+00,1.45349121e+00,1.42913818e+00,  1.41744995e+00,1.44110107e+00,1.45352173e+00,1.44793701e+00,  1.43359375e+00,1.40246582e+00,1.43402100e+00,1.46166992e+00,  1.45724487e+00,1.44830322e+00,1.42437744e+00,1.30468750e+00,  1.26419067e+00,1.34094238e+00,1.35714722e+00,1.36340332e+00,  1.35140991e+00,1.28924561e+00,1.23480225e+00,1.23193359e+00,  1.25222778e+00,1.27807617e+00,1.27481079e+00,1.27639771e+00,  1.26071167e+00,1.13220215e+00,7.64862061e-01,8.30581665e-01,  1.11706543e+00,1.18988037e+00,1.18457031e+00,1.18118286e+00,  1.16690063e+00,1.15441895e+00,1.15219116e+00,1.13500977e+00,  1.09661865e+00,1.03140259e+00,9.98855591e-01,1.01504517e+00,  1.02734375e+00,1.03143311e+00,1.04196167e+00,1.02493286e+00,  9.81719971e-01,9.74929810e-01,9.79492188e-01,9.58496094e-01,  9.72824097e-01,9.63226318e-01,9.70001221e-01,9.75631714e-01,  9.43954468e-01,8.88488770e-01,8.42910767e-01,8.30612183e-01,  8.24066162e-01,8.02291870e-01,8.13827515e-01,8.12072754e-01,  6.46896362e-01,5.11138916e-01,5.23361206e-01,5.33004761e-01,  5.45104980e-01,5.69885254e-01,6.10214233e-01,6.68029785e-01,  7.11181641e-01,7.21817017e-01,7.37915039e-01,7.58209229e-01,  7.66220093e-01,7.64404297e-01,7.53753662e-01,7.43011475e-01,  7.41012573e-01,7.37503052e-01,7.30178833e-01,7.22167969e-01,  7.12432861e-01,7.02560425e-01,6.94366455e-01,6.85104370e-01,  6.75186157e-01,6.62536621e-01,6.48757935e-01,6.36276245e-01,  6.29928589e-01,6.27975464e-01,6.20468140e-01,6.10580444e-01,  5.98205566e-01,5.80551147e-01,5.63446045e-01,5.41625977e-01,  4.85931396e-01,3.87313843e-01,3.05061340e-01,2.81082153e-01,  2.92579651e-01,3.18069458e-01,3.47343445e-01,3.35227966e-01,  3.49143982e-01,3.98368835e-01,4.47761536e-01,4.82437134e-01,  4.97581482e-01,4.95964050e-01,4.87373352e-01,4.83901978e-01,  4.91828918e-01,4.90180969e-01,4.76150513e-01,4.71618652e-01,  4.76890564e-01,4.82887268e-01,4.82330322e-01,4.80918884e-01,  4.81719971e-01,4.79774475e-01,4.73480225e-01,4.66079712e-01,  4.56008911e-01,4.36279297e-01,4.00047302e-01,3.63403320e-01,  3.70079041e-01,4.04190063e-01,4.17060852e-01,4.27703857e-01,  4.32830811e-01,4.28031921e-01,4.16275024e-01,4.01268005e-01,  3.83354187e-01,3.67187500e-01,3.56742859e-01,3.37799072e-01,  2.99621582e-01,2.88139343e-01,2.67768860e-01,1.92279816e-01,  9.62696075e-02,3.67136002e-02,1.86452866e-02,1.61552429e-02,  1.84402466e-02,2.58111954e-02,2.24957466e-02,2.46295929e-02,  3.28083038e-02,3.32880020e-02,3.76691818e-02,4.20379639e-02,  5.49545288e-02,7.11421967e-02,8.73279572e-02,1.03858948e-01,  1.04763031e-01,1.04415894e-01,1.16497040e-01,1.33506775e-01,  1.55025482e-01,1.89273834e-01,1.95182800e-01,1.71390533e-01,  1.59103394e-01,1.72683716e-01,1.87641144e-01,2.05875397e-01,  2.33512878e-01,2.55989075e-01,2.62580872e-01,2.67944336e-01,  2.79144287e-01,2.82218933e-01,2.83683777e-01,2.86399841e-01,  2.87406921e-01,2.86590576e-01,2.86437988e-01,2.87300110e-01,  2.85675049e-01,2.82554626e-01,2.79083252e-01,2.70431519e-01,  2.53349304e-01,2.44831085e-01,2.51701355e-01,2.58888245e-01,  2.60734558e-01,2.57720947e-01,2.46154785e-01,2.41428375e-01,  2.44899750e-01,2.52746582e-01,2.57118225e-01,2.60108948e-01,  2.60513306e-01,2.59468079e-01,2.58247375e-01,2.56996155e-01,  2.56248474e-01,2.54318237e-01,2.51045227e-01,2.47123718e-01,  2.43843079e-01,2.40093231e-01,2.35469818e-01,2.30506897e-01,  2.27306366e-01,2.23014832e-01,2.21687317e-01],
        [3.45146179e-01,3.58062744e-01,3.81401062e-01,4.50614929e-01,  4.83108521e-01,4.96009827e-01,5.00259399e-01,5.02792358e-01,  5.49819946e-01,6.33148193e-01,8.47274780e-01,9.61990356e-01,  1.01950073e+00,1.06872559e+00,1.08142090e+00,1.05789185e+00,  1.04827881e+00,1.14218140e+00,1.24786377e+00,1.34918213e+00,  1.42785645e+00,1.46640015e+00,1.49594116e+00,1.50131226e+00,  1.51278687e+00,1.54443359e+00,1.56005859e+00,1.51031494e+00,  1.52682495e+00,1.55587769e+00,1.53619385e+00,1.54370117e+00,  1.54281616e+00,1.48840332e+00,1.49264526e+00,1.52566528e+00,  1.54806519e+00,1.55459595e+00,1.54330444e+00,1.54721069e+00,  1.54769897e+00,1.53173828e+00,1.50558472e+00,1.49176025e+00,  1.47912598e+00,1.47598267e+00,1.49142456e+00,1.49185181e+00,  1.47048950e+00,1.47299194e+00,1.46261597e+00,1.46057129e+00,  1.44992065e+00,1.44021606e+00,1.44281006e+00,1.41778564e+00,  1.40618896e+00,1.43069458e+00,1.44342041e+00,1.44024658e+00,  1.42550659e+00,1.39324951e+00,1.42654419e+00,1.45404053e+00,  1.45013428e+00,1.44238281e+00,1.41809082e+00,1.29891968e+00,  1.25640869e+00,1.33486938e+00,1.35183716e+00,1.35501099e+00,  1.34622192e+00,1.28433228e+00,1.22811890e+00,1.22747803e+00,  1.24609375e+00,1.27258301e+00,1.27178955e+00,1.27218628e+00,  1.25799561e+00,1.12805176e+00,7.58239746e-01,8.24447632e-01,  1.11071777e+00,1.18478394e+00,1.18276978e+00,1.17657471e+00,  1.16046143e+00,1.15231323e+00,1.14636230e+00,1.13491821e+00,  1.09692383e+00,1.02627563e+00,9.97573853e-01,1.00955200e+00,  1.01696777e+00,1.02838135e+00,1.03543091e+00,1.02105713e+00,  9.78424072e-01,9.68414307e-01,9.84085083e-01,9.64843750e-01,  9.71893311e-01,9.69345093e-01,9.68826294e-01,9.68200684e-01,  9.47509766e-01,8.87741089e-01,8.43933105e-01,8.40728760e-01,  8.20907593e-01,8.06564331e-01,8.20098877e-01,7.93991089e-01,  6.37222290e-01,5.04501343e-01,5.16693115e-01,5.26260376e-01,  5.38818359e-01,5.63781738e-01,6.04217529e-01,6.63162231e-01,  7.07153320e-01,7.18338013e-01,7.35076904e-01,7.56134033e-01,  7.64266968e-01,7.62786865e-01,7.52380371e-01,7.41622925e-01,  7.39395142e-01,7.36007690e-01,7.28668213e-01,7.20748901e-01,  7.11151123e-01,7.01187134e-01,6.93023682e-01,6.83761597e-01,  6.73736572e-01,6.60781860e-01,6.46957397e-01,6.34353638e-01,  6.28021240e-01,6.26342773e-01,6.19049072e-01,6.08947754e-01,  5.96603394e-01,5.78735352e-01,5.61019897e-01,5.38696289e-01,  4.82460022e-01,3.82957458e-01,3.00407410e-01,2.76222229e-01,  2.87803650e-01,3.13697815e-01,3.43139648e-01,3.30696106e-01,  3.44680786e-01,3.94546509e-01,4.44374084e-01,4.79667664e-01,  4.95254517e-01,4.93789673e-01,4.85237122e-01,4.81781006e-01,  4.89761353e-01,4.88273621e-01,4.74273682e-01,4.69802856e-01,  4.75280762e-01,4.81506348e-01,4.80903625e-01,4.79522705e-01,  4.80506897e-01,4.78485107e-01,4.72122192e-01,4.64958191e-01,  4.54795837e-01,4.34677124e-01,3.97903442e-01,3.60733032e-01,  3.67492676e-01,4.02229309e-01,4.15565491e-01,4.26567078e-01,  4.31770325e-01,4.26795959e-01,4.14901733e-01,3.99765015e-01,  3.81454468e-01,3.65226746e-01,3.54614258e-01,3.35517883e-01,  2.96798706e-01,2.85362244e-01,2.64862061e-01,1.89022064e-01,  9.33914185e-02,3.48291397e-02,1.76033974e-02,1.52506828e-02,  1.73683167e-02,2.42929459e-02,2.10962296e-02,2.30789185e-02,  3.08060646e-02,3.12776566e-02,3.57723236e-02,4.00066376e-02,  5.27811050e-02,6.88209534e-02,8.45909119e-02,1.00788116e-01,  1.01850510e-01,1.01356506e-01,1.13405228e-01,1.30489349e-01,  1.52046204e-01,1.86538696e-01,1.92386627e-01,1.68525696e-01,  1.56192780e-01,1.69757843e-01,1.85009003e-01,2.03510284e-01,  2.31410980e-01,2.54158020e-01,2.61001587e-01,2.66487122e-01,  2.77923584e-01,2.81066895e-01,2.82821655e-01,2.85507202e-01,  2.86430359e-01,2.85697937e-01,2.85507202e-01,2.86445618e-01,  2.84896851e-01,2.81784058e-01,2.78411865e-01,2.69462585e-01,  2.52120972e-01,2.43556976e-01,2.50534058e-01,2.58140564e-01,  2.59864807e-01,2.56690979e-01,2.44850159e-01,2.39894867e-01,  2.43518829e-01,2.51533508e-01,2.56172180e-01,2.59056091e-01,  2.59170532e-01,2.57804871e-01,2.56103516e-01,2.54737854e-01,  2.53875732e-01,2.51823425e-01,2.47932434e-01,2.42660522e-01,  2.38906860e-01,2.34024048e-01,2.30033875e-01,2.26043701e-01,  2.22110748e-01,2.17506409e-01,2.13901520e-01],
        [3.36692810e-01,3.48785400e-01,3.67904663e-01,4.37156677e-01,  4.65286255e-01,4.79957581e-01,4.86846924e-01,4.85649109e-01,  5.35293579e-01,6.16958618e-01,8.26385498e-01,9.39437866e-01,  9.96093750e-01,1.04620361e+00,1.05868530e+00,1.03610229e+00,  1.02847290e+00,1.12091064e+00,1.22573853e+00,1.32629395e+00,  1.40393066e+00,1.44345093e+00,1.47241211e+00,1.47824097e+00,  1.49041748e+00,1.52239990e+00,1.53875732e+00,1.48999023e+00,  1.50723267e+00,1.53704834e+00,1.51727295e+00,1.52532959e+00,  1.52496338e+00,1.47238159e+00,1.47708130e+00,1.50921631e+00,  1.53195190e+00,1.53839111e+00,1.52761841e+00,1.53179932e+00,  1.53207397e+00,1.51828003e+00,1.49142456e+00,1.47619629e+00,  1.46383667e+00,1.45962524e+00,1.47631836e+00,1.47912598e+00,  1.45739746e+00,1.45974731e+00,1.44857788e+00,1.44665527e+00,  1.43753052e+00,1.42752075e+00,1.43103027e+00,1.40628052e+00,  1.39416504e+00,1.42056274e+00,1.43334961e+00,1.43084717e+00,  1.41744995e+00,1.38412476e+00,1.41842651e+00,1.44577026e+00,  1.44274902e+00,1.43688965e+00,1.40951538e+00,1.29000854e+00,  1.24853516e+00,1.32662964e+00,1.34658813e+00,1.35137939e+00,  1.33996582e+00,1.27993774e+00,1.22360229e+00,1.22225952e+00,  1.24157715e+00,1.26550293e+00,1.26586914e+00,1.26733398e+00,  1.25131226e+00,1.12664795e+00,7.51495361e-01,8.15719604e-01,  1.10870361e+00,1.18066406e+00,1.17913818e+00,1.17340088e+00,  1.15420532e+00,1.14990234e+00,1.14498901e+00,1.12936401e+00,  1.09820557e+00,1.02441406e+00,9.93682861e-01,1.01296997e+00,  1.01498413e+00,1.02713013e+00,1.03915405e+00,1.02120972e+00,  9.79995728e-01,9.60998535e-01,9.73297119e-01,9.61868286e-01,  9.58892822e-01,9.64340210e-01,9.71176147e-01,9.60693359e-01,  9.48577881e-01,8.92852783e-01,8.36395264e-01,8.39279175e-01,  8.09036255e-01,7.94158936e-01,8.24050903e-01,7.96020508e-01,  6.34155273e-01,5.01693726e-01,5.14221191e-01,5.23757935e-01,  5.36300659e-01,5.61111450e-01,6.01806641e-01,6.61087036e-01,  7.05627441e-01,7.16812134e-01,7.33779907e-01,7.55416870e-01,  7.63839722e-01,7.62161255e-01,7.51815796e-01,7.40966797e-01,  7.38677979e-01,7.35290527e-01,7.28012085e-01,7.20214844e-01,  7.10601807e-01,7.00973511e-01,6.92581177e-01,6.83334351e-01,  6.73141479e-01,6.60064697e-01,6.45980835e-01,6.33422852e-01,  6.27380371e-01,6.25778198e-01,6.18591309e-01,6.08627319e-01,  5.96160889e-01,5.78170776e-01,5.60226440e-01,5.37643433e-01,  4.81079102e-01,3.81248474e-01,2.98377991e-01,2.74238586e-01,  2.85659790e-01,3.11782837e-01,3.41217041e-01,3.28674316e-01,  3.42758179e-01,3.92822266e-01,4.43061829e-01,4.78546143e-01,  4.94308472e-01,4.92935181e-01,4.84413147e-01,4.80995178e-01,  4.89204407e-01,4.87670898e-01,4.73564148e-01,4.69070435e-01,  4.74533081e-01,4.80842590e-01,4.80278015e-01,4.79087830e-01,  4.80033875e-01,4.78225708e-01,4.71778870e-01,4.64439392e-01,  4.54010010e-01,4.33464050e-01,3.96179199e-01,3.58535767e-01,  3.65318298e-01,4.00863647e-01,4.14924622e-01,4.26116943e-01,  4.31419373e-01,4.26353455e-01,4.14459229e-01,3.99246216e-01,  3.80882263e-01,3.64479065e-01,3.53965759e-01,3.34671021e-01,  2.95730591e-01,2.84225464e-01,2.63641357e-01,1.87683105e-01,  9.22107697e-02,3.40709686e-02,1.70626640e-02,1.48825645e-02,  1.68843269e-02,2.35815048e-02,2.03800201e-02,2.23641396e-02,  2.99658775e-02,3.06229591e-02,3.48863602e-02,3.91960144e-02,  5.16939163e-02,6.76136017e-02,8.33454132e-02,9.95826721e-02,  1.00564957e-01,1.00013733e-01,1.11968994e-01,1.29211426e-01,  1.50901794e-01,1.85398102e-01,1.91291809e-01,1.67320251e-01,  1.55014038e-01,1.68674469e-01,1.83773041e-01,2.02388763e-01,  2.30545044e-01,2.53501892e-01,2.60429382e-01,2.66029358e-01,  2.77526855e-01,2.80723572e-01,2.82447815e-01,2.85247803e-01,  2.86155701e-01,2.85537720e-01,2.85263062e-01,2.86224365e-01,  2.84706116e-01,2.81654358e-01,2.78137207e-01,2.69126892e-01,  2.51495361e-01,2.43003845e-01,2.50015259e-01,2.57667542e-01,  2.59544373e-01,2.56095886e-01,2.43980408e-01,2.38941193e-01,  2.42729187e-01,2.51190186e-01,2.55859375e-01,2.58499146e-01,  2.58331299e-01,2.57118225e-01,2.55378723e-01,2.53921509e-01,  2.53028870e-01,2.50755310e-01,2.46494293e-01,2.40692139e-01,  2.36015320e-01,2.31395721e-01,2.27409363e-01,2.23491669e-01,  2.19650269e-01,2.14794159e-01,2.12020874e-01],
        [3.21548462e-01,3.34144592e-01,3.53851318e-01,4.19807434e-01,  4.47448730e-01,4.64523315e-01,4.69047546e-01,4.71160889e-01,  5.19531250e-01,5.98526001e-01,8.04016113e-01,9.14672852e-01,  9.70565796e-01,1.02050781e+00,1.03308105e+00,1.01342773e+00,  1.00570679e+00,1.09741211e+00,1.20172119e+00,1.30032349e+00,  1.37887573e+00,1.41723633e+00,1.44659424e+00,1.45394897e+00,  1.46536255e+00,1.49813843e+00,1.51461792e+00,1.46795654e+00,  1.48638916e+00,1.51528931e+00,1.49624634e+00,1.50503540e+00,  1.50515747e+00,1.45401001e+00,1.45837402e+00,1.49050903e+00,  1.51266479e+00,1.51904297e+00,1.50958252e+00,1.51339722e+00,  1.51397705e+00,1.50033569e+00,1.47341919e+00,1.45822144e+00,  1.44537354e+00,1.44131470e+00,1.45962524e+00,1.46212769e+00,  1.44119263e+00,1.44287109e+00,1.43212891e+00,1.43164062e+00,  1.42230225e+00,1.41439819e+00,1.41857910e+00,1.39233398e+00,  1.38150024e+00,1.40856934e+00,1.42034912e+00,1.41864014e+00,  1.40573120e+00,1.37417603e+00,1.40939331e+00,1.43572998e+00,  1.43322754e+00,1.42773438e+00,1.40206909e+00,1.28366089e+00,  1.23931885e+00,1.31826782e+00,1.33938599e+00,1.34292603e+00,  1.33306885e+00,1.27102661e+00,1.21188354e+00,1.21255493e+00,  1.23397827e+00,1.25936890e+00,1.26306152e+00,1.26269531e+00,  1.24618530e+00,1.12188721e+00,7.43179321e-01,8.08410645e-01,  1.10205078e+00,1.17431641e+00,1.17459106e+00,1.16632080e+00,  1.15319824e+00,1.14685059e+00,1.13800049e+00,1.12356567e+00,  1.08584595e+00,1.01260376e+00,9.85504150e-01,9.99191284e-01,  1.00491333e+00,1.02639771e+00,1.03176880e+00,1.01593018e+00,  9.76501465e-01,9.57260132e-01,9.77310181e-01,9.61639404e-01,  9.58145142e-01,9.63729858e-01,9.62707520e-01,9.58053589e-01,  9.47647095e-01,8.79776001e-01,8.38027954e-01,8.45245361e-01,  7.97119141e-01,7.90222168e-01,8.19335938e-01,7.84057617e-01,  6.24755859e-01,4.92660522e-01,5.05371094e-01,5.15228271e-01,  5.28152466e-01,5.53680420e-01,5.95184326e-01,6.55609131e-01,  7.01370239e-01,7.12997437e-01,7.30545044e-01,7.52731323e-01,  7.61901855e-01,7.60848999e-01,7.50549316e-01,7.39852905e-01,  7.37792969e-01,7.34420776e-01,7.26989746e-01,7.19070435e-01,  7.09457397e-01,6.99676514e-01,6.91436768e-01,6.82083130e-01,  6.71859741e-01,6.58828735e-01,6.44836426e-01,6.32110596e-01,  6.25869751e-01,6.24404907e-01,6.17050171e-01,6.06979370e-01,  5.94543457e-01,5.76080322e-01,5.57617188e-01,5.34408569e-01,  4.77050781e-01,3.76037598e-01,2.92396545e-01,2.67906189e-01,  2.79655457e-01,3.06167603e-01,3.35632324e-01,3.22837830e-01,  3.37295532e-01,3.88061523e-01,4.39308167e-01,4.75448608e-01,  4.91569519e-01,4.90470886e-01,4.82017517e-01,4.78675842e-01,  4.87091064e-01,4.85832214e-01,4.71611023e-01,4.67102051e-01,  4.72923279e-01,4.79362488e-01,4.78843689e-01,4.77821350e-01,  4.79141235e-01,4.77378845e-01,4.71023560e-01,4.63485718e-01,  4.52873230e-01,4.31953430e-01,3.94042969e-01,3.55987549e-01,  3.63090515e-01,3.99269104e-01,4.13772583e-01,4.25270081e-01,  4.30480957e-01,4.25224304e-01,4.13215637e-01,3.97628784e-01,  3.79173279e-01,3.62457275e-01,3.51570129e-01,3.32000732e-01,  2.92358398e-01,2.80776978e-01,2.60200500e-01,1.83990479e-01,  8.90636444e-02,3.27148438e-02,1.66964531e-02,1.45883560e-02,  1.64213181e-02,2.24003792e-02,1.95989609e-02,2.13661194e-02,  2.82783508e-02,2.90684700e-02,3.33576202e-02,3.73401642e-02,  4.95824814e-02,6.50424957e-02,8.04309845e-02,9.64946747e-02,  9.74941254e-02,9.67407227e-02,1.08514786e-01,1.25755310e-01,  1.47583008e-01,1.82025909e-01,1.87992096e-01,1.64157867e-01,  1.51668549e-01,1.65470123e-01,1.80816650e-01,1.99699402e-01,  2.28252411e-01,2.51663208e-01,2.58995056e-01,2.64984131e-01,  2.76725769e-01,2.80052185e-01,2.81944275e-01,2.84774780e-01,  2.85896301e-01,2.85133362e-01,2.85026550e-01,2.86254883e-01,  2.84858704e-01,2.81646729e-01,2.78129578e-01,2.68859863e-01,  2.51029968e-01,2.42404938e-01,2.49828339e-01,2.57865906e-01,  2.59811401e-01,2.56309509e-01,2.43728638e-01,2.38662720e-01,  2.42523193e-01,2.51037598e-01,2.55889893e-01,2.58811951e-01,  2.58949280e-01,2.57377625e-01,2.55378723e-01,2.53860474e-01,  2.52731323e-01,2.51060486e-01,2.46902466e-01,2.42298126e-01,  2.38731384e-01,2.33951569e-01,2.28897095e-01,2.24170685e-01,  2.20413208e-01,2.15801239e-01,2.13226318e-01],
        [3.04504395e-01,3.16375732e-01,3.41232300e-01,4.01901245e-01,  4.32907104e-01,4.47761536e-01,4.51538086e-01,4.57115173e-01,  5.02075195e-01,5.80902100e-01,7.81112671e-01,8.88442993e-01,  9.45205688e-01,9.93133545e-01,1.00833130e+00,9.90737915e-01,  9.83352661e-01,1.07458496e+00,1.17544556e+00,1.27331543e+00,  1.35104370e+00,1.39019775e+00,1.42141724e+00,1.42868042e+00,  1.44091797e+00,1.47393799e+00,1.49008179e+00,1.44543457e+00,  1.46356201e+00,1.49307251e+00,1.47619629e+00,1.48373413e+00,  1.48483276e+00,1.43469238e+00,1.43905640e+00,1.47131348e+00,  1.49325562e+00,1.50021362e+00,1.48983765e+00,1.49307251e+00,  1.49526978e+00,1.48126221e+00,1.45556641e+00,1.44076538e+00,  1.42645264e+00,1.42333984e+00,1.44064331e+00,1.44369507e+00,  1.42446899e+00,1.42608643e+00,1.41613770e+00,1.41552734e+00,  1.40591431e+00,1.39913940e+00,1.40383911e+00,1.37783813e+00,  1.36813354e+00,1.39492798e+00,1.40921021e+00,1.40853882e+00,  1.39514160e+00,1.36526489e+00,1.39926147e+00,1.42678833e+00,  1.42474365e+00,1.41772461e+00,1.39300537e+00,1.27401733e+00,  1.22949219e+00,1.31100464e+00,1.33139038e+00,1.33566284e+00,  1.32666016e+00,1.26309204e+00,1.20547485e+00,1.20700073e+00,  1.22607422e+00,1.25515747e+00,1.25720215e+00,1.25689697e+00,  1.24398804e+00,1.11581421e+00,7.38220215e-01,8.01818848e-01,  1.09408569e+00,1.16793823e+00,1.16851807e+00,1.16140747e+00,  1.14889526e+00,1.13858032e+00,1.13015747e+00,1.12252808e+00,  1.08200073e+00,1.01196289e+00,9.85549927e-01,9.96078491e-01,  1.00955200e+00,1.02481079e+00,1.02600098e+00,1.01367188e+00,  9.72564697e-01,9.58755493e-01,9.80499268e-01,9.54696655e-01,  9.62310791e-01,9.68490601e-01,9.59060669e-01,9.64843750e-01,  9.47998047e-01,8.79150391e-01,8.32397461e-01,8.27560425e-01,  7.92053223e-01,7.93930054e-01,8.02352905e-01,7.83905029e-01,  6.25717163e-01,4.87785339e-01,5.00335693e-01,5.10299683e-01,  5.23178101e-01,5.48965454e-01,5.90621948e-01,6.52069092e-01,  6.98440552e-01,7.10433960e-01,7.28378296e-01,7.51190186e-01,  7.60589600e-01,7.59552002e-01,7.49450684e-01,7.38861084e-01,  7.36526489e-01,7.33398438e-01,7.25936890e-01,7.18048096e-01,  7.08480835e-01,6.98791504e-01,6.90612793e-01,6.81198120e-01,  6.70837402e-01,6.57501221e-01,6.43341064e-01,6.30706787e-01,  6.24572754e-01,6.23077393e-01,6.15921021e-01,6.05941772e-01,  5.93475342e-01,5.74920654e-01,5.55938721e-01,5.32379150e-01,  4.74426270e-01,3.72840881e-01,2.88879395e-01,2.64381409e-01,  2.76199341e-01,3.02993774e-01,3.32496643e-01,3.19480896e-01,  3.33877563e-01,3.84956360e-01,4.36660767e-01,4.73182678e-01,  4.89738464e-01,4.88754272e-01,4.80392456e-01,4.77226257e-01,  4.85748291e-01,4.84588623e-01,4.70207214e-01,4.65637207e-01,  4.71343994e-01,4.78050232e-01,4.77859497e-01,4.76974487e-01,  4.78454590e-01,4.76699829e-01,4.70230103e-01,4.62646484e-01,  4.51744080e-01,4.30404663e-01,3.91967773e-01,3.53302002e-01,  3.60649109e-01,3.97659302e-01,4.12765503e-01,4.24621582e-01,  4.29862976e-01,4.24568176e-01,4.12361145e-01,3.96713257e-01,  3.78067017e-01,3.61289978e-01,3.50395203e-01,3.30451965e-01,  2.90443420e-01,2.78793335e-01,2.58148193e-01,1.81865692e-01,  8.75949860e-02,3.22580338e-02,1.68042183e-02,1.48572922e-02,  1.65028572e-02,2.21171379e-02,1.93357468e-02,2.10957527e-02,  2.77595520e-02,2.86116600e-02,3.28111649e-02,3.66649628e-02,  4.85296249e-02,6.37264252e-02,7.88440704e-02,9.46903229e-02,  9.58633423e-02,9.51747894e-02,1.06767654e-01,1.23975754e-01,  1.45793915e-01,1.80233002e-01,1.86191559e-01,1.62288666e-01,  1.49990082e-01,1.63829803e-01,1.79302216e-01,1.98295593e-01,  2.27146149e-01,2.50869751e-01,2.58308411e-01,2.64404297e-01,  2.76275635e-01,2.79724121e-01,2.81852722e-01,2.84881592e-01,  2.85896301e-01,2.85179138e-01,2.85102844e-01,2.86407471e-01,  2.84965515e-01,2.81951904e-01,2.78640747e-01,2.69256592e-01,  2.51121521e-01,2.42408752e-01,2.49858856e-01,2.57987976e-01,  2.60078430e-01,2.56515503e-01,2.44018555e-01,2.38899231e-01,  2.42794037e-01,2.51403809e-01,2.56317139e-01,2.59254456e-01,  2.59407043e-01,2.57926941e-01,2.56210327e-01,2.54859924e-01,  2.54127502e-01,2.52891541e-01,2.49309540e-01,2.44186401e-01,  2.40787506e-01,2.36454010e-01,2.32261658e-01,2.27794647e-01,  2.24414825e-01,2.19768524e-01,2.16583252e-01],
        [2.93327332e-01,3.04550171e-01,3.23310852e-01,3.86703491e-01,  4.13101196e-01,4.28977966e-01,4.35729980e-01,4.38041687e-01,  4.83749390e-01,5.58395386e-01,7.51510620e-01,8.56887817e-01,  9.11453247e-01,9.61288452e-01,9.74700928e-01,9.58343506e-01,  9.53262329e-01,1.04159546e+00,1.14385986e+00,1.23928833e+00,  1.31503296e+00,1.35449219e+00,1.38491821e+00,1.39440918e+00,  1.40664673e+00,1.43884277e+00,1.45709229e+00,1.41348267e+00,  1.43322754e+00,1.46331787e+00,1.44537354e+00,1.45419312e+00,  1.45529175e+00,1.40701294e+00,1.41223145e+00,1.44326782e+00,  1.46554565e+00,1.47290039e+00,1.46334839e+00,1.46743774e+00,  1.46841431e+00,1.45608521e+00,1.42944336e+00,1.41424561e+00,  1.40139771e+00,1.39727783e+00,1.41677856e+00,1.42034912e+00,  1.39846802e+00,1.40173340e+00,1.39208984e+00,1.39236450e+00,  1.38531494e+00,1.37857056e+00,1.38372803e+00,1.35760498e+00,  1.34707642e+00,1.37673950e+00,1.39135742e+00,1.39096069e+00,  1.37814331e+00,1.34793091e+00,1.38439941e+00,1.41201782e+00,  1.40939331e+00,1.40457153e+00,1.37850952e+00,1.25903320e+00,  1.21527100e+00,1.29519653e+00,1.31832886e+00,1.32260132e+00,  1.31408691e+00,1.25070190e+00,1.18838501e+00,1.19100952e+00,  1.21273804e+00,1.24182129e+00,1.24740601e+00,1.24707031e+00,  1.23266602e+00,1.10842896e+00,7.25723267e-01,7.89260864e-01,  1.08801270e+00,1.16177368e+00,1.16223145e+00,1.15591431e+00,  1.13803101e+00,1.13165283e+00,1.12252808e+00,1.11154175e+00,  1.07781982e+00,1.00018311e+00,9.74304199e-01,9.87426758e-01,  9.92721558e-01,1.01644897e+00,1.02209473e+00,1.00744629e+00,  9.75402832e-01,9.48989868e-01,9.68750000e-01,9.55947876e-01,  9.50668335e-01,9.64355469e-01,9.55001831e-01,9.45541382e-01,  9.39422607e-01,8.59436035e-01,8.06427002e-01,8.27392578e-01,  7.81341553e-01,7.75375366e-01,8.07678223e-01,7.69287109e-01,  6.08840942e-01,4.73205566e-01,4.86000061e-01,4.95903015e-01,  5.08956909e-01,5.35263062e-01,5.78277588e-01,6.41769409e-01,  6.90338135e-01,7.03155518e-01,7.22091675e-01,7.46444702e-01,  7.56622314e-01,7.55859375e-01,7.45910645e-01,7.35382080e-01,  7.33123779e-01,7.30056763e-01,7.22885132e-01,7.14797974e-01,  7.05459595e-01,6.95907593e-01,6.87500000e-01,6.78100586e-01,  6.67648315e-01,6.54251099e-01,6.39877319e-01,6.27273560e-01,  6.21246338e-01,6.19979858e-01,6.12823486e-01,6.02569580e-01,  5.89630127e-01,5.70617676e-01,5.50659180e-01,5.26016235e-01,  4.66934204e-01,3.63685608e-01,2.78770447e-01,2.53829956e-01,  2.65907288e-01,2.93281555e-01,3.22700500e-01,3.09333801e-01,  3.24188232e-01,3.76434326e-01,4.29718018e-01,4.67475891e-01,  4.84519958e-01,4.83985901e-01,4.75601196e-01,4.72320557e-01,  4.81399536e-01,4.80476379e-01,4.66094971e-01,4.61776733e-01,  4.67872620e-01,4.74861145e-01,4.74891663e-01,4.74212646e-01,  4.76028442e-01,4.74555969e-01,4.68025208e-01,4.60372925e-01,  4.49386597e-01,4.27719116e-01,3.88633728e-01,3.49487305e-01,  3.56872559e-01,3.94622803e-01,4.10392761e-01,4.22538757e-01,  4.27711487e-01,4.22195435e-01,4.09530640e-01,3.93524170e-01,  3.74488831e-01,3.57223511e-01,3.45932007e-01,3.25477600e-01,  2.84355164e-01,2.72689819e-01,2.52037048e-01,1.75472260e-01,  8.24871063e-02,2.95753479e-02,1.55282021e-02,1.37698650e-02,  1.50420666e-02,1.98383331e-02,1.76200867e-02,1.91283226e-02,  2.49290466e-02,2.58388519e-02,2.96835899e-02,3.31487656e-02,  4.44269180e-02,5.90553284e-02,7.36331940e-02,8.92391205e-02,  9.07115936e-02,8.98571014e-02,1.00950241e-01,1.17948532e-01,  1.39774323e-01,1.74182892e-01,1.80385590e-01,1.56444550e-01,  1.44260406e-01,1.58302307e-01,1.73896790e-01,1.93214417e-01,  2.22675323e-01,2.47135162e-01,2.55401611e-01,2.61810303e-01,  2.74055481e-01,2.77809143e-01,2.80204773e-01,2.83340454e-01,  2.84393311e-01,2.83973694e-01,2.84072876e-01,2.85453796e-01,  2.84111023e-01,2.81059265e-01,2.77587891e-01,2.67982483e-01,  2.49511719e-01,2.40852356e-01,2.48558044e-01,2.56996155e-01,  2.59170532e-01,2.55401611e-01,2.42515564e-01,2.37277985e-01,  2.41298676e-01,2.50137329e-01,2.55088806e-01,2.58049011e-01,  2.57957458e-01,2.56416321e-01,2.54577637e-01,2.53097534e-01,  2.52197266e-01,2.50358582e-01,2.46631622e-01,2.41065979e-01,  2.37579346e-01,2.33688354e-01,2.29061127e-01,2.25086212e-01,  2.20706940e-01,2.15763092e-01,2.13222504e-01],
        [2.74719238e-01,2.88215637e-01,3.09608459e-01,3.65631104e-01,  3.96438599e-01,4.09904480e-01,4.15382385e-01,4.20814514e-01,  4.62234497e-01,5.36819458e-01,7.22732544e-01,8.24539185e-01,  8.79272461e-01,9.25979614e-01,9.43145752e-01,9.27642822e-01,  9.23583984e-01,1.01113892e+00,1.10855103e+00,1.20462036e+00,  1.27932739e+00,1.31799316e+00,1.35037231e+00,1.35900879e+00,  1.37365723e+00,1.40603638e+00,1.42300415e+00,1.38375854e+00,  1.40240479e+00,1.43255615e+00,1.41696167e+00,1.42471313e+00,  1.42764282e+00,1.38085938e+00,1.38650513e+00,1.41769409e+00,  1.43829346e+00,1.44668579e+00,1.43807983e+00,1.44192505e+00,  1.44442749e+00,1.43069458e+00,1.40518188e+00,1.38989258e+00,  1.37536621e+00,1.37283325e+00,1.39102173e+00,1.39572144e+00,  1.37731934e+00,1.37863159e+00,1.37088013e+00,1.37167358e+00,  1.36343384e+00,1.35900879e+00,1.36343384e+00,1.33895874e+00,  1.33053589e+00,1.35836792e+00,1.37561035e+00,1.37393188e+00,  1.36135864e+00,1.33489990e+00,1.36978149e+00,1.39880371e+00,  1.39749146e+00,1.39157104e+00,1.36853027e+00,1.24691772e+00,  1.20074463e+00,1.28472900e+00,1.30508423e+00,1.31204224e+00,  1.30499268e+00,1.23825073e+00,1.17825317e+00,1.17715454e+00,  1.19879150e+00,1.23348999e+00,1.23596191e+00,1.23974609e+00,  1.22781372e+00,1.09765625e+00,7.18353271e-01,7.81127930e-01,  1.07742310e+00,1.15536499e+00,1.15444946e+00,1.14883423e+00,  1.13690186e+00,1.12249756e+00,1.12207031e+00,1.10852051e+00,  1.06527710e+00,9.94262695e-01,9.64294434e-01,9.77890015e-01,  9.92660522e-01,1.00601196e+00,1.01818848e+00,1.00363159e+00,  9.59823608e-01,9.55154419e-01,9.72106934e-01,9.41909790e-01,  9.53979492e-01,9.51004028e-01,9.44091797e-01,9.56100464e-01,  9.26818848e-01,8.62884521e-01,8.14361572e-01,8.00445557e-01,  7.81494141e-01,7.80700684e-01,7.93930054e-01,7.79113770e-01,  6.06109619e-01,4.61997986e-01,4.74891663e-01,4.85198975e-01,  4.98497009e-01,5.25299072e-01,5.68710327e-01,6.33728027e-01,  6.83471680e-01,6.96716309e-01,7.16583252e-01,7.42080688e-01,  7.52990723e-01,7.52716064e-01,7.42874146e-01,7.32238770e-01,  7.30224609e-01,7.27127075e-01,7.19757080e-01,7.11959839e-01,  7.02636719e-01,6.92901611e-01,6.84539795e-01,6.75170898e-01,  6.64749146e-01,6.51168823e-01,6.36688232e-01,6.24114990e-01,  6.18209839e-01,6.17065430e-01,6.10031128e-01,5.99639893e-01,  5.86456299e-01,5.66818237e-01,5.46005249e-01,5.20690918e-01,  4.60853577e-01,3.56582642e-01,2.71034241e-01,2.45944977e-01,  2.58064270e-01,2.85835266e-01,3.15116882e-01,3.01597595e-01,  3.16627502e-01,3.69628906e-01,4.23973083e-01,4.62379456e-01,  4.80117798e-01,4.79896545e-01,4.71534729e-01,4.68437195e-01,  4.77699280e-01,4.76951599e-01,4.62570190e-01,4.58190918e-01,  4.64492798e-01,4.71778870e-01,4.71931458e-01,4.71527100e-01,  4.73632812e-01,4.72221375e-01,4.65919495e-01,4.58229065e-01,  4.46899414e-01,4.24903870e-01,3.85116577e-01,3.45359802e-01,  3.53050232e-01,3.91418457e-01,4.07821655e-01,4.20158386e-01,  4.25407410e-01,4.19914246e-01,4.07058716e-01,3.90716553e-01,  3.71246338e-01,3.53652954e-01,3.41972351e-01,3.21044922e-01,  2.79373169e-01,2.67730713e-01,2.47116089e-01,1.70703888e-01,  7.88192749e-02,2.78563499e-02,1.47228241e-02,1.30581856e-02,  1.40790939e-02,1.81913376e-02,1.62706375e-02,1.76830292e-02,  2.28428841e-02,2.39524841e-02,2.77118683e-02,3.08752060e-02,  4.16297913e-02,5.56154251e-02,6.99234009e-02,8.51116180e-02,  8.66222382e-02,8.55693817e-02,9.64374542e-02,1.13502502e-01,  1.35261536e-01,1.69578552e-01,1.75651550e-01,1.51802063e-01,  1.39610291e-01,1.53701782e-01,1.69460297e-01,1.89071655e-01,  2.19058990e-01,2.44052887e-01,2.52647400e-01,2.59269714e-01,  2.71789551e-01,2.75794983e-01,2.78457642e-01,2.81791687e-01,  2.83027649e-01,2.82516479e-01,2.82592773e-01,2.83988953e-01,  2.82882690e-01,2.79899597e-01,2.76428223e-01,2.66769409e-01,  2.47997284e-01,2.39227295e-01,2.46948242e-01,2.55592346e-01,  2.57804871e-01,2.54142761e-01,2.41016388e-01,2.35622406e-01,  2.39761353e-01,2.48741150e-01,2.53761292e-01,2.56446838e-01,  2.56271362e-01,2.54661560e-01,2.52616882e-01,2.51113892e-01,  2.50129700e-01,2.48252869e-01,2.44014740e-01,2.37892151e-01,  2.34142303e-01,2.30175018e-01,2.25677490e-01,2.21603394e-01,  2.17300415e-01,2.12963104e-01,2.09999084e-01],
        [2.56034851e-01,2.72026062e-01,2.91336060e-01,3.45581055e-01,  3.77250671e-01,3.87908936e-01,3.94889832e-01,4.01161194e-01,  4.42527771e-01,5.15701294e-01,6.94244385e-01,7.94418335e-01,  8.48541260e-01,8.94454956e-01,9.11834717e-01,8.97750854e-01,  8.95523071e-01,9.81567383e-01,1.07748413e+00,1.17254639e+00,  1.24615479e+00,1.28546143e+00,1.31832886e+00,1.32693481e+00,  1.34280396e+00,1.37423706e+00,1.39266968e+00,1.35516357e+00,  1.37435913e+00,1.40536499e+00,1.38980103e+00,1.39816284e+00,  1.40243530e+00,1.35638428e+00,1.36306763e+00,1.39331055e+00,  1.41311646e+00,1.42178345e+00,1.41354370e+00,1.41772461e+00,  1.42080688e+00,1.40713501e+00,1.38226318e+00,1.36666870e+00,  1.35244751e+00,1.35058594e+00,1.36895752e+00,1.37475586e+00,  1.35522461e+00,1.35687256e+00,1.35058594e+00,1.35070801e+00,  1.34405518e+00,1.34106445e+00,1.34567261e+00,1.32080078e+00,  1.31164551e+00,1.34112549e+00,1.36010742e+00,1.35858154e+00,  1.34686279e+00,1.32089233e+00,1.35504150e+00,1.38629150e+00,  1.38494873e+00,1.37835693e+00,1.35632324e+00,1.23428345e+00,  1.18847656e+00,1.27316284e+00,1.29421997e+00,1.30313110e+00,  1.29598999e+00,1.22702026e+00,1.16552734e+00,1.16387939e+00,  1.18847656e+00,1.22488403e+00,1.22763062e+00,1.23303223e+00,  1.22097778e+00,1.09060669e+00,7.10479736e-01,7.71560669e-01,  1.07192993e+00,1.15225220e+00,1.14904785e+00,1.14477539e+00,  1.12924194e+00,1.11538696e+00,1.11486816e+00,1.09860229e+00,  1.05578613e+00,9.86846924e-01,9.53735352e-01,9.69757080e-01,  9.85015869e-01,9.99389648e-01,1.01815796e+00,1.00411987e+00,  9.56130981e-01,9.51797485e-01,9.64447021e-01,9.42611694e-01,  9.47448730e-01,9.41177368e-01,9.50256348e-01,9.55657959e-01,  9.19067383e-01,8.59741211e-01,8.01849365e-01,7.96920776e-01,  7.76214600e-01,7.61657715e-01,7.79907227e-01,7.75482178e-01,  5.96282959e-01,4.51416016e-01,4.64347839e-01,4.74510193e-01,  4.88143921e-01,5.15609741e-01,5.60104370e-01,6.26770020e-01,  6.78253174e-01,6.92550659e-01,7.13195801e-01,7.39685059e-01,  7.51602173e-01,7.51312256e-01,7.41348267e-01,7.31002808e-01,  7.28973389e-01,7.25906372e-01,7.18902588e-01,7.11273193e-01,  7.01950073e-01,6.92214966e-01,6.83776855e-01,6.74087524e-01,  6.63482666e-01,6.49948120e-01,6.35177612e-01,6.22528076e-01,  6.16790771e-01,6.15798950e-01,6.08810425e-01,5.98327637e-01,  5.84808350e-01,5.64407349e-01,5.42800903e-01,5.16616821e-01,  4.55841064e-01,3.50234985e-01,2.63717651e-01,2.38407135e-01,  2.50762939e-01,2.79014587e-01,3.08197021e-01,2.94281006e-01,  3.09608459e-01,3.63555908e-01,4.19296265e-01,4.58702087e-01,  4.76837158e-01,4.77088928e-01,4.68864441e-01,4.65774536e-01,  4.75326538e-01,4.74548340e-01,4.59907532e-01,4.55856323e-01,  4.62547302e-01,4.69909668e-01,4.70443726e-01,4.70329285e-01,  4.72640991e-01,4.71565247e-01,4.65065002e-01,4.57160950e-01,  4.45632935e-01,4.23057556e-01,3.82385254e-01,3.41964722e-01,  3.49853516e-01,3.89259338e-01,4.06631470e-01,4.19311523e-01,  4.24552917e-01,4.18807983e-01,4.05494690e-01,3.88839722e-01,  3.69010925e-01,3.50997925e-01,3.38996887e-01,3.17619324e-01,  2.74948120e-01,2.63328552e-01,2.43000031e-01,1.66202545e-01,  7.53116608e-02,2.60386467e-02,1.38683319e-02,1.23927593e-02,  1.30798817e-02,1.65743828e-02,1.50139332e-02,1.63908005e-02,  2.10881233e-02,2.21300125e-02,2.55308151e-02,2.84371376e-02,  3.88889313e-02,5.23118973e-02,6.61525726e-02,8.12015533e-02,  8.28247070e-02,8.17298889e-02,9.22279358e-02,1.09054565e-01,  1.30962372e-01,1.65073395e-01,1.71432495e-01,1.47583008e-01,  1.35318756e-01,1.49639130e-01,1.65519714e-01,1.85504913e-01,  2.15991974e-01,2.41592407e-01,2.50694275e-01,2.57652283e-01,  2.70317078e-01,2.74520874e-01,2.77435303e-01,2.81105042e-01,  2.82440186e-01,2.81936646e-01,2.82188416e-01,2.83615112e-01,  2.82394409e-01,2.79426575e-01,2.76008606e-01,2.66151428e-01,  2.47009277e-01,2.38224030e-01,2.46204376e-01,2.54936218e-01,  2.57247925e-01,2.53204346e-01,2.39727020e-01,2.34355927e-01,  2.38594055e-01,2.47932434e-01,2.53120422e-01,2.55874634e-01,  2.55508423e-01,2.53417969e-01,2.51136780e-01,2.49095917e-01,  2.47955322e-01,2.45731354e-01,2.41741180e-01,2.35458374e-01,  2.31346130e-01,2.26722717e-01,2.21443176e-01,2.17041016e-01,  2.13153839e-01,2.08614349e-01,2.06573486e-01],
        [2.43309021e-01,2.55081177e-01,2.72552490e-01,3.27186584e-01,  3.53782654e-01,3.67164612e-01,3.74076843e-01,3.81172180e-01,  4.22058105e-01,4.91027832e-01,6.62948608e-01,7.59490967e-01,  8.12316895e-01,8.59008789e-01,8.76373291e-01,8.63555908e-01,  8.62792969e-01,9.46807861e-01,1.04104614e+00,1.13348389e+00,  1.20715332e+00,1.24703979e+00,1.27813721e+00,1.28930664e+00,  1.30465698e+00,1.33615112e+00,1.35604858e+00,1.32046509e+00,  1.34045410e+00,1.37112427e+00,1.35723877e+00,1.36602783e+00,  1.36981201e+00,1.32626343e+00,1.33294678e+00,1.36300659e+00,  1.38351440e+00,1.39227295e+00,1.38433838e+00,1.38900757e+00,  1.39202881e+00,1.37985229e+00,1.35424805e+00,1.33868408e+00,  1.32443237e+00,1.32138062e+00,1.34063721e+00,1.34756470e+00,  1.32928467e+00,1.33132935e+00,1.32415771e+00,1.32653809e+00,  1.32156372e+00,1.31750488e+00,1.32284546e+00,1.29904175e+00,  1.28961182e+00,1.32144165e+00,1.34133911e+00,1.33963013e+00,  1.32821655e+00,1.30261230e+00,1.33850098e+00,1.36837769e+00,  1.36828613e+00,1.36407471e+00,1.34191895e+00,1.21966553e+00,  1.17343140e+00,1.25765991e+00,1.28021240e+00,1.28836060e+00,  1.28198242e+00,1.21292114e+00,1.14910889e+00,1.14953613e+00,  1.17599487e+00,1.21316528e+00,1.21908569e+00,1.22180176e+00,  1.20922852e+00,1.08184814e+00,7.01019287e-01,7.62298584e-01,  1.06408691e+00,1.14349365e+00,1.14199829e+00,1.13455200e+00,  1.11764526e+00,1.10858154e+00,1.10748291e+00,1.09643555e+00,  1.05273438e+00,9.72854614e-01,9.47677612e-01,9.64508057e-01,  9.73541260e-01,9.90341187e-01,1.00665283e+00,9.91241455e-01,  9.54528809e-01,9.41024780e-01,9.50454712e-01,9.34722900e-01,  9.46685791e-01,9.43481445e-01,9.42214966e-01,9.39575195e-01,  9.22302246e-01,8.44467163e-01,7.93792725e-01,7.87567139e-01,  7.63885498e-01,7.54776001e-01,7.73651123e-01,7.55493164e-01,  5.83068848e-01,4.39315796e-01,4.52087402e-01,4.62326050e-01,  4.76051331e-01,5.03829956e-01,5.48919678e-01,6.16973877e-01,  6.69784546e-01,6.84555054e-01,7.06130981e-01,7.33856201e-01,  7.46490479e-01,7.46688843e-01,7.36999512e-01,7.26669312e-01,  7.24700928e-01,7.21618652e-01,7.14477539e-01,7.06832886e-01,  6.97723389e-01,6.88095093e-01,6.79794312e-01,6.70028687e-01,  6.59378052e-01,6.45584106e-01,6.30676270e-01,6.17935181e-01,  6.12365723e-01,6.11541748e-01,6.04431152e-01,5.93963623e-01,  5.80322266e-01,5.59509277e-01,5.37017822e-01,5.10162354e-01,  4.48585510e-01,3.42132568e-01,2.55332947e-01,2.29812622e-01,  2.42202759e-01,2.70668030e-01,2.99743652e-01,2.85728455e-01,  3.01147461e-01,3.55773926e-01,4.12330627e-01,4.52598572e-01,  4.71267700e-01,4.71580505e-01,4.63554382e-01,4.60441589e-01,  4.70367432e-01,4.69970703e-01,4.55291748e-01,4.51171875e-01,  4.58076477e-01,4.65736389e-01,4.66438293e-01,4.66545105e-01,  4.69238281e-01,4.68254089e-01,4.61891174e-01,4.53895569e-01,  4.42100525e-01,4.19075012e-01,3.77708435e-01,3.36738586e-01,  3.44833374e-01,3.85063171e-01,4.03091431e-01,4.16198730e-01,  4.21478271e-01,4.15565491e-01,4.02145386e-01,3.85070801e-01,  3.64959717e-01,3.46580505e-01,3.34327698e-01,3.12568665e-01,  2.69439697e-01,2.57850647e-01,2.37411499e-01,1.61064148e-01,  7.16743469e-02,2.45542526e-02,1.32877827e-02,1.20606422e-02,  1.26030445e-02,1.56669617e-02,1.44090652e-02,1.56016350e-02,  1.95846558e-02,2.06179619e-02,2.38666534e-02,2.65278816e-02,  3.63397598e-02,4.93373871e-02,6.27231598e-02,7.74269104e-02,  7.90290833e-02,7.76939392e-02,8.76331329e-02,1.04221344e-01,  1.26091003e-01,1.60034180e-01,1.66366577e-01,1.42814636e-01,  1.30878448e-01,1.45072937e-01,1.60945892e-01,1.80919647e-01,  2.11822510e-01,2.37892151e-01,2.47432709e-01,2.54776001e-01,  2.67776489e-01,2.72155762e-01,2.75276184e-01,2.78953552e-01,  2.80357361e-01,2.80006409e-01,2.80273438e-01,2.81967163e-01,  2.80975342e-01,2.78190613e-01,2.74620056e-01,2.64495850e-01,  2.45071411e-01,2.36072540e-01,2.44316101e-01,2.53433228e-01,  2.55966187e-01,2.51937866e-01,2.38136292e-01,2.32650757e-01,  2.36919403e-01,2.46326447e-01,2.51579285e-01,2.54249573e-01,  2.53997803e-01,2.52082825e-01,2.49752045e-01,2.47928619e-01,  2.46536255e-01,2.44438171e-01,2.40127563e-01,2.33806610e-01,  2.30247498e-01,2.26222992e-01,2.22175598e-01,2.17422485e-01,  2.13291168e-01,2.08171844e-01,2.05085754e-01],
        [2.31853485e-01,2.37403870e-01,2.57041931e-01,3.07434082e-01,  3.32008362e-01,3.49555969e-01,3.54110718e-01,3.62518311e-01,  4.01550293e-01,4.66407776e-01,6.33422852e-01,7.24380493e-01,  7.76046753e-01,8.22784424e-01,8.40164185e-01,8.31344604e-01,  8.29620361e-01,9.12353516e-01,1.00534058e+00,1.09457397e+00,  1.16799927e+00,1.20690918e+00,1.23950195e+00,1.25161743e+00,  1.26712036e+00,1.29940796e+00,1.31878662e+00,1.28549194e+00,  1.30712891e+00,1.33694458e+00,1.32409668e+00,1.33267212e+00,  1.33688354e+00,1.29626465e+00,1.30203247e+00,1.33184814e+00,  1.35226440e+00,1.36120605e+00,1.35513306e+00,1.35922241e+00,  1.36248779e+00,1.35067749e+00,1.32510376e+00,1.31072998e+00,  1.29464722e+00,1.29193115e+00,1.31402588e+00,1.31872559e+00,  1.30133057e+00,1.30520630e+00,1.29803467e+00,1.30187988e+00,  1.29663086e+00,1.29373169e+00,1.30078125e+00,1.27471924e+00,  1.26821899e+00,1.30014038e+00,1.31909180e+00,1.32122803e+00,  1.30932617e+00,1.28536987e+00,1.32269287e+00,1.35040283e+00,  1.35299683e+00,1.34930420e+00,1.32586670e+00,1.20452881e+00,  1.15640259e+00,1.24237061e+00,1.26834106e+00,1.27481079e+00,  1.27059937e+00,1.19891357e+00,1.13391113e+00,1.13778687e+00,  1.16156006e+00,1.20056152e+00,1.20913696e+00,1.20999146e+00,  1.20153809e+00,1.07299805e+00,6.86584473e-01,7.52258301e-01,  1.05410767e+00,1.13339233e+00,1.13638306e+00,1.12414551e+00,  1.11129761e+00,1.10281372e+00,1.09503174e+00,1.09045410e+00,  1.04379272e+00,9.62188721e-01,9.39666748e-01,9.52987671e-01,  9.70855713e-01,9.88449097e-01,9.95925903e-01,9.93576050e-01,  9.50942993e-01,9.33731079e-01,9.56802368e-01,9.32006836e-01,  9.37988281e-01,9.44686890e-01,9.34921265e-01,9.44686890e-01,  9.19952393e-01,8.29223633e-01,7.94006348e-01,7.89581299e-01,  7.54043579e-01,7.66357422e-01,7.64755249e-01,7.39440918e-01,  5.74325562e-01,4.29283142e-01,4.42382812e-01,4.52819824e-01,  4.66705322e-01,4.94781494e-01,5.40618896e-01,6.09924316e-01,  6.64031982e-01,6.79229736e-01,7.01354980e-01,7.30163574e-01,  7.43591309e-01,7.44140625e-01,7.34664917e-01,7.24365234e-01,  7.22381592e-01,7.19345093e-01,7.12448120e-01,7.04772949e-01,  6.95465088e-01,6.85852051e-01,6.77536011e-01,6.67861938e-01,  6.57028198e-01,6.43051147e-01,6.27944946e-01,6.15188599e-01,  6.09802246e-01,6.09085083e-01,6.02127075e-01,5.91506958e-01,  5.77789307e-01,5.56549072e-01,5.33264160e-01,5.05508423e-01,  4.43328857e-01,3.36029053e-01,2.48729706e-01,2.23056793e-01,  2.35515594e-01,2.64244080e-01,2.93182373e-01,2.78953552e-01,  2.94464111e-01,3.49670410e-01,4.07356262e-01,4.48341370e-01,  4.67529297e-01,4.68299866e-01,4.60250854e-01,4.57168579e-01,  4.67285156e-01,4.67025757e-01,4.52232361e-01,4.48219299e-01,  4.55337524e-01,4.63264465e-01,4.64218140e-01,4.64591980e-01,  4.67491150e-01,4.66636658e-01,4.60258484e-01,4.52125549e-01,  4.40017700e-01,4.16351318e-01,3.74137878e-01,3.32504272e-01,  3.40858459e-01,3.82087708e-01,4.01062012e-01,4.14558411e-01,  4.19898987e-01,4.13856506e-01,4.00077820e-01,3.82843018e-01,  3.62480164e-01,3.43788147e-01,3.31298828e-01,3.09173584e-01,  2.65235901e-01,2.53669739e-01,2.33318329e-01,1.57005310e-01,  6.90040588e-02,2.37526894e-02,1.32153034e-02,1.20787621e-02,  1.24776363e-02,1.49433613e-02,1.38635635e-02,1.49042606e-02,  1.85594559e-02,1.97229385e-02,2.27308273e-02,2.51889229e-02,  3.46651077e-02,4.69560623e-02,5.98735809e-02,7.42263794e-02,  7.59525299e-02,7.44724274e-02,8.42170715e-02,1.00702286e-01,  1.22486115e-01,1.56230927e-01,1.62609100e-01,1.39141083e-01,  1.27136230e-01,1.41483307e-01,1.57505035e-01,1.77768707e-01,  2.09148407e-01,2.35626221e-01,2.45559692e-01,2.53135681e-01,  2.66334534e-01,2.70805359e-01,2.74314880e-01,2.78221130e-01,  2.79754639e-01,2.79441833e-01,2.79922485e-01,2.81623840e-01,  2.80639648e-01,2.77755737e-01,2.74215698e-01,2.63946533e-01,  2.44201660e-01,2.35237122e-01,2.43698120e-01,2.53265381e-01,  2.55912781e-01,2.51625061e-01,2.37312317e-01,2.31636047e-01,  2.36221313e-01,2.45941162e-01,2.51441956e-01,2.54386902e-01,  2.54104614e-01,2.52105713e-01,2.49374390e-01,2.47806549e-01,  2.46459961e-01,2.44316101e-01,2.40497589e-01,2.34893799e-01,  2.32410431e-01,2.28916168e-01,2.23800659e-01,2.19577789e-01,  2.15103149e-01,2.09835052e-01,2.06813812e-01],
        [2.07431793e-01,2.22564697e-01,2.38826752e-01,2.85362244e-01,  3.12904358e-01,3.26347351e-01,3.32916260e-01,3.38874817e-01,  3.76869202e-01,4.42893982e-01,5.99349976e-01,6.88568115e-01,  7.38464355e-01,7.83355713e-01,8.01483154e-01,7.92831421e-01,  7.94906616e-01,8.74572754e-01,9.64553833e-01,1.05319214e+00,  1.12438965e+00,1.16427612e+00,1.19696045e+00,1.20968628e+00,  1.22604370e+00,1.25765991e+00,1.27789307e+00,1.24722290e+00,  1.26889038e+00,1.29998779e+00,1.28701782e+00,1.29644775e+00,  1.30303955e+00,1.26358032e+00,1.27087402e+00,1.29956055e+00,  1.31878662e+00,1.32873535e+00,1.32214355e+00,1.32693481e+00,  1.33029175e+00,1.31930542e+00,1.29592896e+00,1.27941895e+00,  1.26452637e+00,1.26126099e+00,1.28204346e+00,1.29113770e+00,  1.27297974e+00,1.27545166e+00,1.27108765e+00,1.27444458e+00,  1.27044678e+00,1.26840210e+00,1.27502441e+00,1.25137329e+00,  1.24261475e+00,1.27633667e+00,1.29876709e+00,1.30026245e+00,  1.29138184e+00,1.26806641e+00,1.30459595e+00,1.33496094e+00,  1.33517456e+00,1.33294678e+00,1.31204224e+00,1.18798828e+00,  1.14047241e+00,1.22711182e+00,1.25219727e+00,1.26275635e+00,  1.25640869e+00,1.18569946e+00,1.12155151e+00,1.12213135e+00,  1.14987183e+00,1.19088745e+00,1.20001221e+00,1.20343018e+00,  1.19024658e+00,1.06195068e+00,6.77062988e-01,7.37976074e-01,  1.04483032e+00,1.12664795e+00,1.12503052e+00,1.11691284e+00,  1.10113525e+00,1.09500122e+00,1.09436035e+00,1.08081055e+00,  1.03509521e+00,9.54208374e-01,9.28695679e-01,9.44442749e-01,  9.57672119e-01,9.77508545e-01,9.91668701e-01,9.77706909e-01,  9.45709229e-01,9.33990479e-01,9.43939209e-01,9.24545288e-01,  9.34417725e-01,9.33364868e-01,9.31381226e-01,9.31442261e-01,  9.13848877e-01,8.40301514e-01,7.79388428e-01,7.72354126e-01,  7.47863770e-01,7.36801147e-01,7.63183594e-01,7.48794556e-01,  5.67932129e-01,4.20883179e-01,4.34013367e-01,4.44519043e-01,  4.58442688e-01,4.86625671e-01,5.32714844e-01,6.02905273e-01,  6.57836914e-01,6.73248291e-01,6.96075439e-01,7.25616455e-01,  7.39395142e-01,7.40386963e-01,7.31155396e-01,7.20764160e-01,  7.18872070e-01,7.15927124e-01,7.08923340e-01,7.01202393e-01,  6.91970825e-01,6.82403564e-01,6.73980713e-01,6.64260864e-01,  6.53457642e-01,6.39221191e-01,6.23992920e-01,6.11221313e-01,  6.06018066e-01,6.05545044e-01,5.98739624e-01,5.88104248e-01,  5.74310303e-01,5.52719116e-01,5.29067993e-01,5.01052856e-01,  4.38446045e-01,3.30581665e-01,2.43144989e-01,2.17441559e-01,  2.29873657e-01,2.58705139e-01,2.87460327e-01,2.73117065e-01,  2.88764954e-01,3.44276428e-01,4.02420044e-01,4.43862915e-01,  4.63485718e-01,4.64492798e-01,4.56520081e-01,4.53437805e-01,  4.63745117e-01,4.63577271e-01,4.48738098e-01,4.44709778e-01,  4.52110291e-01,4.60235596e-01,4.61235046e-01,4.61776733e-01,  4.64965820e-01,4.64035034e-01,4.57489014e-01,4.49195862e-01,  4.36752319e-01,4.12658691e-01,3.69529724e-01,3.27140808e-01,  3.35990906e-01,3.78303528e-01,3.98025513e-01,4.11972046e-01,  4.17312622e-01,4.11239624e-01,3.97430420e-01,3.80096436e-01,  3.59512329e-01,3.40744019e-01,3.28178406e-01,3.05763245e-01,  2.61428833e-01,2.49809265e-01,2.29389191e-01,1.53339386e-01,  6.63032532e-02,2.23073959e-02,1.24568939e-02,1.12843513e-02,  1.16462708e-02,1.39322281e-02,1.29332542e-02,1.39861107e-02,  1.72243118e-02,1.81803703e-02,2.11868286e-02,2.35648155e-02,  3.26890945e-02,4.47387695e-02,5.72776794e-02,7.14168549e-02,  7.31143951e-02,7.13958740e-02,8.08982849e-02,9.73834991e-02,  1.19113922e-01,1.52683258e-01,1.59126282e-01,1.35818481e-01,  1.23933792e-01,1.38278961e-01,1.54216766e-01,1.74552917e-01,  2.06130981e-01,2.32967377e-01,2.43221283e-01,2.50938416e-01,  2.64396667e-01,2.69020081e-01,2.72575378e-01,2.76550293e-01,  2.77969360e-01,2.77740479e-01,2.78106689e-01,2.80090332e-01,  2.79335022e-01,2.76489258e-01,2.73078918e-01,2.62519836e-01,  2.42206573e-01,2.33020782e-01,2.41676331e-01,2.51441956e-01,  2.54272461e-01,2.49839783e-01,2.35237122e-01,2.29492188e-01,  2.34184265e-01,2.44098663e-01,2.49576569e-01,2.52288818e-01,  2.51762390e-01,2.49740601e-01,2.46871948e-01,2.44941711e-01,  2.43331909e-01,2.41424561e-01,2.37228394e-01,2.30571747e-01,  2.27188110e-01,2.22702026e-01,2.18227386e-01,2.13378906e-01,  2.08728790e-01,2.03823090e-01,1.99794769e-01],
        [1.95964813e-01,2.06069946e-01,2.21984863e-01,2.66983032e-01,  2.89741516e-01,3.04550171e-01,3.11271667e-01,3.18061829e-01,  3.55545044e-01,4.16854858e-01,5.66467285e-01,6.52679443e-01,  7.01370239e-01,7.45468140e-01,7.64205933e-01,7.57843018e-01,  7.60803223e-01,8.38363647e-01,9.26223755e-01,1.01202393e+00,  1.08230591e+00,1.12252808e+00,1.15490723e+00,1.16958618e+00,  1.18652344e+00,1.21697998e+00,1.23791504e+00,1.21023560e+00,  1.23281860e+00,1.26394653e+00,1.25268555e+00,1.26211548e+00,  1.26861572e+00,1.23110962e+00,1.23904419e+00,1.26687622e+00,  1.28579712e+00,1.29544067e+00,1.29025269e+00,1.29559326e+00,  1.29901123e+00,1.28829956e+00,1.26455688e+00,1.24835205e+00,  1.23294067e+00,1.22988892e+00,1.25299072e+00,1.26306152e+00,  1.24493408e+00,1.24765015e+00,1.24179077e+00,1.24688721e+00,  1.24444580e+00,1.24295044e+00,1.25000000e+00,1.22714233e+00,  1.22082520e+00,1.25582886e+00,1.27746582e+00,1.28073120e+00,  1.27188110e+00,1.24975586e+00,1.28683472e+00,1.31774902e+00,  1.32006836e+00,1.31762695e+00,1.29565430e+00,1.17172241e+00,  1.12445068e+00,1.21368408e+00,1.23980713e+00,1.24902344e+00,  1.24536133e+00,1.17333984e+00,1.10659790e+00,1.10873413e+00,  1.13647461e+00,1.17745972e+00,1.18856812e+00,1.19354248e+00,  1.18280029e+00,1.05462646e+00,6.65481567e-01,7.26440430e-01,  1.03488159e+00,1.11703491e+00,1.11868286e+00,1.11282349e+00,  1.09722900e+00,1.08801270e+00,1.08193970e+00,1.07086182e+00,  1.02871704e+00,9.45083618e-01,9.20562744e-01,9.33944702e-01,  9.49768066e-01,9.73495483e-01,9.87930298e-01,9.81079102e-01,  9.43069458e-01,9.25491333e-01,9.38796997e-01,9.17785645e-01,  9.28466797e-01,9.33029175e-01,9.25765991e-01,9.25704956e-01,  9.09744263e-01,8.29956055e-01,7.73803711e-01,7.72125244e-01,  7.35885620e-01,7.30728149e-01,7.54760742e-01,7.26577759e-01,  5.57189941e-01,4.12895203e-01,4.26078796e-01,4.36874390e-01,  4.50866699e-01,4.79393005e-01,5.26031494e-01,5.97305298e-01,  6.53335571e-01,6.69342041e-01,6.92642212e-01,7.23114014e-01,  7.37625122e-01,7.38922119e-01,7.29736328e-01,7.19543457e-01,  7.17742920e-01,7.14859009e-01,7.07839966e-01,7.00225830e-01,  6.91024780e-01,6.81335449e-01,6.72882080e-01,6.63085938e-01,  6.52023315e-01,6.37527466e-01,6.21948242e-01,6.09008789e-01,  6.04095459e-01,6.04049683e-01,5.97351074e-01,5.86822510e-01,  5.72921753e-01,5.50888062e-01,5.26290894e-01,4.97581482e-01,  4.34249878e-01,3.25576782e-01,2.37758636e-01,2.11917877e-01,  2.24578857e-01,2.53791809e-01,2.82447815e-01,2.67944336e-01,  2.83607483e-01,3.39324951e-01,3.98086548e-01,4.40299988e-01,  4.60395813e-01,4.61738586e-01,4.54071045e-01,4.51049805e-01,  4.61616516e-01,4.61715698e-01,4.46609497e-01,4.42565918e-01,  4.50088501e-01,4.58312988e-01,4.59465027e-01,4.60365295e-01,  4.63752747e-01,4.63104248e-01,4.56634521e-01,4.48104858e-01,  4.35264587e-01,4.10324097e-01,3.65997314e-01,3.22746277e-01,  3.31901550e-01,3.75396729e-01,3.96316528e-01,4.11003113e-01,  4.16549683e-01,4.10408020e-01,3.96415710e-01,3.78601074e-01,  3.57719421e-01,3.38661194e-01,3.25752258e-01,3.03092957e-01,  2.58285522e-01,2.46673584e-01,2.26413727e-01,1.50276184e-01,  6.42929077e-02,2.18629837e-02,1.25105381e-02,1.14216805e-02,  1.16143227e-02,1.35228634e-02,1.27506256e-02,1.37412548e-02,  1.67732239e-02,1.77869797e-02,2.05159187e-02,2.26964951e-02,  3.15008163e-02,4.30574417e-02,5.52492142e-02,6.91280365e-02,  7.07988739e-02,6.90212250e-02,7.82737732e-02,9.45701599e-02,  1.16310120e-01,1.49700165e-01,1.56318665e-01,1.33098602e-01,  1.21202469e-01,1.35532379e-01,1.51641846e-01,1.72122955e-01,  2.04032898e-01,2.31307983e-01,2.41958618e-01,2.50053406e-01,  2.63664246e-01,2.68440247e-01,2.72247314e-01,2.76329041e-01,  2.77885437e-01,2.77656555e-01,2.78221130e-01,2.80258179e-01,  2.79617310e-01,2.76893616e-01,2.73384094e-01,2.62512207e-01,  2.41661072e-01,2.32425690e-01,2.41546631e-01,2.51747131e-01,  2.54783630e-01,2.50160217e-01,2.34954834e-01,2.29183197e-01,  2.33970642e-01,2.44197845e-01,2.50091553e-01,2.52914429e-01,  2.52418518e-01,2.50076294e-01,2.47268677e-01,2.45426178e-01,  2.44064331e-01,2.42473602e-01,2.38571167e-01,2.32578278e-01,  2.29488373e-01,2.25402832e-01,2.20771790e-01,2.15515137e-01,  2.11688995e-01,2.06943512e-01,2.04200745e-01],
        [1.72889709e-01,1.86000824e-01,2.02804565e-01,2.40940094e-01,  2.67677307e-01,2.80380249e-01,2.87712097e-01,2.95768738e-01,  3.29132080e-01,3.88206482e-01,5.27328491e-01,6.08749390e-01,  6.56951904e-01,6.98760986e-01,7.19467163e-01,7.13745117e-01,  7.18154907e-01,7.93563843e-01,8.77777100e-01,9.61975098e-01,  1.02947998e+00,1.06958008e+00,1.10241699e+00,1.11624146e+00,  1.13537598e+00,1.16580200e+00,1.18716431e+00,1.16275024e+00,  1.18508911e+00,1.21658325e+00,1.20596313e+00,1.21520996e+00,  1.22357178e+00,1.18823242e+00,1.19622803e+00,1.22387695e+00,  1.24240112e+00,1.25347900e+00,1.24847412e+00,1.25350952e+00,  1.25857544e+00,1.24777222e+00,1.22540283e+00,1.20910645e+00,  1.19271851e+00,1.19006348e+00,1.21316528e+00,1.22463989e+00,  1.20852661e+00,1.21023560e+00,1.20700073e+00,1.21194458e+00,  1.21008301e+00,1.21060181e+00,1.21734619e+00,1.19583130e+00,  1.18966675e+00,1.22534180e+00,1.24920654e+00,1.25103760e+00,  1.24374390e+00,1.22451782e+00,1.26101685e+00,1.29364014e+00,  1.29611206e+00,1.29376221e+00,1.27392578e+00,1.14855957e+00,  1.10119629e+00,1.19165039e+00,1.21762085e+00,1.22988892e+00,  1.22592163e+00,1.15148926e+00,1.08639526e+00,1.08544922e+00,  1.11654663e+00,1.16140747e+00,1.17163086e+00,1.17926025e+00,  1.16580200e+00,1.03558350e+00,6.53335571e-01,7.10601807e-01,  1.01855469e+00,1.10610962e+00,1.10467529e+00,1.10006714e+00,  1.08648682e+00,1.07684326e+00,1.07229614e+00,1.05764771e+00,  1.01162720e+00,9.32800293e-01,9.01367188e-01,9.26345825e-01,  9.40689087e-01,9.56756592e-01,9.79614258e-01,9.64675903e-01,  9.22958374e-01,9.19891357e-01,9.31686401e-01,9.13421631e-01,  9.31930542e-01,9.23522949e-01,9.22531128e-01,9.23339844e-01,  8.89846802e-01,8.24432373e-01,7.59536743e-01,7.50183105e-01,  7.35229492e-01,7.13088989e-01,7.37442017e-01,7.33383179e-01,  5.47683716e-01,3.96522522e-01,4.09614563e-01,4.20631409e-01,  4.34928894e-01,4.63760376e-01,5.11291504e-01,5.84609985e-01,  6.42684937e-01,6.59576416e-01,6.84188843e-01,7.16094971e-01,  7.31796265e-01,7.33398438e-01,7.24182129e-01,7.14141846e-01,  7.12432861e-01,7.09732056e-01,7.02911377e-01,6.95526123e-01,  6.86569214e-01,6.76940918e-01,6.68502808e-01,6.58569336e-01,  6.47125244e-01,6.32278442e-01,6.16592407e-01,6.03637695e-01,  5.98907471e-01,5.99060059e-01,5.92544556e-01,5.81985474e-01,  5.67718506e-01,5.44876099e-01,5.19165039e-01,4.89395142e-01,  4.25094604e-01,3.15330505e-01,2.26882935e-01,2.00782776e-01,  2.13466644e-01,2.43000031e-01,2.71362305e-01,2.56721497e-01,  2.72628784e-01,3.29292297e-01,3.89526367e-01,4.32769775e-01,  4.53674316e-01,4.55444336e-01,4.47700500e-01,4.44770813e-01,  4.55734253e-01,4.56146240e-01,4.41040039e-01,4.37126160e-01,  4.45106506e-01,4.53788757e-01,4.55131531e-01,4.56382751e-01,  4.60250854e-01,4.59640503e-01,4.53147888e-01,4.44458008e-01,  4.31213379e-01,4.05677795e-01,3.60366821e-01,3.16429138e-01,  3.25859070e-01,3.70552063e-01,3.92601013e-01,4.07638550e-01,  4.13154602e-01,4.06768799e-01,3.92356873e-01,3.74153137e-01,  3.52851868e-01,3.33343506e-01,3.20083618e-01,2.96913147e-01,  2.51144409e-01,2.39532471e-01,2.19505310e-01,1.43779755e-01,  5.99737167e-02,2.02517509e-02,1.19407177e-02,1.10163689e-02,  1.10666752e-02,1.23884678e-02,1.18474960e-02,1.27503872e-02,  1.51934624e-02,1.62649155e-02,1.87029839e-02,2.05178261e-02,  2.86827087e-02,3.93724442e-02,5.07507324e-02,6.40506744e-02,  6.59313202e-02,6.40411377e-02,7.27863312e-02,8.88004303e-02,  1.10311508e-01,1.43241882e-01,1.49829865e-01,1.26945496e-01,  1.15236282e-01,1.29642487e-01,1.45683289e-01,1.66378021e-01,  1.98886871e-01,2.26997375e-01,2.38300323e-01,2.46803284e-01,  2.60688782e-01,2.65800476e-01,2.70027161e-01,2.74322510e-01,  2.76062012e-01,2.75917053e-01,2.76649475e-01,2.78907776e-01,  2.78320312e-01,2.75642395e-01,2.72079468e-01,2.61009216e-01,  2.39780426e-01,2.30445862e-01,2.39788055e-01,2.50411987e-01,  2.53555298e-01,2.48607635e-01,2.33043671e-01,2.27233887e-01,  2.32158661e-01,2.42713928e-01,2.48741150e-01,2.51457214e-01,  2.50938416e-01,2.48466492e-01,2.45506287e-01,2.43515015e-01,  2.42069244e-01,2.40657806e-01,2.36083984e-01,2.29900360e-01,  2.27458954e-01,2.24067688e-01,2.19459534e-01,2.14572906e-01,  2.09976196e-01,2.04593658e-01,2.01828003e-01],
        [1.56616211e-01,1.67255402e-01,1.82025909e-01,2.19291687e-01,  2.42446899e-01,2.55638123e-01,2.63786316e-01,2.71530151e-01,  3.03924561e-01,3.59687805e-01,4.92172241e-01,5.69869995e-01,  6.15737915e-01,6.57455444e-01,6.77490234e-01,6.74362183e-01,  6.80419922e-01,7.52593994e-01,8.33709717e-01,9.14367676e-01,  9.80346680e-01,1.01959229e+00,1.05194092e+00,1.06738281e+00,  1.08593750e+00,1.11535645e+00,1.13723755e+00,1.11511230e+00,  1.13787842e+00,1.16857910e+00,1.15905762e+00,1.16845703e+00,  1.17678833e+00,1.14419556e+00,1.15252686e+00,1.17855835e+00,  1.19631958e+00,1.20727539e+00,1.20355225e+00,1.20932007e+00,  1.21475220e+00,1.20584106e+00,1.18414307e+00,1.16864014e+00,  1.15258789e+00,1.14999390e+00,1.17437744e+00,1.18817139e+00,  1.17150879e+00,1.17422485e+00,1.17181396e+00,1.17782593e+00,  1.17834473e+00,1.18029785e+00,1.18832397e+00,1.16659546e+00,  1.16067505e+00,1.19772339e+00,1.22323608e+00,1.22659302e+00,  1.21923828e+00,1.20092773e+00,1.23971558e+00,1.27261353e+00,  1.27645874e+00,1.27526855e+00,1.25534058e+00,1.13113403e+00,  1.08358765e+00,1.17391968e+00,1.20230103e+00,1.21478271e+00,  1.21139526e+00,1.13806152e+00,1.07064819e+00,1.07150269e+00,  1.10299683e+00,1.14944458e+00,1.16262817e+00,1.17028809e+00,  1.15933228e+00,1.03076172e+00,6.40762329e-01,6.97006226e-01,  1.01092529e+00,1.09762573e+00,1.09799194e+00,1.09133911e+00,  1.07406616e+00,1.06503296e+00,1.06234741e+00,1.04928589e+00,  1.00341797e+00,9.16946411e-01,8.90884399e-01,9.12322998e-01,  9.28100586e-01,9.51080322e-01,9.69589233e-01,9.57183838e-01,  9.19845581e-01,9.10888672e-01,9.23553467e-01,9.03930664e-01,  9.10690308e-01,9.12796021e-01,9.11987305e-01,9.08172607e-01,  8.82263184e-01,8.02703857e-01,7.42904663e-01,7.43377686e-01,  7.15225220e-01,7.02499390e-01,7.31384277e-01,7.11471558e-01,  5.31631470e-01,3.83598328e-01,3.96514893e-01,4.07531738e-01,  4.21752930e-01,4.51072693e-01,4.99427795e-01,5.73974609e-01,  6.33468628e-01,6.51138306e-01,6.76651001e-01,7.10037231e-01,  7.26547241e-01,7.28622437e-01,7.19589233e-01,7.09564209e-01,  7.08114624e-01,7.05398560e-01,6.98699951e-01,6.91390991e-01,  6.82403564e-01,6.72882080e-01,6.64398193e-01,6.54174805e-01,  6.42608643e-01,6.27609253e-01,6.11587524e-01,5.98602295e-01,  5.94177246e-01,5.94635010e-01,5.88348389e-01,5.77789307e-01,  5.63186646e-01,5.39642334e-01,5.12924194e-01,4.82231140e-01,  4.17289734e-01,3.07075500e-01,2.18456268e-01,1.92161560e-01,  2.04914093e-01,2.34695435e-01,2.62573242e-01,2.47817993e-01,  2.63786316e-01,3.20632935e-01,3.81973267e-01,4.26071167e-01,  4.47616577e-01,4.49974060e-01,4.42428589e-01,4.39506531e-01,  4.50790405e-01,4.51370239e-01,4.36080933e-01,4.32189941e-01,  4.40399170e-01,4.49172974e-01,4.50973511e-01,4.52651978e-01,  4.56886292e-01,4.56619263e-01,4.50065613e-01,4.41085815e-01,  4.27398682e-01,4.01062012e-01,3.54499817e-01,3.09700012e-01,  3.19503784e-01,3.65417480e-01,3.88748169e-01,4.04449463e-01,  4.10110474e-01,4.03526306e-01,3.88893127e-01,3.70468140e-01,  3.48770142e-01,3.28842163e-01,3.15307617e-01,2.91763306e-01,  2.45197296e-01,2.33612061e-01,2.13745117e-01,1.38473511e-01,  5.66577911e-02,1.89261436e-02,1.14023685e-02,1.06432438e-02,  1.04749203e-02,1.15063190e-02,1.11892223e-02,1.19788647e-02,  1.40402317e-02,1.50640011e-02,1.71818733e-02,1.88283920e-02,  2.65541077e-02,3.65552902e-02,4.74166870e-02,6.03084564e-02,  6.21252060e-02,6.00309372e-02,6.82926178e-02,8.40969086e-02,  1.05415344e-01,1.37973785e-01,1.44634247e-01,1.21997833e-01,  1.10422134e-01,1.24877930e-01,1.40876770e-01,1.61712646e-01,  1.94652557e-01,2.23224640e-01,2.35061646e-01,2.43885040e-01,  2.58064270e-01,2.63427734e-01,2.67845154e-01,2.72476196e-01,  2.74253845e-01,2.74177551e-01,2.75146484e-01,2.77542114e-01,  2.77076721e-01,2.74429321e-01,2.70820618e-01,2.59429932e-01,  2.37590790e-01,2.28355408e-01,2.37918854e-01,2.48920441e-01,  2.52342224e-01,2.47135162e-01,2.30972290e-01,2.25009918e-01,  2.30258942e-01,2.41241455e-01,2.47360229e-01,2.50068665e-01,  2.49359131e-01,2.46673584e-01,2.43713379e-01,2.41416931e-01,  2.40253448e-01,2.38628387e-01,2.34752655e-01,2.28607178e-01,  2.25467682e-01,2.21515656e-01,2.16636658e-01,2.11204529e-01,  2.07042694e-01,2.02060699e-01,2.00542450e-01],
        [1.40785217e-01,1.49349213e-01,1.62803650e-01,1.98249817e-01,  2.18452454e-01,2.31243134e-01,2.39269257e-01,2.46921539e-01,  2.77168274e-01,3.27911377e-01,4.49676514e-01,5.22369385e-01,  5.66284180e-01,6.06735229e-01,6.26480103e-01,6.25747681e-01,  6.32598877e-01,7.01126099e-01,7.78961182e-01,8.55514526e-01,  9.19784546e-01,9.58755493e-01,9.90585327e-01,1.00750732e+00,  1.02600098e+00,1.05529785e+00,1.07821655e+00,1.05902100e+00,  1.08261108e+00,1.11303711e+00,1.10504150e+00,1.11483765e+00,  1.12426758e+00,1.09463501e+00,1.10314941e+00,1.12823486e+00,  1.14599609e+00,1.15731812e+00,1.15469360e+00,1.16073608e+00,  1.16604614e+00,1.15847778e+00,1.13732910e+00,1.12158203e+00,  1.10537720e+00,1.10287476e+00,1.12844849e+00,1.14312744e+00,  1.12744141e+00,1.13052368e+00,1.12857056e+00,1.13647461e+00,  1.13812256e+00,1.14093018e+00,1.14996338e+00,1.12823486e+00,  1.12356567e+00,1.16210938e+00,1.18838501e+00,1.19348145e+00,  1.18746948e+00,1.17062378e+00,1.21011353e+00,1.24246216e+00,  1.24734497e+00,1.24752808e+00,1.22821045e+00,1.10488892e+00,  1.05657959e+00,1.14773560e+00,1.17846680e+00,1.19113159e+00,  1.18853760e+00,1.11532593e+00,1.04611206e+00,1.04757690e+00,  1.07983398e+00,1.12820435e+00,1.14425659e+00,1.15219116e+00,  1.14193726e+00,1.01510620e+00,6.24862671e-01,6.80267334e-01,  9.95498657e-01,1.08251953e+00,1.08468628e+00,1.07629395e+00,  1.06079102e+00,1.05267334e+00,1.04864502e+00,1.03750610e+00,  9.89517212e-01,9.00741577e-01,8.77136230e-01,8.97171021e-01,  9.14321899e-01,9.41818237e-01,9.60388184e-01,9.49523926e-01,  9.11941528e-01,8.99841309e-01,9.17602539e-01,8.99063110e-01,  9.04998779e-01,9.10369873e-01,9.06829834e-01,9.04418945e-01,  8.80630493e-01,7.96646118e-01,7.39212036e-01,7.38174438e-01,  7.02972412e-01,6.94732666e-01,7.21664429e-01,6.98028564e-01,  5.16708374e-01,3.68186951e-01,3.80996704e-01,3.92082214e-01,  4.06555176e-01,4.36096191e-01,4.84725952e-01,5.60699463e-01,  6.21414185e-01,6.39724731e-01,6.66351318e-01,7.01065063e-01,  7.18719482e-01,7.21466064e-01,7.12570190e-01,7.02758789e-01,  7.01232910e-01,6.98745728e-01,6.92138672e-01,6.84799194e-01,  6.75842285e-01,6.66458130e-01,6.57897949e-01,6.47659302e-01,  6.35864258e-01,6.20803833e-01,6.04537964e-01,5.91445923e-01,  5.87448120e-01,5.88363647e-01,5.81939697e-01,5.71228027e-01,  5.56457520e-01,5.32272339e-01,5.04608154e-01,4.73220825e-01,  4.07722473e-01,2.97065735e-01,2.08480835e-01,1.82025909e-01,  1.94709778e-01,2.24575043e-01,2.51960754e-01,2.37213135e-01,  2.53265381e-01,3.10554504e-01,3.72726440e-01,4.17701721e-01,  4.39720154e-01,4.42550659e-01,4.35256958e-01,4.32182312e-01,  4.43748474e-01,4.44778442e-01,4.29428101e-01,4.25743103e-01,  4.34280396e-01,4.43367004e-01,4.45350647e-01,4.47326660e-01,  4.51904297e-01,4.51713562e-01,4.45266724e-01,4.36065674e-01,  4.22004700e-01,3.94882202e-01,3.47213745e-01,3.01689148e-01,  3.11820984e-01,3.59054565e-01,3.83514404e-01,3.99925232e-01,  4.05609131e-01,3.98902893e-01,3.84078979e-01,3.65364075e-01,  3.43330383e-01,3.23005676e-01,3.09127808e-01,2.85179138e-01,  2.38067627e-01,2.26486206e-01,2.06897736e-01,1.32503510e-01,  5.30805588e-02,1.77235603e-02,1.08795166e-02,1.01087093e-02,  9.74488258e-03,1.04405880e-02,1.02682114e-02,1.10857487e-02,  1.27964020e-02,1.37937069e-02,1.57346725e-02,1.72090530e-02,  2.42772102e-02,3.35149765e-02,4.36115265e-02,5.59654236e-02,  5.78613281e-02,5.57060242e-02,6.34002686e-02,7.88574219e-02,  9.98554230e-02,1.31759644e-01,1.38408661e-01,1.16235733e-01,  1.04925156e-01,1.19354248e-01,1.35292053e-01,1.56044006e-01,  1.89319611e-01,2.18475342e-01,2.30808258e-01,2.40070343e-01,  2.54570007e-01,2.60192871e-01,2.65007019e-01,2.69668579e-01,  2.71522522e-01,2.71560669e-01,2.72483826e-01,2.75177002e-01,  2.74887085e-01,2.72453308e-01,2.68867493e-01,2.57133484e-01,  2.34745026e-01,2.25234985e-01,2.35122681e-01,2.46555328e-01,  2.50091553e-01,2.44773865e-01,2.28084564e-01,2.22000122e-01,  2.27443695e-01,2.38746643e-01,2.44960785e-01,2.47688293e-01,  2.46875763e-01,2.43972778e-01,2.40600586e-01,2.38327026e-01,  2.36579895e-01,2.34626770e-01,2.30670929e-01,2.24311829e-01,  2.21263885e-01,2.17166901e-01,2.12249756e-01,2.06836700e-01,  2.01931000e-01,1.96601868e-01,1.94042206e-01],
        [1.23958588e-01,1.32068634e-01,1.44054413e-01,1.76738739e-01,  1.94355011e-01,2.06615448e-01,2.13935852e-01,2.21099854e-01,  2.50030518e-01,2.96844482e-01,4.08134460e-01,4.75494385e-01,  5.16479492e-01,5.55511475e-01,5.75149536e-01,5.76080322e-01,  5.84655762e-01,6.49459839e-01,7.23937988e-01,7.97149658e-01,  8.58993530e-01,8.97598267e-01,9.28604126e-01,9.47097778e-01,  9.66705322e-01,9.94903564e-01,1.01834106e+00,1.00238037e+00,  1.02642822e+00,1.05700684e+00,1.05044556e+00,1.06097412e+00,  1.07083130e+00,1.04406738e+00,1.05352783e+00,1.07745361e+00,  1.09490967e+00,1.10629272e+00,1.10430908e+00,1.11087036e+00,  1.11636353e+00,1.10977173e+00,1.08950806e+00,1.07318115e+00,  1.05715942e+00,1.05413818e+00,1.08071899e+00,1.09701538e+00,  1.08163452e+00,1.08526611e+00,1.08444214e+00,1.09274292e+00,  1.09646606e+00,1.10110474e+00,1.11053467e+00,1.09002686e+00,  1.08547974e+00,1.12594604e+00,1.15396118e+00,1.15963745e+00,  1.15478516e+00,1.13973999e+00,1.18041992e+00,1.21328735e+00,  1.21884155e+00,1.22109985e+00,1.20220947e+00,1.07803345e+00,  1.03039551e+00,1.12228394e+00,1.15420532e+00,1.16839600e+00,  1.16635132e+00,1.09207153e+00,1.02185059e+00,1.02304077e+00,  1.05673218e+00,1.10748291e+00,1.12551880e+00,1.13424683e+00,  1.12402344e+00,9.98855591e-01,6.08947754e-01,6.62109375e-01,  9.79278564e-01,1.06726074e+00,1.06997681e+00,1.06344604e+00,  1.04614258e+00,1.03915405e+00,1.03335571e+00,1.02282715e+00,  9.75143433e-01,8.83544922e-01,8.61328125e-01,8.83438110e-01,  8.99230957e-01,9.28405762e-01,9.47235107e-01,9.38980103e-01,  9.05380249e-01,8.92501831e-01,9.07135010e-01,8.90914917e-01,  8.93478394e-01,8.98208618e-01,8.96499634e-01,8.91693115e-01,  8.69735718e-01,7.83706665e-01,7.22442627e-01,7.22915649e-01,  6.84219360e-01,6.79168701e-01,7.13302612e-01,6.86233521e-01,  5.02243042e-01,3.52890015e-01,3.65608215e-01,3.76731873e-01,  3.91395569e-01,4.21310425e-01,4.70397949e-01,5.47729492e-01,  6.10351562e-01,6.29501343e-01,6.57226562e-01,6.93557739e-01,  7.12310791e-01,7.15469360e-01,7.07092285e-01,6.97235107e-01,  6.95816040e-01,6.93466187e-01,6.86935425e-01,6.79687500e-01,  6.70944214e-01,6.61605835e-01,6.52984619e-01,6.42639160e-01,  6.30737305e-01,6.14929199e-01,5.98281860e-01,5.85174561e-01,  5.81436157e-01,5.82748413e-01,5.76705933e-01,5.66070557e-01,  5.51086426e-01,5.26199341e-01,4.97215271e-01,4.64775085e-01,  3.98620605e-01,2.87216187e-01,1.98375702e-01,1.71844482e-01,  1.84570312e-01,2.14508057e-01,2.41535187e-01,2.26619720e-01,  2.42851257e-01,3.00392151e-01,3.63655090e-01,4.09606934e-01,  4.32624817e-01,4.35935974e-01,4.28756714e-01,4.25842285e-01,  4.37744141e-01,4.38880920e-01,4.23484802e-01,4.19960022e-01,  4.28741455e-01,4.38217163e-01,4.40475464e-01,4.42924500e-01,  4.48112488e-01,4.48074341e-01,4.41474915e-01,4.32090759e-01,  4.17312622e-01,3.89236450e-01,3.40148926e-01,2.93632507e-01,  3.04168701e-01,3.53057861e-01,3.78967285e-01,3.96141052e-01,  4.01969910e-01,3.95141602e-01,3.80020142e-01,3.60893250e-01,  3.38317871e-01,3.17657471e-01,3.03482056e-01,2.78968811e-01,  2.31224060e-01,2.19665527e-01,2.00122833e-01,1.26483917e-01,  4.92973328e-02,1.61991119e-02,1.01871490e-02,9.50646400e-03,  8.99791718e-03,9.43088531e-03,9.36436653e-03,1.00429058e-02,  1.14967823e-02,1.23016834e-02,1.39789581e-02,1.53157711e-02,  2.17814445e-02,3.03373337e-02,3.98607254e-02,5.16538620e-02,  5.35860062e-02,5.11198044e-02,5.83534241e-02,7.34844208e-02,  9.41410065e-02,1.25617981e-01,1.32331848e-01,1.10452652e-01,  9.93633270e-02,1.13712311e-01,1.29520416e-01,1.50535583e-01,  1.84215546e-01,2.14096069e-01,2.27180481e-01,2.36797333e-01,  2.51525879e-01,2.57301331e-01,2.62390137e-01,2.67341614e-01,  2.69279480e-01,2.69592285e-01,2.70713806e-01,2.73605347e-01,  2.73521423e-01,2.71018982e-01,2.67318726e-01,2.55187988e-01,  2.32051849e-01,2.22629547e-01,2.32921600e-01,2.44953156e-01,  2.48752594e-01,2.42919922e-01,2.25700378e-01,2.19242096e-01,  2.24899292e-01,2.36755371e-01,2.43247986e-01,2.45822906e-01,  2.44731903e-01,2.41664886e-01,2.37880707e-01,2.35595703e-01,  2.33871460e-01,2.32059479e-01,2.27588654e-01,2.20207214e-01,  2.16594696e-01,2.11692810e-01,2.06432343e-01,2.02106476e-01,  1.97853088e-01,1.92653656e-01,1.89334869e-01],
        [1.09479904e-01,1.15451813e-01,1.25701904e-01,1.54850006e-01,  1.70619965e-01,1.82090759e-01,1.88369751e-01,1.95194244e-01,  2.22175598e-01,2.64465332e-01,3.65447998e-01,4.26902771e-01,  4.65324402e-01,5.02746582e-01,5.21774292e-01,5.24368286e-01,  5.33676147e-01,5.94741821e-01,6.64962769e-01,7.33825684e-01,  7.93655396e-01,8.31420898e-01,8.62258911e-01,8.81454468e-01,  9.00466919e-01,9.28329468e-01,9.52560425e-01,9.39529419e-01,  9.64492798e-01,9.94384766e-01,9.89257812e-01,9.99816895e-01,  1.01083374e+00,9.87335205e-01,9.96994019e-01,1.02001953e+00,  1.03686523e+00,1.04867554e+00,1.04800415e+00,1.05459595e+00,  1.06057739e+00,1.05511475e+00,1.03530884e+00,1.01943970e+00,  1.00280762e+00,9.99389648e-01,1.02734375e+00,1.04486084e+00,  1.03012085e+00,1.03414917e+00,1.03390503e+00,1.04403687e+00,  1.04910278e+00,1.05429077e+00,1.06512451e+00,1.04421997e+00,  1.04080200e+00,1.08316040e+00,1.11221313e+00,1.11923218e+00,  1.11538696e+00,1.10247803e+00,1.14419556e+00,1.17749023e+00,  1.18453979e+00,1.18753052e+00,1.16928101e+00,1.04614258e+00,  9.97161865e-01,1.08972168e+00,1.12341309e+00,1.13818359e+00,  1.13754272e+00,1.06179810e+00,9.89181519e-01,9.91470337e-01,  1.02645874e+00,1.08041382e+00,1.10144043e+00,1.11038208e+00,  1.10134888e+00,9.78164673e-01,5.89279175e-01,6.41006470e-01,  9.56817627e-01,1.04556274e+00,1.05145264e+00,1.04345703e+00,  1.02526855e+00,1.02020264e+00,1.01358032e+00,1.00460815e+00,  9.56054688e-01,8.62106323e-01,8.41094971e-01,8.63220215e-01,  8.79241943e-01,9.11041260e-01,9.29595947e-01,9.23309326e-01,  8.90686035e-01,8.75961304e-01,8.94470215e-01,8.76419067e-01,  8.79852295e-01,8.88626099e-01,8.83850098e-01,8.77807617e-01,  8.60382080e-01,7.66540527e-01,7.07229614e-01,7.10556030e-01,  6.65100098e-01,6.63650513e-01,6.99462891e-01,6.66885376e-01,  4.81910706e-01,3.32687378e-01,3.44963074e-01,3.56338501e-01,  3.71322632e-01,4.01611328e-01,4.51370239e-01,5.30456543e-01,  5.94833374e-01,6.14776611e-01,6.43936157e-01,6.82281494e-01,  7.02529907e-01,7.06497192e-01,6.98410034e-01,6.88842773e-01,  6.87515259e-01,6.85485840e-01,6.78894043e-01,6.71829224e-01,  6.63330078e-01,6.53900146e-01,6.45202637e-01,6.34780884e-01,  6.22482300e-01,6.06567383e-01,5.89523315e-01,5.76431274e-01,  5.72982788e-01,5.74707031e-01,5.68939209e-01,5.58074951e-01,  5.42663574e-01,5.16815186e-01,4.86434937e-01,4.52949524e-01,  3.86100769e-01,2.74291992e-01,1.85573578e-01,1.58950806e-01,  1.71512604e-01,2.01347351e-01,2.27848053e-01,2.13016510e-01,  2.29358673e-01,2.87261963e-01,3.51631165e-01,3.98559570e-01,  4.22386169e-01,4.26452637e-01,4.19403076e-01,4.16618347e-01,  4.29115295e-01,4.30618286e-01,4.15161133e-01,4.11659241e-01,  4.20837402e-01,4.30641174e-01,4.33219910e-01,4.36363220e-01,  4.42062378e-01,4.42382812e-01,4.35813904e-01,4.26139832e-01,  4.10949707e-01,3.81759644e-01,3.31275940e-01,2.83798218e-01,  2.94860840e-01,3.45130920e-01,3.72550964e-01,3.90571594e-01,  3.96507263e-01,3.89381409e-01,3.73954773e-01,3.54309082e-01,  3.31092834e-01,3.09875488e-01,2.95333862e-01,2.70256042e-01,  2.21759796e-01,2.10311890e-01,1.91215515e-01,1.18936539e-01,  4.49876785e-02,1.47480965e-02,9.66763496e-03,9.06872749e-03,  8.43214989e-03,8.54063034e-03,8.56304169e-03,9.24420357e-03,  1.03945732e-02,1.11498833e-02,1.25181675e-02,1.35552883e-02,  1.93228722e-02,2.67992020e-02,3.54156494e-02,4.64992523e-02,  4.85811234e-02,4.60243225e-02,5.24940491e-02,6.70013428e-02,  8.70265961e-02,1.17475510e-01,1.24197006e-01,1.03010178e-01,  9.23900604e-02,1.06639862e-01,1.22383118e-01,1.43238068e-01,  1.77307129e-01,2.07874298e-01,2.21641541e-01,2.31716156e-01,  2.46856689e-01,2.53013611e-01,2.58834839e-01,2.64060974e-01,  2.66159058e-01,2.66555786e-01,2.67738342e-01,2.70828247e-01,  2.70980835e-01,2.68585205e-01,2.64976501e-01,2.52563477e-01,  2.28942871e-01,2.19230652e-01,2.29804993e-01,2.42115021e-01,  2.46105194e-01,2.40230560e-01,2.22454071e-01,2.15953827e-01,  2.21897125e-01,2.33936310e-01,2.40600586e-01,2.43034363e-01,  2.41874695e-01,2.38708496e-01,2.34821320e-01,2.32589722e-01,  2.30587006e-01,2.28687286e-01,2.23934174e-01,2.16060638e-01,  2.12863922e-01,2.08248138e-01,2.03113556e-01,1.97776794e-01,  1.93019867e-01,1.88758850e-01,1.85451508e-01],
        [9.21611786e-02,9.76009369e-02,1.08985901e-01,1.32545471e-01,  1.47315979e-01,1.57943726e-01,1.63074493e-01,1.70673370e-01,  1.93603516e-01,2.32177734e-01,3.21731567e-01,3.76731873e-01,  4.13192749e-01,4.47319031e-01,4.66682434e-01,4.70825195e-01,  4.80758667e-01,5.38406372e-01,6.03103638e-01,6.68167114e-01,  7.24884033e-01,7.61184692e-01,7.92251587e-01,8.11508179e-01,  8.31314087e-01,8.58215332e-01,8.82202148e-01,8.73977661e-01,  8.98986816e-01,9.28634644e-01,9.25262451e-01,9.35363770e-01,  9.47891235e-01,9.27505493e-01,9.37316895e-01,9.59960938e-01,  9.75494385e-01,9.88052368e-01,9.88693237e-01,9.94842529e-01,  1.00219727e+00,9.97177124e-01,9.78286743e-01,9.62738037e-01,  9.44839478e-01,9.42306519e-01,9.70932007e-01,9.89181519e-01,  9.76791382e-01,9.80026245e-01,9.81460571e-01,9.92614746e-01,  9.98458862e-01,1.00668335e+00,1.01739502e+00,9.97283936e-01,  9.95666504e-01,1.03872681e+00,1.07043457e+00,1.07821655e+00,  1.07476807e+00,1.06558228e+00,1.10723877e+00,1.14151001e+00,  1.15014648e+00,1.15277100e+00,1.13690186e+00,1.01425171e+00,  9.64065552e-01,1.05902100e+00,1.09277344e+00,1.10922241e+00,  1.11120605e+00,1.03335571e+00,9.61166382e-01,9.63195801e-01,  9.98779297e-01,1.05673218e+00,1.07812500e+00,1.08837891e+00,  1.08187866e+00,9.57046509e-01,5.71701050e-01,6.22070312e-01,  9.37438965e-01,1.02966309e+00,1.03314209e+00,1.02462769e+00,  1.00936890e+00,1.00088501e+00,9.96948242e-01,9.88204956e-01,  9.34570312e-01,8.42849731e-01,8.21395874e-01,8.41583252e-01,  8.65203857e-01,8.93966675e-01,9.15695190e-01,9.14642334e-01,  8.76678467e-01,8.66958618e-01,8.85971069e-01,8.63952637e-01,  8.76617432e-01,8.81362915e-01,8.72940063e-01,8.74938965e-01,  8.46054077e-01,7.55065918e-01,6.98318481e-01,6.88446045e-01,  6.56860352e-01,6.55364990e-01,6.79702759e-01,6.56921387e-01,  4.70230103e-01,3.18138123e-01,3.30337524e-01,3.41720581e-01,  3.56613159e-01,3.87199402e-01,4.37469482e-01,5.17517090e-01,  5.83435059e-01,6.04278564e-01,6.34613037e-01,6.74606323e-01,  6.96212769e-01,7.00759888e-01,6.92733765e-01,6.83258057e-01,  6.82174683e-01,6.79992676e-01,6.73629761e-01,6.66717529e-01,  6.58233643e-01,6.49002075e-01,6.40289307e-01,6.29577637e-01,  6.16958618e-01,6.00479126e-01,5.82977295e-01,5.69427490e-01,  5.66619873e-01,5.69000244e-01,5.63476562e-01,5.52978516e-01,  5.37384033e-01,5.10635376e-01,4.79179382e-01,4.44694519e-01,  3.77075195e-01,2.64900208e-01,1.76269531e-01,1.49539948e-01,  1.61983490e-01,1.92001343e-01,2.17956543e-01,2.03189850e-01,  2.19486237e-01,2.77320862e-01,3.42567444e-01,3.90548706e-01,  4.15039062e-01,4.19776917e-01,4.13002014e-01,4.10224915e-01,  4.23095703e-01,4.24942017e-01,4.09294128e-01,4.05914307e-01,  4.15435791e-01,4.25376892e-01,4.28245544e-01,4.31938171e-01,  4.38140869e-01,4.38819885e-01,4.32151794e-01,4.21943665e-01,  4.05929565e-01,3.75450134e-01,3.23196411e-01,2.74688721e-01,  2.86392212e-01,3.38417053e-01,3.67904663e-01,3.86886597e-01,  3.92913818e-01,3.85742188e-01,3.70018005e-01,3.49929810e-01,  3.26385498e-01,3.04679871e-01,2.89863586e-01,2.64511108e-01,  2.15347290e-01,2.03823090e-01,1.84803009e-01,1.13368988e-01,  4.18882370e-02,1.37612820e-02,9.18459892e-03,8.67605209e-03,  8.15224648e-03,8.00251961e-03,8.20851326e-03,8.67462158e-03,  9.49716568e-03,1.01778507e-02,1.13017559e-02,1.23875141e-02,  1.77917480e-02,2.45990753e-02,3.25183868e-02,4.30812836e-02,  4.49714661e-02,4.22039032e-02,4.82320786e-02,6.22615814e-02,  8.20865631e-02,1.11818314e-01,1.18692398e-01,9.79518890e-02,  8.74633789e-02,1.01593018e-01,1.17099762e-01,1.38038635e-01,  1.72508240e-01,2.03674316e-01,2.18124390e-01,2.28755951e-01,  2.44213104e-01,2.50740051e-01,2.56782532e-01,2.62207031e-01,  2.64396667e-01,2.64762878e-01,2.66349792e-01,2.69744873e-01,  2.70050049e-01,2.67707825e-01,2.64030457e-01,2.51075745e-01,  2.26608276e-01,2.16789246e-01,2.27924347e-01,2.40932465e-01,  2.45296478e-01,2.38777161e-01,2.20310211e-01,2.13672638e-01,  2.19760895e-01,2.32414246e-01,2.39337921e-01,2.41817474e-01,  2.40455627e-01,2.36919403e-01,2.32975006e-01,2.30567932e-01,  2.28733063e-01,2.26730347e-01,2.21218109e-01,2.13081360e-01,  2.09308624e-01,2.05345154e-01,2.00481415e-01,1.95724487e-01,  1.91734314e-01,1.85420990e-01,1.81758881e-01],
        [7.69405365e-02,8.16078186e-02,9.06143188e-02,1.10807419e-01,  1.24307632e-01,1.32446289e-01,1.38050079e-01,1.45503998e-01,  1.65321350e-01,1.99256897e-01,2.76977539e-01,3.25653076e-01,  3.58810425e-01,3.90083313e-01,4.08790588e-01,4.14237976e-01,  4.24766541e-01,4.77607727e-01,5.37322998e-01,5.97320557e-01,  6.50253296e-01,6.85424805e-01,7.15759277e-01,7.35397339e-01,  7.55584717e-01,7.81341553e-01,8.05679321e-01,8.01055908e-01,  8.25912476e-01,8.55133057e-01,8.53210449e-01,8.63677979e-01,  8.77029419e-01,8.59832764e-01,8.70407104e-01,8.91510010e-01,  9.06250000e-01,9.19540405e-01,9.21127319e-01,9.27642822e-01,  9.35485840e-01,9.31152344e-01,9.13253784e-01,8.97964478e-01,  8.80599976e-01,8.77761841e-01,9.06585693e-01,9.26696777e-01,  9.15420532e-01,9.19021606e-01,9.21539307e-01,9.33471680e-01,  9.40856934e-01,9.50363159e-01,9.61364746e-01,9.42977905e-01,  9.42260742e-01,9.86160278e-01,1.01965332e+00,1.02874756e+00,  1.02691650e+00,1.02001953e+00,1.06250000e+00,1.09716797e+00,  1.10665894e+00,1.11108398e+00,1.09646606e+00,9.74105835e-01,  9.23690796e-01,1.01950073e+00,1.05477905e+00,1.07278442e+00,  1.07519531e+00,9.96917725e-01,9.24926758e-01,9.27032471e-01,  9.63653564e-01,1.02383423e+00,1.04769897e+00,1.06015015e+00,  1.05386353e+00,9.30770874e-01,5.48858643e-01,5.95687866e-01,  9.12063599e-01,1.00555420e+00,1.00845337e+00,1.00030518e+00,  9.86053467e-01,9.77722168e-01,9.73449707e-01,9.65408325e-01,  9.11773682e-01,8.18374634e-01,7.96447754e-01,8.20678711e-01,  8.43139648e-01,8.73291016e-01,8.99246216e-01,8.96072388e-01,  8.58474731e-01,8.48770142e-01,8.66607666e-01,8.48526001e-01,  8.57162476e-01,8.60305786e-01,8.57467651e-01,8.61389160e-01,  8.29544067e-01,7.33810425e-01,6.74194336e-01,6.68106079e-01,  6.34414673e-01,6.35604858e-01,6.61575317e-01,6.38153076e-01,  4.50241089e-01,2.98645020e-01,3.10234070e-01,3.21777344e-01,  3.36730957e-01,3.67637634e-01,4.18136597e-01,4.99633789e-01,  5.67214966e-01,5.88668823e-01,6.20132446e-01,6.62078857e-01,  6.84967041e-01,6.90322876e-01,6.82800293e-01,6.73690796e-01,  6.72714233e-01,6.70913696e-01,6.64794922e-01,6.57928467e-01,  6.49642944e-01,6.40274048e-01,6.31378174e-01,6.20590210e-01,  6.07543945e-01,5.90667725e-01,5.72616577e-01,5.59173584e-01,  5.56747437e-01,5.59829712e-01,5.54870605e-01,5.44204712e-01,  5.28198242e-01,5.00549316e-01,4.67559814e-01,4.32090759e-01,  3.64295959e-01,2.52296448e-01,1.64047241e-01,1.37561798e-01,  1.49848938e-01,1.79370880e-01,2.04456329e-01,1.89891815e-01,  2.06127167e-01,2.63946533e-01,3.29978943e-01,3.79013062e-01,  4.04411316e-01,4.09782410e-01,4.03190613e-01,4.00352478e-01,  4.13528442e-01,4.15809631e-01,4.00238037e-01,3.96980286e-01,  4.06845093e-01,4.17167664e-01,4.20410156e-01,4.24484253e-01,  4.31419373e-01,4.32128906e-01,4.25399780e-01,4.14985657e-01,  3.98139954e-01,3.66546631e-01,3.12568665e-01,2.63168335e-01,  2.75360107e-01,3.29017639e-01,3.60450745e-01,3.80386353e-01,  3.86611938e-01,3.79386902e-01,3.63456726e-01,3.43078613e-01,  3.18870544e-01,2.96707153e-01,2.81478882e-01,2.55668640e-01,  2.06054688e-01,1.94564819e-01,1.76013947e-01,1.06428146e-01,  3.86495590e-02,1.32868290e-02,9.34886932e-03,8.82148743e-03,  8.14294815e-03,7.78901577e-03,7.99298286e-03,8.54563713e-03,  9.23418999e-03,9.83905792e-03,1.08582973e-02,1.16312504e-02,  1.62568092e-02,2.20608711e-02,2.91328430e-02,3.89070511e-02,  4.07905579e-02,3.78866196e-02,4.32996750e-02,5.67855835e-02,  7.57427216e-02,1.04465485e-01,1.11227036e-01,9.12895203e-02,  8.12797546e-02,9.50794220e-02,1.10303879e-01,1.31046295e-01,  1.65790558e-01,1.97765350e-01,2.12909698e-01,2.23983765e-01,  2.39738464e-01,2.46498108e-01,2.53051758e-01,2.58766174e-01,  2.61100769e-01,2.61642456e-01,2.63496399e-01,2.67311096e-01,  2.67875671e-01,2.65655518e-01,2.61894226e-01,2.48466492e-01,  2.23075867e-01,2.13218689e-01,2.24891663e-01,2.38822937e-01,  2.43526459e-01,2.36755371e-01,2.17163086e-01,2.10281372e-01,  2.16686249e-01,2.29827881e-01,2.37358093e-01,2.39841461e-01,  2.38559723e-01,2.34909058e-01,2.30705261e-01,2.27813721e-01,  2.25982666e-01,2.24830627e-01,2.20615387e-01,2.13157654e-01,  2.10159302e-01,2.06367493e-01,2.01259613e-01,1.95472717e-01,  1.90361023e-01,1.85443878e-01,1.82140350e-01],
        [6.45217896e-02,6.81781769e-02,7.40718842e-02,9.16252136e-02,  1.01026535e-01,1.09167099e-01,1.15177155e-01,1.20426178e-01,  1.38351440e-01,1.66767120e-01,2.32509613e-01,2.74932861e-01,  3.03909302e-01,3.32969666e-01,3.50151062e-01,3.56155396e-01,  3.67607117e-01,4.14276123e-01,4.68627930e-01,5.23117065e-01,  5.71716309e-01,6.05758667e-01,6.33773804e-01,6.53915405e-01,  6.73797607e-01,6.97814941e-01,7.22366333e-01,7.20153809e-01,  7.45315552e-01,7.74139404e-01,7.73117065e-01,7.84164429e-01,  7.97882080e-01,7.84530640e-01,7.95516968e-01,8.14422607e-01,  8.28964233e-01,8.41964722e-01,8.44131470e-01,8.51593018e-01,  8.59039307e-01,8.56262207e-01,8.39828491e-01,8.24020386e-01,  8.07174683e-01,8.03985596e-01,8.33496094e-01,8.54782104e-01,  8.44085693e-01,8.48876953e-01,8.52005005e-01,8.64685059e-01,  8.74908447e-01,8.85009766e-01,8.97811890e-01,8.80172729e-01,  8.79318237e-01,9.26071167e-01,9.60662842e-01,9.70687866e-01,  9.71130371e-01,9.66247559e-01,1.00976562e+00,1.04498291e+00,  1.05593872e+00,1.06268311e+00,1.04794312e+00,9.27322388e-01,  8.77593994e-01,9.72274780e-01,1.01065063e+00,1.03027344e+00,  1.03283691e+00,9.55413818e-01,8.81179810e-01,8.83239746e-01,  9.22653198e-01,9.83917236e-01,1.01229858e+00,1.02578735e+00,  1.01901245e+00,9.01458740e-01,5.24139404e-01,5.67321777e-01,  8.82369995e-01,9.75250244e-01,9.79934692e-01,9.72061157e-01,  9.54711914e-01,9.49874878e-01,9.44229126e-01,9.34158325e-01,  8.84246826e-01,7.88619995e-01,7.68096924e-01,7.93563843e-01,  8.13796997e-01,8.50585938e-01,8.74816895e-01,8.70697021e-01,  8.40957642e-01,8.29788208e-01,8.46969604e-01,8.34365845e-01,  8.38790894e-01,8.45199585e-01,8.40866089e-01,8.36669922e-01,  8.13537598e-01,7.11746216e-01,6.46926880e-01,6.53335571e-01,  6.09619141e-01,6.02752686e-01,6.40914917e-01,6.06521606e-01,  4.24331665e-01,2.77000427e-01,2.88154602e-01,2.99789429e-01,  3.14903259e-01,3.45848083e-01,3.96614075e-01,4.79133606e-01,  5.48080444e-01,5.70419312e-01,6.03698730e-01,6.47918701e-01,  6.72485352e-01,6.78787231e-01,6.71859741e-01,6.62857056e-01,  6.62063599e-01,6.60446167e-01,6.54464722e-01,6.47674561e-01,  6.39541626e-01,6.30371094e-01,6.21459961e-01,6.10427856e-01,  5.97045898e-01,5.79299927e-01,5.60745239e-01,5.47302246e-01,  5.45288086e-01,5.49194336e-01,5.44723511e-01,5.34027100e-01,  5.17791748e-01,4.89143372e-01,4.54589844e-01,4.17938232e-01,  3.49876404e-01,2.38212585e-01,1.50489807e-01,1.24235153e-01,  1.36318207e-01,1.65214539e-01,1.89453125e-01,1.75369263e-01,  1.91558838e-01,2.49038696e-01,3.15803528e-01,3.65638733e-01,  3.92105103e-01,3.98315430e-01,3.91860962e-01,3.89083862e-01,  4.02839661e-01,4.05601501e-01,3.90029907e-01,3.86970520e-01,  3.97239685e-01,4.07966614e-01,4.11621094e-01,4.16198730e-01,  4.23866272e-01,4.25056458e-01,4.18228149e-01,4.07241821e-01,  3.89541626e-01,3.56414795e-01,3.00575256e-01,2.50137329e-01,  2.62954712e-01,3.18626404e-01,3.52195740e-01,3.73397827e-01,  3.79905701e-01,3.72413635e-01,3.56124878e-01,3.35121155e-01,  3.10279846e-01,2.87666321e-01,2.72140503e-01,2.45994568e-01,  1.95873260e-01,1.84494019e-01,1.66278839e-01,9.85927582e-02,  3.47366333e-02,1.20575428e-02,8.88180733e-03,8.47315788e-03,  7.75611401e-03,7.35676289e-03,7.62319565e-03,8.09001923e-03,  8.67581367e-03,9.05561447e-03,9.82999802e-03,1.04799271e-02,  1.44021511e-02,1.93672180e-02,2.54592896e-02,3.43179703e-02,  3.62310410e-02,3.31926346e-02,3.78561020e-02,5.05323410e-02,  6.86798096e-02,9.60521698e-02,1.02863312e-01,8.38603973e-02,  7.43350983e-02,8.78505707e-02,1.02621078e-01,1.23199463e-01,  1.58168793e-01,1.90799713e-01,2.06928253e-01,2.18650818e-01,  2.34897614e-01,2.42019653e-01,2.49118805e-01,2.55096436e-01,  2.57461548e-01,2.58155823e-01,2.60337830e-01,2.64533997e-01,  2.65441895e-01,2.63458252e-01,2.59704590e-01,2.45727539e-01,  2.19257355e-01,2.09209442e-01,2.21359253e-01,2.36034393e-01,  2.41226196e-01,2.33978271e-01,2.13768005e-01,2.06691742e-01,  2.13504791e-01,2.27027893e-01,2.34687805e-01,2.37312317e-01,  2.35679626e-01,2.31975555e-01,2.27573395e-01,2.25067139e-01,  2.23098755e-01,2.21561432e-01,2.17151642e-01,2.09342957e-01,  2.06943512e-01,2.03540802e-01,1.98654175e-01,1.94049835e-01,  1.88644409e-01,1.83483124e-01,1.81133270e-01],
        [5.19809723e-02,5.45253754e-02,6.09703064e-02,7.39879608e-02,  8.20255280e-02,8.90407562e-02,9.25903320e-02,9.81540680e-02,  1.12379074e-01,1.36104584e-01,1.91215515e-01,2.26570129e-01,  2.52372742e-01,2.77145386e-01,2.92968750e-01,2.99774170e-01,  3.10653687e-01,3.52500916e-01,4.00321960e-01,4.48852539e-01,  4.93415833e-01,5.24536133e-01,5.51788330e-01,5.71624756e-01,  5.90606689e-01,6.13540649e-01,6.37145996e-01,6.38656616e-01,  6.63574219e-01,6.90719604e-01,6.91375732e-01,7.02255249e-01,  7.17025757e-01,7.07031250e-01,7.18002319e-01,7.36480713e-01,  7.49969482e-01,7.62741089e-01,7.66677856e-01,7.74215698e-01,  7.82440186e-01,7.80456543e-01,7.65167236e-01,7.50564575e-01,  7.33154297e-01,7.29858398e-01,7.59811401e-01,7.82089233e-01,  7.73361206e-01,7.77908325e-01,7.82501221e-01,7.96798706e-01,  8.08013916e-01,8.19915771e-01,8.32412720e-01,8.15246582e-01,  8.16146851e-01,8.63403320e-01,8.99871826e-01,9.11743164e-01,  9.12811279e-01,9.11468506e-01,9.54956055e-01,9.90524292e-01,  1.00332642e+00,1.01007080e+00,9.97604370e-01,8.79043579e-01,  8.28750610e-01,9.24682617e-01,9.63241577e-01,9.84466553e-01,  9.89349365e-01,9.09881592e-01,8.36166382e-01,8.39324951e-01,  8.78768921e-01,9.43832397e-01,9.73800659e-01,9.88220215e-01,  9.84024048e-01,8.68240356e-01,4.97207642e-01,5.36987305e-01,  8.48220825e-01,9.43359375e-01,9.49508667e-01,9.41360474e-01,  9.24667358e-01,9.18823242e-01,9.14398193e-01,9.08309937e-01,  8.54263306e-01,7.56591797e-01,7.37365723e-01,7.62939453e-01,  7.86544800e-01,8.23074341e-01,8.48937988e-01,8.50402832e-01,  8.17321777e-01,8.08120728e-01,8.29284668e-01,8.13537598e-01,  8.21334839e-01,8.24584961e-01,8.18115234e-01,8.18939209e-01,  7.90054321e-01,6.86370850e-01,6.27227783e-01,6.24481201e-01,  5.84899902e-01,5.81237793e-01,6.11175537e-01,5.85128784e-01,  4.02252197e-01,2.53967285e-01,2.64366150e-01,2.75802612e-01,  2.91038513e-01,3.21975708e-01,3.73023987e-01,4.56443787e-01,  5.26626587e-01,5.49835205e-01,5.84365845e-01,6.30554199e-01,  6.56951904e-01,6.64154053e-01,6.57440186e-01,6.48773193e-01,  6.48239136e-01,6.46987915e-01,6.41296387e-01,6.34719849e-01,  6.26693726e-01,6.17614746e-01,6.08459473e-01,5.97091675e-01,  5.83236694e-01,5.64987183e-01,5.45745850e-01,5.32302856e-01,  5.31066895e-01,5.35964966e-01,5.32119751e-01,5.21530151e-01,  5.05050659e-01,4.75379944e-01,4.39025879e-01,4.01382446e-01,  3.33412170e-01,2.22625732e-01,1.36436462e-01,1.10683441e-01,  1.22303009e-01,1.50501251e-01,1.73698425e-01,1.59828186e-01,  1.75693512e-01,2.32505798e-01,2.99522400e-01,3.50296021e-01,  3.77746582e-01,3.84803772e-01,3.78669739e-01,3.75999451e-01,  3.90243530e-01,3.93463135e-01,3.77838135e-01,3.74786377e-01,  3.85322571e-01,3.96369934e-01,4.00291443e-01,4.05593872e-01,  4.14016724e-01,4.15534973e-01,4.08645630e-01,3.97094727e-01,  3.78234863e-01,3.43521118e-01,2.85789490e-01,2.34394073e-01,  2.47848511e-01,3.05397034e-01,3.41362000e-01,3.64006042e-01,  3.70849609e-01,3.63189697e-01,3.46885681e-01,3.25584412e-01,  3.00186157e-01,2.77000427e-01,2.61085510e-01,2.34580994e-01,  1.84173584e-01,1.72809601e-01,1.55246735e-01,8.98971558e-02,  3.02228928e-02,1.02367401e-02,7.61532784e-03,7.31313229e-03,  6.52003288e-03,5.91671467e-03,6.28018379e-03,6.81769848e-03,  7.21788406e-03,7.53176212e-03,8.05425644e-03,8.65721703e-03,  1.18935108e-02,1.60012245e-02,2.12087631e-02,2.92310715e-02,  3.10530663e-02,2.78592110e-02,3.18822861e-02,4.36697006e-02,  6.07423782e-02,8.67481232e-02,9.34543610e-02,7.55176544e-02,  6.66103363e-02,7.97500610e-02,9.39998627e-02,1.14212036e-01,  1.49074554e-01,1.82193756e-01,1.99069977e-01,2.11456299e-01,  2.28061676e-01,2.35469818e-01,2.43141174e-01,2.49427795e-01,  2.51785278e-01,2.52685547e-01,2.55012512e-01,2.59674072e-01,  2.60910034e-01,2.58972168e-01,2.55256653e-01,2.40459442e-01,  2.12917328e-01,2.02865601e-01,2.15637207e-01,2.31170654e-01,  2.36560822e-01,2.28809357e-01,2.07469940e-01,1.99871063e-01,  2.07027435e-01,2.21302032e-01,2.29732513e-01,2.32215881e-01,  2.30052948e-01,2.25849152e-01,2.20623016e-01,2.17739105e-01,  2.15152740e-01,2.13211060e-01,2.08053589e-01,1.99211121e-01,  1.96647644e-01,1.91776276e-01,1.86874390e-01,1.81591034e-01,  1.75807953e-01,1.70272827e-01,1.66393280e-01],
        [3.93762589e-02,4.36258316e-02,4.79059219e-02,5.78680038e-02,  6.54144287e-02,6.92996979e-02,7.36846924e-02,7.77091980e-02,  8.86955261e-02,1.08242035e-01,1.51325226e-01,1.80980682e-01,  2.02159882e-01,2.22946167e-01,2.37468719e-01,2.43896484e-01,  2.54791260e-01,2.90634155e-01,3.31535339e-01,3.74519348e-01,  4.13421631e-01,4.42153931e-01,4.67399597e-01,4.86152649e-01,  5.04928589e-01,5.25588989e-01,5.48675537e-01,5.53604126e-01,  5.77484131e-01,6.03759766e-01,6.05606079e-01,6.16531372e-01,  6.32049561e-01,6.24923706e-01,6.36627197e-01,6.53305054e-01,  6.65557861e-01,6.79122925e-01,6.83822632e-01,6.91772461e-01,  7.00531006e-01,6.99386597e-01,6.85852051e-01,6.71432495e-01,  6.54739380e-01,6.51626587e-01,6.81213379e-01,7.05520630e-01,  6.98257446e-01,7.03201294e-01,7.09426880e-01,7.23617554e-01,  7.36572266e-01,7.50122070e-01,7.62603760e-01,7.47253418e-01,  7.48397827e-01,7.97073364e-01,8.35845947e-01,8.48098755e-01,  8.51547241e-01,8.52966309e-01,8.96148682e-01,9.32968140e-01,  9.46472168e-01,9.55581665e-01,9.45373535e-01,8.27499390e-01,  7.77664185e-01,8.73916626e-01,9.13574219e-01,9.36965942e-01,  9.42276001e-01,8.64166260e-01,7.91610718e-01,7.93319702e-01,  8.34899902e-01,9.02465820e-01,9.35073853e-01,9.52667236e-01,  9.47753906e-01,8.34625244e-01,4.72244263e-01,5.06027222e-01,  8.16085815e-01,9.13482666e-01,9.18609619e-01,9.12414551e-01,  8.94332886e-01,8.88107300e-01,8.86245728e-01,8.77059937e-01,  8.23501587e-01,7.27233887e-01,7.06832886e-01,7.35519409e-01,  7.59460449e-01,7.95745850e-01,8.26980591e-01,8.27133179e-01,  7.97195435e-01,7.92831421e-01,8.07357788e-01,7.93426514e-01,  8.02093506e-01,8.03680420e-01,8.04336548e-01,7.98400879e-01,  7.66265869e-01,6.69891357e-01,5.99807739e-01,6.02066040e-01,  5.65948486e-01,5.53787231e-01,5.95382690e-01,5.70098877e-01,  3.80607605e-01,2.33837128e-01,2.43675232e-01,2.55043030e-01,  2.70027161e-01,3.00842285e-01,3.51928711e-01,4.35890198e-01,  5.07293701e-01,5.31402588e-01,5.67565918e-01,6.16149902e-01,  6.44531250e-01,6.52786255e-01,6.46530151e-01,6.38305664e-01,  6.37924194e-01,6.36749268e-01,6.30950928e-01,6.24465942e-01,  6.16668701e-01,6.07467651e-01,5.98419189e-01,5.86853027e-01,  5.72219849e-01,5.53115845e-01,5.32958984e-01,5.18981934e-01,  5.18890381e-01,5.24826050e-01,5.21652222e-01,5.11505127e-01,  4.94827271e-01,4.64050293e-01,4.26269531e-01,3.87535095e-01,  3.19137573e-01,2.09247589e-01,1.24317169e-01,9.87911224e-02,  1.10013962e-01,1.37451172e-01,1.59328461e-01,1.46060944e-01,  1.61659241e-01,2.17750549e-01,2.85324097e-01,3.37020874e-01,  3.65417480e-01,3.73435974e-01,3.67576599e-01,3.64814758e-01,  3.79493713e-01,3.83331299e-01,3.67675781e-01,3.64776611e-01,  3.75823975e-01,3.87176514e-01,3.91456604e-01,3.97430420e-01,  4.06356812e-01,4.08126831e-01,4.01054382e-01,3.88710022e-01,  3.68598938e-01,3.31771851e-01,2.71446228e-01,2.19341278e-01,  2.33455658e-01,2.93182373e-01,3.32199097e-01,3.56552124e-01,  3.63883972e-01,3.56323242e-01,3.39767456e-01,3.17901611e-01,  2.91793823e-01,2.68066406e-01,2.51831055e-01,2.25086212e-01,  1.74263000e-01,1.62811279e-01,1.45767212e-01,8.28666687e-02,  2.72212029e-02,9.68217850e-03,7.50327110e-03,7.17532635e-03,  6.34849072e-03,5.71036339e-03,6.05881214e-03,6.51717186e-03,  6.93559647e-03,7.29560852e-03,7.65514374e-03,8.04281235e-03,  1.08404160e-02,1.40957832e-02,1.85074806e-02,2.57096291e-02,  2.72717476e-02,2.41961479e-02,2.75807381e-02,3.84626389e-02,  5.45749664e-02,7.90824890e-02,8.56742859e-02,6.88400269e-02,  6.05335236e-02,7.31811523e-02,8.68740082e-02,1.06670380e-01,  1.41685486e-01,1.75380707e-01,1.93275452e-01,2.06302643e-01,  2.23247528e-01,2.31262207e-01,2.39471436e-01,2.45971680e-01,  2.48371124e-01,2.49370575e-01,2.52090454e-01,2.57362366e-01,  2.59048462e-01,2.57293701e-01,2.53334045e-01,2.37850189e-01,  2.09014893e-01,1.98535919e-01,2.12196350e-01,2.28725433e-01,  2.34794617e-01,2.26428986e-01,2.03723907e-01,1.95960999e-01,  2.03487396e-01,2.18791962e-01,2.27523804e-01,2.30007172e-01,  2.27714539e-01,2.23041534e-01,2.17781067e-01,2.14630127e-01,  2.12295532e-01,2.11078644e-01,2.05814362e-01,1.96746826e-01,  1.93244934e-01,1.89327240e-01,1.84425354e-01,1.78768158e-01,  1.73454285e-01,1.67648315e-01,1.64936066e-01],
        [3.19147110e-02,3.49445343e-02,3.67975235e-02,4.34741974e-02,  4.91743088e-02,5.16510010e-02,5.53264618e-02,5.78947067e-02,  6.61373138e-02,8.11023712e-02,1.13225937e-01,1.36089325e-01,  1.52606964e-01,1.68991089e-01,1.80816650e-01,1.86634064e-01,  1.96304321e-01,2.25196838e-01,2.58056641e-01,2.93197632e-01,  3.25767517e-01,3.50837708e-01,3.72703552e-01,3.90007019e-01,  4.07241821e-01,4.25041199e-01,4.46411133e-01,4.53506470e-01,  4.75639343e-01,4.99893188e-01,5.03082275e-01,5.13534546e-01,  5.28884888e-01,5.25482178e-01,5.37078857e-01,5.51834106e-01,  5.63110352e-01,5.76431274e-01,5.81985474e-01,5.90148926e-01,  5.99014282e-01,5.99334717e-01,5.88150024e-01,5.74630737e-01,  5.58532715e-01,5.55801392e-01,5.84609985e-01,6.09329224e-01,  6.03897095e-01,6.09085083e-01,6.16836548e-01,6.31576538e-01,  6.45797729e-01,6.60095215e-01,6.72286987e-01,6.58096313e-01,  6.59973145e-01,7.08724976e-01,7.48794556e-01,7.61810303e-01,  7.67288208e-01,7.71179199e-01,8.13385010e-01,8.49884033e-01,  8.65005493e-01,8.75656128e-01,8.67233276e-01,7.54089355e-01,  7.05490112e-01,7.99758911e-01,8.40454102e-01,8.65188599e-01,  8.71810913e-01,7.95837402e-01,7.24197388e-01,7.25509644e-01,  7.68142700e-01,8.37066650e-01,8.71749878e-01,8.91082764e-01,  8.88183594e-01,7.81387329e-01,4.33692932e-01,4.62699890e-01,  7.62969971e-01,8.58795166e-01,8.65844727e-01,8.59664917e-01,  8.41812134e-01,8.36090088e-01,8.33847046e-01,8.26019287e-01,  7.73071289e-01,6.75537109e-01,6.56402588e-01,6.88598633e-01,  7.12600708e-01,7.49404907e-01,7.84301758e-01,7.85629272e-01,  7.58331299e-01,7.56317139e-01,7.71194458e-01,7.57995605e-01,  7.65655518e-01,7.66204834e-01,7.65716553e-01,7.60559082e-01,  7.30133057e-01,6.34033203e-01,5.64331055e-01,5.61660767e-01,  5.26763916e-01,5.17807007e-01,5.57159424e-01,5.32836914e-01,  3.47930908e-01,2.05463409e-01,2.14099884e-01,2.24964142e-01,  2.39482880e-01,2.69920349e-01,3.20091248e-01,4.03411865e-01,  4.75013733e-01,4.99443054e-01,5.36865234e-01,5.87478638e-01,  6.17370605e-01,6.26892090e-01,6.21475220e-01,6.13464355e-01,  6.13418579e-01,6.12823486e-01,6.07391357e-01,6.01181030e-01,  5.93704224e-01,5.84838867e-01,5.75653076e-01,5.63705444e-01,  5.48400879e-01,5.28350830e-01,5.07537842e-01,4.93606567e-01,  4.94338989e-01,5.01831055e-01,4.99839783e-01,4.90119934e-01,  4.73587036e-01,4.42001343e-01,4.02740479e-01,3.63212585e-01,  2.95875549e-01,1.89079285e-01,1.07482910e-01,8.31928253e-02,  9.38701630e-02,1.19487762e-01,1.39698029e-01,1.27288818e-01,  1.42173767e-01,1.96308136e-01,2.62794495e-01,3.14414978e-01,  3.43826294e-01,3.52905273e-01,3.47412109e-01,3.44680786e-01,  3.59825134e-01,3.64158630e-01,3.48762512e-01,3.46008301e-01,  3.57299805e-01,3.68835449e-01,3.73420715e-01,3.80119324e-01,  3.89923096e-01,3.92021179e-01,3.84773254e-01,3.71856689e-01,  3.50357056e-01,3.11645508e-01,2.49832153e-01,1.97643280e-01,  2.12463379e-01,2.73468018e-01,3.15086365e-01,3.41178894e-01,  3.48937988e-01,3.41392517e-01,3.24913025e-01,3.02780151e-01,  2.76145935e-01,2.52128601e-01,2.35988617e-01,2.09350586e-01,  1.58973694e-01,1.47590637e-01,1.31473541e-01,7.27653503e-02,  2.27799416e-02,8.14342499e-03,6.47747517e-03,6.18088245e-03,  5.34558296e-03,4.72831726e-03,5.04124165e-03,5.61702251e-03,  5.98800182e-03,6.13641739e-03,6.33335114e-03,6.70492649e-03,  8.76760483e-03,1.13036633e-02,1.47709846e-02,2.07262039e-02,  2.22034454e-02,1.92165375e-02,2.18930244e-02,3.14016342e-02,  4.60224152e-02,6.82601929e-02,7.45658875e-02,5.93585968e-02,  5.19113541e-02,6.37435913e-02,7.65552521e-02,9.56497192e-02,  1.29833221e-01,1.63696289e-01,1.82285309e-01,1.95941925e-01,  2.13245392e-01,2.21584320e-01,2.30567932e-01,2.37224579e-01,  2.39635468e-01,2.40840912e-01,2.43808746e-01,2.49542236e-01,  2.51686096e-01,2.50244141e-01,2.46433258e-01,2.30068207e-01,  2.00141907e-01,1.89525604e-01,2.03826904e-01,2.21366882e-01,  2.27870941e-01,2.18952179e-01,1.95209503e-01,1.87171936e-01,  1.95117950e-01,2.10872650e-01,2.20043182e-01,2.22339630e-01,  2.19844818e-01,2.14950562e-01,2.09426880e-01,2.06050873e-01,  2.03048706e-01,2.01358795e-01,1.95846558e-01,1.85813904e-01,  1.83265686e-01,1.79443359e-01,1.73995972e-01,1.67846680e-01,  1.62277222e-01,1.55975342e-01,1.51706696e-01],
        [2.52180099e-02,2.59232521e-02,2.78525352e-02,3.32565308e-02,  3.56044769e-02,3.85580063e-02,4.00724411e-02,4.20551300e-02,  4.85010147e-02,5.86776733e-02,8.29219818e-02,9.90219116e-02,  1.11246109e-01,1.24181747e-01,1.33106232e-01,1.38633728e-01,  1.46148682e-01,1.68601990e-01,1.95014954e-01,2.22377777e-01,  2.49092102e-01,2.69409180e-01,2.87666321e-01,3.03337097e-01,  3.17565918e-01,3.32916260e-01,3.51776123e-01,3.59901428e-01,  3.80310059e-01,4.01100159e-01,4.05006409e-01,4.14756775e-01,  4.29145813e-01,4.28634644e-01,4.39186096e-01,4.51988220e-01,  4.62020874e-01,4.74128723e-01,4.80781555e-01,4.88700867e-01,  4.97474670e-01,4.99046326e-01,4.89418030e-01,4.77767944e-01,  4.62791443e-01,4.59579468e-01,4.88151550e-01,5.12664795e-01,  5.08987427e-01,5.14709473e-01,5.22537231e-01,5.38314819e-01,  5.53054810e-01,5.67306519e-01,5.79895020e-01,5.66452026e-01,  5.69854736e-01,6.17462158e-01,6.57028198e-01,6.72058105e-01,  6.77917480e-01,6.84906006e-01,7.27416992e-01,7.62344360e-01,  7.79006958e-01,7.91259766e-01,7.84622192e-01,6.78665161e-01,  6.31362915e-01,7.22549438e-01,7.63702393e-01,7.88818359e-01,  7.97897339e-01,7.24273682e-01,6.54418945e-01,6.57150269e-01,  6.99005127e-01,7.70004272e-01,8.08593750e-01,8.28338623e-01,  8.26950073e-01,7.27325439e-01,3.95355225e-01,4.20478821e-01,  7.09594727e-01,8.05068970e-01,8.14300537e-01,8.05938721e-01,  7.89596558e-01,7.85751343e-01,7.81906128e-01,7.77420044e-01,  7.21862793e-01,6.23016357e-01,6.09558105e-01,6.38412476e-01,  6.64672852e-01,7.06604004e-01,7.38372803e-01,7.43499756e-01,  7.18322754e-01,7.12265015e-01,7.32131958e-01,7.19573975e-01,  7.25906372e-01,7.33963013e-01,7.28576660e-01,7.26013184e-01,  6.95434570e-01,5.86303711e-01,5.23590088e-01,5.22247314e-01,  4.77722168e-01,4.75914001e-01,5.13168335e-01,4.85557556e-01,  3.13423157e-01,1.78657532e-01,1.85974121e-01,1.96067810e-01,  2.10151672e-01,2.39707947e-01,2.88566589e-01,3.70941162e-01,  4.42466736e-01,4.67475891e-01,5.06149292e-01,5.58959961e-01,  5.91079712e-01,6.01959229e-01,5.97335815e-01,5.89874268e-01,  5.90133667e-01,5.89797974e-01,5.84777832e-01,5.78765869e-01,  5.71533203e-01,5.62789917e-01,5.53237915e-01,5.40756226e-01,  5.24810791e-01,5.03555298e-01,4.81636047e-01,4.67712402e-01,  4.69871521e-01,4.79148865e-01,4.78385925e-01,4.69169617e-01,  4.52583313e-01,4.19921875e-01,3.79089355e-01,3.38722229e-01,  2.72766113e-01,1.69837952e-01,9.18712616e-02,6.91490173e-02,  7.89928436e-02,1.02556229e-01,1.20756149e-01,1.09407425e-01,  1.23224258e-01,1.74690247e-01,2.40245819e-01,2.91946411e-01,  3.22227478e-01,3.32588196e-01,3.27384949e-01,3.24707031e-01,  3.40484619e-01,3.45268250e-01,3.30062866e-01,3.27529907e-01,  3.38897705e-01,3.50646973e-01,3.55735779e-01,3.63151550e-01,  3.73847961e-01,3.76167297e-01,3.68614197e-01,3.54804993e-01,  3.31649780e-01,2.90786743e-01,2.27394104e-01,1.75395966e-01,  1.90803528e-01,2.53097534e-01,2.97737122e-01,3.25927734e-01,  3.34251404e-01,3.26927185e-01,3.10592651e-01,2.88078308e-01,  2.60742188e-01,2.36396790e-01,2.20245361e-01,1.93798065e-01,  1.44039154e-01,1.32854462e-01,1.17650986e-01,6.34937286e-02,  1.96843147e-02,7.74288177e-03,6.43062592e-03,6.14821911e-03,  5.42271137e-03,4.84848022e-03,5.18083572e-03,5.62739372e-03,  5.97298145e-03,6.15167618e-03,6.30247593e-03,6.53553009e-03,  7.98106194e-03,9.80043411e-03,1.24394894e-02,1.73048973e-02,  1.84841156e-02,1.56903267e-02,1.77764893e-02,2.60686874e-02,  3.87687683e-02,5.83600998e-02,6.43672943e-02,5.09471893e-02,  4.45346832e-02,5.54361343e-02,6.68430328e-02,8.50448608e-02,  1.18587494e-01,1.52568817e-01,1.72065735e-01,1.86279297e-01,  2.03968048e-01,2.12726593e-01,2.22209930e-01,2.29228973e-01,  2.31658936e-01,2.33154297e-01,2.36595154e-01,2.43118286e-01,  2.45883942e-01,2.44670868e-01,2.40787506e-01,2.23445892e-01,  1.92043304e-01,1.81545258e-01,1.96784973e-01,2.15572357e-01,  2.22755432e-01,2.12921143e-01,1.87728882e-01,1.79428101e-01,  1.87801361e-01,2.04597473e-01,2.14862823e-01,2.17365265e-01,  2.14355469e-01,2.09102631e-01,2.02938080e-01,1.99256897e-01,  1.96971893e-01,1.96384430e-01,1.92241669e-01,1.82605743e-01,  1.80198669e-01,1.76242828e-01,1.70734406e-01,1.65592194e-01,  1.59484863e-01,1.55036926e-01,1.52484894e-01],
        [1.95102692e-02,2.05430984e-02,2.12154388e-02,2.53853798e-02,  2.69598961e-02,2.78415680e-02,2.89387703e-02,2.97050476e-02,  3.39698792e-02,4.06408310e-02,5.66644669e-02,6.74934387e-02,  7.53707886e-02,8.39672089e-02,8.99219513e-02,9.36164856e-02,  9.91535187e-02,1.14564896e-01,1.32884979e-01,1.52256012e-01,  1.71390533e-01,1.86832428e-01,2.00439453e-01,2.12631226e-01,  2.23945618e-01,2.35378265e-01,2.50732422e-01,2.59170532e-01,  2.75810242e-01,2.92808533e-01,2.97073364e-01,3.05740356e-01,  3.18481445e-01,3.20121765e-01,3.29841614e-01,3.40332031e-01,  3.48960876e-01,3.60122681e-01,3.66668701e-01,3.74046326e-01,  3.82255554e-01,3.84727478e-01,3.77906799e-01,3.68095398e-01,  3.55026245e-01,3.52134705e-01,3.77593994e-01,4.01016235e-01,  3.99139404e-01,4.04937744e-01,4.13558960e-01,4.28398132e-01,  4.42817688e-01,4.56512451e-01,4.67613220e-01,4.56123352e-01,  4.59259033e-01,5.03646851e-01,5.41503906e-01,5.56030273e-01,  5.63430786e-01,5.72509766e-01,6.11801147e-01,6.44729614e-01,  6.61224365e-01,6.74163818e-01,6.69876099e-01,5.74096680e-01,  5.29647827e-01,6.13662720e-01,6.53335571e-01,6.78695679e-01,  6.88369751e-01,6.20590210e-01,5.55877686e-01,5.57968140e-01,  5.98327637e-01,6.67068481e-01,7.06405640e-01,7.27783203e-01,  7.27523804e-01,6.39663696e-01,3.41148376e-01,3.57994080e-01,  6.22314453e-01,7.13729858e-01,7.22930908e-01,7.15682983e-01,  6.99600220e-01,6.96517944e-01,6.93847656e-01,6.89315796e-01,  6.37863159e-01,5.44876099e-01,5.32012939e-01,5.61813354e-01,  5.86715698e-01,6.27731323e-01,6.62033081e-01,6.69082642e-01,  6.48269653e-01,6.44378662e-01,6.61941528e-01,6.52832031e-01,  6.58935547e-01,6.64932251e-01,6.62338257e-01,6.58416748e-01,  6.27883911e-01,5.23956299e-01,4.61158752e-01,4.60906982e-01,  4.21218872e-01,4.16145325e-01,4.55657959e-01,4.32373047e-01,  2.73002625e-01,1.46907806e-01,1.52614594e-01,1.61689758e-01,  1.74606323e-01,2.02800751e-01,2.49671936e-01,3.29879761e-01,  4.00382996e-01,4.25529480e-01,4.65049744e-01,5.19821167e-01,  5.54107666e-01,5.66238403e-01,5.62698364e-01,5.55938721e-01,  5.56411743e-01,5.56564331e-01,5.52032471e-01,5.46539307e-01,  5.39703369e-01,5.31173706e-01,5.21392822e-01,5.08209229e-01,  4.91355896e-01,4.68864441e-01,4.45701599e-01,4.31694031e-01,  4.35279846e-01,4.46746826e-01,4.47822571e-01,4.39704895e-01,  4.23072815e-01,3.89862061e-01,3.47877502e-01,3.07235718e-01,  2.43530273e-01,1.46274567e-01,7.37686157e-02,5.30014038e-02,  6.18000031e-02,8.25939178e-02,9.83009338e-02,8.84113312e-02,  1.01123810e-01,1.48838043e-01,2.11967468e-01,2.62794495e-01,  2.93716431e-01,3.05328369e-01,3.00537109e-01,2.97874451e-01,  3.14048767e-01,3.19595337e-01,3.04893494e-01,3.02513123e-01,  3.14056396e-01,3.25828552e-01,3.31207275e-01,3.39248657e-01,  3.50723267e-01,3.53385925e-01,3.45405579e-01,3.30406189e-01,  3.05503845e-01,2.62596130e-01,1.98337555e-01,1.47739410e-01,  1.63585663e-01,2.26219177e-01,2.73773193e-01,3.04077148e-01,  3.13102722e-01,3.05999756e-01,2.89978027e-01,2.67463684e-01,  2.39833832e-01,2.15534210e-01,1.99508667e-01,1.73507690e-01,  1.25122070e-01,1.14080429e-01,1.00242615e-01,5.20391464e-02,  1.52413845e-02,6.07454777e-03,5.13243675e-03,4.90093231e-03,  4.15694714e-03,3.63981724e-03,3.90475988e-03,4.29570675e-03,  4.69875336e-03,4.79543209e-03,4.82749939e-03,5.11741638e-03,  6.00647926e-03,7.21943378e-03,9.11235809e-03,1.26092434e-02,  1.35004520e-02,1.10802650e-02,1.23510361e-02,1.90448761e-02,  2.96154022e-02,4.62722778e-02,5.18312454e-02,4.06618118e-02,  3.53584290e-02,4.48579788e-02,5.46808243e-02,7.14721680e-02,  1.03483200e-01,1.36875153e-01,1.57226562e-01,1.72206879e-01,  1.90116882e-01,1.99337006e-01,2.09499359e-01,2.16625214e-01,  2.18883514e-01,2.20561981e-01,2.24662781e-01,2.31891632e-01,  2.35389709e-01,2.34493256e-01,2.30590820e-01,2.12261200e-01,  1.79260254e-01,1.68579102e-01,1.84486389e-01,2.04662323e-01,  2.12654114e-01,2.02018738e-01,1.75460815e-01,1.66648865e-01,  1.75434113e-01,1.92947388e-01,2.03678131e-01,2.06378937e-01,  2.02884674e-01,1.97189331e-01,1.90700531e-01,1.86523438e-01,  1.83597565e-01,1.82426453e-01,1.77314758e-01,1.66366577e-01,  1.64279938e-01,1.60919189e-01,1.55090332e-01,1.49326324e-01,  1.42906189e-01,1.37332916e-01,1.36085510e-01],
        [1.49724483e-02,1.49660110e-02,1.60784721e-02,1.88627243e-02,  1.99589729e-02,2.07204819e-02,2.05168724e-02,2.10685730e-02,  2.35052109e-02,2.76632309e-02,3.81984711e-02,4.51097488e-02,  5.03158569e-02,5.56364059e-02,5.91526031e-02,6.14910126e-02,  6.48994446e-02,7.51285553e-02,8.71753693e-02,1.00152969e-01,  1.13285065e-01,1.24074936e-01,1.34056091e-01,1.43299103e-01,  1.51634216e-01,1.59988403e-01,1.71920776e-01,1.79912567e-01,  1.93420410e-01,2.06993103e-01,2.11399078e-01,2.18681335e-01,  2.29991913e-01,2.33482361e-01,2.42084503e-01,2.50907898e-01,  2.58201599e-01,2.68287659e-01,2.75222778e-01,2.82363892e-01,  2.90443420e-01,2.93754578e-01,2.89024353e-01,2.81158447e-01,  2.69348145e-01,2.66876221e-01,2.90405273e-01,3.12644958e-01,  3.12576294e-01,3.18641663e-01,3.28056335e-01,3.42948914e-01,  3.56872559e-01,3.70567322e-01,3.80599976e-01,3.69567871e-01,  3.73222351e-01,4.15878296e-01,4.53689575e-01,4.68917847e-01,  4.76966858e-01,4.88441467e-01,5.26657104e-01,5.58288574e-01,  5.75714111e-01,5.89691162e-01,5.87524414e-01,4.98092651e-01,  4.54360962e-01,5.35537720e-01,5.75012207e-01,6.01058960e-01,  6.12869263e-01,5.47698975e-01,4.85389709e-01,4.87709045e-01,  5.27847290e-01,5.97808838e-01,6.40136719e-01,6.63803101e-01,  6.65451050e-01,5.83068848e-01,3.03421021e-01,3.15177917e-01,  5.67001343e-01,6.59194946e-01,6.70043945e-01,6.61987305e-01,  6.46697998e-01,6.43539429e-01,6.40731812e-01,6.38610840e-01,  5.85510254e-01,4.93141174e-01,4.83055115e-01,5.12496948e-01,  5.40100098e-01,5.83099365e-01,6.18804932e-01,6.30996704e-01,  6.10702515e-01,6.08078003e-01,6.28265381e-01,6.20361328e-01,  6.28128052e-01,6.33560181e-01,6.28463745e-01,6.25030518e-01,  5.94024658e-01,4.87411499e-01,4.24354553e-01,4.22431946e-01,  3.82247925e-01,3.78753662e-01,4.15199280e-01,3.90167236e-01,  2.34405518e-01,1.16630554e-01,1.20328903e-01,1.28452301e-01,  1.40575409e-01,1.66431427e-01,2.10594177e-01,2.87879944e-01,  3.56559753e-01,3.81385803e-01,4.21768188e-01,4.78416443e-01,  5.14984131e-01,5.29067993e-01,5.26626587e-01,5.20568848e-01,  5.21499634e-01,5.22171021e-01,5.17990112e-01,5.12908936e-01,  5.06500244e-01,4.97848511e-01,4.87846375e-01,4.73991394e-01,  4.55612183e-01,4.31266785e-01,4.06379700e-01,3.92402649e-01,  3.98025513e-01,4.12345886e-01,4.15641785e-01,4.08714294e-01,  3.92730713e-01,3.58810425e-01,3.15536499e-01,2.74459839e-01,  2.13500977e-01,1.22825623e-01,5.69257736e-02,3.87125015e-02,  4.63008881e-02,6.39419556e-02,7.71331787e-02,6.88133240e-02,  7.98225403e-02,1.23079300e-01,1.82968140e-01,2.32860565e-01,  2.64434814e-01,2.77221680e-01,2.73063660e-01,2.70408630e-01,  2.86964417e-01,2.93426514e-01,2.79067993e-01,2.76710510e-01,  2.88368225e-01,3.00056458e-01,3.05610657e-01,3.14369202e-01,  3.26583862e-01,3.29360962e-01,3.20976257e-01,3.04481506e-01,  2.77297974e-01,2.32067108e-01,1.67240143e-01,1.18648529e-01,  1.34719849e-01,1.96949005e-01,2.47695923e-01,2.80952454e-01,  2.91259766e-01,2.84805298e-01,2.69447327e-01,2.46650696e-01,  2.18456268e-01,1.93992615e-01,1.78115845e-01,1.52809143e-01,  1.06357574e-01,9.55505371e-02,8.31985474e-02,4.15439606e-02,  1.14374161e-02,4.65893745e-03,3.94463539e-03,3.68297100e-03,  3.12548876e-03,2.57658958e-03,2.91126966e-03,3.40574980e-03,  3.70556116e-03,3.78739834e-03,3.74799967e-03,3.87722254e-03,  4.48298454e-03,5.23769855e-03,6.42824173e-03,8.84413719e-03,  9.38224792e-03,7.49754906e-03,8.47816467e-03,1.34503841e-02,  2.16350555e-02,3.51867676e-02,4.00009155e-02,3.11598778e-02,  2.70738602e-02,3.51276398e-02,4.33330536e-02,5.85422516e-02,  8.86688232e-02,1.21215820e-01,1.42181396e-01,1.58023834e-01,  1.76296234e-01,1.86016083e-01,1.97124481e-01,2.04406738e-01,  2.06485748e-01,2.08290100e-01,2.12963104e-01,2.21233368e-01,  2.25616455e-01,2.25196838e-01,2.21176147e-01,2.01435089e-01,  1.66393280e-01,1.55548096e-01,1.72710419e-01,1.94534302e-01,  2.03334808e-01,1.91696167e-01,1.63120270e-01,1.54041290e-01,  1.63227081e-01,1.81827545e-01,1.93496704e-01,1.96048737e-01,  1.92264557e-01,1.85840607e-01,1.78764343e-01,1.74438477e-01,  1.70780182e-01,1.70021057e-01,1.64577484e-01,1.53099060e-01,  1.50173187e-01,1.46446228e-01,1.40899658e-01,1.35665894e-01,  1.29085541e-01,1.23401642e-01,1.20685577e-01],
        [1.06706619e-02,1.09853745e-02,1.16293430e-02,1.32491589e-02,  1.41165257e-02,1.43530369e-02,1.45323277e-02,1.46806240e-02,  1.61147118e-02,1.89213753e-02,2.56171227e-02,2.97927856e-02,  3.26976776e-02,3.54890823e-02,3.73029709e-02,3.81622314e-02,  3.98302078e-02,4.56199646e-02,5.24892807e-02,6.00223541e-02,  6.75029755e-02,7.36446381e-02,7.94811249e-02,8.48560333e-02,  8.99848938e-02,9.49668884e-02,1.02554321e-01,1.08192444e-01,  1.17122650e-01,1.26289368e-01,1.29577637e-01,1.34700775e-01,  1.42963409e-01,1.46331787e-01,1.52801514e-01,1.59088135e-01,  1.64596558e-01,1.72561646e-01,1.78344727e-01,1.84177399e-01,  1.91051483e-01,1.94602966e-01,1.92108154e-01,1.86424255e-01,  1.77200317e-01,1.75106049e-01,1.94240570e-01,2.13157654e-01,  2.13989258e-01,2.19833374e-01,2.29412079e-01,2.42427826e-01,  2.54661560e-01,2.66166687e-01,2.73666382e-01,2.64312744e-01,  2.67234802e-01,3.03855896e-01,3.37738037e-01,3.51142883e-01,  3.58985901e-01,3.71543884e-01,4.05052185e-01,4.33341980e-01,  4.49890137e-01,4.63356018e-01,4.63333130e-01,3.87672424e-01,  3.48197937e-01,4.17541504e-01,4.52941895e-01,4.78012085e-01,  4.90203857e-01,4.31831360e-01,3.76121521e-01,3.77738953e-01,  4.15016174e-01,4.82040405e-01,5.25177002e-01,5.50292969e-01,  5.53634644e-01,4.84092712e-01,2.45857239e-01,2.51762390e-01,  4.71405029e-01,5.56930542e-01,5.68649292e-01,5.60913086e-01,  5.45730591e-01,5.42587280e-01,5.40222168e-01,5.39123535e-01,  4.88403320e-01,4.01901245e-01,3.94157410e-01,4.23461914e-01,  4.50210571e-01,4.92637634e-01,5.31158447e-01,5.45852661e-01,  5.31066895e-01,5.31097412e-01,5.49438477e-01,5.43716431e-01,  5.52581787e-01,5.57022095e-01,5.52917480e-01,5.48873901e-01,  5.14511108e-01,4.10949707e-01,3.51158142e-01,3.47183228e-01,  3.09471130e-01,3.04061890e-01,3.41102600e-01,3.18855286e-01,  1.82033539e-01,8.07704926e-02,8.19473267e-02,8.81271362e-02,  9.84115601e-02,1.20874405e-01,1.59748077e-01,2.30445862e-01,  2.94593811e-01,3.18618774e-01,3.59474182e-01,4.17526245e-01,  4.56817627e-01,4.73159790e-01,4.72320557e-01,4.67323303e-01,  4.69001770e-01,4.70314026e-01,4.66804504e-01,4.62013245e-01,  4.56184387e-01,4.48020935e-01,4.37530518e-01,4.23118591e-01,  4.03404236e-01,3.77334595e-01,3.51219177e-01,3.37226868e-01,  3.44963074e-01,3.61915588e-01,3.67774963e-01,3.62342834e-01,  3.46900940e-01,3.12522888e-01,2.68264771e-01,2.27745056e-01,  1.72042847e-01,9.30881500e-02,3.80897522e-02,2.38318443e-02,  2.96115875e-02,4.23097610e-02,5.16662598e-02,4.58221436e-02,  5.42030334e-02,8.97827148e-02,1.43211365e-01,1.90402985e-01,  2.22187042e-01,2.36694336e-01,2.33062744e-01,2.30384827e-01,  2.46944427e-01,2.54127502e-01,2.41016388e-01,2.39097595e-01,  2.50473022e-01,2.61878967e-01,2.68234253e-01,2.77778625e-01,  2.90588379e-01,2.93769836e-01,2.84934998e-01,2.67135620e-01,  2.38014221e-01,1.91852570e-01,1.29753113e-01,8.61816406e-02,  1.01528168e-01,1.60095215e-01,2.12329865e-01,2.47356415e-01,  2.58209229e-01,2.52281189e-01,2.37762451e-01,2.15286255e-01,  1.86901093e-01,1.62883759e-01,1.47605896e-01,1.23733521e-01,  8.11710358e-02,7.08827972e-02,6.08577728e-02,2.90555954e-02,  8.01205635e-03,3.81433964e-03,3.41165066e-03,3.30764055e-03,  2.76154280e-03,2.19291449e-03,2.36898661e-03,2.69937515e-03,  2.96688080e-03,3.17043066e-03,3.18503380e-03,3.43734026e-03,  3.75908613e-03,3.99684906e-03,4.57262993e-03,5.77414036e-03,  5.82444668e-03,4.66418266e-03,5.27513027e-03,8.32557678e-03,  1.35850906e-02,2.26235390e-02,2.63695717e-02,2.05307007e-02,  1.81665421e-02,2.38637924e-02,2.95543671e-02,4.22058105e-02,  6.88419342e-02,9.91992950e-02,1.20391846e-01,1.36978149e-01,  1.55357361e-01,1.65611267e-01,1.77673340e-01,1.85195923e-01,  1.87427521e-01,1.89884186e-01,1.95365906e-01,2.04681396e-01,  2.10105896e-01,2.10456848e-01,2.06287384e-01,1.85188293e-01,  1.48731232e-01,1.38122559e-01,1.56242371e-01,1.79565430e-01,  1.89182281e-01,1.76460266e-01,1.46369934e-01,1.36943817e-01,  1.46579742e-01,1.66213989e-01,1.79103851e-01,1.81602478e-01,  1.77185059e-01,1.70398712e-01,1.62765503e-01,1.57836914e-01,  1.54487610e-01,1.54708862e-01,1.49902344e-01,1.38111115e-01,  1.35135651e-01,1.31454468e-01,1.26712799e-01,1.21259689e-01,  1.15535736e-01,1.10025406e-01,1.07645035e-01],
        [6.87313080e-03,7.21621513e-03,7.53581524e-03,8.73970985e-03,  9.44924355e-03,9.54031944e-03,9.57369804e-03,9.65714455e-03,  1.05776787e-02,1.22919083e-02,1.64895058e-02,1.89256668e-02,  2.03790665e-02,2.16808319e-02,2.22845078e-02,2.21710205e-02,  2.24299431e-02,2.49195099e-02,2.77833939e-02,3.07464600e-02,  3.34882736e-02,3.53193283e-02,3.68986130e-02,3.82280350e-02,  3.94296646e-02,4.05664444e-02,4.24308777e-02,4.34932709e-02,  4.60062027e-02,4.85477448e-02,4.87365723e-02,4.96950150e-02,  5.19685745e-02,5.25379181e-02,5.41429520e-02,5.55734634e-02,  5.67731857e-02,5.90772629e-02,6.06288910e-02,6.22129440e-02,  6.42204285e-02,6.50901794e-02,6.38084412e-02,6.12659454e-02,  5.74693680e-02,5.63678741e-02,6.30455017e-02,6.97231293e-02,  6.97822571e-02,7.17830658e-02,7.54585266e-02,8.03508759e-02,  8.48045349e-02,8.89320374e-02,9.13677216e-02,8.75263214e-02,  8.83712769e-02,1.02186203e-01,1.15303040e-01,1.20172501e-01,  1.23155594e-01,1.28643036e-01,1.41822815e-01,1.52988434e-01,  1.59679413e-01,1.65073395e-01,1.65321350e-01,1.36447906e-01,  1.20042801e-01,1.45633698e-01,1.59156799e-01,1.69433594e-01,  1.74713135e-01,1.50924683e-01,1.28051758e-01,1.28021240e-01,  1.43211365e-01,1.71440125e-01,1.90856934e-01,2.02632904e-01,  2.04631805e-01,1.78287506e-01,8.85925293e-02,8.91399384e-02,  1.73225403e-01,2.07996368e-01,2.12909698e-01,2.08980560e-01,  2.01869965e-01,2.00275421e-01,1.98596954e-01,1.98249817e-01,  1.76761627e-01,1.40655518e-01,1.37805939e-01,1.50100708e-01,  1.61643982e-01,1.80511475e-01,1.98162079e-01,2.06104279e-01,  2.01690674e-01,2.02590942e-01,2.10395813e-01,2.08934784e-01,  2.12898254e-01,2.14771271e-01,2.12711334e-01,2.10460663e-01,  1.94473267e-01,1.48578644e-01,1.22346878e-01,1.21185303e-01,  1.05789185e-01,1.02766037e-01,1.18579865e-01,1.08423233e-01,  5.57632446e-02,2.11844444e-02,2.05316544e-02,2.20127106e-02,  2.50387192e-02,3.20997238e-02,4.50830460e-02,7.00988770e-02,  9.33380127e-02,1.02523804e-01,1.19112015e-01,1.43302917e-01,  1.60202026e-01,1.67865753e-01,1.68125153e-01,1.66400909e-01,  1.67240143e-01,1.68075562e-01,1.66923523e-01,1.65203094e-01,  1.63166046e-01,1.59893036e-01,1.55498505e-01,1.49307251e-01,  1.40724182e-01,1.29638672e-01,1.18730545e-01,1.13157272e-01,  1.16876602e-01,1.24578476e-01,1.27647400e-01,1.25934601e-01,  1.19838715e-01,1.05737686e-01,8.75415802e-02,7.15847015e-02,  5.19132614e-02,2.61368752e-02,9.39130783e-03,5.42998314e-03,  6.84273243e-03,9.89365578e-03,1.20689869e-02,1.06968880e-02,  1.29017830e-02,2.35295296e-02,4.14066315e-02,5.85546494e-02,  7.11135864e-02,7.75585175e-02,7.61127472e-02,7.48023987e-02,  8.15105438e-02,8.47816467e-02,8.01334381e-02,7.98473358e-02,  8.40969086e-02,8.85639191e-02,9.14859772e-02,9.54475403e-02,  1.00736618e-01,1.02195740e-01,9.86938477e-02,9.11827087e-02,  7.91034698e-02,6.08129501e-02,3.80020142e-02,2.32405663e-02,  2.86555290e-02,4.94871140e-02,7.00836182e-02,8.46176147e-02,  8.91075134e-02,8.68530273e-02,8.13407898e-02,7.22293854e-02,  6.06708527e-02,5.13277054e-02,4.54816818e-02,3.68213654e-02,  2.22654343e-02,1.85489655e-02,1.57828331e-02,7.32159615e-03,  2.11143494e-03,1.15916133e-03,1.07112527e-03,1.04126334e-03,  9.10073519e-04,6.99877739e-04,7.68840313e-04,8.72418284e-04,  9.86099243e-04,1.01441145e-03,1.04230642e-03,1.09022856e-03,  1.21164322e-03,1.32060051e-03,1.46049261e-03,1.61868334e-03,  1.50382519e-03,1.23131275e-03,1.38506293e-03,1.91605091e-03,  2.95436382e-03,4.96685505e-03,5.93054295e-03,4.80818748e-03,  4.41122055e-03,5.69474697e-03,6.95335865e-03,1.07538700e-02,  1.92627907e-02,2.98547745e-02,3.81412506e-02,4.50420380e-02,  5.22584915e-02,5.65052032e-02,6.18238449e-02,6.50386810e-02,  6.60858154e-02,6.73847198e-02,6.98871613e-02,7.40852356e-02,  7.67421722e-02,7.72361755e-02,7.56168365e-02,6.67915344e-02,  5.17225266e-02,4.75025177e-02,5.50489426e-02,6.49108887e-02,  6.91051483e-02,6.36425018e-02,5.11569977e-02,4.73451614e-02,  5.12771606e-02,5.93681335e-02,6.48212433e-02,6.58721924e-02,  6.38999939e-02,6.10523224e-02,5.80368042e-02,5.59864044e-02,  5.45272827e-02,5.47056198e-02,5.27858734e-02,4.83713150e-02,  4.76799011e-02,4.70113754e-02,4.51812744e-02,4.30669785e-02,  4.06026840e-02,3.84178162e-02,3.76501083e-02],
        [4.39155102e-03,4.64224815e-03,4.92477417e-03,5.71000576e-03,  6.14571571e-03,6.26266003e-03,6.26420975e-03,6.35194778e-03,  6.92903996e-03,8.06784630e-03,1.08962059e-02,1.24788284e-02,  1.33929253e-02,1.41441822e-02,1.44414902e-02,1.42502785e-02,  1.42545700e-02,1.56974792e-02,1.72777176e-02,1.88641548e-02,  2.01778412e-02,2.08492279e-02,2.13184357e-02,2.15835571e-02,  2.18076706e-02,2.19945908e-02,2.21648216e-02,2.18639374e-02,  2.23603249e-02,2.27675438e-02,2.20031738e-02,2.16283798e-02,  2.18319893e-02,2.12130547e-02,2.09903717e-02,2.07457542e-02,  2.02956200e-02,2.02689171e-02,2.00581551e-02,1.98101997e-02,  1.96604729e-02,1.91974640e-02,1.81365013e-02,1.69653893e-02,  1.57194138e-02,1.51965618e-02,1.62672997e-02,1.73277855e-02,  1.70550346e-02,1.68480873e-02,1.67999268e-02,1.72743797e-02,  1.78961754e-02,1.85508728e-02,1.89070702e-02,1.81403160e-02,  1.81651115e-02,2.02822685e-02,2.22597122e-02,2.30231285e-02,  2.33569145e-02,2.38089561e-02,2.55732536e-02,2.70438194e-02,  2.78205872e-02,2.83203125e-02,2.80833244e-02,2.34899521e-02,  2.10328102e-02,2.48317719e-02,2.65245438e-02,2.75549889e-02,  2.79331207e-02,2.46782303e-02,2.16131210e-02,2.15182304e-02,  2.31404305e-02,2.61569023e-02,2.79531479e-02,2.89320946e-02,  2.88276672e-02,2.49557495e-02,1.29146576e-02,1.31587982e-02,  2.36515999e-02,2.76904106e-02,2.80847549e-02,2.73628235e-02,  2.64897346e-02,2.60329247e-02,2.56900787e-02,2.53429413e-02,  2.26898193e-02,1.88336372e-02,1.84202194e-02,1.93271637e-02,  2.04238892e-02,2.19502449e-02,2.36778259e-02,2.43782997e-02,  2.36220360e-02,2.35362053e-02,2.42724419e-02,2.36406326e-02,  2.42929459e-02,2.45356560e-02,2.38881111e-02,2.36659050e-02,  2.19106674e-02,1.67717934e-02,1.43828392e-02,1.41062737e-02,  1.29065514e-02,1.25391483e-02,1.32436752e-02,1.23441219e-02,  7.34353065e-03,3.71611118e-03,3.64464521e-03,3.76725197e-03,  4.04787064e-03,4.71365452e-03,5.94508648e-03,8.31365585e-03,  1.04703903e-02,1.12888813e-02,1.28474236e-02,1.51274204e-02,  1.67069435e-02,1.74098015e-02,1.73711777e-02,1.71294212e-02,  1.71432495e-02,1.71637535e-02,1.69615746e-02,1.66954994e-02,  1.64008141e-02,1.59716606e-02,1.54294968e-02,1.46963596e-02,  1.37388706e-02,1.25145912e-02,1.13351345e-02,1.07192993e-02,  1.10938549e-02,1.19171143e-02,1.22938156e-02,1.21419430e-02,  1.15768909e-02,1.02677345e-02,8.58259201e-03,7.11488724e-03,  5.35404682e-03,3.04764509e-03,1.51786208e-03,1.13302469e-03,  1.25065446e-03,1.54626369e-03,1.77744031e-03,1.62759423e-03,  1.83048844e-03,2.72560120e-03,4.18448448e-03,5.62000275e-03,  6.71517849e-03,7.28249550e-03,7.11357594e-03,6.95419312e-03,  7.51376152e-03,7.78317451e-03,7.32362270e-03,7.22885132e-03,  7.59303570e-03,7.98487663e-03,8.18586349e-03,8.50462914e-03,  8.92949104e-03,9.01174545e-03,8.63695145e-03,7.88569450e-03,  6.74176216e-03,5.11550903e-03,3.23247910e-03,2.07549334e-03,  2.50345469e-03,4.19461727e-03,5.97405434e-03,7.30597973e-03,  7.76183605e-03,7.59124756e-03,7.13467598e-03,6.35182858e-03,  5.34319878e-03,4.53186035e-03,4.02951241e-03,3.32599878e-03,  2.17163563e-03,1.87313557e-03,1.64544582e-03,9.38326120e-04,  3.97548079e-04,2.40381807e-04,2.07711011e-04,1.95045024e-04,  1.78977847e-04,1.65924430e-04,1.71136111e-04,1.83258206e-04,  2.00990587e-04,2.06936151e-04,2.07316130e-04,2.22131610e-04,  2.56940722e-04,2.91392207e-04,3.18795443e-04,3.62358987e-04,  3.45483422e-04,3.00541520e-04,3.39686871e-04,4.11428511e-04,  5.31241298e-04,7.24956393e-04,8.03112984e-04,6.87077641e-04,  6.37188554e-04,7.38039613e-04,8.54700804e-04,1.14515424e-03,  1.76262856e-03,2.54571438e-03,3.16971540e-03,3.71593237e-03,  4.28295135e-03,4.59241867e-03,5.00953197e-03,5.27155399e-03,  5.36251068e-03,5.47814369e-03,5.69486618e-03,6.08086586e-03,  6.34479523e-03,6.38794899e-03,6.24442101e-03,5.46061993e-03,  4.15766239e-03,3.79538536e-03,4.44495678e-03,5.31625748e-03,  5.69009781e-03,5.21421432e-03,4.13429737e-03,3.81505489e-03,  4.14454937e-03,4.81009483e-03,5.29849529e-03,5.42747974e-03,  5.28264046e-03,5.05876541e-03,4.81867790e-03,4.70066071e-03,  4.65583801e-03,4.86314297e-03,4.92393970e-03,4.69446182e-03,  4.79304790e-03,4.82010841e-03,4.82797623e-03,4.76086140e-03,  4.62591648e-03,4.53686714e-03,4.50706482e-03],
    ]),
    index=weather.index,
    columns=[350.0, 355.0, 360.0, 365.0, 370.0, 375.0, 380.0, 385.0, 390.0, 395.0, 400.0, 405.0, 410.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440.0, 445.0, 450.0, 455.0, 460.0, 465.0, 470.0, 475.0, 480.0, 485.0, 490.0, 495.0, 500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545.0, 550.0, 555.0, 560.0, 565.0, 570.0, 575.0, 580.0, 585.0, 590.0, 595.0, 600.0, 605.0, 610.0, 615.0, 620.0, 625.0, 630.0, 635.0, 640.0, 645.0, 650.0, 655.0, 660.0, 665.0, 670.0, 675.0, 680.0, 685.0, 690.0, 695.0, 700.0, 705.0, 710.0, 715.0, 720.0, 725.0, 730.0, 735.0, 740.0, 745.0, 750.0, 755.0, 760.0, 765.0, 770.0, 775.0, 780.0, 785.0, 790.0, 795.0, 800.0, 805.0, 810.0, 815.0, 820.0, 825.0, 830.0, 835.0, 840.0, 845.0, 850.0, 855.0, 860.0, 865.0, 870.0, 875.0, 880.0, 885.0, 890.0, 895.0, 900.0, 905.0, 910.0, 915.0, 920.0, 925.0, 930.0, 935.0, 940.0, 945.0, 950.0, 955.0, 960.0, 965.0, 970.0, 975.0, 980.0, 985.0, 990.0, 995.0, 1000.0, 1005.0, 1010.0, 1015.0, 1020.0, 1025.0, 1030.0, 1035.0, 1040.0, 1045.0, 1050.0, 1055.0, 1060.0, 1065.0, 1070.0, 1075.0, 1080.0, 1085.0, 1090.0, 1095.0, 1100.0, 1105.0, 1110.0, 1115.0, 1120.0, 1125.0, 1130.0, 1135.0, 1140.0, 1145.0, 1150.0, 1155.0, 1160.0, 1165.0, 1170.0, 1175.0, 1180.0, 1185.0, 1190.0, 1195.0, 1200.0, 1205.0, 1210.0, 1215.0, 1220.0, 1225.0, 1230.0, 1235.0, 1240.0, 1245.0, 1250.0, 1255.0, 1260.0, 1265.0, 1270.0, 1275.0, 1280.0, 1285.0, 1290.0, 1295.0, 1300.0, 1305.0, 1310.0, 1315.0, 1320.0, 1325.0, 1330.0, 1335.0, 1340.0, 1345.0, 1350.0, 1355.0, 1360.0, 1365.0, 1370.0, 1375.0, 1380.0, 1385.0, 1390.0, 1395.0, 1400.0, 1405.0, 1410.0, 1415.0, 1420.0, 1425.0, 1430.0, 1435.0, 1440.0, 1445.0, 1450.0, 1455.0, 1460.0, 1465.0, 1470.0, 1475.0, 1480.0, 1485.0, 1490.0, 1495.0, 1500.0, 1505.0, 1510.0, 1515.0, 1520.0, 1525.0, 1530.0, 1535.0, 1540.0, 1545.0, 1550.0, 1555.0, 1560.0, 1565.0, 1570.0, 1575.0, 1580.0, 1585.0, 1590.0, 1595.0, 1600.0, 1605.0, 1610.0, 1615.0, 1620.0, 1625.0, 1630.0, 1635.0, 1640.0, 1645.0, 1650.0, 1655.0, 1660.0, 1665.0, 1670.0, 1675.0, 1680.0, 1685.0, 1690.0, 1695.0, 1700.0],
)

# PVfit has a spectral-irradiance class for spectra on a finite wavelength interval.
spectra_to_extend = SpectralIrradiance(
    lambda_nm=spectra.columns,
    E_W_per_m2_nm=spectra.to_numpy(),
)

# Because we must scale each spectrum to integrate to corresponding broadband total effective irradiance,
# we extrapolate all spectra using scaled STC spectrum.
spectra_extended = extrapolate_spectral_irradiance(
    spectral_irradiance=spectra_to_extend,
    extrapolant=E_60904_3_hemispherical_tilted,  # STC spectrum (with tail to infinity) scaled to integrate to 1000 W/m2.
    E_total_W_per_m2=poa_components["total_eff"].to_numpy(),
)

#### Plot example extended spectra

In [ ]:
idx_mid = 58

fig, ax = pyplot.subplots(nrows=3, ncols=1)

# Dawn
ax[0].plot(spectra_extended.lambda_nm, spectra_extended.E_W_per_m2_nm[0,:], '-', label="Extrapolation")
ax[0].plot(
    spectra_to_extend.lambda_nm,
    spectra_extended.get_E_total_subinterval_W_per_m2(
            lambda_min_nm=spectra_to_extend.lambda_nm[0],
            lambda_max_nm=spectra_to_extend.lambda_nm[-1]
        )[0] / spectra_to_extend.E_total_W_per_m2[0] * spectra_to_extend.E_W_per_m2_nm[0,:],
    '-', label="Measurement",
)
ax[0].annotate(
    f"Integrates to $E_\\mathrm{{eff}}$={spectra_extended.E_total_W_per_m2[0]:.3g} W/m$^2$,\n"
    f"with {spectra_extended.E_tail_W_per_m2[0]:.3g} W/m$^2$ for $\\lambda>4$ $\\mu$m.",
    (1150, 0.55e-5),
)
ax[0].set_ylabel("$E_\\lambda$ [W/m$^2$/nm]")
ax[0].set_title("Measured Spectrum Extrapolation Dawn")
ax[0].legend(loc="upper right")

# Noon
ax[1].plot(spectra_extended.lambda_nm, spectra_extended.E_W_per_m2_nm[idx_mid,:], '-', label="Extrapolation")
ax[1].plot(
    spectra_to_extend.lambda_nm,
    spectra_extended.get_E_total_subinterval_W_per_m2(
            lambda_min_nm=spectra_to_extend.lambda_nm[0],
            lambda_max_nm=spectra_to_extend.lambda_nm[-1]
        )[idx_mid] / spectra_to_extend.E_total_W_per_m2[idx_mid] * spectra_to_extend.E_W_per_m2_nm[idx_mid,:],
    '-', label="Measurement",
)
ax[1].annotate(
    f"Integrates to $E_\\mathrm{{eff}}$={spectra_extended.E_total_W_per_m2[idx_mid]:.1f} W/m$^2$,\n"
    f"with {spectra_extended.E_tail_W_per_m2[idx_mid]:.3g} W/m$^2$ for $\\lambda>4$ $\\mu$m.",
    (1150, 0.725),
)
ax[1].set_ylabel("$E_\\lambda$ [W/m$^2$/nm]")
ax[1].set_title("Measured Spectrum Extrapolation Midday")
ax[1].legend(loc="upper right")

# Dusk
ax[2].plot(spectra_extended.lambda_nm, spectra_extended.E_W_per_m2_nm[-1,:], '-', label="Extrapolation")
ax[2].plot(
    spectra_to_extend.lambda_nm,
    spectra_extended.get_E_total_subinterval_W_per_m2(
            lambda_min_nm=spectra_to_extend.lambda_nm[0],
            lambda_max_nm=spectra_to_extend.lambda_nm[-1]
        )[-1] / spectra_to_extend.E_total_W_per_m2[-1] * spectra_to_extend.E_W_per_m2_nm[-1,:],
    '-', label="Measurement",
)
ax[2].annotate(
    f"Integrates to $E_\\mathrm{{eff}}$={spectra_extended.E_total_W_per_m2[-1]:.3g} W/m$^2$,\n"
    f"with {spectra_extended.E_tail_W_per_m2[-1]:.3g} W/m$^2$ for $\\lambda>4$ $\\mu$m.",
    (1150, 2.25e-4)
)
ax[2].set_ylabel("$E_\\lambda$ [W/m$^2$/nm]")
ax[2].set_title("Measured Spectrum Extrapolation Dusk")
ax[2].legend(loc="upper right")
ax[2].set_xlabel("Wavelength [nm]")

fig.tight_layout()

#### Computation of F

In [ ]:
F = inner_product(f1=spectral_responsivity, f2=spectra_extended) / \
    inner_product(f1=spectral_responsivity, f2=E_60904_3_hemispherical_tilted) * \
    (1 + module_spec["dI_sc_dT_A_per_degC_0"] / module_spec["I_sc_A_0"] * (T_degC - T_degC_stc))

##### Plot

In [ ]:
fig, ax = pyplot.subplots()
ax.plot_date(weather.index, F, '.', label="F")
ax.legend(loc="upper left")
ax.set_title("Effective Irradiance Ratio")
ax.set_ylabel("$F$ [1]")
fig.autofmt_xdate()

### Maximum Power

In [ ]:
# Compute the maximum DC power for the F-T timeseries. Note that power is selected from larger result dictionary.
P_mp_W = sde_simple_sim.P_mp(
    model_parameters=sdm_simple_ae.compute_sde_model_parameters(
        ft_data=FTData(F=F, T_degC=T_degC),
        model_parameters=sdm_simple_fit_matrix["model_parameters"],
    ),
)["P_mp_W"]


# These are the "fenceposts" of the piecewise-constant power, used to plot accumulated total energy.
energy_timestamps = pandas.date_range(start="2014-01-16 07:10:00", end="2014-01-16 17:15:00", freq="5min", tz="Etc/GMT+7")
energy_W_h = 5 / 60 * numpy.hstack((0.0, numpy.cumsum(P_mp_W)))

power_sim_5_min = pandas.DataFrame(
    data={
        "F": F,
        "T_degC": T_degC,
        "P_mp_W": P_mp_W,
        "energy_W_h": energy_W_h[:-1],
    },
    index=weather.index,
)
power_sim_5_min

##### Summary Plots

In [ ]:
fig, ax_left = pyplot.subplots(nrows=2, ncols=1)
ax_left[0].plot(weather.index, power_sim_5_min["F"], '.', label="F")
ax_left[0].set_ylabel("$F$ [1]")
ax_left[0].legend(loc="upper left")

ax_right = ax_left[0].twinx()
color = next(ax_left[0]._get_lines.prop_cycler)["color"]
ax_right.plot(weather.index, power_sim_5_min["T_degC"], 'x', label="T_degC", color=color)
ax_right.set_ylabel("$T$ [$^\\circ$C]")
ax_right.legend(loc="upper right")

color = next(ax_left[0]._get_lines.prop_cycler)["color"]
ax_left[1].plot(weather.index, power_sim_5_min["P_mp_W"], '.', label="P_pm_W", color=color)
ax_left[1].set_ylabel("$P_\\mathrm{mp}$ [W]")
ax_left[1].legend(loc="upper left")

ax_right = ax_left[1].twinx()
color = next(ax_left[0]._get_lines.prop_cycler)["color"]
color = next(ax_left[0]._get_lines.prop_cycler)["color"]  # Avoid red next to green.
ax_right.plot(energy_timestamps, power_sim_5_min["energy_W_h"], 'x-', label="energy_W_h", color=color)
ax_right.set_ylabel("Energy [W$\\cdot$h]")
ax_right.legend(loc=(0.525, 0.275))

ax_left[0].set_title('Average 5-min $(F,T)$ "weather" and $P_\\mathrm{mp}$ & Energy')
fig.autofmt_xdate()
fig.tight_layout()